## 数据准备

### 获取降水数据

In [ ]:
import xarray as xr
import numpy as np
source_zarr = "D:\\Data_Store\\Dataset\\Original_Data\\0.1_Data.zarr"
output_npy_path = "D:\\Data_Store\\Dataset\\Reconstruction\\37SM\\hourlyPrecipRateGC.npy"
ds = xr.open_zarr(source_zarr)
hourlyPrecipRateGC = ds['hourlyPrecipRateGC'].values
np.save(output_npy_path, hourlyPrecipRateGC)

data = np.load("D:\\Data_Store\\Dataset\\Reconstruction\\37SM\\hourlyPrecipRateGC.npy")
print(data.shape)

### (弃用) 站点指标cal（单网络）

In [ ]:
import sys
sys.path.append(r"C:\Users\Administrator\Desktop\code\ST-Conv")
import argparse
import torch
import pandas as pd
import xarray as xr
import numpy as np
from Original_Point_Data.tool.origin_utils import OriginUtils
from tool.utils import Util
from metrics.metrics import CustomMetricCollection

def cal_point_data(cal_time, mode, csv_path, reconstruct_path, start_time,find_mode='0.1'):
    # 点位信息
    ali_list = OriginUtils.find_grid_points(mode, csv_path,find_mode=find_mode)
    point_list = [[item[2], item[1], item[0]] for item in ali_list]  # 重新排列为 [Latitude, Longitude, 'Point Name']
    point_list.sort(key=lambda x: (-x[0], x[1])) # 纬度降序，经度升序

    # 指标字典
    metrics_dict = {}
    # 数据列表
    point_data_list = []
    for lat, lon, point in point_list:      ## ！！！！这里似乎逻辑有问题
        # 点位数据和lon_ind, lat_ind
        point_data = OriginUtils.cal_point_data_specific(ali_list, point)
        _, (lat_ind, lon_ind) = OriginUtils.find_grid_coordinates(lon, lat)

        # 重建数据并进行时间对齐
        reconstruct_data = np.load(reconstruct_path)[:, lat_ind, lon_ind]
        reconstruct_data = np.where(reconstruct_data < 0, 0, reconstruct_data)
        aligned_data = OriginUtils.align_time_series(point_data, reconstruct_data, start_time)
        if cal_time == '12':
            aligned_data['Time'] = pd.to_datetime(aligned_data['Time'])
            aligned_data = aligned_data[aligned_data['Time'].dt.hour.isin([6, 18])]
        point_data_list.append(aligned_data)

        # 评价指标
        metrics_collector = CustomMetricCollection()
        if not aligned_data.empty:
            valid_data = aligned_data.dropna(subset=['Point Data SM', 'Reconstructed SM'])
            if not valid_data.empty:
                output = torch.tensor(valid_data['Reconstructed SM'].values)
                target = torch.tensor(valid_data['Point Data SM'].values)
                metrics_collector.update(output, target)

                metrics_dict[f"{point}_{lon}_{lat}"] = metrics_collector.calculate_averages()
            else:
                metrics_dict[f"{point}_{lon}_{lat}"] = {metric: np.nan for metric in CustomMetricCollection().metrics.keys()}
        metrics_collector.reset()
    return point_list, point_data_list , metrics_dict

def main(args):
    Original_Point_path_argu = Util.load_config(args.Original_Point_path_config)
    path_argu = Util.load_config(args.path_config_path)
    return Original_Point_path_argu, path_argu

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--Original_Point_path_config', type=str, default='../Original_Point_Data/config/path_config.yaml')
    parser.add_argument('--path_config_path', type=str, default='../config/path_config.yaml')
    args = parser.parse_known_args()[0]
    Original_Point_path_argu, path_argu = main(args)
    
    # 初始指标
    start_time = '2016-01-01 03:00:00'
    mode = 'Shiquanhe'  #（Shiquanhe、Ali、Maqu、Naqu、CTP）
    cal_time = '3'
    csv_path = Original_Point_path_argu['info']

    #reconstruct_path = r"D:\Data_Store\Dataset\Reconstruction\RF\RF_reconstructed_data.npy"
    reconstruct_path = r"D:\Data_Store\Dataset\Reconstruction\LSTM\LSTM1.npy"
    point_list, point_data_list , metrics_dict = cal_point_data(cal_time, mode, csv_path, reconstruct_path, start_time,find_mode='0.1')

### (弃用) 查看偏移是否更优

In [ ]:
import sys
sys.path.append(r"C:\Users\Administrator\Desktop\code\ST-Conv")
import argparse
import torch
import os
import pandas as pd
import xarray as xr
import numpy as np
from Original_Point_Data.tool.origin_utils import OriginUtils
from tool.utils import Util
from metrics.metrics import CustomMetricCollection

def tensor_to_scalar(value):
    """如果值是tensor，则转换为标量"""
    if isinstance(value, torch.Tensor):
        return value.item()
    return value

def find_best_shift(output, target, metrics_collector, max_shift=1600):
    best_metrics = None
    best_shift = 0
    for shift in range(max_shift + 1):
        if shift > 0:
            shifted_output = output[shift:]
            shifted_target = target[:len(target) - shift]
        else:
            shifted_output = output
            shifted_target = target
        metrics_collector.reset()
        metrics_collector.update(shifted_output, shifted_target)
        current_metrics = metrics_collector.calculate_averages()
        if best_metrics is None or current_metrics['PearsonR'] > best_metrics['PearsonR']:  # 假设以RMSE为优化指标
            best_metrics = current_metrics
            best_shift = shift
    print(f"Best Shift: {best_shift}")  # 打印最佳偏移量
    return best_metrics, best_shift

def cal_all_modes_data(start_time, modes, csv_path, reconstruct_path, cal_time='3', find_mode='0.1', str_param='RF'):
    all_metrics_dict = {}
    for mode in modes:
        ali_list = OriginUtils.find_grid_points(mode, csv_path, find_mode=find_mode)
        point_list = [[item[2], item[1], item[0]] for item in ali_list]
        point_list.sort(key=lambda x: (-x[0], x[1]))

        metrics_dict = {}
        for lat, lon, point in point_list:
            point_data = OriginUtils.cal_point_data_specific(ali_list, point)
            _, (lat_ind, lon_ind) = OriginUtils.find_grid_coordinates(lon, lat)
            reconstruct_data = np.load(reconstruct_path)[:, lat_ind, lon_ind]
            reconstruct_data = np.where(reconstruct_data < 0, 0, reconstruct_data)
            aligned_data = OriginUtils.align_time_series(point_data, reconstruct_data, start_time)
            if cal_time == '12':
                aligned_data['Time'] = pd.to_datetime(aligned_data['Time'])
                aligned_data = aligned_data[aligned_data['Time'].dt.hour.isin([6, 18])]

            metrics_collector = CustomMetricCollection()
            if not aligned_data.empty:
                valid_data = aligned_data.dropna(subset=['Point Data SM', 'Reconstructed SM'])
                if not valid_data.empty:
                    output = torch.tensor(valid_data['Reconstructed SM'].values, dtype=torch.float32)
                    target = torch.tensor(valid_data['Point Data SM'].values, dtype=torch.float32)
                    best_metrics, best_shift = find_best_shift(output, target, metrics_collector)
                    metrics_dict[f"{point}_{lon}_{lat}"] = {k: tensor_to_scalar(v) for k, v in best_metrics.items()}
                else:
                    metrics_dict[f"{point}_{lon}_{lat}"] = {metric: np.nan for metric in CustomMetricCollection().metrics.keys()}
            metrics_collector.reset()

        all_metrics_dict[mode] = metrics_dict

    folder_path = os.path.dirname(reconstruct_path)
    filename = f"{str_param}_station_metrics_cor.xlsx"
    full_path = os.path.join(folder_path, filename)

    with pd.ExcelWriter(full_path) as writer:
        for mode, metrics in all_metrics_dict.items():
            df = pd.DataFrame.from_dict(metrics, orient='index').reset_index()
            df.columns = ['站点名'] + list(metrics[next(iter(metrics))].keys())  # 重置列名以包括站点名
            df.to_excel(writer, sheet_name=mode, index=False)

def main(args):
    Original_Point_path_argu = Util.load_config(args.Original_Point_path_config)
    path_argu = Util.load_config(args.path_config_path)
    return Original_Point_path_argu, path_argu

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--Original_Point_path_config', type=str, default='../Original_Point_Data/config/path_config.yaml')
    parser.add_argument('--path_config_path', type=str, default='../config/path_config.yaml')
    args = parser.parse_known_args()[0]
    Original_Point_path_argu, path_argu = main(args)
    
    start_time = '2016-01-01 03:00:00'
    modes = ['Shiquanhe', 'Ali', 'Maqu', 'Naqu', 'CTP']  # 所有模式
    cal_time = '3'
    csv_path = Original_Point_path_argu['info']
    reconstruct_path_lstm = r"D:\Data_Store\Dataset\Reconstruction\LSTM\LSTM1.npy"
    reconstruct_path_rf = r"D:\Data_Store\Dataset\Reconstruction\RF\RF_reconstructed_data.npy"
    file_path1 = "D:\Data_Store\Dataset\Reconstruction\ST\RES\spatial_resnet34_temporal_usage_position_embedding_init256.npy"
    file_path2 = "D:\Data_Store\Dataset\Reconstruction\ST\RES\spatial_unet_temporal_usage_position_embedding_init256.npy"
    file_path3 = "D:\Data_Store\Dataset\Reconstruction\ST\RES\ST_resnet34_temporal_usage_position_embedding_init256.npy"
    file_path4 = "D:\Data_Store\Dataset\Reconstruction\ST\RES\ST_unet_temporal_usage_position_embedding_init256.npy"
    file_path5 = "D:\Data_Store\Dataset\Reconstruction\ST\RES\ST_ViT_serial_temporal_usage_position_embedding_individual_init256.n16.npy"
    file_path6 = r"D:\Data_Store\Dataset\Reconstruction\ST\RES\temporal_temporal_usage_position_embedding.npy"
    # 调用函数处理所有模式并保存数据
    cal_all_modes_data(start_time, modes, csv_path, reconstruct_path_lstm, cal_time, find_mode='0.1',str_param='LSTM')


### 站点指标cal（所有网络）-获取全网络指标.XLSX文件

In [ ]:
import sys
sys.path.append(r"C:\Users\Administrator\Desktop\code\ST-Conv")
import argparse
import torch
import os
import pandas as pd
import xarray as xr
import numpy as np
from Original_Point_Data.tool.origin_utils import OriginUtils
from tool.utils import Util
from metrics.metrics import CustomMetricCollection

def tensor_to_scalar(value):
    """如果值是tensor，则转换为标量"""
    if isinstance(value, torch.Tensor):
        return value.item()
    return value

def cal_all_modes_data(start_time, modes, csv_path, reconstruct_path, cal_time='3', find_mode='0.1', str_param='RF'):
    all_metrics_dict = {}
    for mode in modes:  # 迭代处理每个模式
        # 点位信息
        ali_list = OriginUtils.find_grid_points(mode, csv_path, find_mode=find_mode)
        point_list = [[item[2], item[1], item[0]] for item in ali_list]
        point_list.sort(key=lambda x: (-x[0], x[1]))  # 纬度降序，经度升序

        metrics_dict = {}  # 对每个模式重置指标字典

        for lat, lon, point in point_list:
            point_data = OriginUtils.cal_point_data_specific(ali_list, point)
            _, (lat_ind, lon_ind) = OriginUtils.find_grid_coordinates(lon, lat, find_mode=find_mode)

            reconstruct_data = np.load(reconstruct_path)[:, lat_ind, lon_ind]
            reconstruct_data = np.where(reconstruct_data < 0, 0, reconstruct_data)
            aligned_data = OriginUtils.align_time_series(point_data, reconstruct_data, start_time)
            if cal_time == '12':
                aligned_data['Time'] = pd.to_datetime(aligned_data['Time'])
                aligned_data = aligned_data[aligned_data['Time'].dt.hour.isin([6, 18])]

            metrics_collector = CustomMetricCollection()
            if not aligned_data.empty:
                valid_data = aligned_data.dropna(subset=['Point Data SM', 'Reconstructed SM'])
                if not valid_data.empty:
                    output = torch.tensor(valid_data['Reconstructed SM'].values)
                    target = torch.tensor(valid_data['Point Data SM'].values)
                    metrics_collector.update(output, target)

                    # 将所有tensor对象转换为标量值
                    metrics = metrics_collector.calculate_averages()
                    metrics = {k: tensor_to_scalar(v) for k, v in metrics.items()}
                    metrics_dict[f"{point}_{lon}_{lat}"] = metrics
                else:
                    metrics_dict[f"{point}_{lon}_{lat}"] = {metric: np.nan for metric in CustomMetricCollection().metrics.keys()}
            metrics_collector.reset()

        all_metrics_dict[mode] = metrics_dict  # 将当前模式的metrics字典保存到总字典中

    folder_path = os.path.dirname(reconstruct_path)

    # 保存到CSV
    filename = f"{str_param}_station_metrics.xlsx"  # 构造文件名
    full_path = os.path.join(folder_path, filename)  # 将文件夹路径和文件名拼接起来

    with pd.ExcelWriter(full_path) as writer:
        for mode, metrics in all_metrics_dict.items():
            # 调整DataFrame以确保站点名称作为第一列显示
            df = pd.DataFrame(metrics).T.reset_index()
            df.rename(columns={'index': '站点名'}, inplace=True)
            df.to_excel(writer, sheet_name=mode, index=False)  # 每个模式一个子表

def main(args):
    Original_Point_path_argu = Util.load_config(args.Original_Point_path_config)
    path_argu = Util.load_config(args.path_config_path)
    return Original_Point_path_argu, path_argu

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--Original_Point_path_config', type=str, default='../Original_Point_Data/config/path_config.yaml')
    parser.add_argument('--path_config_path', type=str, default='../config/path_config.yaml')
    args = parser.parse_known_args()[0]
    Original_Point_path_argu, path_argu = main(args)
    
    start_time = '2016-01-01 03:00:00'
    modes = ['Shiquanhe', 'Ali', 'Maqu', 'Naqu', 'CTP']  # 所有模式
    cal_time = '3'
    csv_path = Original_Point_path_argu['info']
    Original_path = r"D:\Data_Store\Dataset\Reconstruction\37SM\soil_moisture.npy"
    reconstruct_path_lstm = r"D:\Data_Store\Dataset\Reconstruction\LSTM\LSTM1.npy"
    reconstruct_path_rf = r"D:\Data_Store\Dataset\Reconstruction\RF\RF_reconstructed_data.npy"
    reconstruct_path_STPSCFENet = r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\ST_PSC-FENet.npy"
    reconstruct_path_SPPSCFENet = r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SP_PSC-FENet.npy"
    reconstruct_path_STPSCFENet36 = r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\ST_PSC-FENet36.npy"
    reconstruct_path_SPPSCFENet36 = r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SP_PSC-FENet36.npy"
    reconstruct_path_SPRESNET = r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SP_ResNet.npy"
    reconstruct_path_SPUNET = r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SP_UNet.npy"
    reconstruct_path_STRESNET = r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\ST_ResNet.npy"
    reconstruct_path_STUNET = r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\ST_UNet.npy"
    reconstruct_path_STVIT = r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\ST_ViT.npy"
    reconstruct_path_SPVIT = r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SP_ViT.npy"
    reconstruct_path_TEMPORAL = r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\Temporal.npy"

    reconstruction_paths = {
    'PSC-FENet-SP': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SP_PSC-FENet.npy",
    'PSC-FENet-ST': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\ST_PSC-FENet.npy",
    'PSC-FENet-SP36': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SP_PSC-FENet36.npy",
    'PSC-FENet-ST36': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\ST_PSC-FENet36.npy",
    'LSTM': r"D:\Data_Store\Dataset\Reconstruction\LSTM\LSTM1.npy",
    'RF': r"D:\Data_Store\Dataset\Reconstruction\RF\RF_reconstructed_data.npy",
    'ResNet-SP': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SP_ResNet.npy",
    'ResNet-ST': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\ST_ResNet.npy",
    'UNet-SP': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SP_UNet.npy",
    'UNet-ST': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\ST_UNet.npy",
    'VIT-SP': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SP_ViT.npy",
    'VIT-ST': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\ST_ViT.npy",
    'Temporal': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\Temporal.npy"
}

    # KPNET重建数据
    #cal_all_modes_data(start_time, modes, csv_path, reconstruct_path_STPSCFENet, cal_time, find_mode='0.1', str_param='STPSC-FENet')
    #cal_all_modes_data(start_time, modes, csv_path, reconstruct_path_SPPSCFENet, cal_time, find_mode='0.1', str_param='SPPSC-FENet')
    cal_all_modes_data(start_time, modes, csv_path, reconstruct_path_STPSCFENet36, cal_time, find_mode='0.1', str_param='STPSC-FENet36')
    cal_all_modes_data(start_time, modes, csv_path, reconstruct_path_SPPSCFENet36, cal_time, find_mode='0.1', str_param='SPPSC-FENet36')
    # RF重建数据
    #cal_all_modes_data(start_time, modes, csv_path, reconstruct_path_rf, cal_time, find_mode='0.1', str_param='RF')
    # LSTM重建数据
    #cal_all_modes_data(start_time, modes, csv_path, reconstruct_path_lstm, cal_time, find_mode='0.1', str_param='LSTM')
    # RESNET重建数据
    #cal_all_modes_data(start_time, modes, csv_path, reconstruct_path_SPRESNET, cal_time, find_mode='0.1', str_param='SPRESNET')
    #cal_all_modes_data(start_time, modes, csv_path, reconstruct_path_STRESNET, cal_time, find_mode='0.1', str_param='STRESNET')
    # UNET重建数据
    #cal_all_modes_data(start_time, modes, csv_path, reconstruct_path_SPUNET, cal_time, find_mode='0.1', str_param='SPUNET')
    #cal_all_modes_data(start_time, modes, csv_path, reconstruct_path_STUNET, cal_time, find_mode='0.1', str_param='STUNET')
    # VIT重建数据
    #cal_all_modes_data(start_time, modes, csv_path, reconstruct_path_STVIT, cal_time, find_mode='0.1', str_param='STVIT')
    #cal_all_modes_data(start_time, modes, csv_path, reconstruct_path_SPVIT, cal_time, find_mode='0.1', str_param='SPVIT')
    # TEMPORAL重建数据
    #cal_all_modes_data(start_time, modes, csv_path, reconstruct_path_TEMPORAL, cal_time, find_mode='0.1', str_param='TEMPORAL')
    # 卫星数据
    #cal_all_modes_data(start_time, modes, csv_path, Original_path, cal_time, find_mode='0.37', str_param='Original')

### 计算降尺度指标Gdown、GEFFI、GPREC、GACCU

In [ ]:
import sys
sys.path.append(r"C:\Users\Administrator\Desktop\code\ST-Conv")
import argparse
import torch
import os
import pandas as pd
import xarray as xr
import numpy as np
from Original_Point_Data.tool.origin_utils import OriginUtils
from tool.utils import Util
from metrics.metrics import SoilMoistureMetrics
import copy

def tensor_to_scalar(value):
    """如果值是tensor，则转换为标量"""
    if isinstance(value, torch.Tensor):
        return value.item()
    return value

def cal_all_modes_data(start_time, modes, csv_path, Original_path, reconstruct_path, cal_time='3', find_mode='0.1', str_param='RF'):
    all_metrics_dict = {}
    for mode in modes:  # 迭代处理每个模式
        # 点位信息
        ali_list = OriginUtils.find_grid_points(mode, csv_path, find_mode=find_mode)
        point_list = [[item[2], item[1], item[0]] for item in ali_list]
        point_list.sort(key=lambda x: (-x[0], x[1]))  # 纬度降序，经度升序

        metrics_dict = {}  # 对每个模式重置指标字典

        for lat, lon, point in point_list:
            point_data = OriginUtils.cal_point_data_specific(ali_list, point)
            _, (lat_ind, lon_ind) = OriginUtils.find_grid_coordinates(lon, lat, find_mode=find_mode)
            _, (lat_ind_37, lon_ind_37) = OriginUtils.find_grid_coordinates(lon, lat, find_mode='0.37')

            reconstruct_data = np.load(reconstruct_path)[:, lat_ind, lon_ind]
            reconstruct_data = np.where(reconstruct_data < 0, 0, reconstruct_data)

            Original_data = np.load(Original_path)[:, lat_ind_37, lon_ind_37]
            Original_data = np.where(Original_data < 0, 0, Original_data)

            aligned_data = OriginUtils.align_time_series(point_data, reconstruct_data, start_time)
            aligned_Original_data = OriginUtils.align_time_series(point_data, Original_data, start_time)

            aligned_Original_data.rename(columns={'Reconstructed SM': 'Original SM'}, inplace=True)
            merged_data = pd.merge(aligned_data, aligned_Original_data[['Time', 'Original SM']], on='Time', how='outer')

            if not merged_data.empty:
                merged_data.dropna(subset=['Point Data SM', 'Reconstructed SM', 'Original SM'], inplace=True)
                metrics_collector = SoilMoistureMetrics()
                if not merged_data.empty:
                    hr = merged_data['Reconstructed SM'].values.astype(np.float32)
                    lr = merged_data['Original SM'].values.astype(np.float32)
                    is_data = merged_data['Point Data SM'].values.astype(np.float32)

                    metrics_collector.update(hr, lr, is_data)
                    metrics = metrics_collector.get_metrics()
                    metrics_dict[f"{point}_{lon}_{lat}"] = {key: (val[0] if isinstance(val, list) and len(val) == 1 else np.nan if not val else val) 
                                                           for key, val in copy.deepcopy(metrics).items()}

                metrics_collector.reset()

        all_metrics_dict[mode] = metrics_dict

    folder_path = os.path.dirname(reconstruct_path)
    filename = f"{str_param}_station_G_metrics.xlsx"
    full_path = os.path.join(folder_path, filename)

    with pd.ExcelWriter(full_path) as writer:
        for mode, metrics in all_metrics_dict.items():
            # 调整DataFrame以确保站点名称作为第一列显示
            df = pd.DataFrame(metrics).T.reset_index()
            df.rename(columns={'index': '站点名'}, inplace=True)
            df.to_excel(writer, sheet_name=mode, index=False)

def main(args):
    Original_Point_path_argu = Util.load_config(args.Original_Point_path_config)
    path_argu = Util.load_config(args.path_config_path)
    return Original_Point_path_argu, path_argu

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--Original_Point_path_config', type=str, default='../Original_Point_Data/config/path_config.yaml')
    parser.add_argument('--path_config_path', type=str, default='../config/path_config.yaml')
    args = parser.parse_known_args()[0]
    Original_Point_path_argu, path_argu = main(args)
    
    start_time = '2016-01-01 03:00:00'
    modes = ['Shiquanhe', 'Ali', 'Maqu', 'Naqu', 'CTP']  # 所有模式
    cal_time = '3'
    csv_path = Original_Point_path_argu['info']
    Original_path = r"D:\Data_Store\Dataset\Reconstruction\37SM\soil_moisture.npy"
    reconstruct_path_lstm = r"D:\Data_Store\Dataset\Reconstruction\LSTM\LSTM1.npy"
    reconstruct_path_rf = r"D:\Data_Store\Dataset\Reconstruction\RF\RF_reconstructed_data.npy"
    reconstruct_path_STPSCFENet = r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\ST_PSC-FENet.npy"
    reconstruct_path_SPPSCFENet = r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SP_PSC-FENet.npy"
    reconstruct_path_STPSCFENet36 = r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\ST_PSC-FENet36.npy"
    reconstruct_path_SPPSCFENet36 = r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SP_PSC-FENet36.npy"
    reconstruct_path_SPRESNET = r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SP_ResNet.npy"
    reconstruct_path_SPUNET = r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SP_UNet.npy"
    reconstruct_path_STRESNET = r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\ST_ResNet.npy"
    reconstruct_path_STUNET = r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\ST_UNet.npy"
    reconstruct_path_STVIT = r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\ST_ViT.npy"
    reconstruct_path_SPVIT = r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SP_ViT.npy"
    reconstruct_path_TEMPORAL = r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\Temporal.npy"

    # KPNET重建数据
    #cal_all_modes_data(start_time, modes, csv_path, Original_path, reconstruct_path_STPSCFENet, cal_time, find_mode='0.1', str_param='STPSC-FENet')
    #cal_all_modes_data(start_time, modes, csv_path, Original_path, reconstruct_path_SPPSCFENet, cal_time, find_mode='0.1', str_param='SPPSC-FENet')
    cal_all_modes_data(start_time, modes, csv_path, Original_path, reconstruct_path_STPSCFENet36, cal_time, find_mode='0.1', str_param='STPSC-FENet36')
    cal_all_modes_data(start_time, modes, csv_path, Original_path, reconstruct_path_SPPSCFENet36, cal_time, find_mode='0.1', str_param='SPPSC-FENet36')
    # RF重建数据
    #cal_all_modes_data(start_time, modes, csv_path, Original_path, reconstruct_path_rf, cal_time, find_mode='0.1', str_param='RF')
    # LSTM重建数据
    #cal_all_modes_data(start_time, modes, csv_path, Original_path, reconstruct_path_lstm, cal_time, find_mode='0.1', str_param='LSTM')
    # RESNET重建数据
    #cal_all_modes_data(start_time, modes, csv_path, Original_path, reconstruct_path_SPRESNET, cal_time, find_mode='0.1', str_param='SPRESNET')
    #cal_all_modes_data(start_time, modes, csv_path, Original_path, reconstruct_path_STRESNET, cal_time, find_mode='0.1', str_param='STRESNET')
    # UNET重建数据
    #cal_all_modes_data(start_time, modes, csv_path, Original_path, reconstruct_path_SPUNET, cal_time, find_mode='0.1', str_param='SPUNET')
    #cal_all_modes_data(start_time, modes, csv_path, Original_path, reconstruct_path_STUNET, cal_time, find_mode='0.1', str_param='STUNET')
    # VIT重建数据
    #cal_all_modes_data(start_time, modes, csv_path, Original_path, reconstruct_path_STVIT, cal_time, find_mode='0.1', str_param='STVIT')
    #cal_all_modes_data(start_time, modes, csv_path, Original_path, reconstruct_path_SPVIT, cal_time, find_mode='0.1', str_param='SPVIT')
    # TEMPORAL重建数据
    #cal_all_modes_data(start_time, modes, csv_path, Original_path, reconstruct_path_TEMPORAL, cal_time, find_mode='0.1', str_param='TEMPORAL')

In [ ]:
import sys
sys.path.append(r"C:\Users\Administrator\Desktop\code\ST-Conv")
import argparse
import torch
import os
import pandas as pd
import xarray as xr
import numpy as np
from Original_Point_Data.tool.origin_utils import OriginUtils
from tool.utils import Util
from metrics.metrics import CustomMetricCollection
import numpy as np
import matplotlib.pyplot as plt
def main():
    # 加载数据
    data_path = r"D:\Data_Store\Dataset\Reconstruction\37SM\soil_moisture.npy"
    data = np.load(data_path)

    # 计算非NaN的数据数目
    non_nan_counts = np.sum(~np.isnan(data), axis=0)  # 在时间维度上求和非NaN值

    # 绘制非NaN数据数目
    plt.figure(figsize=(10, 6))
    im = plt.imshow(non_nan_counts, cmap='viridis', origin='lower')
    plt.colorbar(im, label='Non-NaN Count')
    plt.title('Non-NaN Data Counts at Each Latitude and Longitude')
    plt.xlabel('Longitude Index')
    plt.ylabel('Latitude Index')
    plt.show()

if __name__ == "__main__":
    main()

### (弃用，多时间尺度指标计算中已算过) 计算12hour的时频数据

In [ ]:
import sys
sys.path.append(r"C:\Users\Administrator\Desktop\code\ST-Conv")
import argparse
import torch
import os
import pandas as pd
import xarray as xr
import numpy as np
from Original_Point_Data.tool.origin_utils import OriginUtils
from tool.utils import Util
from metrics.metrics import CustomMetricCollection

def tensor_to_scalar(value):
    """如果值是tensor，则转换为标量"""
    if isinstance(value, torch.Tensor):
        return value.item()
    return value

def cal_all_modes_data(start_time, modes, csv_path, reconstruct_path, cal_time='3', find_mode='0.1', str_param='RF'):
    all_metrics_dict = {}
    for mode in modes:  # 迭代处理每个模式
        # 点位信息
        ali_list = OriginUtils.find_grid_points(mode, csv_path, find_mode=find_mode)
        point_list = [[item[2], item[1], item[0]] for item in ali_list]
        point_list.sort(key=lambda x: (-x[0], x[1]))  # 纬度降序，经度升序

        metrics_dict = {}  # 对每个模式重置指标字典

        for lat, lon, point in point_list:
            point_data = OriginUtils.cal_point_data_specific(ali_list, point)
            _, (lat_ind, lon_ind) = OriginUtils.find_grid_coordinates(lon, lat, find_mode=find_mode)

            reconstruct_data = np.load(reconstruct_path)[:, lat_ind, lon_ind]
            reconstruct_data = np.where(reconstruct_data < 0, 0, reconstruct_data)
            aligned_data = OriginUtils.align_time_series(point_data, reconstruct_data, start_time)
            if cal_time == '12':
                aligned_data['Time'] = pd.to_datetime(aligned_data['Time'])
                aligned_data = aligned_data[aligned_data['Time'].dt.hour.isin([6, 18])]

            metrics_collector = CustomMetricCollection()
            if not aligned_data.empty:
                valid_data = aligned_data.dropna(subset=['Point Data SM', 'Reconstructed SM'])
                if not valid_data.empty:
                    output = torch.tensor(valid_data['Reconstructed SM'].values)
                    target = torch.tensor(valid_data['Point Data SM'].values)
                    metrics_collector.update(output, target)

                    # 将所有tensor对象转换为标量值
                    metrics = metrics_collector.calculate_averages()
                    metrics = {k: tensor_to_scalar(v) for k, v in metrics.items()}
                    metrics_dict[f"{point}_{lon}_{lat}"] = metrics
                else:
                    metrics_dict[f"{point}_{lon}_{lat}"] = {metric: np.nan for metric in CustomMetricCollection().metrics.keys()}
            metrics_collector.reset()

        all_metrics_dict[mode] = metrics_dict  # 将当前模式的metrics字典保存到总字典中

    folder_path = os.path.dirname(reconstruct_path)

    # 保存到CSV
    filename = f"{str_param}_station_metrics_12.xlsx"  # 构造文件名
    full_path = os.path.join(folder_path, filename)  # 将文件夹路径和文件名拼接起来

    with pd.ExcelWriter(full_path) as writer:
        for mode, metrics in all_metrics_dict.items():
            # 调整DataFrame以确保站点名称作为第一列显示
            df = pd.DataFrame(metrics).T.reset_index()
            df.rename(columns={'index': '站点名'}, inplace=True)
            df.to_excel(writer, sheet_name=mode, index=False)  # 每个模式一个子表

def main(args):
    Original_Point_path_argu = Util.load_config(args.Original_Point_path_config)
    path_argu = Util.load_config(args.path_config_path)
    return Original_Point_path_argu, path_argu

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--Original_Point_path_config', type=str, default='../Original_Point_Data/config/path_config.yaml')
    parser.add_argument('--path_config_path', type=str, default='../config/path_config.yaml')
    args = parser.parse_known_args()[0]
    Original_Point_path_argu, path_argu = main(args)
    
    start_time = '2016-01-01 03:00:00'
    modes = ['Shiquanhe', 'Ali', 'Maqu', 'Naqu', 'CTP']  # 所有模式
    cal_time = '12'
    csv_path = Original_Point_path_argu['info']
    Original_path = r"D:\Data_Store\Dataset\Reconstruction\37SM\soil_moisture.npy"
    reconstruct_path_lstm = r"D:\Data_Store\Dataset\Reconstruction\LSTM\LSTM1.npy"
    reconstruct_path_rf = r"D:\Data_Store\Dataset\Reconstruction\RF\RF_reconstructed_data.npy"
    reconstruct_path_STPSCFENet = r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\ST_PSC-FENet.npy"
    reconstruct_path_SPPSCFENet = r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SP_PSC-FENet.npy"
    reconstruct_path_SPRESNET = r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SP_ResNet.npy"
    reconstruct_path_SPUNET = r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SP_UNet.npy"
    reconstruct_path_STRESNET = r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\ST_ResNet.npy"
    reconstruct_path_STUNET = r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\ST_UNet.npy"
    reconstruct_path_STVIT = r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\ST_ViT.npy"
    reconstruct_path_SPVIT = r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SP_ViT.npy"
    reconstruct_path_TEMPORAL = r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\Temporal.npy"

    # KPNET重建数据
    cal_all_modes_data(start_time, modes, csv_path, reconstruct_path_STPSCFENet, cal_time, find_mode='0.1', str_param='STPSCFENet')
    cal_all_modes_data(start_time, modes, csv_path, reconstruct_path_SPPSCFENet, cal_time, find_mode='0.1', str_param='SPPSCFENet')
    # RF重建数据
    cal_all_modes_data(start_time, modes, csv_path, reconstruct_path_rf, cal_time, find_mode='0.1', str_param='RF')
    # LSTM重建数据
    cal_all_modes_data(start_time, modes, csv_path, reconstruct_path_lstm, cal_time, find_mode='0.1', str_param='LSTM')
    # RESNET重建数据
    cal_all_modes_data(start_time, modes, csv_path, reconstruct_path_SPRESNET, cal_time, find_mode='0.1', str_param='SPRESNET')
    cal_all_modes_data(start_time, modes, csv_path, reconstruct_path_STRESNET, cal_time, find_mode='0.1', str_param='STRESNET')
    # UNET重建数据
    cal_all_modes_data(start_time, modes, csv_path, reconstruct_path_SPUNET, cal_time, find_mode='0.1', str_param='SPUNET')
    cal_all_modes_data(start_time, modes, csv_path, reconstruct_path_STUNET, cal_time, find_mode='0.1', str_param='STUNET')
    # VIT重建数据
    cal_all_modes_data(start_time, modes, csv_path, reconstruct_path_STVIT, cal_time, find_mode='0.1', str_param='STVIT')
    cal_all_modes_data(start_time, modes, csv_path, reconstruct_path_SPVIT, cal_time, find_mode='0.1', str_param='SPVIT')
    # TEMPORAL重建数据
    cal_all_modes_data(start_time, modes, csv_path, reconstruct_path_TEMPORAL, cal_time, find_mode='0.1', str_param='TEMPORAL')
    # 卫星数据 
    cal_all_modes_data(start_time, modes, csv_path, Original_path, cal_time, find_mode='0.37', str_param='Original')

In [ ]:
import sys
sys.path.append(r"C:\Users\Administrator\Desktop\code\ST-Conv")
import argparse
import torch
import os
import pandas as pd
import xarray as xr
import numpy as np
from Original_Point_Data.tool.origin_utils import OriginUtils
from tool.utils import Util
from metrics.metrics import CustomMetricCollection

def tensor_to_scalar(value):
    """如果值是tensor，则转换为标量"""
    if isinstance(value, torch.Tensor):
        return value.item()
    return value

def cal_all_modes_data(start_time, modes, csv_path, reconstruct_path, cal_time='3', find_mode='0.1', str_param='RF'):
    all_metrics_dict = {}
    for mode in modes:  # 迭代处理每个模式
        # 点位信息
        ali_list = OriginUtils.find_grid_points(mode, csv_path, find_mode=find_mode)
        point_list = [[item[2], item[1], item[0]] for item in ali_list]
        point_list.sort(key=lambda x: (-x[0], x[1]))  # 纬度降序，经度升序

        metrics_dict = {}  # 对每个模式重置指标字典

        for lat, lon, point in point_list:
            point_data = OriginUtils.cal_point_data_specific(ali_list, point)
            _, (lat_ind, lon_ind) = OriginUtils.find_grid_coordinates(lon, lat, find_mode=find_mode)

            reconstruct_data = np.load(reconstruct_path)[:, lat_ind, lon_ind]
            reconstruct_data = np.where(reconstruct_data < 0, 0, reconstruct_data)
            aligned_data = OriginUtils.align_time_series(point_data, reconstruct_data, start_time)
            if cal_time == '12':
                aligned_data['Time'] = pd.to_datetime(aligned_data['Time'])
                aligned_data = aligned_data[aligned_data['Time'].dt.hour.isin([6, 18])]

            metrics_collector = CustomMetricCollection()
            if not aligned_data.empty:
                valid_data = aligned_data.dropna(subset=['Point Data SM', 'Reconstructed SM'])
                if not valid_data.empty:
                    output = torch.tensor(valid_data['Reconstructed SM'].values)
                    target = torch.tensor(valid_data['Point Data SM'].values)
                    metrics_collector.update(output, target)

                    # 将所有tensor对象转换为标量值
                    metrics = metrics_collector.calculate_averages()
                    metrics = {k: tensor_to_scalar(v) for k, v in metrics.items()}
                    metrics_dict[f"{point}_{lon}_{lat}"] = metrics
                else:
                    metrics_dict[f"{point}_{lon}_{lat}"] = {metric: np.nan for metric in CustomMetricCollection().metrics.keys()}
            metrics_collector.reset()

        all_metrics_dict[mode] = metrics_dict  # 将当前模式的metrics字典保存到总字典中

    folder_path = os.path.dirname(reconstruct_path)

    # 保存到CSV
    filename = f"{str_param}_station_metrics.xlsx"  # 构造文件名
    full_path = os.path.join(folder_path, filename)  # 将文件夹路径和文件名拼接起来

    with pd.ExcelWriter(full_path) as writer:
        for mode, metrics in all_metrics_dict.items():
            # 调整DataFrame以确保站点名称作为第一列显示
            df = pd.DataFrame(metrics).T.reset_index()
            df.rename(columns={'index': '站点名'}, inplace=True)
            df.to_excel(writer, sheet_name=mode, index=False)  # 每个模式一个子表

def main(args):
    Original_Point_path_argu = Util.load_config(args.Original_Point_path_config)
    path_argu = Util.load_config(args.path_config_path)
    return Original_Point_path_argu, path_argu

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--Original_Point_path_config', type=str, default='../Original_Point_Data/config/path_config.yaml')
    parser.add_argument('--path_config_path', type=str, default='../config/path_config.yaml')
    args = parser.parse_known_args()[0]
    Original_Point_path_argu, path_argu = main(args)
    
    start_time = '2016-01-01 03:00:00'
    modes = ['Shiquanhe', 'Ali', 'Maqu', 'Naqu', 'CTP']  # 所有模式
    cal_time = '3'
    csv_path = Original_Point_path_argu['info']
    Original_path = r"D:\Data_Store\Dataset\Reconstruction\37SM\soil_moisture.npy"
    reconstruct_path_lstm = r"D:\Data_Store\Dataset\Reconstruction\LSTM\LSTM1.npy"
    reconstruct_path_rf = r"D:\Data_Store\Dataset\Reconstruction\RF\RF_reconstructed_data.npy"
    reconstruct_path_KPNET = r"D:\Data_Store\Dataset\Reconstruction\ST\RES\ST_kpnet_Encoder_lite_serial_temporal_usage_position_embedding_individual_init256_11_121_131_dropout_rate0.3168.npy"
    reconstruct_path_SPKPNET = r"D:\Data_Store\Dataset\Reconstruction\ST\RES\spatial_kpnet_Encoder_lite_serial_temporal_usage_position_embedding_individual_init256168.npy"   
    reconstruct_path_SPRESNET = r"D:\Data_Store\Dataset\Reconstruction\ST\RES\spatial_resnet34_temporal_usage_position_embedding_init256168.npy"
    reconstruct_path_SPUNET = r"D:\Data_Store\Dataset\Reconstruction\ST\RES\spatial_unet_temporal_usage_position_embedding_init256168.npy"
    reconstruct_path_STRESNET = r"D:\Data_Store\Dataset\Reconstruction\ST\RES\ST_resnet34_temporal_usage_position_embedding_init256168.npy"
    reconstruct_path_STUNET = r"D:\Data_Store\Dataset\Reconstruction\ST\RES\ST_unet_temporal_usage_position_embedding_init256168.npy"
    reconstruct_path_STVIT = r"D:\Data_Store\Dataset\Reconstruction\ST\RES\ST_ViT_serial_temporal_usage_position_embedding_individual_init256168.npy"
    reconstruct_path_SPVIT = r"D:\Data_Store\Dataset\Reconstruction\ST\RES\spatial_VIT_serial_temporal_usage_position_embedding_individual_init256168.npy"  
    reconstruct_path_TEMPORAL = r"D:\Data_Store\Dataset\Reconstruction\ST\RES\temporal_temporal_usage_position_embedding168.npy"

    # KPNET重建数据
    cal_all_modes_data(start_time, modes, csv_path, reconstruct_path_KPNET, cal_time, find_mode='0.1', str_param='STKPNET')
    cal_all_modes_data(start_time, modes, csv_path, reconstruct_path_SPKPNET, cal_time, find_mode='0.1', str_param='SPKPNET')
    # RF重建数据
    cal_all_modes_data(start_time, modes, csv_path, reconstruct_path_rf, cal_time, find_mode='0.1', str_param='RF')
    # LSTM重建数据
    cal_all_modes_data(start_time, modes, csv_path, reconstruct_path_lstm, cal_time, find_mode='0.1', str_param='LSTM')
    # RESNET重建数据
    cal_all_modes_data(start_time, modes, csv_path, reconstruct_path_SPRESNET, cal_time, find_mode='0.1', str_param='SPRESNET')
    cal_all_modes_data(start_time, modes, csv_path, reconstruct_path_STRESNET, cal_time, find_mode='0.1', str_param='STRESNET')
    # UNET重建数据
    cal_all_modes_data(start_time, modes, csv_path, reconstruct_path_SPUNET, cal_time, find_mode='0.1', str_param='SPUNET')
    cal_all_modes_data(start_time, modes, csv_path, reconstruct_path_STUNET, cal_time, find_mode='0.1', str_param='STUNET')
    # VIT重建数据
    cal_all_modes_data(start_time, modes, csv_path, reconstruct_path_STVIT, cal_time, find_mode='0.1', str_param='STVIT')
    cal_all_modes_data(start_time, modes, csv_path, reconstruct_path_SPVIT, cal_time, find_mode='0.1', str_param='SPVIT')
    # TEMPORAL重建数据
    cal_all_modes_data(start_time, modes, csv_path, reconstruct_path_TEMPORAL, cal_time, find_mode='0.1', str_param='TEMPORAL')
    # 卫星数据 
    cal_all_modes_data(start_time, modes, csv_path, Original_path, cal_time, find_mode='0.37', str_param='Original')

### 站点多时间尺度指标（获取全网络全时刻指标.xlsx文件）

In [ ]:
import sys
sys.path.append(r"C:\Users\Administrator\Desktop\code\ST-Conv")
import argparse
import torch
import os
import pandas as pd
import xarray as xr
import numpy as np
from Original_Point_Data.tool.origin_utils import OriginUtils
from tool.utils import Util
from metrics.metrics import CustomMetricCollection

def tensor_to_scalar(value):
    """如果值是tensor，则转换为标量"""
    if isinstance(value, torch.Tensor):
        return value.item()
    return value

def cal_all_modes_data_time_specific(start_time, modes, csv_path, reconstruct_path, find_mode='0.1', str_param='RF'):
    # 定义时间点
    time_points = [0, 3, 6, 9, 12, 15, 18, 21]
    all_metrics_dict = {}
    
    for mode in modes:
        ali_list = OriginUtils.find_grid_points(mode, csv_path, find_mode='0.1')
        point_list = [[item[2], item[1], item[0]] for item in ali_list]
        point_list.sort(key=lambda x: (-x[0], x[1]))

        for time_point in time_points:
            metrics_dict = {}
            
            for lat, lon, point in point_list:
                point_data = OriginUtils.cal_point_data_specific(ali_list, point)
                _, (lat_ind, lon_ind) = OriginUtils.find_grid_coordinates(lon, lat, find_mode='0.1')

                reconstruct_data = np.load(reconstruct_path)[:, lat_ind, lon_ind]
                reconstruct_data = np.where(reconstruct_data < 0, 0, reconstruct_data)
                aligned_data = OriginUtils.align_time_series(point_data, reconstruct_data, start_time)

                # 提取特定时间点的数据
                aligned_data['Time'] = pd.to_datetime(aligned_data['Time'])
                specific_time_data = aligned_data[aligned_data['Time'].dt.hour.isin([time_point])]

                metrics_collector = CustomMetricCollection()
                if not specific_time_data.empty:
                    valid_data = specific_time_data.dropna(subset=['Point Data SM', 'Reconstructed SM'])
                    if not valid_data.empty:
                        output = torch.tensor(valid_data['Reconstructed SM'].values)
                        target = torch.tensor(valid_data['Point Data SM'].values)
                        metrics_collector.update(output, target)

                        metrics = metrics_collector.calculate_averages()
                        metrics = {k: tensor_to_scalar(v) for k, v in metrics.items()}
                        metrics_dict[f"{point}_{lon}_{lat}"] = metrics
                    else:
                        metrics_dict[f"{point}_{lon}_{lat}"] = {metric: np.nan for metric in CustomMetricCollection().metrics.keys()}
                metrics_collector.reset()

            all_metrics_dict[f"{mode}_{time_point}"] = metrics_dict

    folder_path = os.path.dirname(reconstruct_path)

    # 保存到Excel
    filename = f"{str_param}_station_time_metrics.xlsx"
    full_path = os.path.join(folder_path, filename)

    with pd.ExcelWriter(full_path) as writer:
        for time_point, metrics in all_metrics_dict.items():
            df = pd.DataFrame(metrics).T.reset_index()
            df.rename(columns={'index': '站点名'}, inplace=True)
            df.to_excel(writer, sheet_name=f"{time_point}h", index=False)

    return all_metrics_dict

def main(args):
    Original_Point_path_argu = Util.load_config(args.Original_Point_path_config)
    path_argu = Util.load_config(args.path_config_path)
    return Original_Point_path_argu, path_argu

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--Original_Point_path_config', type=str, default='../Original_Point_Data/config/path_config.yaml')
    parser.add_argument('--path_config_path', type=str, default='../config/path_config.yaml')
    args = parser.parse_known_args()[0]
    Original_Point_path_argu, path_argu = main(args)
    
    start_time = '2016-01-01 03:00:00'
    modes = ['Shiquanhe', 'Ali', 'Maqu', 'Naqu', 'CTP']  # 所有模式
    cal_time = '3'
    csv_path = Original_Point_path_argu['info']
    Original_path = r"D:\Data_Store\Dataset\Reconstruction\37SM\soil_moisture.npy"
    reconstruct_path_lstm = r"D:\Data_Store\Dataset\Reconstruction\LSTM\LSTM1.npy"
    reconstruct_path_rf = r"D:\Data_Store\Dataset\Reconstruction\RF\RF_reconstructed_data.npy"
    reconstruct_path_STPSCFENet = r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\ST_PSC-FENet.npy"
    reconstruct_path_SPPSCFENet = r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SP_PSC-FENet.npy"
    reconstruct_path_STPSCFENet36 = r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\ST_PSC-FENet36.npy"
    reconstruct_path_SPPSCFENet36 = r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SP_PSC-FENet36.npy"
    reconstruct_path_SPRESNET = r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SP_ResNet.npy"
    reconstruct_path_SPUNET = r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SP_UNet.npy"
    reconstruct_path_STRESNET = r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\ST_ResNet.npy"
    reconstruct_path_STUNET = r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\ST_UNet.npy"
    reconstruct_path_STVIT = r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\ST_ViT.npy"
    reconstruct_path_SPVIT = r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SP_ViT.npy"
    reconstruct_path_TEMPORAL = r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\Temporal.npy"


    # KPNET重建数据
    #cal_all_modes_data_time_specific(start_time, modes, csv_path, reconstruct_path_STPSCFENet, find_mode='0.1', str_param='STPSCFENet')
    #cal_all_modes_data_time_specific(start_time, modes, csv_path, reconstruct_path_SPPSCFENet, find_mode='0.1', str_param='SPPSCFENet')
    cal_all_modes_data_time_specific(start_time, modes, csv_path, reconstruct_path_STPSCFENet36, find_mode='0.1', str_param='STPSCFENet36')
    cal_all_modes_data_time_specific(start_time, modes, csv_path, reconstruct_path_SPPSCFENet36, find_mode='0.1', str_param='SPPSCFENet36')
    # RF重建数据
    #cal_all_modes_data_time_specific(start_time, modes, csv_path, reconstruct_path_rf, find_mode='0.1', str_param='RF')
    # LSTM重建数据
    #cal_all_modes_data_time_specific(start_time, modes, csv_path, reconstruct_path_lstm, find_mode='0.1', str_param='LSTM')
    # RESNET重建数据
    #cal_all_modes_data_time_specific(start_time, modes, csv_path, reconstruct_path_SPRESNET, find_mode='0.1', str_param='SPRESNET')
    #cal_all_modes_data_time_specific(start_time, modes, csv_path, reconstruct_path_STRESNET, find_mode='0.1', str_param='STRESNET')
    # UNET重建数据
    #cal_all_modes_data_time_specific(start_time, modes, csv_path, reconstruct_path_SPUNET, find_mode='0.1', str_param='SPUNET')
    #cal_all_modes_data_time_specific(start_time, modes, csv_path, reconstruct_path_STUNET, find_mode='0.1', str_param='STUNET')
    # VIT重建数据
    #cal_all_modes_data_time_specific(start_time, modes, csv_path, reconstruct_path_STVIT, find_mode='0.1', str_param='STVIT')
    #cal_all_modes_data_time_specific(start_time, modes, csv_path, reconstruct_path_SPVIT, find_mode='0.1', str_param='SPVIT')
    # TEMPORAL重建数据
    #cal_all_modes_data_time_specific(start_time, modes, csv_path, reconstruct_path_TEMPORAL, find_mode='0.1', str_param='TEMPORAL')
    # 卫星数据 
    #cal_all_modes_data_time_specific(start_time, modes, csv_path, Original_path, find_mode='0.37', str_param='Original')


## 绘制散点密度图(未用到)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
from sklearn.metrics import explained_variance_score,r2_score,median_absolute_error,mean_squared_error,mean_absolute_error
from sklearn.linear_model import LinearRegression
from matplotlib.font_manager import FontProperties
from scipy import stats
from matplotlib import rcParams
from statistics import mean
from scipy.stats import pearsonr,t

def preprocess_and_merge_data(all_networks_data):
    merged_data = pd.DataFrame()

    for network, data in all_networks_data.items():
        relevant_data = data[['Original_Data'] + [col for col in data.columns if col not in ['Original_Data', 'Precipitation']]]
        relevant_data = relevant_data.dropna(subset=['Original_Data'])
        if merged_data.empty:
            merged_data = relevant_data
        else:
            merged_data = merged_data.reset_index(drop=True)
            relevant_data = relevant_data.reset_index(drop=True)
            merged_data = pd.concat([merged_data, relevant_data], axis=0, ignore_index=True)
    return merged_data



def plot_density_scatter(x_data, y_data, title, xlabel, ylabel, text_fontsize=14):
    # 设置字体
    font = FontProperties()
    font.set_family('Times New Roman')
    font.set_size(text_fontsize)
    font_prop = FontProperties(family='Times New Roman', size=14)
    # 准备数据
    df = pd.DataFrame({'x': x_data, 'y': y_data}).dropna()
    x, y = df['x'], df['y']
    xy = np.vstack([x, y])
    z = gaussian_kde(xy)(xy)
    idx = z.argsort()
    x, y, z = x.iloc[idx], y.iloc[idx], z[idx]

    # 计算拟合线参数
    def slope(xs, ys):
        m = (((np.mean(xs) * np.mean(ys)) - np.mean(xs * ys)) /
             ((np.mean(xs) * np.mean(xs)) - np.mean(xs * xs)))
        b = np.mean(ys) - m * np.mean(xs)
        return m, b
    k, b = slope(x, y)

    # 构建拟合线和95%置信区间线
    regression_line = k * x + b

    # 计算统计指标
    BIAS = np.mean(x - y)
    MSE = mean_squared_error(x, y)
    RMSE = np.sqrt(MSE)
    ubRMSE = np.sqrt(np.sum((y - np.mean(y))**2)/ len(y))
    R2 = pearsonr(x, y).statistic
    n = len(y)
    # 95置信带
    n = 1  # 数据点的数量
    t_value = 1.96  # 95%置信区间对应的t值
    y_fit = k * x + b  # 拟合的y值
    std_err = np.std(y - y_fit) / np.sqrt(n)  # 标准误差
    margin_of_error = t_value * std_err  # 误差幅度

    # 计算置信区间
    lower_confidence_bound = y_fit - margin_of_error
    upper_confidence_bound = y_fit + margin_of_error

    # 计算x轴的最小值和最大值及其padding
    min_val_x = x.min()
    max_val_x = x.max()
    padding_x = (max_val_x - min_val_x) * 0.01  # 添加10%的padding以避免数据点紧贴边缘

    # 计算y轴的最小值和最大值及其padding
    min_val_y = y.min()
    max_val_y = y.max()
    padding_y = (max_val_y - min_val_y) * 0.01  # 添加10%的padding以避免数据点紧贴边缘

    # 绘图
    fig, ax = plt.subplots(figsize=(8, 6), dpi=300)
    #ax.plot(x, lower_confidence_bound, linestyle='--', color='black', label='95% Prediction Band')
    #ax.plot(x, upper_confidence_bound, linestyle='--', color='black')

    # 使用蓝色色系的颜色映射来表示数据点的密度
    scatter = ax.scatter(x, y, marker='o', c=z, edgecolors=None, s=15, cmap='RdBu_r',  alpha=0.8)
    cbar = plt.colorbar(scatter, shrink=1, orientation='vertical', extend='both', pad=0.015, aspect=30, label='Frequency')

    # 绘制回归线和1:1线
    ax.plot(x, regression_line, 'black', lw=1.5, label='Regression Line')
    ax.plot([min_val_x - padding_x, max_val_x + padding_x], [min_val_y - padding_y, max_val_y + padding_y], 'red', lw=1.5, linestyle='--', label='1:1 Line')

    # 设置图形的范围
    ax.axis([min_val_x - padding_x, max_val_x + padding_x, min_val_y - padding_y, max_val_y + padding_y])

    # 在右下角添加统计指标文本
    text_position_x = max_val_x - padding_x * 2
    text_position_y = min_val_y + padding_y * 2

    step = 0.018

    # 使用FontProperties实例绘制文本，去除$符号避免斜体
    ax.text(text_position_x, text_position_y, f'R²={R2:.3f}', fontproperties=font_prop, horizontalalignment='right', verticalalignment='bottom')
    ax.text(text_position_x, text_position_y + step, f'RMSE={RMSE:.3f}', fontproperties=font_prop, horizontalalignment='right', verticalalignment='bottom')
    ax.text(text_position_x, text_position_y + 2*step, f'ubRMSE={ubRMSE:.3f}', fontproperties=font_prop, horizontalalignment='right', verticalalignment='bottom')
    ax.text(text_position_x, text_position_y + 3*step, f'bias={BIAS:.3f}', fontproperties=font_prop, horizontalalignment='right', verticalalignment='bottom')
    ax.text(text_position_x, text_position_y + 4*step, f'N={n}', fontproperties=font_prop, horizontalalignment='right', verticalalignment='bottom')

    #ax.set_title(title, fontproperties=font,fontsize=16)
    ax.set_xlabel(xlabel, fontproperties=font,fontsize=14)
    ax.set_ylabel(ylabel, fontproperties=font,fontsize=14)

    for label in ax.get_xticklabels() + ax.get_yticklabels():
        label.set_fontsize(12)
    ax.legend(loc='upper left', frameon=False)
    ax.grid(True, linestyle='--', alpha=0.2)
    plt.legend(prop=font_prop, loc='upper left', fancybox=False, shadow=False,frameon=False)
    plt.show()
processed_data = preprocess_and_merge_data(all_networks_data)
for method in processed_data.columns:
    if method != 'Satellite_Data':# 'Satellite_Data''Original_Data'
        plot_density_scatter(processed_data['Satellite_Data'], processed_data[method], f'{method}', 'Satellite_Data($m^3/m^{-3}$)', f'{method}($m^3/m^{-3}$)')

## SMAP卫星原位验证图-密度散点图指标图、二维指标图、重建结果图

#### (数据处理) 0.1 降尺度产品下采样 -得到与卫星数据一致的数据lon，lat，times
为确保数据匹配准确性，每次绘图之前需正确设置selected_model_names，在后续中selected_model_names可能会多次修改
二维指标图只绘制 KPNET-SP、LSTM、RF、Temporal 

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

def adjust_boundaries(lat_min, lat_max, lon_min, lon_max, lat_points, lon_points):
    lat_step = (lat_max - lat_min) / (lat_points - 1)
    lon_step = (lon_max - lon_min) / (lon_points - 1)

    lat_min_adjusted = lat_min - lat_step / 2
    lat_max_adjusted = lat_max + lat_step / 2
    lon_min_adjusted = lon_min - lon_step / 2
    lon_max_adjusted = lon_max + lon_step / 2

    return lat_min_adjusted, lat_max_adjusted, lon_min_adjusted, lon_max_adjusted
def map_high_res_to_low_res(high_res_params, low_res_params):
    high_lat_min, high_lat_max, high_lon_min, high_lon_max, high_lat_points, high_lon_points = high_res_params
    low_lat_min, low_lat_max, low_lon_min, low_lon_max, low_lat_points, low_lon_points = low_res_params
    
    high_lat_linspace = np.linspace(high_lat_min, high_lat_max, high_lat_points)
    high_lon_linspace = np.linspace(high_lon_min, high_lon_max, high_lon_points)
    high_lon_grid, high_lat_grid = np.meshgrid(high_lon_linspace, high_lat_linspace)
    
    low_lat_linspace = np.linspace(low_lat_min, low_lat_max, low_lat_points)
    low_lon_linspace = np.linspace(low_lon_min, low_lon_max, low_lon_points)
    low_lon_grid, low_lat_grid = np.meshgrid(low_lon_linspace, low_lat_linspace)
    
    mapping_matrix = np.zeros((high_lat_points, high_lon_points, 2), dtype=int)

    for i in range(high_lat_points):
        for j in range(high_lon_points):
            high_lon, high_lat = high_lon_grid[i, j], high_lat_grid[i, j]
            dist = (low_lon_grid - high_lon) ** 2 + (low_lat_grid - high_lat) ** 2
            lat_ind, lon_ind = np.unravel_index(np.argmin(dist, axis=None), dist.shape)
            mapping_matrix[i, j] = [lat_ind, lon_ind]
    
    return mapping_matrix

def aggregate_to_low_res2(high_res_data, mapping_matrix, low_res_shape):
    low_res_data = np.zeros((high_res_data.shape[0], low_res_shape[0], low_res_shape[1]))
    
    for t in range(high_res_data.shape[0]):
        accumulator = np.zeros(low_res_shape)
        counter = np.zeros(low_res_shape, dtype=int)
        for i in range(mapping_matrix.shape[0]):
            for j in range(mapping_matrix.shape[1]):
                low_i, low_j = mapping_matrix[i, j]
                accumulator[low_i, low_j] += high_res_data[t, i, j]
                counter[low_i, low_j] += 1

        low_res_data[t] = np.where(counter > 0, accumulator / counter, 0)
    
    return low_res_data
def aggregate_to_low_res(high_res_data, mapping_matrix, low_res_shape):
    """
    将高分辨率数据汇总到低分辨率网格。
    :param high_res_data: 高分辨率数据，形状为 (time, lat, lon)
    :param mapping_matrix: 映射矩阵，形状为 (high_lat, high_lon, 2)
    :param low_res_shape: 低分辨率数据的形状 (lat, lon)
    :return: 聚合后的低分辨率数据
    """
    low_res_data = np.zeros((high_res_data.shape[0], low_res_shape[0], low_res_shape[1]))
    low_i_indices, low_j_indices = mapping_matrix[:, :, 0], mapping_matrix[:, :, 1]

    for t in range(high_res_data.shape[0]):
        np.add.at(low_res_data[t], (low_i_indices, low_j_indices), high_res_data[t])
        counter = np.zeros(low_res_shape, dtype=int)
        np.add.at(counter, (low_i_indices, low_j_indices), 1)
        low_res_data[t] = np.where(counter > 0, low_res_data[t] / counter, 0)

    return low_res_data

def generate_clipping_mask(lon_grid, lat_grid, shp_path):
    """
    使用.shp文件生成数据掩码：在边界外的数据掩码为0，在边界内的数据掩码为1。
    """
    gdf = gpd.read_file(shp_path)
    poly = gdf.unary_union
    mask = np.zeros_like(lon_grid, dtype=bool)

    for i in range(lon_grid.shape[0]):
        for j in range(lon_grid.shape[1]):
            point = Point(lon_grid[i, j], lat_grid[i, j])
            if point.within(poly):
                mask[i, j] = True

    return mask

# 建立高分数据到低分数据映射矩阵
high_res_params = (25.550117, 40.450184, 66.95031, 104.95048, 150, 381)
low_res_params = (25.293533, 40.604774, 66.84705, 105.31184, 42, 104)
high_lat_min_adj, high_lat_max_adj, high_lon_min_adj, high_lon_max_adj = adjust_boundaries(*high_res_params)
low_lat_min_adj, low_lat_max_adj, low_lon_min_adj, low_lon_max_adj = adjust_boundaries(*low_res_params)
high_res_params_adj = (high_lat_min_adj, high_lat_max_adj, high_lon_min_adj, high_lon_max_adj, high_res_params[-2], high_res_params[-1])
low_res_params_adj = (low_lat_min_adj, low_lat_max_adj, low_lon_min_adj, low_lon_max_adj, low_res_params[-2], low_res_params[-1])
mapping_matrix = map_high_res_to_low_res(high_res_params_adj, low_res_params_adj)

reconstruction_paths = {
    'PSC-FENet-SP': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SP_PSC-FENet.npy",
    'PSC-FENet-ST': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\ST_PSC-FENet.npy",
    'PSC-FENet-SP32': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SP_PSC-FENet36.npy",
    'PSC-FENet-ST32': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\ST_PSC-FENet36.npy",
    'LSTM': r"D:\Data_Store\Dataset\Reconstruction\LSTM\LSTM1.npy",
    'RF': r"D:\Data_Store\Dataset\Reconstruction\RF\RF_reconstructed_data.npy",
    'ResNet-SP': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SP_ResNet.npy",
    'ResNet-ST': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\ST_ResNet.npy",
    'UNet-SP': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SP_UNet.npy",
    'UNet-ST': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\ST_UNet.npy",
    'VIT-SP': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SP_ViT.npy",
    'VIT-ST': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\ST_ViT.npy",
    'Temporal': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\Temporal.npy"
}

dates = pd.date_range(start='2016-01-01 03:00', periods=8768, freq='3H')
filter_mask = ((dates.hour == 6) | (dates.hour == 18))
filtered_indices = np.where(filter_mask)[0]

# 第一张图
#selected_model_names = ['PSC-FENet-ST','PSC-FENet-ST32', 'ResNet-ST', 'UNet-ST', 'VIT-ST', 'PSC-FENet-SP','PSC-FENet-SP32', 'ResNet-SP', 'UNet-SP', 'VIT-SP']
# 第二张图
#selected_model_names = ['PSC-FENet-ST','PSC-FENet-ST36', 'LSTM', 'RF', 'Temporal']
selected_model_names = ['PSC-FENet-ST', 'LSTM', 'RF', 'Temporal']

selected_paths = [reconstruction_paths[name] for name in selected_model_names if name in reconstruction_paths]
model_data = [np.load(path)[filtered_indices, :, :] for path in selected_paths]

# 计算均值
low_res_shape = (42, 104)
low_res_model_data = [aggregate_to_low_res(data, mapping_matrix, low_res_shape) for data in model_data]

# 使用掩码矩阵-划分TP
shp_path = r"D:\\Data_Store\\TPBoundary_new(2021)\\TPBoundary_new(2021).shp"

settings = {
    0.37: (25.293533, 40.604774, 66.84705, 105.31184, 42, 104)
}

masks = {}
mode = 0.37
for mode, (lat_min, lat_max, lon_min, lon_max, lat_points, lon_points) in settings.items():
    lon_step = (lon_max - lon_min) / (lon_points - 1)
    lat_step = (lat_max - lat_min) / (lat_points - 1)
    new_lon_min = lon_min - lon_step / 2
    new_lon_max = lon_max + lon_step / 2
    new_lat_min = lat_min - lat_step / 2
    new_lat_max = lat_max + lat_step / 2
    lon_range, lat_range = np.linspace(new_lon_min, new_lon_max, lon_points), np.linspace(new_lat_min, new_lat_max, lat_points)
    lon_grid, lat_grid = np.meshgrid(lon_range, lat_range)
    masks[mode] = generate_clipping_mask(lon_grid, lat_grid, shp_path)

mask = masks[mode]
mask_expanded = mask[np.newaxis, :, :]
low_res_model_data = [np.where(mask_expanded, data, np.nan) for data in low_res_model_data]

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import pearsonr

def cal_data(Original_path, low_res_model_data):
    num_models = len(low_res_model_data)
    # 定义时间筛选
    dates = pd.date_range(start='2016-01-01 03:00', periods=8768, freq='3H')
    filter_mask = ((dates.hour == 6) | (dates.hour == 18))
    filtered_indices = np.where(filter_mask)[0]
    Original_data_low_res_full = np.load(Original_path) 
    Original_data_low_res = Original_data_low_res_full[filtered_indices, :, :]
    
    Original_mean = np.nanmean(Original_data_low_res, axis=0)
    
    # 初始化r_values数组来存储每个模型数据和原始数据之间的相关系数
    r_values = np.zeros((Original_data_low_res.shape[1], Original_data_low_res.shape[2], num_models)) * np.nan
    
    for k, model_data in enumerate(low_res_model_data):
        for i in range(Original_data_low_res.shape[1]):  # 遍历纬度
            for j in range(Original_data_low_res.shape[2]):  # 遍历经度
                original_series = Original_data_low_res[:, i, j]
                model_series = model_data[:, i, j]
                mask = ~np.isnan(original_series) & ~np.isnan(model_series)
                original_series = original_series[mask]
                model_series = model_series[mask]
                if not len(original_series) or not len(model_series):
                    continue
                r, _ = pearsonr(original_series, model_series)
                r_values[i, j, k] = r

    return num_models, Original_mean, r_values
Original_path = r"D:\Data_Store\Dataset\Reconstruction\37SM\soil_moisture.npy"
num_models, Original_mean, r_values = cal_data(Original_path, low_res_model_data)

#### (数据处理) 0.37 卫星和重建数据图和指标 -得到与卫星数据一致的数据lon，lat，times
select_model_names和reconstruction_paths 自动匹配
后续实验中没有生成0.37的数据

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from scipy import stats
import seaborn as sns
from matplotlib.colors import Normalize
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.collections import PolyCollection

low_res_params = (25.293533, 40.604774, 66.84705, 105.31184, 42, 104)
def cal_data(selected_model_names, Original_path, reconstruction_paths):
    dates = pd.date_range(start='2016-01-01 03:00', periods=8768, freq='3H')
    filter_mask = ((dates.hour == 6) | (dates.hour == 18))
    filtered_indices = np.where(filter_mask)[0]
    Original_data = np.load(Original_path)[filtered_indices, :, :]

    #选择模型数据
    selected_paths = [reconstruction_paths[name] for name in selected_model_names if name in reconstruction_paths]
    model_data = [np.load(path)[filtered_indices, :, :] for path in selected_paths]
    num_models = len(model_data)

    Original_mean = np.nanmean(Original_data, axis=0)
    
    # 初始化r_values数组来存储每个模型数据和原始数据之间的相关系数
    r_values = np.zeros((Original_data.shape[1], Original_data.shape[2], num_models)) * np.nan
    for k,model_data in enumerate(model_data):
        for i in range(Original_data.shape[1]):
            for j in range(Original_data.shape[2]):
                original_series = Original_data[:, i, j]
                model_series = model_data[:, i, j]
                mask = ~np.isnan(original_series) & ~np.isnan(model_series)
                original_series = original_series[mask]
                model_series = model_series[mask]
                if np.isnan(original_series).all() or np.isnan(model_series).all():
                    continue
                r, _ = pearsonr(original_series, model_series)
                r_values[i, j, k] = r
    return num_models, Original_mean, r_values

reconstruction_paths = {
    'PSC-FENet-SP': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SP_PSC-FENet.npy",
    'PSC-FENet-ST': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\ST_PSC-FENet.npy",
    'LSTM': r"D:\Data_Store\Dataset\Reconstruction\LSTM\LSTM1.npy",
    'RF': r"D:\Data_Store\Dataset\Reconstruction\RF\RF_reconstructed_data.npy",
    'ResNet-SP': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SP_ResNet.npy",
    'ResNet-ST': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\ST_ResNet.npy",
    'UNet-SP': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SP_UNet.npy",
    'UNet-ST': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\ST_UNet.npy",
    'VIT-SP': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SP_ViT.npy",
    'VIT-ST': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\ST_ViT.npy",
    'Temporal': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\Temporal.npy"
}

model_names = list(reconstruction_paths.keys())
Original_path = r"D:\Data_Store\Dataset\Reconstruction\37SM\soil_moisture.npy"

# 模型选择
# 第一张图
selected_model_names = ['PSC-FENet-ST', 'ResNet-ST', 'UNet-ST', 'VIT-ST', 'PSC-FENet-SP', 'ResNet-SP', 'UNet-SP', 'VIT-SP']
# 第二张图
#selected_model_names = ['KPNet-SP', 'LSTM', 'RF', 'Temporal']

num_models, Original_mean, r_values = cal_data(selected_model_names, Original_path, reconstruction_paths)

#### (数据处理) 0.37数据密度散点图指标计算 R-bias-RMSE-ubRMSE 
使用0.37数据和原始数据计算，先连接所有数据，然后统一计算，每个指标一个numpyarray

In [ ]:
import numpy as np

def load_data(Original_path, reconstruction_paths):
    # 加载原始数据
    dates = pd.date_range(start='2016-01-01 03:00', periods=8768, freq='3H')
    filter_mask = ((dates.hour == 6) | (dates.hour == 18))
    filtered_indices = np.where(filter_mask)[0]
    Original_data = np.load(Original_path)[filtered_indices, :, :]
    model_datas = [np.load(path)[filtered_indices, :, :] for path in reconstruction_paths.values()]

    return Original_data, model_datas

def filter_data(Original_data, model_data):
    # 扁平化数据
    Original_flat = Original_data.flatten()
    model_flat = model_data.flatten()
    
    # 获取非NaN的索引
    valid_idx = ~np.isnan(Original_flat) & ~np.isnan(model_flat)
    
    # 过滤出所有非NaN的数据点
    filtered_original = Original_flat[valid_idx]
    filtered_model = model_flat[valid_idx]
    
    return filtered_original, filtered_model

def calculate_metrics(Original_data, model_datas, model_names):
    metrics = {
        'R': [],
        'Bias': [],
        'RMSE': [],
        'ubRMSE': []
    }
    
    for model_name, model_data in zip(model_names, model_datas):
        # 过滤出有效的数据点
        filtered_original, filtered_model = filter_data(Original_data, model_data)
        
        # 计算相关系数 R
        R = np.corrcoef(filtered_original, filtered_model)[0, 1]
        metrics['R'].append(R)
        
        # 计算偏差 Bias
        Bias = np.mean(filtered_model - filtered_original)
        metrics['Bias'].append(Bias)
        
        # 计算均方根误差 RMSE
        RMSE = np.sqrt(np.mean((filtered_model - filtered_original) ** 2))
        metrics['RMSE'].append(RMSE)
        
        # 计算无偏均方根误差 ubRMSE
        mean_original = np.mean(filtered_original)
        mean_model = np.mean(filtered_model)
        deviations_original = filtered_original - mean_original
        deviations_model = filtered_model - mean_model
        ubRMSE = np.sqrt(np.mean((deviations_model - deviations_original) ** 2))
        metrics['ubRMSE'].append(ubRMSE)
        
    return metrics

#selected_model_names 前有定义
selected_reconstruction_paths = {name: paths for name, paths in reconstruction_paths.items() if name in selected_model_names}
Original_data, reconstruction_data = load_data(Original_path, selected_reconstruction_paths)
metrics = calculate_metrics(Original_data, reconstruction_data, selected_model_names)

# 转为numpy数组
for key in metrics:
    metrics[key] = np.array(metrics[key])
for key, value in metrics.items():
    print(f"{key}: {value}")
np.save(r"C:\Users\Administrator\Desktop\Draw\reconstruction_result.npy",metrics)

#### (数据处理) 0.1数据密度散点图指标计算 R-bias-RMSE-ubRMSE 
绘图步骤中第一步的后续步骤哦

In [ ]:
def calculate_metrics(Original_data, model_datas, model_names):
    metrics = {
        'R': [],
        'Bias': [],
        'RMSE': [],
        'ubRMSE': []
    }
    
    for model_name, model_data in zip(model_names, model_datas):
        # 过滤出有效的数据点
        filtered_original, filtered_model = filter_data(Original_data, model_data)
        
        # 计算相关系数 R
        R = np.corrcoef(filtered_original, filtered_model)[0, 1]
        metrics['R'].append(R)
        
        # 计算偏差 Bias
        Bias = np.mean(filtered_model - filtered_original)
        metrics['Bias'].append(Bias)
        
        # 计算均方根误差 RMSE
        RMSE = np.sqrt(np.mean((filtered_model - filtered_original) ** 2))
        metrics['RMSE'].append(RMSE)
        
        # 计算无偏均方根误差 ubRMSE
        mean_original = np.mean(filtered_original)
        mean_model = np.mean(filtered_model)
        deviations_original = filtered_original - mean_original
        deviations_model = filtered_model - mean_model
        ubRMSE = np.sqrt(np.mean((deviations_model - deviations_original) ** 2))
        metrics['ubRMSE'].append(ubRMSE)
        
    return metrics
def filter_data(Original_data, model_data):
    # 扁平化数据
    Original_flat = Original_data.flatten()
    model_flat = model_data.flatten()
    
    # 获取非NaN的索引
    valid_idx = ~np.isnan(Original_flat) & ~np.isnan(model_flat)
    
    # 过滤出所有非NaN的数据点
    filtered_original = Original_flat[valid_idx]
    filtered_model = model_flat[valid_idx]
    
    return filtered_original, filtered_model
def load_data(Original_path):
    # 加载原始数据
    dates = pd.date_range(start='2016-01-01 03:00', periods=8768, freq='3H')
    filter_mask = ((dates.hour == 6) | (dates.hour == 18))
    filtered_indices = np.where(filter_mask)[0]
    Original_data = np.load(Original_path)[filtered_indices, :, :]

    return Original_data

Original_data = load_data(Original_path)
metrics = calculate_metrics(Original_data, low_res_model_data, selected_model_names)

# 将结果转换为numpy数组
for key in metrics:
    metrics[key] = np.array(metrics[key])

# 打印结果
for key, value in metrics.items():
    print(f"{key}: {value}")
np.save(r"C:\Users\Administrator\Desktop\Draw\reconstruction_result.npy",metrics)

### (中间过程查看) 

In [ ]:
# 绘制图像DPI=300
plt.figure(dpi=150)
plt.imshow(Original_data[3], cmap='viridis', origin='lower',vmin=0,vmax=0.5)
plt.colorbar(label='Value')
plt.title(f"Data at Time Index: {5}")
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()
# 绘制图像DPI=300
plt.figure(dpi=150)
plt.imshow(low_res_model_data[0][3], cmap='viridis', origin='lower',vmin=0,vmax=0.5)
plt.colorbar(label='Value')
plt.title(f"Data at Time Index: {5}")
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

In [ ]:
reconstruction_paths = {
    #'KPNet-SP': r"D:\Data_Store\Dataset\Reconstruction\ST\RES1616\spatial_kpnet_Encoder_lite_serial_temporal_usage_position_embedding_individual_init256371616.npy" ,
    'KPNet-ST': r"D:\Data_Store\Dataset\Reconstruction\ST\RES1616\ST_kpnet_Encoder_lite_serial_temporal_usage_position_embedding_individual_init256_11_121_131_dropout_rate0.3371616.npy",
    'LSTM': r"D:\Data_Store\Dataset\Reconstruction\LSTM\LSTM_0.37.npy",
    'RF': r"D:\Data_Store\Dataset\Reconstruction\RF\RF_reconstructed_data_0.37.npy",
    'ResNet-SP': r"D:\Data_Store\Dataset\Reconstruction\ST\RES1616\spatial_resnet34_temporal_usage_position_embedding_init256371616.npy",
    #'ResNet-ST': r"D:\Data_Store\Dataset\Reconstruction\ST\RES1616\ST_resnet34_temporal_usage_position_embedding_init256371616.npy",
    'UNet-SP': r"D:\Data_Store\Dataset\Reconstruction\ST\RES37\spatial_unet_temporal_usage_position_embedding_init25637168.npy",
    #'UNet-ST': r"D:\Data_Store\Dataset\Reconstruction\ST\RES1616\ST_unet_temporal_usage_position_embedding_init256371616.npy",
    'VIT-SP': r"D:\Data_Store\Dataset\Reconstruction\ST\RES1616\spatial_VIT_serial_temporal_usage_position_embedding_individual_init256371616.npy",
    #'VIT-ST': r"D:\Data_Store\Dataset\Reconstruction\ST\RES1616\ST_ViT_serial_temporal_usage_position_embedding_individual_init256371616.npy",
    'Temporal': r"D:\Data_Store\Dataset\Reconstruction\ST\RES1616\temporal_temporal_usage_position_embedding371616.npy"
}
Original_path = r"D:\Data_Store\Dataset\Reconstruction\37SM\soil_moisture.npy"
model_names = ['KPNet-ST', 'LSTM', 'RF', 'ResNet-SP', 'UNet-SP', 'VIT-SP', 'Temporal']
data = np.load(r"D:\Data_Store\Dataset\Reconstruction\ST\RES1616\ST_kpnet_Encoder_lite_serial_temporal_usage_position_embedding_individual_init256_11_121_131_dropout_rate0.3371616.npy")
plt.figure(dpi=150)
plt.imshow(data[3], cmap='viridis', origin='lower',vmin=0,vmax=0.5)
plt.colorbar(label='Value')
plt.title(f"Data at Time Index: {5}")
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

In [ ]:
data = low_res_model_data[0]
plt.figure(dpi=150)
plt.imshow(data[0], cmap='viridis', origin='lower',vmin=0,vmax=0.5)
plt.colorbar(label='Value')
plt.title(f"Data at Time Index: {5}")
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
data = np.load(r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\ST_PSC-FENet.npy")
plt.figure(dpi=150)
plt.imshow(data[3], cmap='viridis', origin='lower',vmin=0,vmax=0.5)
plt.colorbar(label='Value')
plt.title(f"Data at Time Index: {5}")
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
data = np.load(r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\ST_PSC-FENet.npy")
plt.figure(dpi=150)
plt.imshow(data[3], cmap='viridis', origin='lower',vmin=0,vmax=0.5)
plt.colorbar(label='Value')
plt.title(f"Data at Time Index: {5}")
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.grid(False)
plt.show()

### (绘图) V1.0 3-4 密度散点图-完整绘制
SM不按均值区间分割

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from scipy import stats
import seaborn as sns
from matplotlib.colors import Normalize
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.collections import PolyCollection
import matplotlib.colors as mcolors
from scipy.stats import gaussian_kde
from matplotlib.colors import to_rgba
from matplotlib.font_manager import FontProperties

def clean_axis(ax):
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.tick_params(axis='x', which='both', length=0, labelbottom=False)
    ax.tick_params(axis='y', which='both', length=0, labelleft=False)
    ax.grid(False)
def create_subplots(num_models, Original_mean,r_values, lon_range, lat_range, model_names,  metric, plt_name=None):
    # 创建图形对象,绘制顺序依赖于fig定义顺序
    #colors = ['#f8d1b4', '#f69b72', '#99f2d1', '#315c4f', '#b096b6', '#3e2248', '#555555']['#acebe2', '#9fe6dd', '#93e1d8' ,'#6d87bc','#5673ae', '#3e5fa0', '#274b91']
    colors = ['#1f7ea1', '#1f7ea1', '#1f7ea1' ,'#274b91','#274b91', '#274b91', '#274b91']
    fig = plt.figure(figsize=(18, 24))

    ratio_h = 4 # 竖直比例
    ratio_v = 3 # 水平比例

    absolute_dis_h = 0.45/ratio_h # 中央子图绝对大小（竖直）0.15
    absolute_dis_v = 0.45/ratio_v # 中央子图绝对大小（水平）0.2
    absolute_dis_v_third = 0.45/ratio_v*2.5 # 第三层子图绝对大小（水平）0.2

    edge_h = 0.29 # 竖直边缘间隙
    edge_v = 0.1 # 水平边缘间隙

    mid_h = (1-edge_h*2-absolute_dis_h*3)/2 # 竖直间隙,可以通过竖直边缘间隙调节
    mid_v = (1-edge_v*2-absolute_dis_v*4)/3 # 水平间隙……

    right_offset_first = 0.10 # 第一层向右偏移量

    up_offset = 0.1129 # 上方子图向上偏移
    right_offset = 0.1505 # 右方子图向右偏移
    up_ratio = 0.12  # 上方子图长度比例
    right_ratio = 0.25  # 上方子图宽度比例

    # 计算第一层图起点:上层子图  左起点、下起点、宽度比例、高度比例
    ax2 = fig.add_axes([edge_v + right_offset_first, 1 - edge_h - absolute_dis_h + up_offset, absolute_dis_v, absolute_dis_h * up_ratio])  # 左上上
    ax3 = fig.add_axes([edge_v + right_offset_first + right_offset, 1 - edge_h - absolute_dis_h, absolute_dis_v * right_ratio, absolute_dis_h])  # 左上右
    ax1 = fig.add_axes([edge_v + right_offset_first, 1 - edge_h - absolute_dis_h, absolute_dis_v, absolute_dis_h])  # 左上

    ax5 = fig.add_axes([edge_v + right_offset_first + absolute_dis_v + mid_v, 1 - edge_h - absolute_dis_h + up_offset, absolute_dis_v, absolute_dis_h * up_ratio])  # 中上上 32
    ax6 = fig.add_axes([edge_v + right_offset_first + right_offset + absolute_dis_v + mid_v, 1 - edge_h - absolute_dis_h, absolute_dis_v * right_ratio, absolute_dis_h])  # 中上右 32
    ax4 = fig.add_axes([edge_v + right_offset_first + absolute_dis_v + mid_v, 1 - edge_h - absolute_dis_h, absolute_dis_v, absolute_dis_h])  # 中上 32

    ax8 = fig.add_axes([edge_v + right_offset_first + 2*(absolute_dis_v + mid_v), 1 - edge_h - absolute_dis_h + up_offset, absolute_dis_v, absolute_dis_h * up_ratio])  # 右上上
    ax9 = fig.add_axes([edge_v + right_offset_first + right_offset + 2*(absolute_dis_v + mid_v), 1 - edge_h - absolute_dis_h, absolute_dis_v * right_ratio, absolute_dis_h])  # 右上右
    ax7 = fig.add_axes([edge_v + right_offset_first + 2*(absolute_dis_v + mid_v), 1 - edge_h - absolute_dis_h, absolute_dis_v, absolute_dis_h])  # 右上

    # 中层子图
    ax11 = fig.add_axes([edge_v, 1 - edge_h - 2*absolute_dis_h - mid_h + up_offset, absolute_dis_v, absolute_dis_h * up_ratio])  # 左中上
    ax12 = fig.add_axes([edge_v + right_offset, 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v * right_ratio, absolute_dis_h])  # 左中右
    ax10 = fig.add_axes([edge_v , 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v, absolute_dis_h])  # 左中

    ax14 = fig.add_axes([edge_v + absolute_dis_v + mid_v, 1 - edge_h - 2*absolute_dis_h - mid_h + up_offset, absolute_dis_v, absolute_dis_h * up_ratio])  # 中左中上
    ax15 = fig.add_axes([edge_v + right_offset + absolute_dis_v + mid_v, 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v * right_ratio, absolute_dis_h])  # 中左中右
    ax13 = fig.add_axes([edge_v + absolute_dis_v + mid_v, 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v, absolute_dis_h])  # 中左中

    ax17 = fig.add_axes([edge_v + 2*(absolute_dis_v + mid_v), 1 - edge_h - 2*absolute_dis_h - mid_h + up_offset, absolute_dis_v, absolute_dis_h * up_ratio])  # 中右中上
    ax18 = fig.add_axes([edge_v + right_offset + 2*(absolute_dis_v + mid_v), 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v * right_ratio, absolute_dis_h])  # 中右中右
    ax16 = fig.add_axes([edge_v + 2*(absolute_dis_v + mid_v), 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v, absolute_dis_h])  # 中右中

    ax20 = fig.add_axes([edge_v + 3*(absolute_dis_v + mid_v), 1 - edge_h - 2*absolute_dis_h - mid_h + up_offset, absolute_dis_v, absolute_dis_h * up_ratio])  # 右中上
    ax21 = fig.add_axes([edge_v + right_offset + 3*(absolute_dis_v + mid_v), 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v * right_ratio, absolute_dis_h])  # 右中右
    ax19 = fig.add_axes([edge_v + 3*(absolute_dis_v + mid_v), 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v, absolute_dis_h])  # 右中

    axes = [ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8, ax9, ax10, ax11, ax12, ax13, ax14, ax15, ax16, ax17, ax18, ax19, ax20, ax21]

    color_scheme1 = ['#c4f5ed', '#1f7ea1']
    color_scheme2 = ['#9bafd9', '#103783']

    r_set = metric['R']
    bias_set = metric['Bias']
    rmse_set = metric['RMSE']
    ubrmse_set = metric['ubRMSE']

    for i in range(num_models):
        if i < 3:
            color_scheme = color_scheme1
        else:
            color_scheme = color_scheme2
        draw_subplot(axes[3*i], Original_mean, r_values[:,:,i], model_names[i], color_scheme,r_set[i],bias_set[i],rmse_set[i],ubrmse_set[i])
        draw_subplot_up(axes[3*i+1], Original_mean, r_values[:,:,i], colors[i])
        draw_subplot_left(axes[3*i+2], Original_mean, r_values[:,:,i], colors[i])  
    fig.savefig(r"C:\Users\Administrator\Desktop\Draw\sat_original.jpeg", dpi=1000, format='jpeg', bbox_inches='tight')
    plt.show()


def draw_subplot(ax, Original_mean, r_value, model_name, color_scheme, r, bias, rmse, ubrmse):
    cmap = mcolors.LinearSegmentedColormap.from_list('custom_cmap', color_scheme)
    model_names = ['KPNet-ST', 'LSTM', 'RF', 'ResNet-SP', 'UNet-SP', 'VIT-SP', 'Temporal']
    font_prop = FontProperties(family='Times New Roman', size=14)
    r_value = r_value.flatten()
    Original_mean = Original_mean.flatten()
    mask = ~np.isnan(r_value) & ~np.isnan(Original_mean)
    x = Original_mean[mask]
    y = r_value[mask]

    xy = np.vstack([x, y])
    z = gaussian_kde(xy)(xy)

    scatter = ax.scatter(x, y, c=z, s=12, cmap=cmap, edgecolor='face', linewidth=0, alpha=1)
    sns.regplot(x=x, y=y, ax=ax, order=3, scatter=False, color='#522226', line_kws={'label':f'Quad Fit: {model_name}'}, ci=95)
    for artist in ax.findobj(PolyCollection):
        artist.set_alpha(0.4)

    x_min,x_max = np.percentile(x, 1), np.percentile(x, 99)
    y_min,y_max = np.percentile(y, 1), np.percentile(y, 99)

    padding_x = (x_max - x_min) * 0.01
    padding_y = (0.9999 - y_min) * 0.01
    # 指标文本
    x_range = np.arange(0,4)*6
    text_position_x = x_max - padding_x * 0.5
    text_position_y_r = y_min + padding_y * x_range[3] + padding_y * 0.5
    text_position_y_bias = y_min + padding_y * x_range[2] + padding_y * 0.5
    text_position_y_rmse = y_min + padding_y * x_range[1] + padding_y * 0.5
    text_position_y_ubrmse = y_min + padding_y * x_range[0] + padding_y * 0.5
    # 使用FontProperties实例绘制文本，去除$符号避免斜体
    ax.text(text_position_x, text_position_y_r, f'R={r:.3f}', fontproperties=font_prop, horizontalalignment='right', verticalalignment='bottom')
    ax.text(text_position_x, text_position_y_rmse, f'RMSE={rmse:.3f}', fontproperties=font_prop, horizontalalignment='right', verticalalignment='bottom')
    ax.text(text_position_x, text_position_y_bias, f'bias={bias:.3f}', fontproperties=font_prop, horizontalalignment='right', verticalalignment='bottom')
    ax.text(text_position_x, text_position_y_ubrmse, f'ubRMSE={ubrmse:.3f}', fontproperties=font_prop, horizontalalignment='right', verticalalignment='bottom')
    prefix = "(" + chr(ord('a') + model_names.index(model_name)) + ")"
    ax.text(0.01, 0.11, f'{prefix} {model_name}', transform=ax.transAxes, fontsize=18, fontname='Times New Roman', verticalalignment='top')

    # 设置刻度
    ax.set_xlim(x_min,x_max)
    ax.set_ylim(y_min,0.9999)
    ax.xaxis.set_major_locator(MaxNLocator(nbins=4))
    ax.yaxis.set_major_locator(MaxNLocator(nbins=4))
    ax.tick_params(axis='x', which='both', colors='black', length=3, width=1.5, labelsize=14, direction='out', pad=2, bottom=True, top=False, labelbottom=True,labeltop=False)
    ax.tick_params(axis='y', which='both', colors='black', length=3, width=1.5, labelsize=14, direction='out', pad=6, left=True, right=False, labelleft=True,labelright=False)
    # 设置轴标签
    ax.set_xlabel('Original mean SM (m³/m³)', size=16, fontname='Times New Roman')
    ax.set_ylabel('Pearson R',size=16)
    plt.setp(ax.get_xticklabels(), rotation=0,rotation_mode="anchor")
    plt.setp(ax.get_yticklabels(), rotation=90,ha="center", rotation_mode="anchor")
    # 设置轴参数
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_color('black')
    ax.spines['bottom'].set_linewidth(1.5)
    ax.spines['left'].set_color('black')
    ax.spines['left'].set_linewidth(1.5)
    ax.grid(True, color='black', linestyle='--',alpha=0.5,zorder=0.1)

def draw_subplot1(ax, Original_mean, r_value, model_name, color_scheme,r,bias,rmse,ubrmse):
    cmap = mcolors.LinearSegmentedColormap.from_list('custom_cmap', color_scheme)
    model_names = ['KPNet-ST', 'LSTM', 'RF', 'ResNet-SP', 'UNet-SP', 'VIT-SP', 'Temporal']
    font_prop = FontProperties(family='Times New Roman', size=14)
    r_value = r_value.flatten()
    Original_mean = Original_mean.flatten()
    mask = ~np.isnan(r_value) & ~np.isnan(Original_mean)
    x = Original_mean[mask]
    y = r_value[mask]

    # ax.scatter(x, y, color=color, alpha=0.5)
    # 分割Original_mean并计算每个区间的r_value均值
    bins = np.arange(np.min(x), np.max(x) + 0.0001, 0.0001)
    digitized = np.digitize(x, bins)
    bin_means = np.array([y[digitized == i].mean() for i in range(1, len(bins))])
    bin_centers = (bins[:-1] + bins[1:]) / 2

    valid_mask = ~np.isnan(bin_means)
    bin_centers_clean = bin_centers[valid_mask]
    bin_means_clean = bin_means[valid_mask]

    xy_means = np.vstack([bin_centers_clean, bin_means_clean])
    z_means = gaussian_kde(xy_means)(xy_means)

    scatter = ax.scatter(bin_centers_clean, bin_means_clean, c=z_means, s=12, cmap=cmap, edgecolor='face', linewidth=0, alpha=1)
    sns.regplot(x=bin_centers_clean, y=bin_means_clean, ax=ax, order=3, scatter=False, color='#522226', line_kws={'label':f'Quad Fit: {model_name}'}, ci=95)
    for artist in ax.findobj(PolyCollection):
        artist.set_alpha(0.4)

    x_min,x_max = np.percentile(x, 1), np.percentile(x, 99)
    y_min,y_max = np.percentile(y, 5), np.percentile(y, 95)

    padding_x = (x_max - x_min) * 0.01
    padding_y = (0.9999 - y_min) * 0.01
    # 指标文本
    x_range = np.arange(0,4)*6
    text_position_x = x_max - padding_x * 0.5
    text_position_y_r = y_min + padding_y * x_range[3] + padding_y * 0.5
    text_position_y_bias = y_min + padding_y * x_range[2] + padding_y * 0.5
    text_position_y_rmse = y_min + padding_y * x_range[1] + padding_y * 0.5
    text_position_y_ubrmse = y_min + padding_y * x_range[0] + padding_y * 0.5
    # 使用FontProperties实例绘制文本，去除$符号避免斜体
    ax.text(text_position_x, text_position_y_r, f'R={r:.3f}', fontproperties=font_prop, horizontalalignment='right', verticalalignment='bottom')
    ax.text(text_position_x, text_position_y_rmse, f'RMSE={rmse:.3f}', fontproperties=font_prop, horizontalalignment='right', verticalalignment='bottom')
    ax.text(text_position_x, text_position_y_bias, f'bias={bias:.3f}', fontproperties=font_prop, horizontalalignment='right', verticalalignment='bottom')
    ax.text(text_position_x, text_position_y_ubrmse, f'ubRMSE={ubrmse:.3f}', fontproperties=font_prop, horizontalalignment='right', verticalalignment='bottom')
    prefix = "(" + chr(ord('a') + model_names.index(model_name)) + ")"
    ax.text(0.01, 0.11, f'{prefix} {model_name}', transform=ax.transAxes, fontsize=18, fontname='Times New Roman', verticalalignment='top')

    # 设置刻度
    ax.set_xlim(x_min,x_max)
    ax.set_ylim(y_min,0.9999)
    ax.xaxis.set_major_locator(MaxNLocator(nbins=4))
    ax.yaxis.set_major_locator(MaxNLocator(nbins=4))
    ax.tick_params(axis='x', which='both', colors='black', length=3, width=1.5, labelsize=14, direction='out', pad=2, bottom=True, top=False, labelbottom=True,labeltop=False)
    ax.tick_params(axis='y', which='both', colors='black', length=3, width=1.5, labelsize=14, direction='out', pad=6, left=True, right=False, labelleft=True,labelright=False)
    # 设置轴标签
    ax.set_xlabel('Original mean SM (m³/m³)', size=16, fontname='Times New Roman')
    ax.set_ylabel('Pearson R',size=16)
    plt.setp(ax.get_xticklabels(), rotation=0,rotation_mode="anchor")
    plt.setp(ax.get_yticklabels(), rotation=90,ha="center", rotation_mode="anchor")
    # 设置轴参数
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_color('black')
    ax.spines['bottom'].set_linewidth(1.5)
    ax.spines['left'].set_color('black')
    ax.spines['left'].set_linewidth(1.5)
    ax.grid(True, color='black', linestyle='--',alpha=0.5,zorder=0.1)

def draw_subplot_up(ax, Original_mean, r_value, color):
    r_value = r_value.flatten()
    Original_mean = Original_mean.flatten()
    mask = ~np.isnan(r_value) & ~np.isnan(Original_mean)
    x = Original_mean[mask]
    y = r_value[mask]
    x_min,x_max = np.percentile(x, 1), np.percentile(x, 99)

    step = 0.001
    mask = ~np.isnan(Original_mean) & ~np.isnan(r_value)
    Original_mean = Original_mean[mask].flatten()
    min_val, max_val = np.min(Original_mean), np.max(Original_mean)
    bins = np.arange(min_val, max_val + step, step)
    counts, _ = np.histogram(Original_mean, bins=bins)
    centers = (bins[:-1] + bins[1:]) / 2
    counts_min,counts_max = min(counts),max(counts)
    ax.bar(centers, counts, width=step, color=color, edgecolor=color, alpha=0.8)
    ax.set_ylim(0,counts_max)
    ax.set_xlim(x_min,x_max)
    clean_axis(ax)

def draw_subplot_left(ax, Original_mean, r_value, color):
    r_value = r_value.flatten()
    Original_mean = Original_mean.flatten()
    mask = ~np.isnan(r_value) & ~np.isnan(Original_mean)
    x = Original_mean[mask]
    y = r_value[mask]

    step = 0.001
    mask = ~np.isnan(r_value) & ~np.isnan(Original_mean)
    r_value = r_value[mask].flatten()
    min_val, max_val = np.min(r_value), np.max(r_value)
    bins = np.arange(min_val, max_val + step, step)
    counts, _ = np.histogram(r_value, bins=bins)
    centers = (bins[:-1] + bins[1:]) / 2
    ax.barh(centers, counts, height=step, color=color, edgecolor=color, alpha=0.8)
    counts_min,counts_max = min(counts),max(counts)
    y_min,y_max = np.percentile(y, 1), np.percentile(y, 99)
    ax.set_xlim(0, counts_max)
    ax.set_ylim(y_min, 0.9999)
    clean_axis(ax)
    
metric = np.load(r"C:\Users\Administrator\Desktop\Draw\reconstruction_result.npy",allow_pickle=True).item() 
create_subplots(num_models, Original_mean,r_values, lon_range, lat_range, model_names, metric, plt_name = 'a')

### (绘图) V1.0 密度散点图-分割绘制 
Mean SM按0.001步长均值然后绘制密度散点

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import Normalize
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.collections import PolyCollection
import matplotlib.colors as mcolors
from scipy.stats import gaussian_kde
from matplotlib.colors import to_rgba
from matplotlib.font_manager import FontProperties

def clean_axis(ax):
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.tick_params(axis='x', which='both', length=0, labelbottom=False)
    ax.tick_params(axis='y', which='both', length=0, labelleft=False)
    ax.grid(False)
def create_subplots(num_models, Original_mean,r_values, lon_range, lat_range, model_names,  metric, plt_name=None):
    # 创建图形对象,绘制顺序依赖于fig定义顺序
    colors = ['#1f7ea1', '#1f7ea1', '#1f7ea1' ,'#274b91','#274b91', '#274b91', '#274b91']
    fig = plt.figure(figsize=(18, 24))

    ratio_h = 4 # 竖直比例
    ratio_v = 3 # 水平比例

    absolute_dis_h = 0.45/ratio_h # 中央子图绝对大小（竖直）0.15
    absolute_dis_v = 0.45/ratio_v # 中央子图绝对大小（水平）0.2

    edge_h = 0.29 # 竖直边缘间隙
    edge_v = 0.1 # 水平边缘间隙

    mid_h = (1-edge_h*2-absolute_dis_h*3)/2 # 竖直间隙,可以通过竖直边缘间隙调节
    mid_v = (1-edge_v*2-absolute_dis_v*4)/3 # 水平间隙……

    right_offset_first = 0.10 # 第一层向右偏移量

    up_offset = 0.1129 # 上方子图向上偏移
    right_offset = 0.1505 # 右方子图向右偏移
    up_ratio = 0.12  # 上方子图长度比例
    right_ratio = 0.25  # 上方子图宽度比例

    # 计算第一层图起点:上层子图  左起点、下起点、宽度比例、高度比例
    ax2 = fig.add_axes([edge_v + right_offset_first, 1 - edge_h - absolute_dis_h + up_offset, absolute_dis_v, absolute_dis_h * up_ratio])  # 左上上
    ax3 = fig.add_axes([edge_v + right_offset_first + right_offset, 1 - edge_h - absolute_dis_h, absolute_dis_v * right_ratio, absolute_dis_h])  # 左上右
    ax1 = fig.add_axes([edge_v + right_offset_first, 1 - edge_h - absolute_dis_h, absolute_dis_v, absolute_dis_h])  # 左上

    ax5 = fig.add_axes([edge_v + right_offset_first + absolute_dis_v + mid_v, 1 - edge_h - absolute_dis_h + up_offset, absolute_dis_v, absolute_dis_h * up_ratio])  # 中上上 32
    ax6 = fig.add_axes([edge_v + right_offset_first + right_offset + absolute_dis_v + mid_v, 1 - edge_h - absolute_dis_h, absolute_dis_v * right_ratio, absolute_dis_h])  # 中上右 32
    ax4 = fig.add_axes([edge_v + right_offset_first + absolute_dis_v + mid_v, 1 - edge_h - absolute_dis_h, absolute_dis_v, absolute_dis_h])  # 中上 32

    ax8 = fig.add_axes([edge_v + right_offset_first + 2*(absolute_dis_v + mid_v), 1 - edge_h - absolute_dis_h + up_offset, absolute_dis_v, absolute_dis_h * up_ratio])  # 右上上
    ax9 = fig.add_axes([edge_v + right_offset_first + right_offset + 2*(absolute_dis_v + mid_v), 1 - edge_h - absolute_dis_h, absolute_dis_v * right_ratio, absolute_dis_h])  # 右上右
    ax7 = fig.add_axes([edge_v + right_offset_first + 2*(absolute_dis_v + mid_v), 1 - edge_h - absolute_dis_h, absolute_dis_v, absolute_dis_h])  # 右上

    # 中层子图
    ax11 = fig.add_axes([edge_v, 1 - edge_h - 2*absolute_dis_h - mid_h + up_offset, absolute_dis_v, absolute_dis_h * up_ratio])  # 左中上
    ax12 = fig.add_axes([edge_v + right_offset, 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v * right_ratio, absolute_dis_h])  # 左中右
    ax10 = fig.add_axes([edge_v , 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v, absolute_dis_h])  # 左中

    ax14 = fig.add_axes([edge_v + absolute_dis_v + mid_v, 1 - edge_h - 2*absolute_dis_h - mid_h + up_offset, absolute_dis_v, absolute_dis_h * up_ratio])  # 中左中上
    ax15 = fig.add_axes([edge_v + right_offset + absolute_dis_v + mid_v, 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v * right_ratio, absolute_dis_h])  # 中左中右
    ax13 = fig.add_axes([edge_v + absolute_dis_v + mid_v, 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v, absolute_dis_h])  # 中左中

    ax17 = fig.add_axes([edge_v + 2*(absolute_dis_v + mid_v), 1 - edge_h - 2*absolute_dis_h - mid_h + up_offset, absolute_dis_v, absolute_dis_h * up_ratio])  # 中右中上
    ax18 = fig.add_axes([edge_v + right_offset + 2*(absolute_dis_v + mid_v), 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v * right_ratio, absolute_dis_h])  # 中右中右
    ax16 = fig.add_axes([edge_v + 2*(absolute_dis_v + mid_v), 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v, absolute_dis_h])  # 中右中

    ax20 = fig.add_axes([edge_v + 3*(absolute_dis_v + mid_v), 1 - edge_h - 2*absolute_dis_h - mid_h + up_offset, absolute_dis_v, absolute_dis_h * up_ratio])  # 右中上
    ax21 = fig.add_axes([edge_v + right_offset + 3*(absolute_dis_v + mid_v), 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v * right_ratio, absolute_dis_h])  # 右中右
    ax19 = fig.add_axes([edge_v + 3*(absolute_dis_v + mid_v), 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v, absolute_dis_h])  # 右中

    axes = [ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8, ax9, ax10, ax11, ax12, ax13, ax14, ax15, ax16, ax17, ax18, ax19, ax20, ax21]

    color_scheme1 = ['#c4f5ed', '#1f7ea1']
    color_scheme2 = ['#9bafd9', '#103783']

    r_set = metric['R']
    bias_set = metric['Bias']
    rmse_set = metric['RMSE']
    ubrmse_set = metric['ubRMSE']

    for i in range(num_models):
        if i < 3:
            color_scheme = color_scheme1
        else:
            color_scheme = color_scheme2
        draw_subplot(axes[3*i], Original_mean, r_values[:,:,i], model_names[i], color_scheme,r_set[i],bias_set[i],rmse_set[i],ubrmse_set[i])
        draw_subplot_up(axes[3*i+1], Original_mean, r_values[:,:,i], colors[i])
        draw_subplot_left(axes[3*i+2], Original_mean, r_values[:,:,i], colors[i])  
    fig.savefig(r"C:\Users\Administrator\Desktop\Draw\sat_original.jpeg", dpi=500, format='jpeg', bbox_inches='tight')
    plt.show()

def draw_subplot(ax, Original_mean, r_value, model_name, color_scheme,r,bias,rmse,ubrmse):
    cmap = mcolors.LinearSegmentedColormap.from_list('custom_cmap', color_scheme)
    model_names = ['KPNet-ST', 'LSTM', 'RF', 'ResNet-SP', 'UNet-SP', 'VIT-SP', 'Temporal']
    font_prop = FontProperties(family='Times New Roman', size=14)
    r_value = r_value.flatten()
    Original_mean = Original_mean.flatten()
    mask = ~np.isnan(r_value) & ~np.isnan(Original_mean)
    x = Original_mean[mask]
    y = r_value[mask]

    # ax.scatter(x, y, color=color, alpha=0.5)
    # 分割Original_mean并计算每个区间的r_value均值
    bins = np.arange(np.min(x), np.max(x) + 0.0005, 0.0005)
    digitized = np.digitize(x, bins)
    bin_means = np.array([y[digitized == i].mean() if not y[digitized == i].size == 0 else np.nan for i in range(1, len(bins))])
    bin_centers = (bins[:-1] + bins[1:]) / 2

    valid_mask = ~np.isnan(bin_means)
    bin_centers_clean = bin_centers[valid_mask]
    bin_means_clean = bin_means[valid_mask]

    xy_means = np.vstack([bin_centers_clean, bin_means_clean])
    z_means = gaussian_kde(xy_means)(xy_means)

    scatter = ax.scatter(bin_centers_clean, bin_means_clean, c=z_means, s=20, cmap=cmap, edgecolor='face', linewidth=0, alpha=1)
    sns.regplot(x=bin_centers_clean, y=bin_means_clean, ax=ax, order=3, scatter=False, color='#522226', line_kws={'label':f'Quad Fit: {model_name}'}, ci=95)
    for artist in ax.findobj(PolyCollection):
        artist.set_alpha(0.4)

    x_min,x_max = np.percentile(x, 1), np.percentile(x, 99)
    y_min,y_max = np.percentile(y, 5), np.percentile(y, 95)

    padding_x = (x_max - x_min) * 0.01
    padding_y = (0.9999 - y_min) * 0.01
    # 指标文本
    x_range = np.arange(0,4)*6
    text_position_x = x_max - padding_x * 0.5
    text_position_y_r = y_min + padding_y * x_range[3] + padding_y * 0.5
    text_position_y_bias = y_min + padding_y * x_range[2] + padding_y * 0.5
    text_position_y_rmse = y_min + padding_y * x_range[1] + padding_y * 0.5
    text_position_y_ubrmse = y_min + padding_y * x_range[0] + padding_y * 0.5
    # 使用FontProperties实例绘制文本，去除$符号避免斜体
    ax.text(text_position_x, text_position_y_r, f'R={r:.3f}', fontproperties=font_prop, horizontalalignment='right', verticalalignment='bottom')
    ax.text(text_position_x, text_position_y_rmse, f'RMSE={rmse:.3f}', fontproperties=font_prop, horizontalalignment='right', verticalalignment='bottom')
    ax.text(text_position_x, text_position_y_bias, f'bias={bias:.3f}', fontproperties=font_prop, horizontalalignment='right', verticalalignment='bottom')
    ax.text(text_position_x, text_position_y_ubrmse, f'ubRMSE={ubrmse:.3f}', fontproperties=font_prop, horizontalalignment='right', verticalalignment='bottom')
    prefix = "(" + chr(ord('a') + model_names.index(model_name)) + ")"
    ax.text(0.01, 0.11, f'{prefix} {model_name}', transform=ax.transAxes, fontsize=18, fontname='Times New Roman', verticalalignment='top')

    # 设置刻度
    ax.set_xlim(x_min,x_max)
    ax.set_ylim(y_min,0.9999)
    ax.xaxis.set_major_locator(MaxNLocator(nbins=4))
    ax.yaxis.set_major_locator(MaxNLocator(nbins=4))
    ax.tick_params(axis='x', which='both', colors='black', length=3, width=1.5, labelsize=14, direction='out', pad=2, bottom=True, top=False, labelbottom=True,labeltop=False)
    ax.tick_params(axis='y', which='both', colors='black', length=3, width=1.5, labelsize=14, direction='out', pad=6, left=True, right=False, labelleft=True,labelright=False)
    # 设置轴标签
    ax.set_xlabel('Original mean SM (m³/m³)', size=16, fontname='Times New Roman')
    ax.set_ylabel('Pearson R',size=16)
    plt.setp(ax.get_xticklabels(), rotation=0,rotation_mode="anchor")
    plt.setp(ax.get_yticklabels(), rotation=90,ha="center", rotation_mode="anchor")
    # 设置轴参数
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_color('black')
    ax.spines['bottom'].set_linewidth(1.5)
    ax.spines['left'].set_color('black')
    ax.spines['left'].set_linewidth(1.5)
    ax.grid(True, color='black', linestyle='--',alpha=0.5,zorder=0.1)

def draw_subplot_up(ax, Original_mean, r_value, color):
    r_value = r_value.flatten()
    Original_mean = Original_mean.flatten()
    mask = ~np.isnan(r_value) & ~np.isnan(Original_mean)
    x = Original_mean[mask]
    y = r_value[mask]
    x_min,x_max = np.percentile(x, 1), np.percentile(x, 99)

    step = 0.001
    mask = ~np.isnan(Original_mean) & ~np.isnan(r_value)
    Original_mean = Original_mean[mask].flatten()
    min_val, max_val = np.min(Original_mean), np.max(Original_mean)
    bins = np.arange(min_val, max_val + step, step)
    counts, _ = np.histogram(Original_mean, bins=bins)
    centers = (bins[:-1] + bins[1:]) / 2
    counts_min,counts_max = min(counts),max(counts)
    ax.bar(centers, counts, width=step, color=color, edgecolor=color, alpha=0.8)
    ax.set_ylim(0,counts_max)
    ax.set_xlim(x_min,x_max)
    clean_axis(ax)

def draw_subplot_left(ax, Original_mean, r_value, color):
    r_value = r_value.flatten()
    Original_mean = Original_mean.flatten()
    mask = ~np.isnan(r_value) & ~np.isnan(Original_mean)
    x = Original_mean[mask]
    y = r_value[mask]

    step = 0.001
    mask = ~np.isnan(r_value) & ~np.isnan(Original_mean)
    r_value = r_value[mask].flatten()
    min_val, max_val = np.min(r_value), np.max(r_value)
    bins = np.arange(min_val, max_val + step, step)
    counts, _ = np.histogram(r_value, bins=bins)
    centers = (bins[:-1] + bins[1:]) / 2
    ax.barh(centers, counts, height=step, color=color, edgecolor=color, alpha=0.8)
    counts_min,counts_max = min(counts),max(counts)
    y_min,y_max = np.percentile(y, 5), np.percentile(y, 95)
    ax.set_xlim(0, counts_max)
    ax.set_ylim(y_min, 0.9999)
    clean_axis(ax)
metric = np.load(r"C:\Users\Administrator\Desktop\Draw\reconstruction_result.npy",allow_pickle=True).item() 
create_subplots(num_models, Original_mean,r_values, lon_range, lat_range, model_names, metric, plt_name = 'a')

### (绘图) V2.0 4*2 密度散点图 不同模型间对比 -完整绘制
SM不按均值区间分割--在v1版本上添加密度刻度

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.collections import PolyCollection
import matplotlib.colors as mcolors
from scipy.stats import gaussian_kde
from matplotlib.colors import to_rgba
from matplotlib.font_manager import FontProperties
from matplotlib.ticker import MaxNLocator
from matplotlib.colors import LinearSegmentedColormap

def clean_axis(ax):
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.tick_params(axis='x', which='both', length=0, labelbottom=False)
    ax.tick_params(axis='y', which='both', length=0, labelleft=False)
    ax.grid(False)

def create_subplots(num_models, Original_mean, r_values, model_names,  metric, plt_name=None):
    sns.set(style="whitegrid", font='Times New Roman')
    matplotlib.rcParams['font.family'] = 'Times New Roman'
    matplotlib.rcParams['axes.labelweight'] = 'bold'

    # 创建图形对象
    fig = plt.figure(figsize=(18, 24))

    ratio_h = 4 # 竖直比例
    ratio_v = 3 # 水平比例

    absolute_dis_h = 0.45/ratio_h # 中央子图绝对大小（竖直）0.15
    absolute_dis_v = 0.45/ratio_v # 中央子图绝对大小（水平）0.2

    edge_h = 0.29 # 竖直边缘间隙
    edge_v = 0.1 # 水平边缘间隙

    mid_h = (1-edge_h*2-absolute_dis_h*3)/2 # 竖直间隙,可以通过竖直边缘间隙调节
    mid_v = (1-edge_v*2-absolute_dis_v*4)/3 # 水平间隙……

    right_offset_first = 0 # 第一层向右偏移量

    up_offset = 0.1129 # 上方子图向上偏移
    right_offset = 0.1505 # 右方子图向右偏移
    up_ratio = 0.12  # 上方子图长度比例
    right_ratio = 0.25  # 上方子图宽度比例

    # 计算第一层图起点:上层子图  左起点、下起点、宽度比例、高度比例
    ax2 = fig.add_axes([edge_v + right_offset_first, 1 - edge_h - absolute_dis_h + up_offset, absolute_dis_v, absolute_dis_h * up_ratio])
    ax3 = fig.add_axes([edge_v + right_offset_first + right_offset, 1 - edge_h - absolute_dis_h, absolute_dis_v * right_ratio, absolute_dis_h])
    ax1 = fig.add_axes([edge_v + right_offset_first, 1 - edge_h - absolute_dis_h, absolute_dis_v, absolute_dis_h])

    ax5 = fig.add_axes([edge_v + right_offset_first + absolute_dis_v + mid_v, 1 - edge_h - absolute_dis_h + up_offset, absolute_dis_v, absolute_dis_h * up_ratio])
    ax6 = fig.add_axes([edge_v + right_offset_first + right_offset + absolute_dis_v + mid_v, 1 - edge_h - absolute_dis_h, absolute_dis_v * right_ratio, absolute_dis_h])
    ax4 = fig.add_axes([edge_v + right_offset_first + absolute_dis_v + mid_v, 1 - edge_h - absolute_dis_h, absolute_dis_v, absolute_dis_h])

    ax8 = fig.add_axes([edge_v + right_offset_first + 2*(absolute_dis_v + mid_v), 1 - edge_h - absolute_dis_h + up_offset, absolute_dis_v, absolute_dis_h * up_ratio])
    ax9 = fig.add_axes([edge_v + right_offset_first + right_offset + 2*(absolute_dis_v + mid_v), 1 - edge_h - absolute_dis_h, absolute_dis_v * right_ratio, absolute_dis_h])
    ax7 = fig.add_axes([edge_v + right_offset_first + 2*(absolute_dis_v + mid_v), 1 - edge_h - absolute_dis_h, absolute_dis_v, absolute_dis_h])

    ax11 = fig.add_axes([edge_v + right_offset_first + 3*(absolute_dis_v + mid_v), 1 - edge_h - absolute_dis_h + up_offset, absolute_dis_v, absolute_dis_h * up_ratio])
    ax12 = fig.add_axes([edge_v + right_offset_first + right_offset + 3*(absolute_dis_v + mid_v), 1 - edge_h - absolute_dis_h, absolute_dis_v * right_ratio, absolute_dis_h])
    ax10 = fig.add_axes([edge_v + right_offset_first + 3*(absolute_dis_v + mid_v) , 1 - edge_h - absolute_dis_h, absolute_dis_v, absolute_dis_h])

    # 中层子图
    ax14 = fig.add_axes([edge_v, 1 - edge_h - 2*absolute_dis_h - mid_h + up_offset, absolute_dis_v, absolute_dis_h * up_ratio])
    ax15 = fig.add_axes([edge_v + right_offset, 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v * right_ratio, absolute_dis_h]) 
    ax13 = fig.add_axes([edge_v , 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v, absolute_dis_h])

    ax17 = fig.add_axes([edge_v + absolute_dis_v + mid_v, 1 - edge_h - 2*absolute_dis_h - mid_h + up_offset, absolute_dis_v, absolute_dis_h * up_ratio])
    ax18 = fig.add_axes([edge_v + right_offset + absolute_dis_v + mid_v, 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v * right_ratio, absolute_dis_h])
    ax16 = fig.add_axes([edge_v + absolute_dis_v + mid_v, 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v, absolute_dis_h])

    ax20 = fig.add_axes([edge_v + 2*(absolute_dis_v + mid_v), 1 - edge_h - 2*absolute_dis_h - mid_h + up_offset, absolute_dis_v, absolute_dis_h * up_ratio])
    ax21 = fig.add_axes([edge_v + right_offset + 2*(absolute_dis_v + mid_v), 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v * right_ratio, absolute_dis_h])
    ax19 = fig.add_axes([edge_v + 2*(absolute_dis_v + mid_v), 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v, absolute_dis_h])

    ax23 = fig.add_axes([edge_v + 3*(absolute_dis_v + mid_v), 1 - edge_h - 2*absolute_dis_h - mid_h + up_offset, absolute_dis_v, absolute_dis_h * up_ratio])
    ax24 = fig.add_axes([edge_v + right_offset + 3*(absolute_dis_v + mid_v), 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v * right_ratio, absolute_dis_h])
    ax22 = fig.add_axes([edge_v + 3*(absolute_dis_v + mid_v), 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v, absolute_dis_h])

    axes = [ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8, ax9, ax10, ax11, ax12, ax13, ax14, ax15, ax16, ax17, ax18, ax19, ax20, ax21, ax22, ax23, ax24]

    # 上左色盘 
    colors = ['#36134A', '#103783', '#003C50', '#004D40', '#36134A', '#103783', '#003C50', '#004D40']

    # 中间渐变色色条
    color_scheme1 = ['#B7A0C4', '#36134A']
    color_scheme2 = ['#9bafd9', '#103783']
    color_scheme3 = ['#75CCD8', '#00505F']
    color_scheme4 = ['#80CBC4', '#004D40']
    color_schemes = [color_scheme1, color_scheme2, color_scheme3, color_scheme4]

    r_set = metric['R']
    bias_set = metric['Bias']
    rmse_set = metric['RMSE']
    ubrmse_set = metric['ubRMSE']
    
    for i in range(num_models):
        color_scheme = color_schemes[i % 4]
        draw_subplot(axes[3*i], Original_mean, r_values[:,:,i], model_names[i], color_scheme,r_set[i],bias_set[i],rmse_set[i],ubrmse_set[i])
        draw_subplot_up(axes[3*i+1], Original_mean, r_values[:,:,i], colors[i])
        draw_subplot_left(axes[3*i+2], Original_mean, r_values[:,:,i], colors[i])  
    
    fig.savefig(r"C:\Users\Administrator\Desktop\Draw\sat_original.jpeg", dpi=1000, format='jpeg', bbox_inches='tight')
    plt.show()


def draw_subplot(ax, Original_mean, r_value, model_name, color_scheme, r, bias, rmse, ubrmse):
    # 使用了selected_model_names
    cmap = mcolors.LinearSegmentedColormap.from_list('custom_cmap', color_scheme)
    font_prop = FontProperties(family='Times New Roman', size=14)
    r_value = r_value.flatten()
    Original_mean = Original_mean.flatten()
    mask = ~np.isnan(r_value) & ~np.isnan(Original_mean)
    x = Original_mean[mask]
    y = r_value[mask]

    xy = np.vstack([x, y])
    z = gaussian_kde(xy)(xy)

    scatter = ax.scatter(x, y, c=z, s=8, cmap=cmap, edgecolor='face', linewidth=0, alpha=1)
    sns.regplot(x=x, y=y, ax=ax, order=3, scatter=False, color='#522226', line_kws={'label':f'Quad Fit: {model_name}'}, ci=99)
    for artist in ax.findobj(PolyCollection):
        artist.set_alpha(0.4)

    x_min,x_max = np.percentile(x, 1), np.percentile(x, 99)
    y_min,y_max = np.percentile(y, 1), np.percentile(y, 99)

    padding_x = (x_max - x_min) * 0.01
    padding_y = (0.9999 - y_min) * 0.01

    # 密度散点图bar
    norm = matplotlib.colors.Normalize(vmin=np.min(z), vmax=np.max(z))
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])

    cax = ax.inset_axes([0.05, 0.95, 0.6, 0.04], transform=ax.transAxes)
    cbar = plt.colorbar(sm, cax=cax, orientation='horizontal', format='%d', extend='both', alpha=1)

    cbar.outline.set_visible(False)
    cbar.ax.tick_params(colors='black', length=2, width=1.5, labelsize=12, direction='out', pad=2, bottom=True, top=False, labelbottom=True, labeltop=False)

    cbar.ax.xaxis.set_major_locator(MaxNLocator(nbins=4))
    cbar.solids.set_edgecolor("face")


    # 指标文本
    x_range = np.arange(0,4)*6
    text_position_x = x_max - padding_x * 0.5
    text_position_y_r = y_min + padding_y * x_range[3] + padding_y * 0.5
    text_position_y_bias = y_min + padding_y * x_range[2] + padding_y * 0.5
    text_position_y_rmse = y_min + padding_y * x_range[1] + padding_y * 0.5
    text_position_y_ubrmse = y_min + padding_y * x_range[0] + padding_y * 0.5
    # 使用FontProperties实例绘制文本，去除$符号避免斜体
    ax.text(text_position_x, text_position_y_r, f'R={r:.3f}', fontproperties=font_prop, horizontalalignment='right', verticalalignment='bottom')
    ax.text(text_position_x, text_position_y_rmse, f'RMSE={rmse:.3f}', fontproperties=font_prop, horizontalalignment='right', verticalalignment='bottom')
    ax.text(text_position_x, text_position_y_bias, f'bias={bias:.3f}', fontproperties=font_prop, horizontalalignment='right', verticalalignment='bottom')
    ax.text(text_position_x, text_position_y_ubrmse, f'ubRMSE={ubrmse:.3f}', fontproperties=font_prop, horizontalalignment='right', verticalalignment='bottom')
    prefix = "(" + chr(ord('a') + selected_model_names.index(model_name)) + ")"
    if len(model_name)>11:
        ax.text(0.01, 0.21, f'{prefix}', transform=ax.transAxes, fontsize=18, fontname='Times New Roman', verticalalignment='top')
        ax.text(0.01, 0.11, f'{model_name}', transform=ax.transAxes, fontsize=17, fontname='Times New Roman', verticalalignment='top')
    else:
        ax.text(0.01, 0.11, f'{prefix} {model_name}', transform=ax.transAxes, fontsize=17, fontname='Times New Roman', verticalalignment='top')
    # 设置刻度
    ax.set_xlim(x_min,x_max)
    ax.set_ylim(y_min,0.9999)
    ax.xaxis.set_major_locator(MaxNLocator(nbins=4))
    ax.yaxis.set_major_locator(MaxNLocator(nbins=4))
    ax.tick_params(axis='x', which='both', colors='black', length=3, width=1.5, labelsize=14, direction='out', pad=2, bottom=True, top=False, labelbottom=True,labeltop=False)
    ax.tick_params(axis='y', which='both', colors='black', length=3, width=1.5, labelsize=14, direction='out', pad=6, left=True, right=False, labelleft=True,labelright=False)
    # 设置轴标签
    ax.set_xlabel('Original mean SM (m³/m³)', size=16, fontname='Times New Roman')
    ax.set_ylabel('Pearson R',size=16)
    plt.setp(ax.get_xticklabels(), rotation=0,rotation_mode="anchor")
    plt.setp(ax.get_yticklabels(), rotation=90,ha="center", rotation_mode="anchor")
    # 设置轴参数
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_color('black')
    ax.spines['bottom'].set_linewidth(1.5)
    ax.spines['left'].set_color('black')
    ax.spines['left'].set_linewidth(1.5)
    ax.grid(True, color='black', linestyle='--',alpha=0.5,zorder=0.1)

def draw_subplot_up(ax, Original_mean, r_value, color):
    r_value = r_value.flatten()
    Original_mean = Original_mean.flatten()
    mask = ~np.isnan(r_value) & ~np.isnan(Original_mean)
    x = Original_mean[mask]
    y = r_value[mask]
    x_min,x_max = np.percentile(x, 1), np.percentile(x, 99)

    step = 0.001
    mask = ~np.isnan(Original_mean) & ~np.isnan(r_value)
    Original_mean = Original_mean[mask].flatten()
    min_val, max_val = np.min(Original_mean), np.max(Original_mean)
    bins = np.arange(min_val, max_val + step, step)
    counts, _ = np.histogram(Original_mean, bins=bins)
    centers = (bins[:-1] + bins[1:]) / 2
    counts_min,counts_max = min(counts),max(counts)
    ax.bar(centers, counts, width=step, color=color, edgecolor=color, alpha=0.8)
    ax.set_ylim(0,counts_max)
    ax.set_xlim(x_min,x_max)
    clean_axis(ax)

def draw_subplot_left(ax, Original_mean, r_value, color):
    r_value = r_value.flatten()
    Original_mean = Original_mean.flatten()
    mask = ~np.isnan(r_value) & ~np.isnan(Original_mean)
    x = Original_mean[mask]
    y = r_value[mask]

    step = 0.001
    mask = ~np.isnan(r_value) & ~np.isnan(Original_mean)
    r_value = r_value[mask].flatten()
    min_val, max_val = np.min(r_value), np.max(r_value)
    bins = np.arange(min_val, max_val + step, step)
    counts, _ = np.histogram(r_value, bins=bins)
    centers = (bins[:-1] + bins[1:]) / 2
    ax.barh(centers, counts, height=step, color=color, edgecolor=color, alpha=0.8)
    counts_min,counts_max = min(counts),max(counts)
    y_min,y_max = np.percentile(y, 1), np.percentile(y, 99)
    ax.set_xlim(0, counts_max)
    ax.set_ylim(y_min, 0.9999)

    # 轴绘制
    clean_axis(ax)
    ax.spines['bottom'].set_visible(True)
    ax.spines['bottom'].set_color('black')
    ax.spines['bottom'].set_linewidth(1.5)
    ax.tick_params(axis='x', which='both', colors='black', length=4, width=1.5, labelsize=14, direction='inout', pad=2, bottom=True, top=False, labelbottom=True, labeltop=False)
    ax.set_xticks([counts_max])
    ax.set_xticklabels([str(counts_max)])

metric = np.load(r"C:\Users\Administrator\Desktop\Draw\reconstruction_result.npy",allow_pickle=True).item() 
create_subplots(num_models, Original_mean,r_values, selected_model_names, metric, plt_name = 'a')

### (绘图) V3.0 5*2 密度散点图 不同模型间对比 -完整绘制
SM不按均值区间分割--在v1版本上添加密度刻度

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.collections import PolyCollection
import matplotlib.colors as mcolors
from scipy.stats import gaussian_kde
from matplotlib.colors import to_rgba
from matplotlib.font_manager import FontProperties
from matplotlib.ticker import MaxNLocator
from matplotlib.colors import LinearSegmentedColormap

def clean_axis(ax):
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.tick_params(axis='x', which='both', length=0, labelbottom=False)
    ax.tick_params(axis='y', which='both', length=0, labelleft=False)
    ax.grid(False)

def create_subplots(num_models, Original_mean, r_values, model_names,  metric, plt_name=None):
    sns.set(style="whitegrid", font='Times New Roman')
    matplotlib.rcParams['font.family'] = 'Times New Roman'
    matplotlib.rcParams['axes.labelweight'] = 'bold'

    # 创建图形对象
    fig = plt.figure(figsize=(18, 24))

    ratio_h = 4 # 竖直比例
    ratio_v = 3 # 水平比例

    absolute_dis_h = 0.36/ratio_h # 中央子图绝对大小（竖直）0.15
    absolute_dis_v = 0.36/ratio_v # 中央子图绝对大小（水平）0.2

    edge_h = 0.33 # 竖直边缘间隙
    edge_v = 0.1 # 水平边缘间隙

    mid_h = (1-edge_h*2-absolute_dis_h*3)/2 # 竖直间隙,可以通过竖直边缘间隙调节
    mid_v = (1-edge_v*2-absolute_dis_v*5)/4 # 水平间隙……

    right_offset_first = 0 # 第一层向右偏移量

    up_offset = 0.0900 # 上方子图向上偏移
    right_offset = 0.1200 # 右方子图向右偏移
    up_ratio = 0.12  # 上方子图长度比例
    right_ratio = 0.25  # 上方子图宽度比例

    # 计算第一层图起点:上层子图  左起点、下起点、宽度比例、高度比例
    # 上左一
    ax2 = fig.add_axes([edge_v + right_offset_first, 1 - edge_h - absolute_dis_h + up_offset, absolute_dis_v, absolute_dis_h * up_ratio])
    ax3 = fig.add_axes([edge_v + right_offset_first + right_offset, 1 - edge_h - absolute_dis_h, absolute_dis_v * right_ratio, absolute_dis_h])
    ax1 = fig.add_axes([edge_v + right_offset_first, 1 - edge_h - absolute_dis_h, absolute_dis_v, absolute_dis_h])

    ax5 = fig.add_axes([edge_v + right_offset_first + absolute_dis_v + mid_v, 1 - edge_h - absolute_dis_h + up_offset, absolute_dis_v, absolute_dis_h * up_ratio])
    ax6 = fig.add_axes([edge_v + right_offset_first + right_offset + absolute_dis_v + mid_v, 1 - edge_h - absolute_dis_h, absolute_dis_v * right_ratio, absolute_dis_h])
    ax4 = fig.add_axes([edge_v + right_offset_first + absolute_dis_v + mid_v, 1 - edge_h - absolute_dis_h, absolute_dis_v, absolute_dis_h])

    ax8 = fig.add_axes([edge_v + right_offset_first + 2*(absolute_dis_v + mid_v), 1 - edge_h - absolute_dis_h + up_offset, absolute_dis_v, absolute_dis_h * up_ratio])
    ax9 = fig.add_axes([edge_v + right_offset_first + right_offset + 2*(absolute_dis_v + mid_v), 1 - edge_h - absolute_dis_h, absolute_dis_v * right_ratio, absolute_dis_h])
    ax7 = fig.add_axes([edge_v + right_offset_first + 2*(absolute_dis_v + mid_v), 1 - edge_h - absolute_dis_h, absolute_dis_v, absolute_dis_h])

    ax11 = fig.add_axes([edge_v + right_offset_first + 3*(absolute_dis_v + mid_v), 1 - edge_h - absolute_dis_h + up_offset, absolute_dis_v, absolute_dis_h * up_ratio])
    ax12 = fig.add_axes([edge_v + right_offset_first + right_offset + 3*(absolute_dis_v + mid_v), 1 - edge_h - absolute_dis_h, absolute_dis_v * right_ratio, absolute_dis_h])
    ax10 = fig.add_axes([edge_v + right_offset_first + 3*(absolute_dis_v + mid_v) , 1 - edge_h - absolute_dis_h, absolute_dis_v, absolute_dis_h])

    ax14 = fig.add_axes([edge_v + right_offset_first + 4*(absolute_dis_v + mid_v), 1 - edge_h - absolute_dis_h + up_offset, absolute_dis_v, absolute_dis_h * up_ratio])
    ax15 = fig.add_axes([edge_v + right_offset_first + right_offset + 4*(absolute_dis_v + mid_v), 1 - edge_h - absolute_dis_h, absolute_dis_v * right_ratio, absolute_dis_h])
    ax13 = fig.add_axes([edge_v + right_offset_first + 4*(absolute_dis_v + mid_v) , 1 - edge_h - absolute_dis_h, absolute_dis_v, absolute_dis_h])

    # 下左一
    ax17= fig.add_axes([edge_v, 1 - edge_h - 2*absolute_dis_h - mid_h + up_offset, absolute_dis_v, absolute_dis_h * up_ratio])
    ax18 = fig.add_axes([edge_v + right_offset, 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v * right_ratio, absolute_dis_h]) 
    ax16 = fig.add_axes([edge_v , 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v, absolute_dis_h])

    ax20 = fig.add_axes([edge_v + absolute_dis_v + mid_v, 1 - edge_h - 2*absolute_dis_h - mid_h + up_offset, absolute_dis_v, absolute_dis_h * up_ratio])
    ax21 = fig.add_axes([edge_v + right_offset + absolute_dis_v + mid_v, 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v * right_ratio, absolute_dis_h])
    ax19 = fig.add_axes([edge_v + absolute_dis_v + mid_v, 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v, absolute_dis_h])

    ax23 = fig.add_axes([edge_v + 2*(absolute_dis_v + mid_v), 1 - edge_h - 2*absolute_dis_h - mid_h + up_offset, absolute_dis_v, absolute_dis_h * up_ratio])
    ax24 = fig.add_axes([edge_v + right_offset + 2*(absolute_dis_v + mid_v), 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v * right_ratio, absolute_dis_h])
    ax22 = fig.add_axes([edge_v + 2*(absolute_dis_v + mid_v), 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v, absolute_dis_h])

    ax26 = fig.add_axes([edge_v + 3*(absolute_dis_v + mid_v), 1 - edge_h - 2*absolute_dis_h - mid_h + up_offset, absolute_dis_v, absolute_dis_h * up_ratio])
    ax27 = fig.add_axes([edge_v + right_offset + 3*(absolute_dis_v + mid_v), 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v * right_ratio, absolute_dis_h])
    ax25 = fig.add_axes([edge_v + 3*(absolute_dis_v + mid_v), 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v, absolute_dis_h])

    ax29 = fig.add_axes([edge_v + 4*(absolute_dis_v + mid_v), 1 - edge_h - 2*absolute_dis_h - mid_h + up_offset, absolute_dis_v, absolute_dis_h * up_ratio])
    ax30 = fig.add_axes([edge_v + right_offset + 4*(absolute_dis_v + mid_v), 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v * right_ratio, absolute_dis_h])
    ax28 = fig.add_axes([edge_v + 4*(absolute_dis_v + mid_v), 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v, absolute_dis_h])

    axes = [ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8, ax9, ax10, ax11, ax12, ax13, ax14, ax15, ax16, ax17, ax18, ax19, ax20, ax21, ax22, ax23, ax24, ax25, ax26, ax27, ax28, ax29, ax30]

    # 上左色盘 
    colors = ['#36134A', '#103783', '#003C50', '#004D40', '#375700', '#36134A', '#103783', '#003C50', '#004D40', '#375700']

    # 中间渐变色色条
    color_scheme1 = ['#B7A0C4', '#36134A']
    color_scheme2 = ['#9bafd9', '#103783']
    color_scheme3 = ['#75CCD8', '#00505F']
    color_scheme4 = ['#80CBC4', '#004D40']
    color_scheme5 = ['#7AD732', '#375700']
    color_schemes = [color_scheme1, color_scheme2, color_scheme3, color_scheme4, color_scheme5]

    r_set = metric['R']
    bias_set = metric['Bias']
    rmse_set = metric['RMSE']
    ubrmse_set = metric['ubRMSE']

    for i in range(num_models):
        color_scheme = color_schemes[i % 5]
        draw_subplot(axes[3*i], Original_mean, r_values[:,:,i], model_names[i], color_scheme,r_set[i],bias_set[i],rmse_set[i],ubrmse_set[i])
        draw_subplot_up(axes[3*i+1], Original_mean, r_values[:,:,i], colors[i])
        draw_subplot_left(axes[3*i+2], Original_mean, r_values[:,:,i], colors[i])  
    '''
    color_scheme = color_schemes[1 % 5]
    draw_subplot(axes[3*1], Original_mean, r_values[:,:,1], model_names[1], color_scheme,r_set[1],bias_set[1],rmse_set[1],ubrmse_set[1])
    draw_subplot_up(axes[3*1+1], Original_mean, r_values[:,:,1], colors[1])
    draw_subplot_left(axes[3*1+2], Original_mean, r_values[:,:,1], colors[1])  
    '''
    fig.savefig(r"C:\Users\Administrator\Desktop\Draw\sat_original.jpeg", dpi=1000, format='jpeg', bbox_inches='tight')
    plt.show()


def draw_subplot(ax, Original_mean, r_value, model_name, color_scheme, r, bias, rmse, ubrmse):
    # 使用了selected_model_names
    cmap = mcolors.LinearSegmentedColormap.from_list('custom_cmap', color_scheme)
    font_prop = FontProperties(family='Times New Roman', size=11)
    r_value = r_value.flatten()
    Original_mean = Original_mean.flatten()
    mask = ~np.isnan(r_value) & ~np.isnan(Original_mean)
    x = Original_mean[mask]
    y = r_value[mask]

    xy = np.vstack([x, y])
    z = gaussian_kde(xy)(xy)

    scatter = ax.scatter(x, y, c=z, s=6, cmap=cmap, edgecolor='face', linewidth=0, alpha=1)
    sns.regplot(x=x, y=y, ax=ax, order=3, scatter=False, color='#522226', line_kws={'label':f'Quad Fit: {model_name}'}, ci=99)
    for artist in ax.findobj(PolyCollection):
        artist.set_alpha(0.4)

    x_min,x_max = np.percentile(x, 1), np.percentile(x, 99)
    y_min,y_max = np.percentile(y, 1), np.percentile(y, 99)

    padding_x = (x_max - x_min) * 0.01
    padding_y = (0.9999 - y_min) * 0.01

    # 密度散点图bar
    norm = matplotlib.colors.Normalize(vmin=np.min(z), vmax=np.max(z))
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])

    cax = ax.inset_axes([0.05, 0.95, 0.6, 0.04], transform=ax.transAxes)
    cbar = plt.colorbar(sm, cax=cax, orientation='horizontal', format='%d', extend='both', alpha=1)

    cbar.outline.set_visible(False)
    cbar.ax.tick_params(colors='black', length=2, width=1.5, labelsize=12, direction='out', pad=2, bottom=True, top=False, labelbottom=True, labeltop=False)

    cbar.ax.xaxis.set_major_locator(MaxNLocator(nbins=4))
    cbar.solids.set_edgecolor("face")


    # 指标文本
    x_range = np.arange(0,4)*6
    text_position_x = x_max - padding_x * 0.5
    text_position_y_r = y_min + padding_y * x_range[3] + padding_y * 0.5
    text_position_y_bias = y_min + padding_y * x_range[2] + padding_y * 0.5
    text_position_y_rmse = y_min + padding_y * x_range[1] + padding_y * 0.5
    text_position_y_ubrmse = y_min + padding_y * x_range[0] + padding_y * 0.5
    # 使用FontProperties实例绘制文本，去除$符号避免斜体
    ax.text(text_position_x, text_position_y_r, f'R={r:.3f}', fontproperties=font_prop, horizontalalignment='right', verticalalignment='bottom')
    ax.text(text_position_x, text_position_y_rmse, f'RMSE={rmse:.3f}', fontproperties=font_prop, horizontalalignment='right', verticalalignment='bottom')
    ax.text(text_position_x, text_position_y_bias, f'bias={bias:.3f}', fontproperties=font_prop, horizontalalignment='right', verticalalignment='bottom')
    ax.text(text_position_x, text_position_y_ubrmse, f'ubRMSE={ubrmse:.3f}', fontproperties=font_prop, horizontalalignment='right', verticalalignment='bottom')
    prefix = "(" + chr(ord('a') + selected_model_names.index(model_name)) + ")"
    if len(model_name)>11:
        ax.text(0.01, 0.11, f'{prefix}{model_name}', transform=ax.transAxes, fontsize=11, fontname='Times New Roman', verticalalignment='top')
    else:
        ax.text(0.01, 0.11, f'{prefix} {model_name}', transform=ax.transAxes, fontsize=14, fontname='Times New Roman', verticalalignment='top')
    # 设置刻度
    ax.set_xlim(x_min,x_max)
    ax.set_ylim(y_min,0.9999)
    ax.xaxis.set_major_locator(MaxNLocator(nbins=4))
    ax.yaxis.set_major_locator(MaxNLocator(nbins=4))
    ax.tick_params(axis='x', which='both', colors='black', length=3, width=1.5, labelsize=12, direction='out', pad=2, bottom=True, top=False, labelbottom=True,labeltop=False)
    ax.tick_params(axis='y', which='both', colors='black', length=3, width=1.5, labelsize=12, direction='out', pad=4, left=True, right=False, labelleft=True,labelright=False)
    # 设置轴标签
    ax.set_xlabel('Original mean SM (m³/m³)', size=14, fontname='Times New Roman')
    ax.set_ylabel('Pearson R',size=14, labelpad=1)
    plt.setp(ax.get_xticklabels(), rotation=0,rotation_mode="anchor")
    plt.setp(ax.get_yticklabels(), rotation=90,ha="center", rotation_mode="anchor")
    # 设置轴参数
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_color('black')
    ax.spines['bottom'].set_linewidth(1.5)
    ax.spines['left'].set_color('black')
    ax.spines['left'].set_linewidth(1.5)
    ax.grid(True, color='black', linestyle='--',alpha=0.5,zorder=0.1)

def draw_subplot_up(ax, Original_mean, r_value, color):
    r_value = r_value.flatten()
    Original_mean = Original_mean.flatten()
    mask = ~np.isnan(r_value) & ~np.isnan(Original_mean)
    x = Original_mean[mask]
    y = r_value[mask]
    x_min,x_max = np.percentile(x, 1), np.percentile(x, 99)

    step = 0.001
    mask = ~np.isnan(Original_mean) & ~np.isnan(r_value)
    Original_mean = Original_mean[mask].flatten()
    min_val, max_val = np.min(Original_mean), np.max(Original_mean)
    bins = np.arange(min_val, max_val + step, step)
    counts, _ = np.histogram(Original_mean, bins=bins)
    centers = (bins[:-1] + bins[1:]) / 2
    counts_min,counts_max = min(counts),max(counts)
    ax.bar(centers, counts, width=step, color=color, edgecolor=color, alpha=0.8)
    ax.set_ylim(0,counts_max)
    ax.set_xlim(x_min,x_max)
    clean_axis(ax)

def draw_subplot_left(ax, Original_mean, r_value, color):
    r_value = r_value.flatten()
    Original_mean = Original_mean.flatten()
    mask = ~np.isnan(r_value) & ~np.isnan(Original_mean)
    x = Original_mean[mask]
    y = r_value[mask]

    step = 0.001
    mask = ~np.isnan(r_value) & ~np.isnan(Original_mean)
    r_value = r_value[mask].flatten()
    min_val, max_val = np.min(r_value), np.max(r_value)
    bins = np.arange(min_val, max_val + step, step)
    counts, _ = np.histogram(r_value, bins=bins)
    centers = (bins[:-1] + bins[1:]) / 2
    ax.barh(centers, counts, height=step, color=color, edgecolor=color, alpha=0.8)
    counts_min,counts_max = min(counts),max(counts)
    y_min,y_max = np.percentile(y, 1), np.percentile(y, 99)
    ax.set_xlim(0, counts_max)
    ax.set_ylim(y_min, 0.9999)

    # 轴绘制
    clean_axis(ax)
    ax.spines['bottom'].set_visible(True)
    ax.spines['bottom'].set_color('black')
    ax.spines['bottom'].set_linewidth(1.5)
    ax.tick_params(axis='x', which='both', colors='black', length=4, width=1.5, labelsize=14, direction='inout', pad=2, bottom=True, top=False, labelbottom=True, labeltop=False)
    ax.set_xticks([counts_max])
    ax.set_xticklabels([str(counts_max)])

metric = np.load(r"C:\Users\Administrator\Desktop\Draw\reconstruction_result.npy",allow_pickle=True).item() 
create_subplots(num_models, Original_mean,r_values, selected_model_names, metric, plt_name = 'a')

### (绘图) V2.0 4*1 密度散点图 不同模型间对比 -完整绘制 
SM不按均值区间分割--在v1版本上添加密度刻度
使用前注意数据输入

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.collections import PolyCollection
import matplotlib.colors as mcolors
from scipy.stats import gaussian_kde
from matplotlib.colors import to_rgba
from matplotlib.font_manager import FontProperties
from matplotlib.ticker import MaxNLocator
from matplotlib.colors import LinearSegmentedColormap

def clean_axis(ax):
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.tick_params(axis='x', which='both', length=0, labelbottom=False)
    ax.tick_params(axis='y', which='both', length=0, labelleft=False)
    ax.grid(False)

def create_subplots(num_models, Original_mean, r_values, model_names,  metric, plt_name=None):
    sns.set(style="whitegrid", font='Times New Roman')
    matplotlib.rcParams['font.family'] = 'Times New Roman'
    matplotlib.rcParams['axes.labelweight'] = 'bold'

    # 创建图形对象
    fig = plt.figure(figsize=(18, 24))

    ratio_h = 4 # 竖直比例
    ratio_v = 3 # 水平比例

    absolute_dis_h = 0.45/ratio_h # 中央子图绝对大小（竖直）0.15
    absolute_dis_v = 0.45/ratio_v # 中央子图绝对大小（水平）0.2

    edge_h = 0.29 # 竖直边缘间隙
    edge_v = 0.1 # 水平边缘间隙

    mid_h = 0   # 竖直间隙
    mid_v = (1-edge_v*2-absolute_dis_v*4)/3 # 水平间隙……

    up_offset = 0.1129 # 上方子图向上偏移
    right_offset = 0.1505 # 右方子图向右偏移
    up_ratio = 0.12  # 上方子图长度比例
    right_ratio = 0.25  # 上方子图宽度比例

    # 计算第一层图起点:上层子图  左起点、下起点、宽度比例、高度比例
    ax2 = fig.add_axes([edge_v, 1 - edge_h - absolute_dis_h + up_offset, absolute_dis_v, absolute_dis_h * up_ratio])
    ax3 = fig.add_axes([edge_v + right_offset, 1 - edge_h - absolute_dis_h, absolute_dis_v * right_ratio, absolute_dis_h])
    ax1 = fig.add_axes([edge_v, 1 - edge_h - absolute_dis_h, absolute_dis_v, absolute_dis_h])

    ax5 = fig.add_axes([edge_v + absolute_dis_v + mid_v, 1 - edge_h - absolute_dis_h + up_offset, absolute_dis_v, absolute_dis_h * up_ratio])
    ax6 = fig.add_axes([edge_v + right_offset + absolute_dis_v + mid_v, 1 - edge_h - absolute_dis_h, absolute_dis_v * right_ratio, absolute_dis_h])
    ax4 = fig.add_axes([edge_v + absolute_dis_v + mid_v, 1 - edge_h - absolute_dis_h, absolute_dis_v, absolute_dis_h])

    ax8 = fig.add_axes([edge_v + 2*(absolute_dis_v + mid_v), 1 - edge_h - absolute_dis_h + up_offset, absolute_dis_v, absolute_dis_h * up_ratio])
    ax9 = fig.add_axes([edge_v + right_offset + 2*(absolute_dis_v + mid_v), 1 - edge_h - absolute_dis_h, absolute_dis_v * right_ratio, absolute_dis_h])
    ax7 = fig.add_axes([edge_v + 2*(absolute_dis_v + mid_v), 1 - edge_h - absolute_dis_h, absolute_dis_v, absolute_dis_h])

    ax11 = fig.add_axes([edge_v + 3*(absolute_dis_v + mid_v), 1 - edge_h - absolute_dis_h + up_offset, absolute_dis_v, absolute_dis_h * up_ratio])
    ax12 = fig.add_axes([edge_v + right_offset + 3*(absolute_dis_v + mid_v), 1 - edge_h - absolute_dis_h, absolute_dis_v * right_ratio, absolute_dis_h])
    ax10 = fig.add_axes([edge_v + 3*(absolute_dis_v + mid_v) , 1 - edge_h - absolute_dis_h, absolute_dis_v, absolute_dis_h])


    axes = [ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8, ax9, ax10, ax11, ax12]

    # 上左色盘 
    colors = ['#36134A', '#103783', '#003C50', '#004D40', '#36134A', '#103783', '#003C50', '#004D40']

    # 中间渐变色色条
    color_scheme1 = ['#B7A0C4', '#36134A']
    color_scheme2 = ['#9bafd9', '#103783']
    color_scheme3 = ['#75CCD8', '#00505F']
    color_scheme4 = ['#80CBC4', '#004D40']
    color_schemes = [color_scheme1, color_scheme2, color_scheme3, color_scheme4]

    r_set = metric['R']
    bias_set = metric['Bias']
    rmse_set = metric['RMSE']
    ubrmse_set = metric['ubRMSE']

    for i in range(num_models):
        color_scheme = color_schemes[i % 4]
        draw_subplot(axes[3*i], Original_mean, r_values[:,:,i], model_names[i], color_scheme,r_set[i],bias_set[i],rmse_set[i],ubrmse_set[i])
        draw_subplot_up(axes[3*i+1], Original_mean, r_values[:,:,i], colors[i])
        draw_subplot_left(axes[3*i+2], Original_mean, r_values[:,:,i], colors[i])  

    fig.savefig(r"C:\Users\Administrator\Desktop\Draw\sat_original_b.jpeg", dpi=1000, format='jpeg', bbox_inches='tight')
    plt.show()


def draw_subplot(ax, Original_mean, r_value, model_name, color_scheme, r, bias, rmse, ubrmse):
    # 使用了selected_model_names
    cmap = mcolors.LinearSegmentedColormap.from_list('custom_cmap', color_scheme)
    font_prop = FontProperties(family='Times New Roman', size=14)
    r_value = r_value.flatten()
    Original_mean = Original_mean.flatten()
    mask = ~np.isnan(r_value) & ~np.isnan(Original_mean)
    x = Original_mean[mask]
    y = r_value[mask]

    xy = np.vstack([x, y])
    z = gaussian_kde(xy)(xy)

    scatter = ax.scatter(x, y, c=z, s=8, cmap=cmap, edgecolor='face', linewidth=0, alpha=1)
    sns.regplot(x=x, y=y, ax=ax, order=3, scatter=False, color='#522226', line_kws={'label':f'Quad Fit: {model_name}'}, ci=99)
    for artist in ax.findobj(PolyCollection):
        artist.set_alpha(0.4)

    x_min,x_max = np.percentile(x, 1), np.percentile(x, 99)
    y_min,y_max = np.percentile(y, 1), np.percentile(y, 99)

    padding_x = (x_max - x_min) * 0.01
    padding_y = (0.9999 - y_min) * 0.01

    # 密度散点图bar
    norm = matplotlib.colors.Normalize(vmin=np.min(z), vmax=np.max(z))
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])

    cax = ax.inset_axes([0.05, 0.95, 0.6, 0.04], transform=ax.transAxes)
    cbar = plt.colorbar(sm, cax=cax, orientation='horizontal', format='%d', extend='both', alpha=1)

    cbar.outline.set_visible(False)
    cbar.ax.tick_params(colors='black', length=2, width=1.5, labelsize=12, direction='out', pad=2, bottom=True, top=False, labelbottom=True, labeltop=False)

    cbar.ax.xaxis.set_major_locator(MaxNLocator(nbins=4))
    cbar.solids.set_edgecolor("face")


    # 指标文本
    x_range = np.arange(0,4)*6
    text_position_x = x_max - padding_x * 0.5
    text_position_y_r = y_min + padding_y * x_range[3] + padding_y * 0.5
    text_position_y_bias = y_min + padding_y * x_range[2] + padding_y * 0.5
    text_position_y_rmse = y_min + padding_y * x_range[1] + padding_y * 0.5
    text_position_y_ubrmse = y_min + padding_y * x_range[0] + padding_y * 0.5
    # 使用FontProperties实例绘制文本，去除$符号避免斜体
    ax.text(text_position_x, text_position_y_r, f'R={r:.3f}', fontproperties=font_prop, horizontalalignment='right', verticalalignment='bottom')
    ax.text(text_position_x, text_position_y_rmse, f'RMSE={rmse:.3f}', fontproperties=font_prop, horizontalalignment='right', verticalalignment='bottom')
    ax.text(text_position_x, text_position_y_bias, f'bias={bias:.3f}', fontproperties=font_prop, horizontalalignment='right', verticalalignment='bottom')
    ax.text(text_position_x, text_position_y_ubrmse, f'ubRMSE={ubrmse:.3f}', fontproperties=font_prop, horizontalalignment='right', verticalalignment='bottom')
    prefix = "(" + chr(ord('a') + selected_model_names.index(model_name)) + ")"
    if len(model_name)>11:
        ax.text(0.01, 0.21, f'{prefix}', transform=ax.transAxes, fontsize=18, fontname='Times New Roman', verticalalignment='top')
        ax.text(0.01, 0.11, f'{model_name}', transform=ax.transAxes, fontsize=17, fontname='Times New Roman', verticalalignment='top')
    else:
        ax.text(0.01, 0.11, f'{prefix} {model_name}', transform=ax.transAxes, fontsize=17, fontname='Times New Roman', verticalalignment='top')
    # 设置刻度
    ax.set_xlim(x_min,x_max)
    ax.set_ylim(y_min,0.9999)
    ax.xaxis.set_major_locator(MaxNLocator(nbins=4))
    ax.yaxis.set_major_locator(MaxNLocator(nbins=4))
    ax.tick_params(axis='x', which='both', colors='black', length=3, width=1.5, labelsize=14, direction='out', pad=2, bottom=True, top=False, labelbottom=True,labeltop=False)
    ax.tick_params(axis='y', which='both', colors='black', length=3, width=1.5, labelsize=14, direction='out', pad=6, left=True, right=False, labelleft=True,labelright=False)
    # 设置轴标签
    ax.set_xlabel('Original mean SM (m³/m³)', size=16, fontname='Times New Roman')
    ax.set_ylabel('Pearson R',size=16)
    plt.setp(ax.get_xticklabels(), rotation=0,rotation_mode="anchor")
    plt.setp(ax.get_yticklabels(), rotation=90,ha="center", rotation_mode="anchor")
    # 设置轴参数
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_color('black')
    ax.spines['bottom'].set_linewidth(1.5)
    ax.spines['left'].set_color('black')
    ax.spines['left'].set_linewidth(1.5)
    ax.grid(True, color='black', linestyle='--',alpha=0.5,zorder=0.1)

def draw_subplot_up(ax, Original_mean, r_value, color):
    r_value = r_value.flatten()
    Original_mean = Original_mean.flatten()
    mask = ~np.isnan(r_value) & ~np.isnan(Original_mean)
    x = Original_mean[mask]
    y = r_value[mask]
    x_min,x_max = np.percentile(x, 1), np.percentile(x, 99)

    step = 0.001
    mask = ~np.isnan(Original_mean) & ~np.isnan(r_value)
    Original_mean = Original_mean[mask].flatten()
    min_val, max_val = np.min(Original_mean), np.max(Original_mean)
    bins = np.arange(min_val, max_val + step, step)
    counts, _ = np.histogram(Original_mean, bins=bins)
    centers = (bins[:-1] + bins[1:]) / 2
    counts_min,counts_max = min(counts),max(counts)
    ax.bar(centers, counts, width=step, color=color, edgecolor=color, alpha=0.8)
    ax.set_ylim(0,counts_max)
    ax.set_xlim(x_min,x_max)
    clean_axis(ax)

def draw_subplot_left(ax, Original_mean, r_value, color):
    r_value = r_value.flatten()
    Original_mean = Original_mean.flatten()
    mask = ~np.isnan(r_value) & ~np.isnan(Original_mean)
    x = Original_mean[mask]
    y = r_value[mask]

    step = 0.001
    mask = ~np.isnan(r_value) & ~np.isnan(Original_mean)
    r_value = r_value[mask].flatten()
    min_val, max_val = np.min(r_value), np.max(r_value)
    bins = np.arange(min_val, max_val + step, step)
    counts, _ = np.histogram(r_value, bins=bins)
    centers = (bins[:-1] + bins[1:]) / 2
    ax.barh(centers, counts, height=step, color=color, edgecolor=color, alpha=0.8)
    counts_min,counts_max = min(counts),max(counts)
    y_min,y_max = np.percentile(y, 1), np.percentile(y, 99)
    ax.set_xlim(0, counts_max)
    ax.set_ylim(y_min, 0.9999)

    # 轴绘制
    clean_axis(ax)
    ax.spines['bottom'].set_visible(True)
    ax.spines['bottom'].set_color('black')
    ax.spines['bottom'].set_linewidth(1.5)
    ax.tick_params(axis='x', which='both', colors='black', length=4, width=1.5, labelsize=14, direction='inout', pad=2, bottom=True, top=False, labelbottom=True, labeltop=False)
    ax.set_xticks([counts_max])
    ax.set_xticklabels([str(counts_max)])
    

metric = np.load(r"C:\Users\Administrator\Desktop\Draw\reconstruction_result.npy",allow_pickle=True).item() 
selected_model_names = ['PSC-FENet-ST', 'LSTM', 'RF', 'Temporal']
num_models = len(selected_model_names)
create_subplots(num_models, Original_mean,r_values, selected_model_names, metric, plt_name = 'a')


### (数据处理) 二维指标计算 
在此之前先获取0.1、0.37数据

In [ ]:
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd
from scipy.stats import pearsonr

def generate_clipping_mask(lon_grid, lat_grid, shp_path):
    """
    使用.shp文件生成数据掩码：在边界外的数据掩码为0，在边界内的数据掩码为1。
    """
    gdf = gpd.read_file(shp_path)
    poly = gdf.unary_union
    mask = np.zeros_like(lon_grid, dtype=bool)

    for i in range(lon_grid.shape[0]):
        for j in range(lon_grid.shape[1]):
            point = Point(lon_grid[i, j], lat_grid[i, j])
            if point.within(poly):
                mask[i, j] = True

    return mask

def cal_ind_data(Original_data_low_res, low_res_model_data):
    num_models = len(low_res_model_data)
    # 初始化r_values数组来存储每个模型数据和原始数据之间的相关系数
    values = {name: np.full((Original_data_low_res.shape[1], Original_data_low_res.shape[2], num_models), np.nan) 
            for name in ['r_values', 'bias_values', 'RMSE_values', 'ubRMSE_values']}
    
    for k, model_data in enumerate(low_res_model_data):
        for i in range(Original_data_low_res.shape[1]):  # 遍历纬度
            for j in range(Original_data_low_res.shape[2]):  # 遍历经度
                original_series = Original_data_low_res[:, i, j]
                model_series = model_data[:, i, j]
                mask = ~np.isnan(original_series) & ~np.isnan(model_series)
                original_series = original_series[mask]
                model_series = model_series[mask]
                if not len(original_series) or not len(model_series):
                    continue
                values['r_values'][i, j, k], _ = pearsonr(original_series, model_series)
                values['bias_values'][i, j, k] = np.mean(model_series - original_series)
                values['RMSE_values'][i, j, k] = np.sqrt(np.mean((model_series - original_series)**2))
                mean_model = np.mean(model_series)
                mean_obs = np.mean(original_series)
                values['ubRMSE_values'][i, j, k] = np.sqrt(np.mean((model_series - mean_model - (original_series - mean_obs))**2))

    return values['r_values'], values['bias_values'], values['RMSE_values'], values['ubRMSE_values']

shp_path = r"D:\\Data_Store\\TPBoundary_new(2021)\\TPBoundary_new(2021).shp"
lakes_path = r"D:\Data_Store\TPBoundary_new(2021)\Lake_TP_2010.shp"

selected_model_names = ['PSC-FENet-ST', 'LSTM', 'RF', 'Temporal']
#选择模型数据
dates = pd.date_range(start='2016-01-01 03:00', periods=8768, freq='3H')
filter_mask = ((dates.hour == 6) | (dates.hour == 18))
filtered_indices = np.where(filter_mask)[0]
Original_data_full = np.load(Original_path)
Original_data = Original_data_full[filtered_indices, :, :]

# 0.37 
#selected_paths = [reconstruction_paths[name] for name in selected_model_names if name in reconstruction_paths]
#model_data = [np.load(path)[filtered_indices, :, :] for path in selected_paths]
#r_values, bias_values, RMSE_values, ubRMSE_values= cal_ind_data(Original_data, model_data)

#0.1 需先计算low_res_model_data
r_values, bias_values, RMSE_values, ubRMSE_values = cal_ind_data(Original_data, low_res_model_data)


# 裁剪
settings = {
    0.37: (25.293533, 40.604774, 66.84705, 105.31184, 42, 104)
}

masks = {}
mode = 0.37
for mode, (lat_min, lat_max, lon_min, lon_max, lat_points, lon_points) in settings.items():
    lon_step = (lon_max - lon_min) / (lon_points - 1)
    lat_step = (lat_max - lat_min) / (lat_points - 1)
    new_lon_min = lon_min - lon_step / 2
    new_lon_max = lon_max + lon_step / 2
    new_lat_min = lat_min - lat_step / 2
    new_lat_max = lat_max + lat_step / 2
    lon_range, lat_range = np.linspace(new_lon_min, new_lon_max, lon_points), np.linspace(new_lat_min, new_lat_max, lat_points)
    lon_grid, lat_grid = np.meshgrid(lon_range, lat_range)
    masks[mode] = generate_clipping_mask(lon_grid, lat_grid, shp_path)

mask = masks[mode]
mask_expanded = mask[:, :, np.newaxis]

r_values = np.where(mask_expanded, r_values, np.nan)
bias_values = np.where(mask_expanded, bias_values, np.nan)
RMSE_values = np.where(mask_expanded, RMSE_values, np.nan)
ubRMSE_values = np.where(mask_expanded, ubRMSE_values, np.nan)

### (绘图) 二维指标图  模型(4)*指标(3) 
需要修改，不要比例尺，要添加一个比例柱状图

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.collections import PolyCollection
import matplotlib.colors as mcolors
from scipy.stats import gaussian_kde
from matplotlib.colors import to_rgba
from matplotlib.ticker import FuncFormatter
from matplotlib.font_manager import FontProperties
from matplotlib.ticker import MaxNLocator
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.cm as mcm
import matplotlib.patches as mpatches
import geopandas as gpd
import matplotlib.font_manager as font_manager


sns.set(style="whitegrid", font='Times New Roman')
matplotlib.rcParams['font.family'] = 'Times New Roman'
legend_font = FontProperties(family='Times New Roman', size=14)
font_prop = FontProperties(family='Times New Roman', size=16)
def get_cbar_colors(mappable, bins):
    mid_points = [(bins[i] + bins[i+1])/2 for i in range(len(bins)-1)]
    colors = [mappable.cmap(mappable.norm(mid_point)) for mid_point in mid_points]
    return colors

def add_north(ax, labelsize=12, loc_x=0.97, loc_y=0.96, width=0.03, height=0.15, pad=0.15):
    """
    画一个比例尺带'N'文字注释
    主要参数如下
    :param ax: 要画的坐标区域 Axes实例 plt.gca()获取即可
    :param labelsize: 显示'N'文字的大小
    :param loc_x: 以文字下部为中心的占整个ax横向比例
    :param loc_y: 以文字下部为中心的占整个ax纵向比例
    :param width: 指南针占ax比例宽度
    :param height: 指南针占ax比例高度
    :param pad: 文字符号占ax比例间隙
    :return: None
    """
    minx, maxx = ax.get_xlim()
    miny, maxy = ax.get_ylim()
    ylen = maxy - miny
    xlen = maxx - minx
    left = [minx + xlen*(loc_x - width*.5), miny + ylen*(loc_y - pad)]
    right = [minx + xlen*(loc_x + width*.5), miny + ylen*(loc_y - pad)]
    top = [minx + xlen*loc_x, miny + ylen*(loc_y - pad + height)]
    center = [minx + xlen*loc_x, left[1] + (top[1] - left[1])*.4]
    triangle = mpatches.Polygon([left, top, right, center], color='k')
    ax.text(s='N',
            x=minx + xlen*loc_x,
            y=miny + ylen*(loc_y - pad + height),
            fontsize=labelsize,
            horizontalalignment='center',
            verticalalignment='bottom')
    ax.add_patch(triangle)

def add_scalebar(ax,lon0,lat0,length,size=0.45,lw=1.5):
    '''
    ax: 坐标轴
    lon0: 经度
    lat0: 纬度
    length: 长度
    size: 控制粗细和距离的
    '''
    # style 3
    lw = lw
    ax.hlines(y=lat0,  xmin = lon0, xmax = lon0+length/111, colors="black", ls="-", lw=lw, label='%d km' % (length))
    ax.vlines(x = lon0, ymin = lat0-size, ymax = lat0+size, colors="black", ls="-", lw=lw)
    ax.vlines(x = lon0+length/2/111, ymin = lat0-size, ymax = lat0+size, colors="black", ls="-", lw=lw)
    ax.vlines(x = lon0+length/111, ymin = lat0-size, ymax = lat0+size, colors="black", ls="-", lw=lw)
    ax.text(lon0+length/111,lat0+size+0.12,'%d' % (length),horizontalalignment = 'center',size=12)
    ax.text(lon0+length/2/111,lat0+size+0.12,'%d' % (length/2),horizontalalignment = 'center',size=12)
    ax.text(lon0,lat0+size+0.12,'0',horizontalalignment = 'center',size=12)
    ax.text(lon0+length/111/2*2.5,lat0+size+0.12,'km',horizontalalignment = 'center',size=12)

def custom_formatter(x, pos):
    if x == 0:
        return "0"
    else:
        str_x = "{:.15g}".format(x)
        if 'e' in str_x:
            base, exponent = str_x.split('e')
            base = base.rstrip('0').rstrip('.')
            exponent = exponent.replace('+', '').lstrip('0')
            if exponent:
                return f"{base}e{exponent}"
            else:
                return base
        else:
            return str_x
        
def add_degree_E(x, pos):
    return f"{int(x)}°E"

def add_degree_N(y, pos):
    return f"{int(custom_formatter(y, pos))}°N"

def create_2d_subplots(shp_path, lakes_path, r_values, bias_values, RMSE_values, ubRMSE_values, selected_model_names, plt_name = None):

    # 创建图形对象
    fig = plt.figure(figsize=(18, 24))

    ratio_h = 4 # 竖直比例
    ratio_v = 3 # 水平比例

    absolute_dis_h = 0.35/ratio_h # 中央子图绝对大小（竖直）0.15
    absolute_dis_v = 0.7/ratio_v # 中央子图绝对大小（水平）0.2

    edge_h = 0.32 # 竖直边缘间隙
    edge_v = 0.13 # 水平边缘间隙

    mid_h = (1-edge_h*2-absolute_dis_h*4)/3 # 竖直间隙,可以通过竖直边缘间隙调节
    mid_v = (1-edge_v*2-absolute_dis_v*3)/2 # 水平间隙……

    # 计算第一层图起点:上层子图  左起点、下起点、宽度比例、高度比例
    ax1 = fig.add_axes([edge_v, 1 - edge_h - absolute_dis_h, absolute_dis_v, absolute_dis_h])
    ax2 = fig.add_axes([edge_v + absolute_dis_v + mid_v, 1 - edge_h - absolute_dis_h, absolute_dis_v, absolute_dis_h])
    ax3 = fig.add_axes([edge_v + 2*(absolute_dis_v + mid_v), 1 - edge_h - absolute_dis_h, absolute_dis_v, absolute_dis_h])

    ax4 = fig.add_axes([edge_v, 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v, absolute_dis_h])
    ax5 = fig.add_axes([edge_v + absolute_dis_v + mid_v, 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v, absolute_dis_h])
    ax6 = fig.add_axes([edge_v + 2*(absolute_dis_v + mid_v), 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v, absolute_dis_h])

    ax7 = fig.add_axes([edge_v, 1 - edge_h - 3*absolute_dis_h - 2*mid_h, absolute_dis_v, absolute_dis_h])
    ax8 = fig.add_axes([edge_v + absolute_dis_v + mid_v, 1 - edge_h - 3*absolute_dis_h - 2*mid_h, absolute_dis_v, absolute_dis_h])
    ax9 = fig.add_axes([edge_v + 2*(absolute_dis_v + mid_v), 1 - edge_h - 3*absolute_dis_h - 2*mid_h, absolute_dis_v, absolute_dis_h])

    ax10 = fig.add_axes([edge_v, 1 - edge_h - 4*absolute_dis_h - 3*mid_h, absolute_dis_v, absolute_dis_h])
    ax11 = fig.add_axes([edge_v + absolute_dis_v + mid_v, 1 - edge_h - 4*absolute_dis_h - 3*mid_h, absolute_dis_v, absolute_dis_h])
    ax12 = fig.add_axes([edge_v + 2*(absolute_dis_v + mid_v) , 1 - edge_h - 4*absolute_dis_h - 3*mid_h, absolute_dis_v, absolute_dis_h])

    axes = [ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8, ax9, ax10, ax11, ax12]
    '''
    # cmap
    r_colors = ['#B7A0C4', '#36134A']
    bias_colors = ['#9bafd9', '#a6d96a', '#103783']
    ubrmse_colors = ['#75CCD8', '#abdda4', '#00505F'] 
    # 创建对应的色条
    r_vmin,r_vmax = 0, 1
    bias_vmin, bias_vmax = np.nanpercentile(bias_values.flatten(), 1), np.nanpercentile(bias_values.flatten(), 99)
    ubRMSE_vmin, ubRMSE_vmax = np.nanpercentile(ubRMSE_values.flatten(), 1), np.nanpercentile(ubRMSE_values.flatten(), 99)

    r_cmap = mcolors.LinearSegmentedColormap.from_list("r_cmap", r_colors)
    bias_cmap = mcolors.LinearSegmentedColormap.from_list("bias_cmap", bias_colors)
    ubrmse_cmap = mcolors.LinearSegmentedColormap.from_list("ubrmse_cmap", ubrmse_colors)
    '''
    # 生成子子图的label
    r_bins=[0, 0.2, 0.4, 0.6, 0.8, 1]  
    bias_bins=[-0.08, -0.04, 0, 0.04, 0.08, 0.12]  
    ubRMSE_bins=[0, 0.02, 0.04, 0.06, 0.08, 0.1]
    # cmap
    r_cmap = mcm.terrain
    bias_cmap = mcm.BrBG
    ubrmse_cmap = mcm.terrain_r
    # vmin,vmax
    r_vmin, r_vmax = r_bins[0], r_bins[-1]
    bias_vmin, bias_vmax = bias_bins[0], bias_bins[-1]
    ubRMSE_vmin, ubRMSE_vmax = ubRMSE_bins[0], ubRMSE_bins[-1]
    r_norm = mcolors.Normalize(vmin=r_vmin, vmax=r_vmax)
    midpoint = 0
    bias_norm = mcolors.TwoSlopeNorm(vmin=bias_vmin, vcenter=midpoint, vmax=bias_vmax)
    ubRMSE_norm = mcolors.Normalize(vmin=ubRMSE_vmin, vmax=ubRMSE_vmax)
    r = mcm.ScalarMappable(norm=r_norm, cmap=r_cmap)
    bias = mcm.ScalarMappable(norm=bias_norm, cmap=bias_cmap)
    ubRMSE = mcm.ScalarMappable(norm=ubRMSE_norm, cmap=ubrmse_cmap)
    # 生成子子图颜色
    r_colors = get_cbar_colors(r, r_bins)
    bias_colors = get_cbar_colors(bias, bias_bins)
    ubRMSE_colors = get_cbar_colors(ubRMSE, ubRMSE_bins)
    # 刻度查看和设置
    '''
    bias_vmin, bias_vmax = np.nanmin(bias_values.flatten()), np.nanmax(bias_values.flatten()) #-0.154,0.234
    ubRMSE_vmin, ubRMSE_vmax = np.nanmin(ubRMSE_values.flatten()), np.nanmax(ubRMSE_values.flatten()) #0.0027 0.137
    '''

    for i in range(4):
        bottom_spines = True  if i == 3 else False
        draw_subplot(axes[3*i], r_values[..., i], selected_model_names[i], selected_model_names, shp_path, lakes_path, vmin=r_vmin, vmax=r_vmax, cmap=r_cmap, left_spines=(True if 3*i in [0,3,6,9] else False), bottom_spines=bottom_spines, plt_name = 'a', bins=r_bins, colors=r_colors, invert_order=True),
        draw_subplot(axes[3*i+1], bias_values[..., i], selected_model_names[i], selected_model_names, shp_path, lakes_path, vmin=bias_vmin, vmax=bias_vmax, cmap=bias_cmap, left_spines=(True if 3*i+1 in [0,3,6,9] else False), bottom_spines=bottom_spines, plt_name = 'a', bins=bias_bins, colors=bias_colors, invert_order=False)
        draw_subplot(axes[3*i+2], ubRMSE_values[..., i], selected_model_names[i], selected_model_names, shp_path, lakes_path, vmin=ubRMSE_vmin, vmax=ubRMSE_vmax, cmap=ubrmse_cmap, left_spines=(True if 3*i+2 in [0,3,6,9] else False), bottom_spines=bottom_spines, plt_name = 'a', bins=ubRMSE_bins, colors=ubRMSE_colors, invert_order=False)

    cbar_lengths = [0.21, 0.175, 0.15]
    for i, (mappable, label, bins) in enumerate(zip([r, bias, ubRMSE], ['Person R', 'bias', 'ubRMSE'], [r_bins, bias_bins, ubRMSE_bins])):
        cbar_ax = fig.add_axes([0.2535*i+0.1315, 1 - edge_h - 4*absolute_dis_h - 3*mid_h-0.023, cbar_lengths[i], 0.008])
        extend = 'neither' if i == 0 else ('both' if i == 1 else 'max')
        cbar = plt.colorbar(mappable, cax=cbar_ax, orientation='horizontal', extend=extend)
        # 设置刻度为bins数组的值
        cbar.set_ticks(bins)
        cbar.ax.xaxis.set_tick_params(labelsize=12)
        cbar.ax.xaxis.set_major_locator(MaxNLocator(nbins=len(bins)))
        cbar.ax.tick_params(axis='x', which='both', colors='black', length=2, width=1)
        cbar.ax.xaxis.set_major_formatter(FuncFormatter(custom_formatter))
        cbar.outline.set_linewidth(0.8)
        for label in cbar.ax.get_xticklabels():
            label.set_fontproperties(font_manager.FontProperties(weight='normal'))  # 设置为粗体
            label.set_fontsize(12)
    fig.text(0.35, 0.3045, 'R', fontsize=14, fontname='Times New Roman', verticalalignment='top')
    fig.text(0.565, 0.3045, 'bias (m³/m³)', fontsize=14, fontname='Times New Roman', verticalalignment='top')
    fig.text(0.795, 0.3045, 'ubRMSE (m³/m³)', fontsize=14, fontname='Times New Roman', verticalalignment='top')
    
    fig.savefig(r"C:\Users\Administrator\Desktop\Draw\sat_original_2d.jpeg", dpi=1000, format='jpeg', bbox_inches='tight')
    plt.show()

def draw_sub_subplot(ax, value, bins, colors, invert_order=True):
    flat_value = value.flatten()
    flat_value = flat_value[~np.isnan(flat_value)]
    counts, edges = np.histogram(flat_value, bins=bins)
    centers = (edges[:-1] + edges[1:]) / 2
    widths = np.diff(edges)

    for i, (count, center) in enumerate(zip(counts, centers)):
        ax.bar(center, count, width=widths[i], color=colors[i % len(colors)], edgecolor='black', align='center')
        percentage = f'{count / flat_value.size * 100:.1f}'
        ax.text(center, count, percentage, ha='center', va='bottom', fontsize=8)
   
    if invert_order:
        ax.invert_xaxis()

    # 轴线控制
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.yaxis.set_visible(False)
    ax.grid(False)

    #刻度控制
    ax.tick_params(axis='x', which='both', length=2, width=1, labelsize=9, pad=2, direction='out', bottom=True, top=False, labeltop=False, labelbottom=True)
    ax.set_xticks(edges)
    ax.set_xticklabels(["{:.2f}".format(edge) for edge in edges], fontsize=8)
    ax.xaxis.set_major_formatter(FuncFormatter(custom_formatter))
    ax.spines['bottom'].set_bounds(edges[0], edges[-1])
    ax.spines['bottom'].set_color('black')
    ax.spines['bottom'].set_linewidth(1)
    ax.axis('on')
    ax.set_facecolor('none')


def draw_subplot(ax, value, model_name, model_names, shp_path, lakes_path, vmin=None, vmax=None, cmap=None, left_spines=None, bottom_spines=None, plt_name='a', bins=[0, 0.2, 0.4, 0.6, 0.8, 1], colors=None, invert_order=True):
    ##  sub_draw
    sub_ax = ax.inset_axes([0.02, 0.1, 0.3, 0.3])
    draw_sub_subplot(sub_ax, value, bins=bins, colors=colors,invert_order=invert_order)

    ## draw
    gdf = gpd.read_file(shp_path)
    lakes = gpd.read_file(lakes_path)
    lakes = lakes.to_crs(epsg=4326)
    lakes.plot(ax=ax, edgecolor='white', facecolor='white', linewidth=0)
    gdf.plot(ax=ax, edgecolor='black', facecolor='none', linewidth=1.5)

    lat_min, lat_max, lon_min, lon_max, lat_points, lon_points= 25.293533, 40.604774, 66.84705, 105.31184, 42, 104
    lon_step = (lon_max - lon_min) / (lon_points - 1)
    lat_step = (lat_max - lat_min) / (lat_points - 1)
    new_lon_min = lon_min - lon_step / 2
    new_lon_max = lon_max + lon_step / 2
    new_lat_min = lat_min - lat_step / 2
    new_lat_max = lat_max + lat_step / 2

    image = ax.imshow(value, cmap=cmap, origin='lower', vmin=vmin, vmax=vmax, extent=[new_lon_min, new_lon_max, new_lat_min, new_lat_max])


    # 设置X轴和Y轴的标签
    ax.set_xlabel('Longitude(°)', fontsize=16, fontweight='normal', labelpad=0)
    ax.set_ylabel('Latitude(°)', fontsize=16, fontweight='normal')
    ax.tick_params(axis='x', which='both', colors='black', length=3, width=1.5, labelsize=14, direction='out', pad=2, bottom=True, top=True, labelbottom=True,labeltop=False)
    ax.tick_params(axis='y', which='both', colors='black', length=3, width=1.5, labelsize=14, direction='out', pad=6, left=True, right=True, labelleft=True,labelright=False)
    plt.setp(ax.get_xticklabels(), rotation=0,rotation_mode="anchor")
    plt.setp(ax.get_yticklabels(), rotation=90,ha="center", rotation_mode="anchor")

    # 刻度标签轴绘制
    labelsize = 12
    pad_x = 4
    pad_y = 4
    ax.yaxis.label.set_visible(left_spines)
    ax.xaxis.label.set_visible(bottom_spines)
    ax.xaxis.set_major_formatter(FuncFormatter(add_degree_E))

    # 刻度调节
    plt.setp(ax.get_yticklabels(), rotation=90,ha="center", rotation_mode="anchor")
    # 刻度值格式
    ax.yaxis.set_major_formatter(FuncFormatter(add_degree_N))
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_color('black')
    ax.spines['bottom'].set_linewidth(1.5)
    ax.spines['left'].set_color('black')
    ax.spines['left'].set_linewidth(1.5)
    ax.xaxis.set_major_locator(MaxNLocator(nbins=6))
    ax.yaxis.set_major_locator(MaxNLocator(nbins=4))
    ax.tick_params(axis='y', which='both', colors='black', length=5, width=1.5, labelsize=labelsize, direction='out', pad=pad_y, right=False, labelright=False)
    ax.tick_params(axis='x', which='both', colors='black', length=5, width=1.5, labelsize=labelsize, direction='out', pad=pad_x, top=False, labeltop=False)
    add_north(ax)
    ax.grid(False)
    # 文本
    prefix = "(" + chr(ord('a') + model_names.index(model_name)) + ")" 
    model_str = f'{prefix} {model_name}'
    if left_spines: ax.text(0.24, 0.98, model_str, transform=ax.transAxes, fontsize=14, fontname='Times New Roman', verticalalignment='top')

create_2d_subplots(shp_path, lakes_path, r_values, bias_values, RMSE_values, ubRMSE_values, selected_model_names, plt_name = 'a')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.collections import PolyCollection
import matplotlib.colors as mcolors
from scipy.stats import gaussian_kde
from matplotlib.colors import to_rgba
from matplotlib.ticker import FuncFormatter
from matplotlib.font_manager import FontProperties
from matplotlib.ticker import MaxNLocator
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.cm as mcm
import matplotlib.patches as mpatches
import geopandas as gpd
import matplotlib.font_manager as font_manager


sns.set(style="whitegrid", font='Times New Roman')
matplotlib.rcParams['font.family'] = 'Times New Roman'
legend_font = FontProperties(family='Times New Roman', size=14)
font_prop = FontProperties(family='Times New Roman', size=16)
def get_cbar_colors(mappable, bins):
    mid_points = [(bins[i] + bins[i+1])/2 for i in range(len(bins)-1)]
    colors = [mappable.cmap(mappable.norm(mid_point)) for mid_point in mid_points]
    return colors

def add_north(ax, labelsize=12, loc_x=0.97, loc_y=0.96, width=0.03, height=0.15, pad=0.15):
    """
    画一个比例尺带'N'文字注释
    主要参数如下
    :param ax: 要画的坐标区域 Axes实例 plt.gca()获取即可
    :param labelsize: 显示'N'文字的大小
    :param loc_x: 以文字下部为中心的占整个ax横向比例
    :param loc_y: 以文字下部为中心的占整个ax纵向比例
    :param width: 指南针占ax比例宽度
    :param height: 指南针占ax比例高度
    :param pad: 文字符号占ax比例间隙
    :return: None
    """
    minx, maxx = ax.get_xlim()
    miny, maxy = ax.get_ylim()
    ylen = maxy - miny
    xlen = maxx - minx
    left = [minx + xlen*(loc_x - width*.5), miny + ylen*(loc_y - pad)]
    right = [minx + xlen*(loc_x + width*.5), miny + ylen*(loc_y - pad)]
    top = [minx + xlen*loc_x, miny + ylen*(loc_y - pad + height)]
    center = [minx + xlen*loc_x, left[1] + (top[1] - left[1])*.4]
    triangle = mpatches.Polygon([left, top, right, center], color='k')
    ax.text(s='N',
            x=minx + xlen*loc_x,
            y=miny + ylen*(loc_y - pad + height),
            fontsize=labelsize,
            horizontalalignment='center',
            verticalalignment='bottom')
    ax.add_patch(triangle)

def add_scalebar(ax,lon0,lat0,length,size=0.45,lw=1.5):
    '''
    ax: 坐标轴
    lon0: 经度
    lat0: 纬度
    length: 长度
    size: 控制粗细和距离的
    '''
    # style 3
    lw = lw
    ax.hlines(y=lat0,  xmin = lon0, xmax = lon0+length/111, colors="black", ls="-", lw=lw, label='%d km' % (length))
    ax.vlines(x = lon0, ymin = lat0-size, ymax = lat0+size, colors="black", ls="-", lw=lw)
    ax.vlines(x = lon0+length/2/111, ymin = lat0-size, ymax = lat0+size, colors="black", ls="-", lw=lw)
    ax.vlines(x = lon0+length/111, ymin = lat0-size, ymax = lat0+size, colors="black", ls="-", lw=lw)
    ax.text(lon0+length/111,lat0+size+0.12,'%d' % (length),horizontalalignment = 'center',size=12)
    ax.text(lon0+length/2/111,lat0+size+0.12,'%d' % (length/2),horizontalalignment = 'center',size=12)
    ax.text(lon0,lat0+size+0.12,'0',horizontalalignment = 'center',size=12)
    ax.text(lon0+length/111/2*2.5,lat0+size+0.12,'km',horizontalalignment = 'center',size=12)

def custom_formatter(x, pos):
    if x == 0:
        return "0"
    else:
        # 保留足够的小数位数以确保精度，但不超过e记数法所需的位数
        # 使用'.15g'确保数值在转换为科学计数法时保持精度，但不会因太多不必要的零而冗长
        str_x = "{:.15g}".format(x)
        # 分割指数和底数，如果是科学计数法的形式
        if 'e' in str_x:
            base, exponent = str_x.split('e')
            # 清除底数中尾随的零和小数点
            base = base.rstrip('0').rstrip('.')
            # 格式化指数部分，去除前导+和前导0
            exponent = exponent.replace('+', '').lstrip('0')
            if exponent:  # 如果指数部分不为空
                return f"{base}e{exponent}"
            else:  # 如果指数为0
                return base
        else:
            return str_x
        
def add_degree_E(x, pos):
    return f"{int(x)}°E"

def add_degree_N(y, pos):
    return f"{int(custom_formatter(y, pos))}°N"

def create_2d_subplots(shp_path, lakes_path, r_values, bias_values, RMSE_values, ubRMSE_values, selected_model_names, plt_name = None):

    # 创建图形对象
    fig = plt.figure(figsize=(18, 24))

    ratio_h = 4 # 竖直比例
    ratio_v = 3 # 水平比例

    absolute_dis_h = 0.35/ratio_h # 中央子图绝对大小（竖直）0.15
    absolute_dis_v = 0.7/ratio_v # 中央子图绝对大小（水平）0.2

    edge_h = 0.32 # 竖直边缘间隙
    edge_v = 0.13 # 水平边缘间隙

    mid_h = (1-edge_h*2-absolute_dis_h*4)/3 # 竖直间隙,可以通过竖直边缘间隙调节
    mid_v = (1-edge_v*2-absolute_dis_v*3)/2 # 水平间隙……

    # 计算第一层图起点:上层子图  左起点、下起点、宽度比例、高度比例
    ax1 = fig.add_axes([edge_v, 1 - edge_h - absolute_dis_h, absolute_dis_v, absolute_dis_h])
    ax2 = fig.add_axes([edge_v + absolute_dis_v + mid_v, 1 - edge_h - absolute_dis_h, absolute_dis_v, absolute_dis_h])
    ax3 = fig.add_axes([edge_v + 2*(absolute_dis_v + mid_v), 1 - edge_h - absolute_dis_h, absolute_dis_v, absolute_dis_h])

    ax4 = fig.add_axes([edge_v, 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v, absolute_dis_h])
    ax5 = fig.add_axes([edge_v + absolute_dis_v + mid_v, 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v, absolute_dis_h])
    ax6 = fig.add_axes([edge_v + 2*(absolute_dis_v + mid_v), 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v, absolute_dis_h])

    ax7 = fig.add_axes([edge_v, 1 - edge_h - 3*absolute_dis_h - 2*mid_h, absolute_dis_v, absolute_dis_h])
    ax8 = fig.add_axes([edge_v + absolute_dis_v + mid_v, 1 - edge_h - 3*absolute_dis_h - 2*mid_h, absolute_dis_v, absolute_dis_h])
    ax9 = fig.add_axes([edge_v + 2*(absolute_dis_v + mid_v), 1 - edge_h - 3*absolute_dis_h - 2*mid_h, absolute_dis_v, absolute_dis_h])

    ax10 = fig.add_axes([edge_v, 1 - edge_h - 4*absolute_dis_h - 3*mid_h, absolute_dis_v, absolute_dis_h])
    ax11 = fig.add_axes([edge_v + absolute_dis_v + mid_v, 1 - edge_h - 4*absolute_dis_h - 3*mid_h, absolute_dis_v, absolute_dis_h])
    ax12 = fig.add_axes([edge_v + 2*(absolute_dis_v + mid_v) , 1 - edge_h - 4*absolute_dis_h - 3*mid_h, absolute_dis_v, absolute_dis_h])

    axes = [ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8, ax9, ax10, ax11, ax12]
    '''
    # cmap
    r_colors = ['#B7A0C4', '#36134A']
    bias_colors = ['#9bafd9', '#a6d96a', '#103783']
    ubrmse_colors = ['#75CCD8', '#abdda4', '#00505F'] 
    # 创建对应的色条
    r_vmin,r_vmax = 0, 1
    bias_vmin, bias_vmax = np.nanpercentile(bias_values.flatten(), 1), np.nanpercentile(bias_values.flatten(), 99)
    ubRMSE_vmin, ubRMSE_vmax = np.nanpercentile(ubRMSE_values.flatten(), 1), np.nanpercentile(ubRMSE_values.flatten(), 99)

    r_cmap = mcolors.LinearSegmentedColormap.from_list("r_cmap", r_colors)
    bias_cmap = mcolors.LinearSegmentedColormap.from_list("bias_cmap", bias_colors)
    ubrmse_cmap = mcolors.LinearSegmentedColormap.from_list("ubrmse_cmap", ubrmse_colors)
    '''
    # 生成子子图的label
    r_bins=[0, 0.2, 0.4, 0.6, 0.8, 1]  
    bias_bins=[-0.08, -0.04, 0, 0.04, 0.08, 0.12]  
    ubRMSE_bins=[0, 0.02, 0.04, 0.06, 0.08, 0.1]
    sub_param_r = [0.06, 0.035, 0.20, 0.4]
    sub_param_bias = [0.085, 0.035, 0.20, 0.4]
    sub_param_ubRMSE = [0.085, 0.035, 0.20, 0.4]
    # cmap
    r_cmap = mcm.terrain
    bias_cmap = mcm.BrBG
    ubrmse_cmap = mcm.terrain_r
    # vmin,vmax
    r_vmin, r_vmax = r_bins[0], r_bins[-1]
    bias_vmin, bias_vmax = bias_bins[0], bias_bins[-1]
    ubRMSE_vmin, ubRMSE_vmax = ubRMSE_bins[0], ubRMSE_bins[-1]
    r_norm = mcolors.Normalize(vmin=r_vmin, vmax=r_vmax)
    midpoint = 0
    bias_norm = mcolors.TwoSlopeNorm(vmin=bias_vmin, vcenter=midpoint, vmax=bias_vmax)
    ubRMSE_norm = mcolors.Normalize(vmin=ubRMSE_vmin, vmax=ubRMSE_vmax)
    r = mcm.ScalarMappable(norm=r_norm, cmap=r_cmap)
    bias = mcm.ScalarMappable(norm=bias_norm, cmap=bias_cmap)
    ubRMSE = mcm.ScalarMappable(norm=ubRMSE_norm, cmap=ubrmse_cmap)
    # 生成子子图颜色
    r_colors = get_cbar_colors(r, r_bins)
    bias_colors = get_cbar_colors(bias, bias_bins)
    ubRMSE_colors = get_cbar_colors(ubRMSE, ubRMSE_bins)
    # 刻度查看和设置
    '''
    bias_vmin, bias_vmax = np.nanmin(bias_values.flatten()), np.nanmax(bias_values.flatten()) #-0.154,0.234
    ubRMSE_vmin, ubRMSE_vmax = np.nanmin(ubRMSE_values.flatten()), np.nanmax(ubRMSE_values.flatten()) #0.0027 0.137
    '''

    for i in range(4):
        bottom_spines = True  if i == 3 else False
        draw_subplot(axes[3*i], r_values[..., i], selected_model_names[i], selected_model_names, shp_path, lakes_path, vmin=r_vmin, vmax=r_vmax, cmap=r_cmap, left_spines=(True if 3*i in [0,3,6,9] else False), bottom_spines=bottom_spines, plt_name = 'a', bins=r_bins, colors=r_colors, invert_order=True, sub_param = sub_param_r),
        draw_subplot(axes[3*i+1], bias_values[..., i], selected_model_names[i], selected_model_names, shp_path, lakes_path, vmin=bias_vmin, vmax=bias_vmax, cmap=bias_cmap, left_spines=(True if 3*i+1 in [0,3,6,9] else False), bottom_spines=bottom_spines, plt_name = 'a', bins=bias_bins, colors=bias_colors, invert_order=False, sub_param = sub_param_bias)
        draw_subplot(axes[3*i+2], ubRMSE_values[..., i], selected_model_names[i], selected_model_names, shp_path, lakes_path, vmin=ubRMSE_vmin, vmax=ubRMSE_vmax, cmap=ubrmse_cmap, left_spines=(True if 3*i+2 in [0,3,6,9] else False), bottom_spines=bottom_spines, plt_name = 'a', bins=ubRMSE_bins, colors=ubRMSE_colors, invert_order=False, sub_param = sub_param_ubRMSE)

    cbar_lengths = [0.21, 0.175, 0.15]
    for i, (mappable, label, bins) in enumerate(zip([r, bias, ubRMSE], ['Person R', 'bias', 'ubRMSE'], [r_bins, bias_bins, ubRMSE_bins])):
        cbar_ax = fig.add_axes([0.2535*i+0.1315, 1 - edge_h - 4*absolute_dis_h - 3*mid_h-0.023, cbar_lengths[i], 0.008])
        extend = 'neither' if i == 0 else ('both' if i == 1 else 'max')
        cbar = plt.colorbar(mappable, cax=cbar_ax, orientation='horizontal', extend=extend)
        # 设置刻度为bins数组的值
        cbar.set_ticks(bins)
        cbar.ax.xaxis.set_tick_params(labelsize=12)
        cbar.ax.xaxis.set_major_locator(MaxNLocator(nbins=len(bins)))
        cbar.ax.tick_params(axis='x', which='both', colors='black', length=2, width=1)
        cbar.ax.xaxis.set_major_formatter(FuncFormatter(custom_formatter))
        for label in cbar.ax.get_xticklabels():
            label.set_fontproperties(font_manager.FontProperties(weight='normal'))  # 设置为粗体
            label.set_fontsize(12)
    fig.text(0.35, 0.3045, 'R', fontsize=14, fontname='Times New Roman', verticalalignment='top')
    fig.text(0.565, 0.3045, 'bias (m³/m³)', fontsize=14, fontname='Times New Roman', verticalalignment='top')
    fig.text(0.795, 0.3045, 'ubRMSE (m³/m³)', fontsize=14, fontname='Times New Roman', verticalalignment='top')
    
    fig.savefig(r"C:\Users\Administrator\Desktop\Draw\sat_original_2d.jpeg", dpi=1000, format='jpeg', bbox_inches='tight')
    plt.show()

def draw_sub_subplot(ax, value, bins, colors, invert_order=True):
    flat_value = value.flatten()
    flat_value = flat_value[~np.isnan(flat_value)]
    counts, edges = np.histogram(flat_value, bins=bins)
    centers = (edges[:-1] + edges[1:]) / 2
    heights = np.diff(edges)

    for i, (count, center) in enumerate(zip(counts, centers)):
        ax.barh(center, count, height=heights[i], color=colors[i % len(colors)], edgecolor='black', align='center')
        percentage = f'{count / flat_value.size * 100:.1f}%'
        ax.text(count+40, center, percentage, ha='left', va='center', fontsize=10)
    
    if invert_order:
        ax.invert_yaxis()

    # 轴线控制
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.xaxis.set_visible(False)
    ax.grid(False)

    # 刻度控制
    ax.tick_params(axis='y', which='both', length=2, width=1, labelsize=10, pad=2, direction='out', left=True, right=False, labelleft=True, labelright=False)
    ax.set_yticks(edges)
    ax.set_yticklabels(["{:.2f}".format(edge) for edge in edges], fontsize=10)

    ax.yaxis.set_major_formatter(FuncFormatter(custom_formatter))
    ax.spines['left'].set_bounds(edges[0], edges[-1])
    ax.spines['left'].set_color('black')
    ax.spines['left'].set_linewidth(1)
    ax.axis('on')
    ax.set_facecolor('none')

def draw_subplot(ax, value, model_name, model_names, shp_path, lakes_path, vmin=None, vmax=None, cmap=None, 
                 left_spines=None, bottom_spines=None, plt_name='a', bins=[0, 0.2, 0.4, 0.6, 0.8, 1], colors=None, invert_order=True,
                 sub_param = None):

    ##  sub_draw
    sub_ax = ax.inset_axes(sub_param)
    draw_sub_subplot(sub_ax, value, bins=bins, colors=colors,invert_order=invert_order)

    ## draw
    gdf = gpd.read_file(shp_path)
    lakes = gpd.read_file(lakes_path)
    lakes = lakes.to_crs(epsg=4326)
    lakes.plot(ax=ax, edgecolor='white', facecolor='white', linewidth=0)
    gdf.plot(ax=ax, edgecolor='black', facecolor='none', linewidth=1.5)

    lat_min, lat_max, lon_min, lon_max, lat_points, lon_points= 25.293533, 40.604774, 66.84705, 105.31184, 42, 104
    lon_step = (lon_max - lon_min) / (lon_points - 1)
    lat_step = (lat_max - lat_min) / (lat_points - 1)
    new_lon_min = lon_min - lon_step / 2
    new_lon_max = lon_max + lon_step / 2
    new_lat_min = lat_min - lat_step / 2
    new_lat_max = lat_max + lat_step / 2

    image = ax.imshow(value, cmap=cmap, origin='lower', vmin=vmin, vmax=vmax, extent=[new_lon_min, new_lon_max, new_lat_min, new_lat_max])

    # 设置X轴和Y轴的标签
    ax.set_xlabel('Longitude(°)', fontsize=16, fontweight='normal', labelpad=0)
    ax.set_ylabel('Latitude(°)', fontsize=16, fontweight='normal')

    # 刻度标签轴绘制
    labelsize = 12
    pad_x = 4
    pad_y = 4
    ax.yaxis.label.set_visible(left_spines)
    ax.xaxis.label.set_visible(bottom_spines)
    ax.xaxis.set_major_formatter(FuncFormatter(add_degree_E))

    # 刻度调节
    plt.setp(ax.get_xticklabels(), rotation=0,rotation_mode="anchor")
    plt.setp(ax.get_yticklabels(), rotation=90,ha="center", rotation_mode="anchor")
    # 刻度值格式
    ax.yaxis.set_major_formatter(FuncFormatter(add_degree_N))
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_color('black')
    ax.spines['bottom'].set_linewidth(1.5)
    ax.spines['left'].set_color('black')
    ax.spines['left'].set_linewidth(1.5)
    ax.xaxis.set_major_locator(MaxNLocator(nbins=6))
    ax.yaxis.set_major_locator(MaxNLocator(nbins=4))
    ax.tick_params(axis='y', which='both', colors='black', length=5, width=1.5, labelsize=labelsize, direction='out', pad=pad_y, right=False, labelright=False)
    ax.tick_params(axis='x', which='both', colors='black', length=5, width=1.5, labelsize=labelsize, direction='out', pad=pad_x, top=False, labeltop=False)
    add_north(ax)
    ax.grid(False)

    # 文本
    prefix = "(" + chr(ord('a') + model_names.index(model_name)) + ")" 
    model_str = f'{prefix} {model_name}'
    if left_spines: ax.text(0.02, 1.05, model_str, transform=ax.transAxes, fontsize=16, fontname='Times New Roman', verticalalignment='top')

create_2d_subplots(shp_path, lakes_path, r_values, bias_values, RMSE_values, ubRMSE_values, selected_model_names, plt_name = 'a')

### (数据处理) 重建结果数据提取

In [ ]:
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import xarray as xr
import zarr
def generate_clipping_mask(lon_grid, lat_grid, shp_path):
    """
    使用.shp文件生成数据掩码：在边界外的数据掩码为0，在边界内的数据掩码为1。
    """
    import geopandas as gpd
    from shapely.geometry import Point
    import numpy as np

    gdf = gpd.read_file(shp_path)
    poly = gdf.unary_union
    mask = np.zeros_like(lon_grid, dtype=bool)

    for i in range(lon_grid.shape[0]):
        for j in range(lon_grid.shape[1]):
            point = Point(lon_grid[i, j], lat_grid[i, j])
            if point.within(poly):
                mask[i, j] = True

    return mask

shp_path = r"D:\\Data_Store\\TPBoundary_new(2021)\\TPBoundary_new(2021).shp"
lakes_path = r"D:\Data_Store\TPBoundary_new(2021)\Lake_TP_2010.shp"

reconstruction_paths = {
    "Original": r"D:\Data_Store\Dataset\Reconstruction\37SM\soil_moisture.npy",
    "LSTM": r"D:\Data_Store\Dataset\Reconstruction\LSTM\LSTM1.npy",
    'PSC-FENet-ST': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\ST_PSC-FENet.npy",
    "RF": r"D:\Data_Store\Dataset\Reconstruction\RF\RF_reconstructed_data.npy",
    'Temporal': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\Temporal.npy",
    'PSC-FENet-SP': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SP_PSC-FENet.npy",
    'UNet-ST': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\ST_UNet.npy",
    'ResNet-ST': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\ST_ResNet.npy",
    'VIT-ST': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\ST_ViT.npy",
    'UNet-SP': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SP_UNet.npy",
    'ResNet-SP': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SP_ResNet.npy",
    'VIT-SP': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SP_ViT.npy",
}

settings = {
    0.37: (25.293533, 40.604774, 66.84705, 105.31184, 42, 104),
    0.1: (25.550117, 40.450184, 66.95031, 104.95048, 150, 381)
}

'''
selected_model_names = ['KPNet-ST', 'ResNet-ST', 'UNet-ST', 'VIT-ST',  'LSTM', 'RF', 'Temporal']
selected_reconstruction_paths = {name: paths for name, paths in file_paths.items() if name in selected_model_names}
'''
dem_data = xr.open_zarr(r"D:\Data_Store\Dataset\Original_Data\0.1_Data.zarr").as_dem_3s.isel(time=0).compute()
masks = {}
for mode, (lat_min, lat_max, lon_min, lon_max, lat_points, lon_points) in settings.items():
    lon_step = (lon_max - lon_min) / (lon_points - 1)
    lat_step = (lat_max - lat_min) / (lat_points - 1)
    new_lon_min = lon_min - lon_step / 2
    new_lon_max = lon_max + lon_step / 2
    new_lat_min = lat_min - lat_step / 2
    new_lat_max = lat_max + lat_step / 2
    lon_range, lat_range = np.linspace(new_lon_min, new_lon_max, lon_points), np.linspace(new_lat_min, new_lat_max, lat_points)
    lon_grid, lat_grid = np.meshgrid(lon_range, lat_range)
    masks[mode] = generate_clipping_mask(lon_grid, lat_grid, shp_path)

reconstruction_clip_data = {}
reconstruction_data = {}
for i, (key, data_path) in enumerate(reconstruction_paths.items()):
    mode = 0.37 if i == 0 else 0.1
    mask = masks[mode]
    data = np.load(data_path)[7817]
    clipped_data = np.where(mask, data, np.nan)
    reconstruction_clip_data[key] = clipped_data
    reconstruction_data[key] = data

In [ ]:
dem_data = xr.open_zarr(r"D:\Data_Store\Dataset\Original_Data\0.1_Data.zarr").as_dem_3s.isel(time=0).compute()

In [ ]:
import xarray as xr
import zarr

dem_data = xr.open_zarr(r"D:\Data_Store\Dataset\Original_Data\0.1_Data.zarr").as_dem_3s.isel(time=0)

### (绘图) V1.0 重建结果图 4*3

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
from matplotlib.font_manager import FontProperties
from matplotlib.ticker import FuncFormatter
import geopandas as gpd
from shapely.geometry import Point
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.colors as mcolors
import matplotlib.cm as mcm
from matplotlib.ticker import MaxNLocator

sns.set(style="whitegrid", font='Times New Roman')
matplotlib.rcParams['font.family'] = 'Times New Roman'
legend_font = FontProperties(family='Times New Roman', size=14)
font_prop = FontProperties(family='Times New Roman', size=16)

def add_north(ax, labelsize=12, loc_x=0.97, loc_y=0.96, width=0.03, height=0.15, pad=0.15):
    """
    画一个比例尺带'N'文字注释
    主要参数如下
    :param ax: 要画的坐标区域 Axes实例 plt.gca()获取即可
    :param labelsize: 显示'N'文字的大小
    :param loc_x: 以文字下部为中心的占整个ax横向比例
    :param loc_y: 以文字下部为中心的占整个ax纵向比例
    :param width: 指南针占ax比例宽度
    :param height: 指南针占ax比例高度
    :param pad: 文字符号占ax比例间隙
    :return: None
    """
    minx, maxx = ax.get_xlim()
    miny, maxy = ax.get_ylim()
    ylen = maxy - miny
    xlen = maxx - minx
    left = [minx + xlen*(loc_x - width*.5), miny + ylen*(loc_y - pad)]
    right = [minx + xlen*(loc_x + width*.5), miny + ylen*(loc_y - pad)]
    top = [minx + xlen*loc_x, miny + ylen*(loc_y - pad + height)]
    center = [minx + xlen*loc_x, left[1] + (top[1] - left[1])*.4]
    triangle = mpatches.Polygon([left, top, right, center], color='k')
    ax.text(s='N',
            x=minx + xlen*loc_x,
            y=miny + ylen*(loc_y - pad + height),
            fontsize=labelsize,
            horizontalalignment='center',
            verticalalignment='bottom')
    ax.add_patch(triangle)

def add_scalebar(ax,lon0,lat0,length,size=0.45,lw=1.5):
    '''
    ax: 坐标轴
    lon0: 经度
    lat0: 纬度
    length: 长度
    size: 控制粗细和距离的
    '''
    # style 3
    lw = lw
    ax.hlines(y=lat0,  xmin = lon0, xmax = lon0+length/111, colors="black", ls="-", lw=lw, label='%d km' % (length))
    ax.vlines(x = lon0, ymin = lat0-size, ymax = lat0+size, colors="black", ls="-", lw=lw)
    ax.vlines(x = lon0+length/2/111, ymin = lat0-size, ymax = lat0+size, colors="black", ls="-", lw=lw)
    ax.vlines(x = lon0+length/111, ymin = lat0-size, ymax = lat0+size, colors="black", ls="-", lw=lw)
    ax.text(lon0+length/111,lat0+size+0.12,'%d' % (length),horizontalalignment = 'center',size=12)
    ax.text(lon0+length/2/111,lat0+size+0.12,'%d' % (length/2),horizontalalignment = 'center',size=12)
    ax.text(lon0,lat0+size+0.12,'0',horizontalalignment = 'center',size=12)
    ax.text(lon0+length/111/2*2.5,lat0+size+0.12,'km',horizontalalignment = 'center',size=12)

def custom_formatter(x, pos):
    if x == 0:
        return "0"
    else:
        str_x = "{:.15g}".format(x)
        if 'e' in str_x:
            base, exponent = str_x.split('e')
            base = base.rstrip('0').rstrip('.')
            exponent = exponent.replace('+', '').lstrip('0')
            if exponent:
                return f"{base}e{exponent}"
            else:
                return base
        else:
            return str_x
        
def add_degree_E(x, pos):
    return f"{int(x)}°E"

def add_degree_N(y, pos):
    return f"{int(custom_formatter(y, pos))}°N"

def create_subplots(reconstruction_datas,shp_path,lakes_path,model_names):
    fig = plt.figure(figsize=(24, 24))

    ratio_h = 7 # 竖直比例 3
    ratio_v = 4 # 水平比例

    absolute_dis_h = 0.7/ratio_h # 中央子图绝对大小（竖直）0.7
    absolute_dis_v = 0.7/ratio_v # 中央子图绝对大小（水平）

    edge_h = 0.36 # 竖直边缘间隙
    edge_v = 0.135 # 水平边缘间隙

    mid_h = (1-edge_h*2-absolute_dis_h*3)/2 # 竖直间隙,可以通过竖直边缘间隙调节
    mid_v = (1-edge_v*2-absolute_dis_v*4)/3 # 水平间隙……

    right_offset_first = 0 # 第一层向右偏移量

    # 计算第一层图起点:上层子图  左起点、下起点、宽度比例、高度比例
    ax1 = fig.add_axes([edge_v + right_offset_first, 1 - edge_h - absolute_dis_h, absolute_dis_v, absolute_dis_h])
    ax2 = fig.add_axes([edge_v + right_offset_first + absolute_dis_v + mid_v, 1 - edge_h - absolute_dis_h, absolute_dis_v, absolute_dis_h])
    ax3 = fig.add_axes([edge_v + right_offset_first + 2*(absolute_dis_v + mid_v), 1 - edge_h - absolute_dis_h, absolute_dis_v, absolute_dis_h])
    ax4 = fig.add_axes([edge_v + right_offset_first + 3*(absolute_dis_v + mid_v), 1 - edge_h - absolute_dis_h, absolute_dis_v, absolute_dis_h])

    ax5 = fig.add_axes([edge_v + right_offset_first, 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v, absolute_dis_h])
    ax6 = fig.add_axes([edge_v + right_offset_first + absolute_dis_v + mid_v, 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v, absolute_dis_h])
    ax7 = fig.add_axes([edge_v + right_offset_first + 2*(absolute_dis_v + mid_v), 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v, absolute_dis_h])
    ax8 = fig.add_axes([edge_v + right_offset_first + 3*(absolute_dis_v + mid_v), 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v, absolute_dis_h])

    ax9 = fig.add_axes([edge_v + right_offset_first, 1 - edge_h - 3*absolute_dis_h - 2*mid_h, absolute_dis_v, absolute_dis_h])
    ax10 = fig.add_axes([edge_v + right_offset_first + absolute_dis_v + mid_v, 1 - edge_h - 3*absolute_dis_h - 2*mid_h, absolute_dis_v, absolute_dis_h])
    ax11 = fig.add_axes([edge_v + right_offset_first + 2*(absolute_dis_v + mid_v), 1 - edge_h - 3*absolute_dis_h - 2*mid_h, absolute_dis_v, absolute_dis_h])
    ax12 = fig.add_axes([edge_v + right_offset_first + 3*(absolute_dis_v + mid_v), 1 - edge_h - 3*absolute_dis_h - 2*mid_h, absolute_dis_v, absolute_dis_h])

    axes = [ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8, ax9, ax10, ax11, ax12]
    vmin, vmax = 0, 0.5
    norm = mcolors.Normalize(vmin=vmin, vmax=vmax)
    cmap = mcm.BrBG
    sm = mcm.ScalarMappable(norm=norm, cmap=cmap)

    for i, (key, data_path) in enumerate(reconstruction_datas.items()):
        mode = 0.37 if i == 0 else 0.1
        left_spines = True  if i in [0,4,8] else False
        right_spines = True  if i in [3,7,11] else False
        top_spines = True  if i in [0,1,2,3] else False
        bottom_spines = True  if i in [8,9,10,11] else False
        draw_subplot(axes[i], reconstruction_datas[key],mode,shp_path,lakes_path,model_names,model_name=key,vmin=vmin, vmax=vmax,cmap=cmap,top_spines=top_spines,right_spines=right_spines, bottom_spines=bottom_spines, left_spines=left_spines)

    cbar_ax = fig.add_axes([0.25, 1 - edge_h - 3*absolute_dis_h - 2*mid_h-0.023, 0.5, 0.012])
    cbar = plt.colorbar(sm, cax=cbar_ax, orientation='horizontal')#, extend='both'
    cbar.set_ticks([vmin, (vmin+vmax)/2, vmax])
    cbar.ax.xaxis.set_tick_params(labelsize=14)
    cbar.ax.xaxis.set_major_locator(MaxNLocator(nbins=5))
    cbar.ax.tick_params(axis='x', which='both', colors='black', length=3, width=1.5)
    fig.text(0.757, 0.334, 'm³/m³', fontsize=16, fontname='Times New Roman', verticalalignment='top')
    fig.savefig(r"C:\Users\Administrator\Desktop\Draw\eval_darw.jpeg", dpi=1000, format='jpeg', bbox_inches='tight')
    plt.show()

def draw_subplot(ax, clipped_data, mode, shp_path,lakes_path,model_names,model_name=None,vmin=None, vmax=None,cmap=None,
                 top_spines=True, right_spines=True, bottom_spines=True, left_spines=True):
    gdf = gpd.read_file(shp_path)
    lakes = gpd.read_file(lakes_path)
    lakes = lakes.to_crs(epsg=4326)
    lakes.plot(ax=ax, edgecolor='white', facecolor='white', linewidth=0)
    gdf.plot(ax=ax, edgecolor='black', facecolor='none', linewidth=1.5)

    lat_min, lat_max, lon_min, lon_max, lat_points, lon_points= 25.550117,40.450184,66.95031,104.95048,150,381
    lat_step = (lat_max - lat_min) / (lat_points - 1)
    lon_step = (lon_max - lon_min) / (lon_points - 1)

    lon_min_adj = lon_min - lon_step / 2
    lon_max_adj = lon_max + lon_step / 2
    lat_min_adj = lat_min - lat_step / 2
    lat_max_adj = lat_max + lat_step / 2
    image = ax.imshow(clipped_data, cmap=cmap, origin='lower', vmin=vmin, vmax=vmax, extent=[lon_min_adj, lon_max_adj, lat_min_adj, lat_max_adj])

    # 设置X轴和Y轴的标签
    ax.set_xlabel('Longitude(°)',size=16)
    ax.set_ylabel('Latitude(°)',size=16)
    ax.tick_params(axis='x', which='both', colors='black', length=3, width=1.5, labelsize=14, direction='out', pad=2, bottom=True, top=True, labelbottom=True,labeltop=False)
    ax.tick_params(axis='y', which='both', colors='black', length=3, width=1.5, labelsize=14, direction='out', pad=6, left=True, right=True, labelleft=True,labelright=False)
    plt.setp(ax.get_xticklabels(), rotation=0,rotation_mode="anchor")
    plt.setp(ax.get_yticklabels(), rotation=90,ha="center", rotation_mode="anchor")

    # 刻度标签轴绘制
    labelsize = 12
    pad_x = 4
    pad_y = 4
    ax.yaxis.label.set_visible(left_spines)
    ax.xaxis.label.set_visible(bottom_spines)
    ax.xaxis.set_major_formatter(FuncFormatter(add_degree_E))

    # 刻度调节
    plt.setp(ax.get_yticklabels(), rotation=90,ha="center", rotation_mode="anchor")
    # 刻度值格式
    ax.yaxis.set_major_formatter(FuncFormatter(add_degree_N))
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_color('black')
    ax.spines['bottom'].set_linewidth(1.5)
    ax.spines['left'].set_color('black')
    ax.spines['left'].set_linewidth(1.5)
    ax.xaxis.set_major_locator(MaxNLocator(nbins=6))
    ax.yaxis.set_major_locator(MaxNLocator(nbins=4))
    ax.tick_params(axis='y', which='both', colors='black', length=5, width=1.5, labelsize=labelsize, direction='inout', pad=pad_y, right=False, labelright=False)
    ax.tick_params(axis='x', which='both', colors='black', length=5, width=1.5, labelsize=labelsize, direction='inout', pad=pad_x, top=False, labeltop=False)
    add_north(ax)
    add_scalebar(ax,67.9,26,1200,size=0.5,lw=1.5)
    ax.grid(False)
    # 文本
    prefix = "(" + chr(ord('a') + model_names.index(model_name)) + ")" 
    model_str = f'{prefix} {model_name}'
    ax.text(0.01, 0.32, model_str, transform=ax.transAxes, fontsize=16, fontname='Times New Roman', verticalalignment='top')
model_names = list(reconstruction_paths.keys())
create_subplots(reconstruction_datas,shp_path,lakes_path,model_names)

### (绘图) V2.0 重建结果图 3*4

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
from matplotlib.font_manager import FontProperties
from matplotlib.ticker import FuncFormatter
import geopandas as gpd
from shapely.geometry import Point
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.colors as mcolors
import matplotlib.cm as mcm
from matplotlib.ticker import MaxNLocator

sns.set(style="whitegrid", font='Times New Roman')
matplotlib.rcParams['font.family'] = 'Times New Roman'
legend_font = FontProperties(family='Times New Roman', size=14)
font_prop = FontProperties(family='Times New Roman', size=16)

def add_north(ax, labelsize=12, loc_x=0.97, loc_y=0.96, width=0.03, height=0.15, pad=0.15):
    """
    画一个比例尺带'N'文字注释
    主要参数如下
    :param ax: 要画的坐标区域 Axes实例 plt.gca()获取即可
    :param labelsize: 显示'N'文字的大小
    :param loc_x: 以文字下部为中心的占整个ax横向比例
    :param loc_y: 以文字下部为中心的占整个ax纵向比例
    :param width: 指南针占ax比例宽度
    :param height: 指南针占ax比例高度
    :param pad: 文字符号占ax比例间隙
    :return: None
    """
    minx, maxx = ax.get_xlim()
    miny, maxy = ax.get_ylim()
    ylen = maxy - miny
    xlen = maxx - minx
    left = [minx + xlen*(loc_x - width*.5), miny + ylen*(loc_y - pad)]
    right = [minx + xlen*(loc_x + width*.5), miny + ylen*(loc_y - pad)]
    top = [minx + xlen*loc_x, miny + ylen*(loc_y - pad + height)]
    center = [minx + xlen*loc_x, left[1] + (top[1] - left[1])*.4]
    triangle = mpatches.Polygon([left, top, right, center], color='k')
    ax.text(s='N',
            x=minx + xlen*loc_x,
            y=miny + ylen*(loc_y - pad + height),
            fontsize=labelsize,
            horizontalalignment='center',
            verticalalignment='bottom')
    ax.add_patch(triangle)

def add_scalebar(ax,lon0,lat0,length,size=0.45,lw=1.5):
    '''
    ax: 坐标轴
    lon0: 经度
    lat0: 纬度
    length: 长度
    size: 控制粗细和距离的
    '''
    # style 3
    lw = lw
    ax.hlines(y=lat0,  xmin = lon0, xmax = lon0+length/111, colors="black", ls="-", lw=lw, label='%d km' % (length))
    ax.vlines(x = lon0, ymin = lat0-size, ymax = lat0+size, colors="black", ls="-", lw=lw)
    ax.vlines(x = lon0+length/2/111, ymin = lat0-size, ymax = lat0+size, colors="black", ls="-", lw=lw)
    ax.vlines(x = lon0+length/111, ymin = lat0-size, ymax = lat0+size, colors="black", ls="-", lw=lw)
    ax.text(lon0+length/111,lat0+size+0.12,'%d' % (length),horizontalalignment = 'center',size=12)
    ax.text(lon0+length/2/111,lat0+size+0.12,'%d' % (length/2),horizontalalignment = 'center',size=12)
    ax.text(lon0,lat0+size+0.12,'0',horizontalalignment = 'center',size=12)
    ax.text(lon0+length/111/2*2.5,lat0+size+0.12,'km',horizontalalignment = 'center',size=12)

def custom_formatter(x, pos):
    if x == 0:
        return "0"
    else:
        str_x = "{:.15g}".format(x)
        if 'e' in str_x:
            base, exponent = str_x.split('e')
            base = base.rstrip('0').rstrip('.')
            exponent = exponent.replace('+', '').lstrip('0')
            if exponent:
                return f"{base}e{exponent}"
            else:
                return base
        else:
            return str_x
        
def add_degree_E(x, pos):
    return f"{int(x)}°E"

def add_degree_N(y, pos):
    return f"{int(custom_formatter(y, pos))}°N"

def create_subplots(reconstruction_datas,shp_path,lakes_path,model_names):
    fig = plt.figure(figsize=(24, 24))

    ratio_h = 7 # 竖直比例 3
    ratio_v = 4 # 水平比例

    absolute_dis_h = 0.7/ratio_h # 中央子图绝对大小（竖直）0.7
    absolute_dis_v = 0.7/ratio_v # 中央子图绝对大小（水平）

    edge_h = 0.32 # 竖直边缘间隙
    edge_v = 0.225 # 水平边缘间隙

    mid_h = (1-edge_h*2-absolute_dis_h*4)/3 # 竖直间隙,可以通过竖直边缘间隙调节
    mid_v = (1-edge_v*2-absolute_dis_v*3)/2 # 水平间隙……

    # 计算第一层图起点:上层子图  左起点、下起点、宽度比例、高度比例
    ax1 = fig.add_axes([edge_v, 1 - edge_h - absolute_dis_h, absolute_dis_v, absolute_dis_h])
    ax2 = fig.add_axes([edge_v + absolute_dis_v + mid_v, 1 - edge_h - absolute_dis_h, absolute_dis_v, absolute_dis_h])
    ax3 = fig.add_axes([edge_v + 2*(absolute_dis_v + mid_v), 1 - edge_h - absolute_dis_h, absolute_dis_v, absolute_dis_h])

    ax4 = fig.add_axes([edge_v, 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v, absolute_dis_h])
    ax5 = fig.add_axes([edge_v + absolute_dis_v + mid_v, 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v, absolute_dis_h])
    ax6 = fig.add_axes([edge_v + 2*(absolute_dis_v + mid_v), 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v, absolute_dis_h])

    ax7 = fig.add_axes([edge_v, 1 - edge_h - 3*absolute_dis_h - 2*mid_h, absolute_dis_v, absolute_dis_h])
    ax8 = fig.add_axes([edge_v + absolute_dis_v + mid_v, 1 - edge_h - 3*absolute_dis_h - 2*mid_h, absolute_dis_v, absolute_dis_h])
    ax9 = fig.add_axes([edge_v + 2*(absolute_dis_v + mid_v), 1 - edge_h - 3*absolute_dis_h - 2*mid_h, absolute_dis_v, absolute_dis_h])

    ax10 = fig.add_axes([edge_v, 1 - edge_h - 4*absolute_dis_h - 3*mid_h, absolute_dis_v, absolute_dis_h])
    ax11 = fig.add_axes([edge_v + absolute_dis_v + mid_v, 1 - edge_h - 4*absolute_dis_h - 3*mid_h, absolute_dis_v, absolute_dis_h])
    ax12 = fig.add_axes([edge_v + 2*(absolute_dis_v + mid_v), 1 - edge_h - 4*absolute_dis_h - 3*mid_h, absolute_dis_v, absolute_dis_h])

    axes = [ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8, ax9, ax10, ax11, ax12]
    vmin, vmax = 0, 0.5
    norm = mcolors.Normalize(vmin=vmin, vmax=vmax)
    cmap = mcm.BrBG
    sm = mcm.ScalarMappable(norm=norm, cmap=cmap)

    for i, (key, data_path) in enumerate(reconstruction_datas.items()):
        mode = 0.37 if i == 0 else 0.1
        left_spines = True  if i in [0,3,6,9] else False
        right_spines = True  if i in [2,5,8,11] else False
        top_spines = True  if i in [0,1,2] else False
        bottom_spines = True  if i in [9,10,11] else False
        draw_subplot(axes[i], reconstruction_datas[key],mode,shp_path,lakes_path,model_names,model_name=key,vmin=vmin, vmax=vmax,cmap=cmap,top_spines=top_spines,right_spines=right_spines, bottom_spines=bottom_spines, left_spines=left_spines)

    cbar_ax = fig.add_axes([0.25, 1 - edge_h - 4*absolute_dis_h - 3*mid_h-0.02, 0.5, 0.012])
    cbar = plt.colorbar(sm, cax=cbar_ax, orientation='horizontal')#, extend='both'
    cbar.set_ticks([vmin, (vmin+vmax)/2, vmax])
    cbar.ax.xaxis.set_tick_params(labelsize=14)
    cbar.ax.xaxis.set_major_locator(MaxNLocator(nbins=5))
    cbar.ax.tick_params(axis='x', which='both', colors='black', length=3, width=1.5)
    fig.text(0.752, 0.31, 'm³/m³', fontsize=16, fontname='Times New Roman', verticalalignment='top')

    fig.savefig(r"C:\Users\Administrator\Desktop\Draw\eval_darw.jpeg", dpi=1000, format='jpeg', bbox_inches='tight')
    plt.show()

def draw_subplot(ax, clipped_data, mode, shp_path,lakes_path,model_names,model_name=None,vmin=None, vmax=None,cmap=None,
                 top_spines=True, right_spines=True, bottom_spines=True, left_spines=True):
    gdf = gpd.read_file(shp_path)
    lakes = gpd.read_file(lakes_path)
    lakes = lakes.to_crs(epsg=4326)
    lakes.plot(ax=ax, edgecolor='white', facecolor='white', linewidth=0)
    gdf.plot(ax=ax, edgecolor='black', facecolor='none', linewidth=1.5)

    lat_min, lat_max, lon_min, lon_max, lat_points, lon_points= 25.550117,40.450184,66.95031,104.95048,150,381
    lat_step = (lat_max - lat_min) / (lat_points - 1)
    lon_step = (lon_max - lon_min) / (lon_points - 1)

    lon_min_adj = lon_min - lon_step / 2
    lon_max_adj = lon_max + lon_step / 2
    lat_min_adj = lat_min - lat_step / 2
    lat_max_adj = lat_max + lat_step / 2
    image = ax.imshow(clipped_data, cmap=cmap, origin='lower', vmin=vmin, vmax=vmax, extent=[lon_min_adj, lon_max_adj, lat_min_adj, lat_max_adj])

    # 设置X轴和Y轴的标签
    ax.set_xlabel('Longitude(°)',size=16)
    ax.set_ylabel('Latitude(°)',size=16)
    ax.tick_params(axis='x', which='both', colors='black', length=3, width=1.5, labelsize=14, direction='out', pad=2, bottom=True, top=True, labelbottom=True,labeltop=False)
    ax.tick_params(axis='y', which='both', colors='black', length=3, width=1.5, labelsize=14, direction='out', pad=6, left=True, right=True, labelleft=True,labelright=False)
    plt.setp(ax.get_xticklabels(), rotation=0,rotation_mode="anchor")
    plt.setp(ax.get_yticklabels(), rotation=90,ha="center", rotation_mode="anchor")

    # 刻度标签轴绘制
    labelsize = 12
    pad_x = 4
    pad_y = 6
    ax.yaxis.label.set_visible(left_spines)
    ax.xaxis.label.set_visible(bottom_spines)
    ax.xaxis.set_major_formatter(FuncFormatter(add_degree_E))

    # 刻度调节
    plt.setp(ax.get_yticklabels(), rotation=90,ha="center", rotation_mode="anchor")
    # 刻度值格式
    ax.yaxis.set_major_formatter(FuncFormatter(add_degree_N))
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_color('black')
    ax.spines['bottom'].set_linewidth(1.5)
    ax.spines['left'].set_color('black')
    ax.spines['left'].set_linewidth(1.5)
    ax.xaxis.set_major_locator(MaxNLocator(nbins=6))
    ax.yaxis.set_major_locator(MaxNLocator(nbins=4))
    ax.tick_params(axis='y', which='both', colors='black', length=3, width=1.5, labelsize=labelsize, direction='in', pad=pad_y, right=False, labelright=False)
    ax.tick_params(axis='x', which='both', colors='black', length=3, width=1.5, labelsize=labelsize, direction='in', pad=pad_x, top=False, labeltop=False)
    add_north(ax)
    add_scalebar(ax,67.9,26.3,1200,size=0.5,lw=1.5)
    ax.grid(False)
    # 文本
    prefix = "(" + chr(ord('a') + model_names.index(model_name)) + ")" 
    model_str = f'{prefix} {model_name}'
    ax.text(0.01, 0.32, model_str, transform=ax.transAxes, fontsize=16, fontname='Times New Roman', verticalalignment='top')
model_names = list(reconstruction_paths.keys())
create_subplots(reconstruction_datas,shp_path,lakes_path,model_names)

### (绘图) V3.0 重建结果图 3*4 
inset_axes: 右下角开窗，图像缩小右移

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
from matplotlib.font_manager import FontProperties
from matplotlib.ticker import FuncFormatter
import geopandas as gpd
from shapely.geometry import Point
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.colors as mcolors
import matplotlib.cm as mcm
from matplotlib.ticker import MaxNLocator
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.patches import ConnectionPatch
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from mpl_toolkits.axes_grid1 import make_axes_locatable

sns.set(style="whitegrid", font='Times New Roman')
matplotlib.rcParams['font.family'] = 'Times New Roman'
legend_font = FontProperties(family='Times New Roman', size=14)
font_prop = FontProperties(family='Times New Roman', size=16)

def add_north(ax, labelsize=12, loc_x=0.97, loc_y=1, width=0.03, height=0.15, pad=0.15):
    """
    画一个比例尺带'N'文字注释
    主要参数如下
    :param ax: 要画的坐标区域 Axes实例 plt.gca()获取即可
    :param labelsize: 显示'N'文字的大小
    :param loc_x: 以文字下部为中心的占整个ax横向比例
    :param loc_y: 以文字下部为中心的占整个ax纵向比例
    :param width: 指南针占ax比例宽度
    :param height: 指南针占ax比例高度
    :param pad: 文字符号占ax比例间隙
    :return: None
    """
    minx, maxx = ax.get_xlim()
    miny, maxy = ax.get_ylim()
    ylen = maxy - miny
    xlen = maxx - minx
    left = [minx + xlen*(loc_x - width*.5), miny + ylen*(loc_y - pad)]
    right = [minx + xlen*(loc_x + width*.5), miny + ylen*(loc_y - pad)]
    top = [minx + xlen*loc_x, miny + ylen*(loc_y - pad + height)]
    center = [minx + xlen*loc_x, left[1] + (top[1] - left[1])*.4]
    triangle = mpatches.Polygon([left, top, right, center], color='k')
    ax.text(s='N',
            x=minx + xlen*loc_x,
            y=miny + ylen*(loc_y - pad + height),
            fontsize=labelsize,
            horizontalalignment='center',
            verticalalignment='bottom')
    ax.add_patch(triangle)

def add_scalebar(ax,lon0,lat0,length,size=0.45,lw=1.5):
    '''
    ax: 坐标轴
    lon0: 经度
    lat0: 纬度
    length: 长度
    size: 控制粗细和距离的
    '''
    # style 3
    lw = lw
    ax.hlines(y=lat0,  xmin = lon0, xmax = lon0+length/111, colors="black", ls="-", lw=lw, label='%d km' % (length))
    ax.vlines(x = lon0, ymin = lat0-size, ymax = lat0+size, colors="black", ls="-", lw=lw)
    ax.vlines(x = lon0+length/2/111, ymin = lat0-size, ymax = lat0+size, colors="black", ls="-", lw=lw)
    ax.vlines(x = lon0+length/111, ymin = lat0-size, ymax = lat0+size, colors="black", ls="-", lw=lw)
    ax.text(lon0+length/111,lat0+size+0.14,'%d' % (length),horizontalalignment = 'center',size=12)
    ax.text(lon0+length/2/111,lat0+size+0.14,'%d' % (length/2),horizontalalignment = 'center',size=12)
    ax.text(lon0,lat0+size+0.14,'0',horizontalalignment = 'center',size=12)
    ax.text(lon0+length/111/2*2.35,lat0+size+0.14,'km',horizontalalignment = 'center',size=12)

def custom_formatter(x, pos):
    if x == 0:
        return "0"
    else:
        str_x = "{:.15g}".format(x)
        if 'e' in str_x:
            base, exponent = str_x.split('e')
            base = base.rstrip('0').rstrip('.')
            exponent = exponent.replace('+', '').lstrip('0')
            if exponent:
                return f"{base}e{exponent}"
            else:
                return base
        else:
            return str_x
        
def add_degree_E(x, pos):
    return f"{int(x)}°E"

def add_degree_N(y, pos):
    return f"{int(float(custom_formatter(y, pos)))}°N"

def create_subplots(reconstruction_data, reconstruction_clip_data, dem_data, shp_path, lakes_path, model_names):
    fig = plt.figure(figsize=(24, 24))

    ratio_h = 7 # 竖直比例 3
    ratio_v = 4 # 水平比例

    absolute_dis_h = 0.7/ratio_h # 中央子图绝对大小（竖直）0.7
    absolute_dis_v = 0.7/ratio_v # 中央子图绝对大小（水平）

    edge_h = 0.32 # 竖直边缘间隙
    edge_v = 0.225 # 水平边缘间隙

    mid_h = (1-edge_h*2-absolute_dis_h*4)/3 # 竖直间隙,可以通过竖直边缘间隙调节
    mid_v = (1-edge_v*2-absolute_dis_v*3)/2 # 水平间隙……

    # 计算第一层图起点:上层子图  左起点、下起点、宽度比例、高度比例
    ax1 = fig.add_axes([edge_v, 1 - edge_h - absolute_dis_h, absolute_dis_v, absolute_dis_h])
    ax2 = fig.add_axes([edge_v + absolute_dis_v + mid_v, 1 - edge_h - absolute_dis_h, absolute_dis_v, absolute_dis_h])
    ax3 = fig.add_axes([edge_v + 2*(absolute_dis_v + mid_v), 1 - edge_h - absolute_dis_h, absolute_dis_v, absolute_dis_h])

    ax4 = fig.add_axes([edge_v, 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v, absolute_dis_h])
    ax5 = fig.add_axes([edge_v + absolute_dis_v + mid_v, 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v, absolute_dis_h])
    ax6 = fig.add_axes([edge_v + 2*(absolute_dis_v + mid_v), 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v, absolute_dis_h])

    ax7 = fig.add_axes([edge_v, 1 - edge_h - 3*absolute_dis_h - 2*mid_h, absolute_dis_v, absolute_dis_h])
    ax8 = fig.add_axes([edge_v + absolute_dis_v + mid_v, 1 - edge_h - 3*absolute_dis_h - 2*mid_h, absolute_dis_v, absolute_dis_h])
    ax9 = fig.add_axes([edge_v + 2*(absolute_dis_v + mid_v), 1 - edge_h - 3*absolute_dis_h - 2*mid_h, absolute_dis_v, absolute_dis_h])

    ax10 = fig.add_axes([edge_v, 1 - edge_h - 4*absolute_dis_h - 3*mid_h, absolute_dis_v, absolute_dis_h])
    ax11 = fig.add_axes([edge_v + absolute_dis_v + mid_v, 1 - edge_h - 4*absolute_dis_h - 3*mid_h, absolute_dis_v, absolute_dis_h])
    ax12 = fig.add_axes([edge_v + 2*(absolute_dis_v + mid_v), 1 - edge_h - 4*absolute_dis_h - 3*mid_h, absolute_dis_v, absolute_dis_h])

    axes = [ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8, ax9, ax10, ax11, ax12]
    vmin, vmax = 0, 0.5
    norm = mcolors.Normalize(vmin=vmin, vmax=vmax)
    cmap = mcm.BrBG
    sm = mcm.ScalarMappable(norm=norm, cmap=cmap)

    for i, (key, data_path) in enumerate(reconstruction_data.items()):
        mode = 0.37 if i == 0 else 0.1
        left_spines = True  if i in [0,3,6,9] else False
        right_spines = True  if i in [2,5,8,11] else False
        top_spines = True  if i in [0,1,2] else False
        bottom_spines = True  if i in [9,10,11] else False
        full_mode = 'dem' if i==0 else None
        draw_subplot(axes[i], reconstruction_data[key], dem_data, reconstruction_clip_data[key], mode, shp_path, lakes_path, model_names, model_name=key,
                    vmin=vmin, vmax=vmax, cmap=cmap, top_spines=top_spines, right_spines=right_spines, bottom_spines=bottom_spines, left_spines=left_spines, full_mode=full_mode)

    cbar_ax = fig.add_axes([0.25, 1 - edge_h - 4*absolute_dis_h - 3*mid_h-0.02, 0.5, 0.012])
    cbar = plt.colorbar(sm, cax=cbar_ax, orientation='horizontal')
    cbar.set_ticks([vmin, (vmin+vmax)/2, vmax])
    cbar.ax.xaxis.set_tick_params(labelsize=14)
    cbar.ax.xaxis.set_major_locator(MaxNLocator(nbins=5))
    cbar.ax.tick_params(axis='x', which='both', colors='black', length=3, width=1.5)
    cbar.outline.set_linewidth(0.8)
    fig.text(0.752, 0.31, 'm³/m³', fontsize=16, fontname='Times New Roman', verticalalignment='top')

    fig.savefig(r"C:\Users\Administrator\Desktop\Draw\orginal_darw.jpeg", dpi=1000, format='jpeg', bbox_inches='tight')
    plt.show()

def draw_subplot(ax, full_data, full_dem, clipped_data, mode, shp_path,lakes_path,model_names,model_name=None,vmin=None, vmax=None,cmap=None,
                 top_spines=True, right_spines=True, bottom_spines=True, left_spines=True, full_mode=None):
    
    gdf = gpd.read_file(shp_path)
    lakes = gpd.read_file(lakes_path)
    lakes = lakes.to_crs(epsg=4326)
    lakes.plot(ax=ax, edgecolor='white', facecolor='white', linewidth=0)
    gdf.plot(ax=ax, edgecolor='black', facecolor='none', linewidth=1.5)

    lat_min, lat_max, lon_min, lon_max, lat_points, lon_points= 25.550117,40.450184,66.95031,104.95048,150,381
    lat_step = (lat_max - lat_min) / (lat_points - 1)
    lon_step = (lon_max - lon_min) / (lon_points - 1)

    lon_min_adj = lon_min - lon_step / 2
    lon_max_adj = lon_max + lon_step / 2
    lat_min_adj = lat_min - lat_step / 2
    lat_max_adj = lat_max + lat_step / 2

    # 开窗
    sub_ax = ax.inset_axes((-0.10, 0.043, 0.7, 0.52))
    
    if full_mode is None:
        draw_sub_subplot(sub_ax, gdf, lakes, full_data, full_dem, vmin, vmax, cmap,lon_min_adj,lon_max_adj,lat_min_adj,lat_max_adj,draw_mode=full_mode)
    else:
        sub_ax2 = ax.inset_axes((-0.065, 0.572, 0.45, 0.32))
        draw_sub_subplot(sub_ax, gdf, lakes, full_data, full_dem, vmin, vmax, cmap,lon_min_adj,lon_max_adj,lat_min_adj,lat_max_adj,draw_mode='dem')
        draw_sub_subplot(sub_ax2, gdf, lakes, full_data, full_dem, vmin, vmax, cmap,lon_min_adj,lon_max_adj,lat_min_adj,lat_max_adj,draw_mode='original')

    image = ax.imshow(clipped_data, cmap=cmap, origin='lower', vmin=vmin, vmax=vmax, extent=[lon_min_adj, lon_max_adj, lat_min_adj, lat_max_adj])
    ax.set_xlim([lon_min_adj-11, lon_max_adj])
    ax.set_ylim([lat_min_adj-4, lat_max_adj])
    # 设置X轴和Y轴的标签
    ax.set_xlabel('Longitude(°)',size=16)
    ax.set_ylabel('Latitude(°)',size=16)
    ax.tick_params(axis='x', which='both', colors='black', length=3, width=1.5, labelsize=14, direction='out', pad=2, bottom=True, top=True, labelbottom=True,labeltop=False)
    ax.tick_params(axis='y', which='both', colors='black', length=3, width=1.5, labelsize=14, direction='out', pad=6, left=True, right=True, labelleft=True,labelright=False)
    plt.setp(ax.get_xticklabels(), rotation=0,rotation_mode="anchor")
    plt.setp(ax.get_yticklabels(), rotation=90,ha="center", rotation_mode="anchor")

    # 刻度标签轴绘制
    labelsize = 12
    pad_x = 4
    pad_y = 6
    ax.yaxis.label.set_visible(left_spines)
    ax.xaxis.label.set_visible(bottom_spines)
    ax.xaxis.set_major_formatter(FuncFormatter(add_degree_E))

    # 刻度调节
    plt.setp(ax.get_yticklabels(), rotation=90,ha="center", rotation_mode="anchor")
    # 刻度值格式
    ax.yaxis.set_major_formatter(FuncFormatter(add_degree_N))
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_color('black')
    ax.spines['bottom'].set_linewidth(1.5)
    ax.spines['left'].set_color('black')
    ax.spines['left'].set_linewidth(1.5)
    ax.xaxis.set_major_locator(MaxNLocator(nbins=6))
    ax.yaxis.set_major_locator(MaxNLocator(nbins=4))
    ax.tick_params(axis='y', which='both', colors='black', length=3, width=1.5, labelsize=labelsize, direction='in', pad=pad_y, right=False, labelright=False)
    ax.tick_params(axis='x', which='both', colors='black', length=3, width=1.5, labelsize=labelsize, direction='in', pad=pad_x, top=False, labeltop=False)
    add_north(ax)
    add_scalebar(ax,84,22.6,2000,size=0.5,lw=1.5)
    ax.grid(False)
    # 文本
    prefix = "(" + chr(ord('a') + model_names.index(model_name)) + ")" 
    model_str = f'{prefix} {model_name}'
    ax.text(0.02, 1.02, model_str, transform=ax.transAxes, fontsize=14, fontname='Times New Roman', verticalalignment='top')
    if full_mode is not None: ax.text(0.53, 0.245, 'DEM', transform=ax.transAxes, fontsize=14, fontname='Times New Roman', verticalalignment='top')
    if full_mode is not None: 
        mark_inset(ax, sub_ax, loc1=1, loc2=4, fc="none", ec='black', lw=0.8, ls='dashed')
        mark_inset(ax, sub_ax2, loc1=1, loc2=4, fc="none", ec='black', lw=0.5)
    else: mark_inset(ax, sub_ax, loc1=2, loc2=4, fc="none", ec='black', lw=0.8)

def draw_sub_subplot(ax, gdf=None, lakes=None, full_data=None, full_dem=None, vmin=None, vmax=None, cmap=None, lon_min_adj=None, lon_max_adj=None, lat_min_adj=None, lat_max_adj=None,draw_mode=None):
    
    specific_lon_min, specific_lat_min = 92, 35.5
    specific_lon_max, specific_lat_max = 99, 38.5
    lakes.plot(ax=ax, edgecolor='white', facecolor='white', linewidth=0)
    gdf.plot(ax=ax, edgecolor='black', facecolor='none', linewidth=1.5)
    if draw_mode == None:
        ax.imshow(full_data, extent=[lon_min_adj, lon_max_adj, lat_min_adj, lat_max_adj], origin='lower', cmap=cmap, vmin=vmin, vmax=vmax)
    elif draw_mode == 'dem':
        vmin_new = full_dem.min()
        vmax_new = full_dem.max()
        im = ax.imshow(full_dem, extent=[lon_min_adj, lon_max_adj, lat_min_adj, lat_max_adj], origin='lower', cmap=cmap, vmin=vmin_new, vmax=vmax_new)
        cbar_ax = ax.inset_axes([1.02, 0.0, 0.06, 0.6])
        cbar = plt.colorbar(im, cax=cbar_ax, orientation='vertical')
        cbar.outline.set_linewidth(0.5)
        cbar_ax.yaxis.set_ticks_position('right')
        cbar_ax.yaxis.set_tick_params(which='major', direction='inout', length=3, width=1, pad=1, labelleft=False, labelright=True)
    else:
        ax.imshow(full_data, extent=[lon_min_adj, lon_max_adj, lat_min_adj, lat_max_adj], origin='lower', cmap=cmap, vmin=vmin, vmax=vmax)
    ax.set_xlim([specific_lon_min, specific_lon_max])
    ax.set_ylim([specific_lat_min, specific_lat_max])

    ax.tick_params(axis='x', which='both', length=0, width=0, labelsize=0, labelbottom=False, labeltop=False)
    ax.tick_params(axis='y', which='both', length=0, width=0, labelsize=0, labelleft=False, labelright=False)
    ax.xaxis.label.set_visible(False)
    ax.yaxis.label.set_visible(False)
    ax.spines['bottom'].set_color('black')
    ax.spines['bottom'].set_linewidth(1)
    ax.spines['left'].set_color('black')
    ax.spines['left'].set_linewidth(1)
    ax.spines['top'].set_color('black')
    ax.spines['top'].set_linewidth(1)
    ax.spines['right'].set_color('black')
    ax.spines['right'].set_linewidth(1)
    ax.grid(False)

model_names = list(reconstruction_paths.keys())
create_subplots(reconstruction_data, reconstruction_clip_data, dem_data, shp_path, lakes_path, model_names)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 假设 dem_data 是一个二维数组，形状应该与 lat_points 和 lon_points 一致
# 如果 dem_data 不是numpy数组，需要先将其转换为numpy数组
# 例如: dem_data = np.array(dem_data)
lat_min, lat_max, lon_min, lon_max, lat_points, lon_points= 25.550117,40.450184,66.95031,104.95048,150,381
lat_step = (lat_max - lat_min) / (lat_points - 1)
lon_step = (lon_max - lon_min) / (lon_points - 1)

lon_min_adj = lon_min - lon_step / 2
lon_max_adj = lon_max + lon_step / 2
lat_min_adj = lat_min - lat_step / 2
lat_max_adj = lat_max + lat_step / 2
# 创建经纬度的网格
latitudes = np.linspace(lat_min, lat_max, lat_points)
longitudes = np.linspace(lon_min, lon_max, lon_points)
lon_grid, lat_grid = np.meshgrid(longitudes, latitudes)

# 绘制图像
fig, ax = plt.subplots(figsize=(14, 6))
cmap = 'viridis'  # 选择一个合适的colormap，'terrain'适合地形数据
extent = [lon_min_adj, lon_max_adj, lat_min_adj, lat_max_adj]

# 使用imshow绘制dem_data
# 注意：这里假设dem_data的行对应latitudes，列对应longitudes
im = ax.imshow(dem_data, extent=extent, origin='lower', cmap=cmap)

# 添加colorbar
cbar = fig.colorbar(im, ax=ax, orientation='vertical')
cbar.set_label('Elevation (m)')
cbar.ax.tick_params(axis='y', which='both', direction='inout', colors='black', length=5, width=1.5)
cbar.outline.set_linewidth(0)
# 设置图像标题和坐标轴标签
ax.set_title('DEM Data Visualization')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

# 设置坐标轴的范围（可选，imshow已通过extent参数设置）
ax.set_xlim(lon_min_adj, lon_max_adj)
ax.set_ylim(lat_min_adj, lat_max_adj)

plt.show()

## 全局原位网络站点评估图 - 云雨图 + 柱状折线图 + 时间动态图


### (数据处理) 站点指标统计 GDOWN和常规指标
用于表格填充，指标统计

In [ ]:
import pandas as pd

# xlxs文件路径
file_paths = {
    'PSC-FENet-ST':"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\STPSC-FENet_station_metrics.xlsx",
    'PSC-FENet-SP':"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SPPSC-FENet_station_metrics.xlsx",
    'PSC-FENet-ST32':"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\STPSC-FENet36_station_metrics.xlsx",
    'PSC-FENet-SP32':"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SPPSC-FENet36_station_metrics.xlsx",
    'RF': "D:\Data_Store\Dataset\Reconstruction\RF\RF_station_metrics.xlsx",
    'LSTM': "D:\Data_Store\Dataset\Reconstruction\LSTM\LSTM_station_metrics.xlsx",
    'ResNet-SP': "D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SPRESNET_station_metrics.xlsx",
    'UNet-SP': "D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SPUNET_station_metrics.xlsx",
    'ResNet-ST': "D:\Data_Store\Dataset\Reconstruction\ST\RESALL\STRESNET_station_metrics.xlsx",
    'UNet-ST': "D:\Data_Store\Dataset\Reconstruction\ST\RESALL\STUNET_station_metrics.xlsx",
    'VIT-SP': "D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SPVIT_station_metrics.xlsx",
    'VIT-ST': "D:\Data_Store\Dataset\Reconstruction\ST\RESALL\STVIT_station_metrics.xlsx",
    'Temporal': "D:\Data_Store\Dataset\Reconstruction\ST\RESALL\TEMPORAL_station_metrics.xlsx"
}
file_paths_G = {
    'PSC-FENet-ST':"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\STPSC-FENet_station_G_metrics.xlsx",
    'PSC-FENet-SP':"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SPPSC-FENet_station_G_metrics.xlsx",
    'PSC-FENet-ST32':"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\STPSC-FENet36_station_G_metrics.xlsx",
    'PSC-FENet-SP32':"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SPPSC-FENet36_station_G_metrics.xlsx",
    'RF': "D:\Data_Store\Dataset\Reconstruction\RF\RF_station_G_metrics.xlsx",
    'LSTM': "D:\Data_Store\Dataset\Reconstruction\LSTM\LSTM_station_G_metrics.xlsx",
    'ResNet-SP': "D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SPRESNET_station_G_metrics.xlsx",
    'UNet-SP': "D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SPUNET_station_G_metrics.xlsx",
    'ResNet-ST': "D:\Data_Store\Dataset\Reconstruction\ST\RESALL\STRESNET_station_G_metrics.xlsx",
    'UNet-ST': "D:\Data_Store\Dataset\Reconstruction\ST\RESALL\STUNET_station_G_metrics.xlsx",
    'VIT-SP': "D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SPVIT_station_G_metrics.xlsx",
    'VIT-ST': "D:\Data_Store\Dataset\Reconstruction\ST\RESALL\STVIT_station_G_metrics.xlsx",
    'Temporal': "D:\Data_Store\Dataset\Reconstruction\ST\RESALL\TEMPORAL_station_G_metrics.xlsx"
}

def merge_data(file_paths, file_paths_G):
    all_data = {}

    # 对每个模型执行合并操作
    for model in file_paths:
        all_data[model] = {}

        # 获取对应的文件路径
        file_path_metrics = file_paths[model]
        file_path_g_metrics = file_paths_G[model]
        
        # 对每个子表进行操作
        sheets = ['Shiquanhe', 'Ali', 'Maqu', 'Naqu', 'CTP']
        for sheet in sheets:
            # 读取原始指标和G指标的数据
            df_metrics = pd.read_excel(file_path_metrics, sheet_name=sheet)
            df_g_metrics = pd.read_excel(file_path_g_metrics, sheet_name=sheet)
            
            # 根据站点名合并两个DataFrame
            merged_df = pd.merge(df_metrics, df_g_metrics, on='站点名', suffixes=('', '_G'))
            
            # 将合并后的DataFrame存入字典
            all_data[model][sheet] = merged_df

    return all_data

def summarize_metrics(all_data):
    summary_by_model = {}

    for model in all_data:
        # 创建一个空的DataFrame用于当前模型
        model_summaries = []
        
        for sheet in all_data[model]:
            df = all_data[model][sheet]
            split_data = df['站点名'].str.rsplit('_', n=2, expand=True)
            df['point_name'] = split_data[0]
            df['lon'] = split_data[1].astype(float)
            df['lat'] = split_data[2].astype(float)
            df_sorted = df.sort_values(by=['lat', 'lon'], ascending=[False, True])
            best_records = df_sorted.groupby(['lat', 'lon'], as_index=False).apply(lambda x: x.nlargest(1, 'PearsonR')).reset_index(drop=True)
            
            # 选择所需的指标列
            metrics_df = best_records[['RMSE', 'ubRMSE', 'PearsonR', 'Bias', 'GEFFI', 'GPREC', 'GACCU', 'GDOWN']]
            
            # 计算均值（排除缺失值）
            mean_values = {col: metrics_df[col].mean(skipna=True) for col in ['RMSE', 'ubRMSE', 'PearsonR', 'Bias']}
            
            # 计算数值为正的比例，排除NaN
            positive_ratios = {col: (metrics_df[col] > 0).mean(skipna=True) for col in ['GEFFI', 'GPREC', 'GACCU', 'GDOWN']}
            
            # 创建一个包含所有统计信息的字典
            stats_dict = {
                'Network': sheet,
                'RMSE': mean_values['RMSE'],
                'ubRMSE': mean_values['ubRMSE'],
                'PearsonR': mean_values['PearsonR'],
                'Bias': mean_values['Bias'],
                'GEFFI': positive_ratios['GEFFI'],
                'GPREC': positive_ratios['GPREC'],
                'GACCU': positive_ratios['GACCU'],
                'GDOWN': positive_ratios['GDOWN'],
            }
            
            # 将字典转换为DataFrame并添加到列表中
            model_summaries.append(pd.DataFrame([stats_dict]))
        
        # 使用concat代替append来合并所有的DataFrame
        summary_data = pd.concat(model_summaries, ignore_index=True)
        
        # 将'Network'列作为索引
        summary_data.set_index('Network', inplace=True)
        
        # 将当前模型的DataFrame存储在总字典中
        summary_by_model[model] = summary_data

    return summary_by_model

all_data = merge_data(file_paths, file_paths_G)
summary_data = summarize_metrics(all_data)

In [ ]:
def reorder_and_extract(summary_data, new_order, new_columns_order):
    # 为了按照新的行列顺序重新排列数据，先创建一个空的DataFrame
    reordered_summary = pd.DataFrame(columns=new_columns_order, index=new_order)

    # 遍历所有模型的数据
    for model, data in summary_data.items():
        # 对于每个模型，提取每个指定网络的数据，并且重新排列
        for network in new_order:
            if network in data.index:  # 确保网络名存在于数据中
                network_data = data.loc[network, new_columns_order]
                reordered_summary.loc[network, :] = network_data.values  # 确保分配的是一维数组
            else:
                # 如果网络名在数据中不存在，则在该位置填充NaN
                reordered_summary.loc[network, :] = np.nan

        # 如果需要，此处可以添加输出语句来检查或保存每个模型的数据
        print(f"{model} reordered data:")
        print(reordered_summary)
        print("\n")
pd.options.display.float_format = '{:,.4f}'.format
new_order = ['Maqu', 'Naqu', 'Ali', 'Shiquanhe', 'CTP']
new_columns_order = ['PearsonR', 'Bias', 'RMSE', 'ubRMSE', 'GDOWN']
reordered_summary = reorder_and_extract(summary_data, new_order, new_columns_order)
print(summary_data)

In [ ]:
import pandas as pd
import numpy as np
def create_summary_excel(summary_data, models_st, models_sp):
    networks = ['Maqu', 'Naqu', 'Ali', 'Shiquanhe', 'CTP']
    metrics = ['PearsonR', 'Bias', 'RMSE', 'ubRMSE', 'GDOWN']
    columns = ['Products'] + models_st + ['ST-Mean'] + models_sp + ['SP-Mean']
    
    # 初始化DataFrame
    df_summary = pd.DataFrame(index=pd.MultiIndex.from_product([networks, metrics]), columns=columns)
    
    # 填充数据
    for model in models_st + models_sp:
        if model not in summary_data:
            print(f"Warning: Model '{model}' not found in summary data.")
            continue
        for network in networks:
            if network not in summary_data[model].index:
                print(f"Warning: Network '{network}' not found in model '{model}'.")
                continue
            for metric in metrics:
                if metric not in summary_data[model].columns:
                    print(f"Warning: Metric '{metric}' not found in model '{model}', network '{network}'.")
                    continue
                metric_key = 'GDOWN' if metric == '+Gdown' else metric
                value = summary_data[model].loc[network, metric_key]
                df_summary.loc[(network, metric), model] = value
    
    # 计算平均值
    for metric in metrics:
        st_mean = df_summary.loc[(slice(None), metric), models_st].mean(axis=1, skipna=True)
        sp_mean = df_summary.loc[(slice(None), metric), models_sp].mean(axis=1, skipna=True)
        df_summary.loc[(slice(None), metric), 'ST-Mean'] = st_mean
        df_summary.loc[(slice(None), metric), 'SP-Mean'] = sp_mean
    
    # 保存到Excel
    writer = pd.ExcelWriter('C:/Users/Administrator/Desktop/1.xlsx', engine='openpyxl')
    df_summary.to_excel(writer, sheet_name="Summary")
    writer.save()
    writer.close()

    return df_summary

# 模型名称
models_st = ['PSC-FENet-ST','PSC-FENet-ST32', 'ResNet-ST', 'UNet-ST', 'VIT-ST']
models_sp = ['PSC-FENet-SP','PSC-FENet-SP32', 'ResNet-SP', 'UNet-SP', 'VIT-SP']

# 假设summary_data已经准备好
# summary_data = summarize_metrics(all_data)

# 创建并保存汇总数据
summary_excel = create_summary_excel(summary_data, models_st, models_sp)
print(summary_excel)


In [ ]:
import pandas as pd
import numpy as np

def create_summary_excel(summary_data, models_st, models_sp):
    networks = ['Maqu', 'Naqu', 'Ali', 'Shiquanhe', 'CTP']
    metrics = ['PearsonR', 'Bias', 'RMSE', 'ubRMSE', 'GDOWN']
    columns = ['Products'] + models_st + ['ST-Mean'] + models_sp + ['SP-Mean']
    
    # 初始化DataFrame
    df_summary = pd.DataFrame(index=pd.MultiIndex.from_product([networks, metrics]), columns=columns)
    
    # 填充数据
    for model in models_st + models_sp:
        if model not in summary_data:
            print(f"Warning: Model '{model}' not found in summary data.")
            continue
        for network in networks:
            if network not in summary_data[model].index:
                print(f"Warning: Network '{network}' not found in model '{model}'.")
                continue
            for metric in metrics:
                if metric not in summary_data[model].columns:
                    print(f"Warning: Metric '{metric}' not found in model '{model}', network '{network}'.")
                    continue
                value = summary_data[model].loc[network, metric]
                df_summary.loc[(network, metric), model] = value
    
    # 计算平均值
    for metric in metrics:
        st_mean = df_summary.loc[(slice(None), metric), models_st].mean(axis=1, skipna=True)
        sp_mean = df_summary.loc[(slice(None), metric), models_sp].mean(axis=1, skipna=True)
        df_summary.loc[(slice(None), metric), 'ST-Mean'] = st_mean
        df_summary.loc[(slice(None), metric), 'SP-Mean'] = sp_mean
    
    # 保存到Excel
    writer = pd.ExcelWriter('C:/Users/Administrator/Desktop/2.xlsx', engine='openpyxl')
    df_summary.to_excel(writer, sheet_name="Summary")
    writer.save()
    writer.close()

    return df_summary

# 模型名称
models_st = ['PSC-FENet-ST', 'LSTM', 'RF', 'Temporal']  # Assuming these are all ST models for example
models_sp = []  # No SP models listed

# 假设summary_data已经准备好
# summary_data = summarize_metrics(all_data)

# 创建并保存汇总数据
summary_excel = create_summary_excel(summary_data, models_st, models_sp)
print(summary_excel)


In [ ]:
import pandas as pd

def merge_data(file_paths, file_paths_G):
    all_data = {}
    sheets = ['Shiquanhe', 'Ali', 'Maqu', 'Naqu', 'CTP']

    for model in file_paths:
        model_data = {}
        df_metrics = pd.read_excel(file_paths[model], sheet_name=None)  # Read all sheets
        df_g_metrics = pd.read_excel(file_paths_G[model], sheet_name=None)  # Read all sheets

        for sheet in sheets:
            # Ensure both metrics and G metrics have the sheet
            if sheet in df_metrics and sheet in df_g_metrics:
                # Merge on '站点名' assuming it's a common column
                merged_df = pd.merge(df_metrics[sheet], df_g_metrics[sheet], on='站点名', suffixes=('', '_G'))
                model_data[sheet] = merged_df

        all_data[model] = model_data

    return all_data

def calculate_means(all_data):
    results = {}
    metrics = ['RMSE', 'ubRMSE', 'PearsonR', 'Bias']  # Ensure these are correct and exist in your DataFrame

    for model, data in all_data.items():
        model_metrics = []
        for sheet in data.values():  # Iterate through each sheet's DataFrame
            mean_values = sheet[metrics].mean(skipna=True)
            model_metrics.append(mean_values)
        
        # Average over all sheets if multiple sheets data were collected
        model_means = pd.concat(model_metrics, axis=1).mean(axis=1)
        results[model] = model_means

    results_df = pd.DataFrame(results)
    results_df.to_excel('C:/Users/Administrator/Desktop/3.xlsx', sheet_name='Summary')

    return results_df

# Assuming file_paths and file_paths_G are defined
all_data = merge_data(file_paths, file_paths_G)
results_df = calculate_means(all_data)
print(results_df)

### (数据处理) ST/SP产品站点指标统计(2*2箱线图)
用于STSP模型对比
注：使用的文件是xlsx文件，xlsx文件在最开始的部分生成

In [ ]:
import pandas as pd

# xlxs文件路径
file_paths = {
    'Original': r"D:\Data_Store\Dataset\Reconstruction\37SM\Original_station_metrics.xlsx",
    'PSC-FENet-ST':"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\STPSC-FENet_station_metrics.xlsx",
    'PSC-FENet-SP':"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SPPSC-FENet_station_metrics.xlsx",
    'PSC-FENet-ST32':"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\STPSC-FENet36_station_metrics.xlsx",
    'PSC-FENet-SP32':"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SPPSC-FENet36_station_metrics.xlsx",
    'RF': "D:\Data_Store\Dataset\Reconstruction\RF\RF_station_metrics.xlsx",
    'LSTM': "D:\Data_Store\Dataset\Reconstruction\LSTM\LSTM_station_metrics.xlsx",
    'ResNet-SP': "D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SPRESNET_station_metrics.xlsx",
    'UNet-SP': "D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SPUNET_station_metrics.xlsx",
    'ResNet-ST': "D:\Data_Store\Dataset\Reconstruction\ST\RESALL\STRESNET_station_metrics.xlsx",
    'UNet-ST': "D:\Data_Store\Dataset\Reconstruction\ST\RESALL\STUNET_station_metrics.xlsx",
    'VIT-SP': "D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SPVIT_station_metrics.xlsx",
    'VIT-ST': "D:\Data_Store\Dataset\Reconstruction\ST\RESALL\STVIT_station_metrics.xlsx",
    'Temporal': "D:\Data_Store\Dataset\Reconstruction\ST\RESALL\TEMPORAL_station_metrics.xlsx"
}
selected_model_names = ['PSC-FENet-ST', 'PSC-FENet-SP', 'ResNet-ST', 'ResNet-SP', 'UNet-SP', 'UNet-ST', 'VIT-ST', 'VIT-SP']
selected_file_paths = {name: paths for name, paths in file_paths.items() if name in selected_model_names}

all_metrics_data = pd.DataFrame()
labels = list(selected_file_paths.keys())
# 加载所有子表，第二行读取
for method, path in selected_file_paths.items():
    xls = pd.ExcelFile(path)
    for sheet_name in xls.sheet_names:
        df = pd.read_excel(path, sheet_name=sheet_name, header=0)
        df['Method'] = method
        all_metrics_data = pd.concat([all_metrics_data, df], ignore_index=True)


### (绘图)  V1.0 2*2箱线图

In [ ]:
import matplotlib
from matplotlib.font_manager import FontProperties
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.collections as clt
import ptitprince as pt
from matplotlib.ticker import FuncFormatter
from matplotlib.ticker import MaxNLocator

sns.set(style="whitegrid", font='Times New Roman')
matplotlib.rcParams['font.family'] = 'Times New Roman'
legend_font = FontProperties(family='Times New Roman', size=14)
font_prop = FontProperties(family='Times New Roman', size=16)

def custom_formatter(x, pos):
    if x == 0:
        return "0"
    else:
        str_x = "{:.15g}".format(x)
        if 'e' in str_x:
            base, exponent = str_x.split('e')
            base = base.rstrip('0').rstrip('.')
            exponent = exponent.replace('+', '').lstrip('0')
            if exponent:
                return f"{base}e{exponent}"
            else:
                return base
        else:
            return str_x
def clean_axis(ax):
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.tick_params(axis='x', which='both', length=0, labelbottom=False)
    ax.tick_params(axis='y', which='both', length=0, labelleft=False)
    ax.grid(False)

def filter_outliers(data, dx, dy):
    filtered_data = pd.DataFrame(columns=data.columns)
    groups = data.groupby(dy)
    for name, group in groups:
        Q1 = group[dx].quantile(0.25)
        Q3 = group[dx].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        filtered_group = group[(group[dx] >= lower_bound) & (group[dx] <= upper_bound)]
        filtered_data = pd.concat([filtered_data, filtered_group])
    return filtered_data

def create_2d_subplots(all_metrics_data, plt_name = None):
    # 创建图形对象
    fig = plt.figure(figsize=(18, 24),dpi=500)

    ratio_h = 4 # 竖直比例
    ratio_v = 3 # 水平比例

    absolute_dis_h = 0.4/ratio_h # 中央子图绝对大小（竖直）
    absolute_dis_v = 0.9/ratio_v # 中央子图绝对大小（水平）

    edge_h = 0.393 # 竖直边缘间隙
    edge_v = 0.185 # 水平边缘间隙

    mid_h = (1-edge_h*2-absolute_dis_h*2)/1 # 竖直间隙,可以通过竖直边缘间隙调节
    mid_v = (1-edge_v*2-absolute_dis_v*2)/1 # 水平间隙……

    # 计算第一层图起点:上层子图  左起点、下起点、宽度比例、高度比例
    ax1 = fig.add_axes([edge_v, 1 - edge_h - absolute_dis_h, absolute_dis_v, absolute_dis_h])
    ax2 = fig.add_axes([edge_v + absolute_dis_v + mid_v, 1 - edge_h - absolute_dis_h, absolute_dis_v, absolute_dis_h])
    ax3 = fig.add_axes([edge_v, 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v, absolute_dis_h])
    ax4 = fig.add_axes([edge_v + absolute_dis_v + mid_v, 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v, absolute_dis_h])

    axes = [ax1, ax2, ax3, ax4]
    sub_param_left = [0.0, 0., 0.49, 1]
    sub_param_right = [0.51, 0., 0.49, 1]

    metric_columns = ['PearsonR', 'Bias', 'RMSE', 'ubRMSE']
    for i, metric in enumerate(metric_columns):
        left_spines = True if i in [0,2] else False
        bottom_spines = True if i in [2,3] else False
        draw_subplot(axes[i], all_metrics_data[['Method', metric]], metric, sub_param_left, sub_param_right, left_spines=left_spines, bottom_spines=bottom_spines,i=i)

    fig.savefig(r"C:\Users\Administrator\Desktop\Draw\sat_original_statistics.jpeg", dpi=1000, format='jpeg', bbox_inches='tight')

    plt.show()
    
def draw_subplot(ax, data, metric, sub_param_left, sub_param_right, left_spines, bottom_spines,i=None):
    selected_model_left = ['PSC-FENet-ST', 'ResNet-ST', 'UNet-ST', 'VIT-ST']
    selected_model_right = ['PSC-FENet-SP', 'ResNet-SP', 'UNet-SP', 'VIT-SP']
    data_left = data[data['Method'].isin(selected_model_left)]
    data_right = data[data['Method'].isin(selected_model_right)]
    min_val = data[metric].min()
    max_val = data[metric].max()
    range_val = max_val - min_val
    expand_val = range_val * 0.05

    min_val_expanded = min_val - expand_val
    max_val_expanded = max_val + expand_val
    xlim = [-0.5,1] if metric in ['PearsonR'] else (min_val_expanded, max_val_expanded)

    sub_ax_left = ax.inset_axes(sub_param_left)
    sub_ax_right = ax.inset_axes(sub_param_right)

    draw_sub_subplot(sub_ax_left, data_left, metric, xlim, left_tick_spines = True, left_spines=left_spines, bottom_spines=bottom_spines,i=i)
    draw_sub_subplot(sub_ax_right, data_right, metric, xlim, left_tick_spines  = False, left_spines=left_spines, bottom_spines=bottom_spines,i=i)

    middle_x = 0.5
    ax.axvline(x=middle_x, color='black', linestyle='--', linewidth=1.5, alpha=0.5)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    clean_axis(ax)

# 未划分左右
def draw_sub_subplot(ax, data, metric, xlim, left_tick_spines, left_spines, bottom_spines,i):
    dx = metric
    dy = 'Method'
    ort = "h"

    # Point Color
    color_scheme1 = ['#AA7BC9']
    color_scheme2 = ['#4B6CFB']
    color_scheme3 = ['#048A93']
    color_scheme4 = ['#26A269']

    # Violin Color
    Violin_Color = ['#C293D8','#7790FB','#20969E','#3EAB78']

    all_colors = color_scheme1 + color_scheme2 + color_scheme3 + color_scheme4

    half_violinplot_pal = sns.color_palette(Violin_Color)
    stripplot_pal = sns.color_palette(all_colors)
    ax=pt.half_violinplot(ax=ax, x=dx, y=dy, data=data, palette=half_violinplot_pal, bw=0.4, cut=0., 
                        scale="area", width=1, inner=None, orient=ort, linewidth=1, alpha=0.7)
    
    ax = sns.boxplot(ax=ax, x=dx, y=dy, data=data, color="black", width=0.3, zorder=10,
                    showcaps=True, 
                    boxprops={'facecolor':'none', 'edgecolor':'black', 'linewidth':0.8, 'zorder':10}, 
                    medianprops={'color':'black', 'linewidth':1.5}, 
                    whiskerprops={'color':'grey', 'linestyle':'--', 'linewidth':1.2}, 
                    capprops={'color':'black', 'linewidth':1}, 
                    flierprops={'marker':'o', 'markerfacecolor':'none', 'markeredgecolor':'black', 'markersize':3, 'linewidth':0.6},
                    saturation=1, orient=ort)
    
    filtered_data = pd.DataFrame(columns=data.columns)

    groups = data.groupby(dy)
    for name, group in groups:
        Q1 = group[dx].quantile(0.25)
        Q3 = group[dx].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        filtered_group = group[(group[dx] >= lower_bound) & (group[dx] <= upper_bound)]
        filtered_data = pd.concat([filtered_data, filtered_group])
    sns.stripplot(ax=ax, x=dx, y=dy, data=filtered_data, jitter=0.12, alpha=0.5,
                palette=stripplot_pal, edgecolor='none', size=3, orient=ort, zorder=0)
    
    group_means = data.groupby(dy)[dx].mean().reset_index()
    
    for i, (method, mean) in enumerate(group_means.itertuples(index=False)):
            ax.scatter(x=mean, y=i, s=20, color='#E86565', zorder=10)

    # 设置X轴和Y轴的标签
    ax.spines['bottom'].set_color('black')
    ax.spines['bottom'].set_linewidth(1.5)
    ax.spines['left'].set_color('black')
    ax.spines['left'].set_linewidth(1.5)

    # 刻度标签轴绘制
    locs = ax.get_yticks()
    new_locs = locs -0.2
    labels = [item.get_text() for item in ax.get_yticklabels()]
    new_labels = [label.replace('-ST', '').replace('-SP', '') for label in labels]
    ax.set_yticks(new_locs)
    ax.set_yticklabels(new_labels)

    ax.yaxis.label.set_visible(left_tick_spines)
    ax.spines['left'].set_visible(left_tick_spines)
    ax.xaxis.label.set_visible(False)

    ax.xaxis.set_major_formatter(FuncFormatter(custom_formatter))

    i_nbins = [4,4,4,4]
    ax.xaxis.set_major_locator(MaxNLocator(nbins=i_nbins[i]))

    ax.set_ylim(3.4,-0.8)
    ax.set_xlim(xlim)
    # 刻度调节
    plt.setp(ax.get_xticklabels(), rotation=0,rotation_mode="anchor")
    plt.setp(ax.get_yticklabels(), rotation=90,ha="center", rotation_mode="anchor")

    # 刻度值格式
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(True)
    labelsize = 10
    pad_x = 4
    pad_y = 4
    ax.tick_params(axis='y', which='both', colors='black', length=3, width=1, labelsize=labelsize, direction='out', pad=pad_y, left=left_tick_spines, right=False, labelright=False, labelleft=left_tick_spines)
    ax.tick_params(axis='x', which='both', colors='black', length=3, width=1, labelsize=labelsize, direction='out', pad=pad_x, top=False, bottom=True, labelbottom=True, labeltop=False)

    # text
    model_str='(ST)' if left_tick_spines else '(SP)'
    model_str='ST' if left_tick_spines else 'SP'
    left_start = 0.05
    down_start = 0.95
    ax.text(left_start, down_start, model_str, transform=ax.transAxes, fontsize=12, fontname='Times New Roman', verticalalignment='top')

    # 轴Label
    if left_spines and not bottom_spines:
        ax.set_ylabel('Method-R', fontsize=16, fontweight='normal', labelpad=4)
    elif not left_spines and not bottom_spines:
        ax.set_ylabel('Method-bias(m³/m³)', fontsize=16, fontweight='normal', labelpad=4)
    elif left_spines and bottom_spines:
        ax.set_ylabel('Method-RMSE(m³/m³)', fontsize=16, fontweight='normal', labelpad=4)
    elif not left_spines and bottom_spines:
        ax.set_ylabel('Method-ubRMSE(m³/m³)', fontsize=16, fontweight='normal', labelpad=4)
        
    ax.set_xlabel('')
    ax.grid(False)

create_2d_subplots(all_metrics_data)

### (数据处理) ST/SP产品站点指标统计(4*1箱线图)
用于STSP模型对比
注：使用的文件是xlsx文件，xlsx文件在最开始的部分生成

In [ ]:
import pandas as pd

# xlxs文件路径
file_paths = {
    'Original': r"D:\Data_Store\Dataset\Reconstruction\37SM\Original_station_metrics.xlsx",
    'PSC-FENet-ST':"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\STPSC-FENet_station_metrics.xlsx",
    'PSC-FENet-SP':"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SPPSC-FENet_station_metrics.xlsx",
    'RF': "D:\Data_Store\Dataset\Reconstruction\RF\RF_station_metrics.xlsx",
    'LSTM': "D:\Data_Store\Dataset\Reconstruction\LSTM\LSTM_station_metrics.xlsx",
    'ResNet-SP': "D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SPRESNET_station_metrics.xlsx",
    'UNet-SP': "D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SPUNET_station_metrics.xlsx",
    'ResNet-ST': "D:\Data_Store\Dataset\Reconstruction\ST\RESALL\STRESNET_station_metrics.xlsx",
    'UNet-ST': "D:\Data_Store\Dataset\Reconstruction\ST\RESALL\STUNET_station_metrics.xlsx",
    'VIT-SP': "D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SPVIT_station_metrics.xlsx",
    'VIT-ST': "D:\Data_Store\Dataset\Reconstruction\ST\RESALL\STVIT_station_metrics.xlsx",
    'Temporal': "D:\Data_Store\Dataset\Reconstruction\ST\RESALL\TEMPORAL_station_metrics.xlsx"
}

selected_model_names = ['PSC-FENet-ST', 'PSC-FENet-SP', 'ResNet-ST', 'ResNet-SP', 'UNet-ST', 'UNet-SP', 'VIT-ST', 'VIT-SP']

selected_file_paths = {name: paths for name, paths in file_paths.items() if name in selected_model_names}

all_metrics_data = pd.DataFrame()
labels = list(selected_file_paths.keys())
# 加载所有子表，第二行读取
for method, path in selected_file_paths.items():
    xls = pd.ExcelFile(path)
    for sheet_name in xls.sheet_names:
        df = pd.read_excel(path, sheet_name=sheet_name, header=0)
        df['Method'] = method
        all_metrics_data = pd.concat([all_metrics_data, df], ignore_index=True)
all_metrics_data['Method']

### (绘图) V2.0 4*1 箱线图

In [ ]:
import matplotlib
from matplotlib.font_manager import FontProperties
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.collections as clt
import ptitprince as pt
from matplotlib.ticker import FuncFormatter
from matplotlib.ticker import MaxNLocator

sns.set(style="whitegrid", font='Times New Roman')
matplotlib.rcParams['font.family'] = 'Times New Roman'
legend_font = FontProperties(family='Times New Roman', size=14)
font_prop = FontProperties(family='Times New Roman', size=16)

def custom_formatter(x, pos):
    if x == 0:
        return "0"
    else:
        str_x = "{:.15g}".format(x)
        if 'e' in str_x:
            base, exponent = str_x.split('e')
            base = base.rstrip('0').rstrip('.')
            exponent = exponent.replace('+', '').lstrip('0')
            if exponent:
                return f"{base}e{exponent}"
            else:
                return base
        else:
            return str_x
def clean_axis(ax):
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.tick_params(axis='x', which='both', length=0, labelbottom=False)
    ax.tick_params(axis='y', which='both', length=0, labelleft=False)
    ax.grid(False)

def create_2d_subplots(all_metrics_data, plt_name = None):
    sns.set(style="whitegrid", font='Times New Roman')
    matplotlib.rcParams['font.family'] = 'Times New Roman'
    matplotlib.rcParams['axes.labelweight'] = 'bold'

    # 创建图形对象
    fig = plt.figure(figsize=(18, 24))

    ratio_h = 4 # 竖直比例
    ratio_v = 2.5 # 水平比例

    absolute_dis_h = 0.25/ratio_h # 中央子图绝对大小（竖直）0.15
    absolute_dis_v = 1/ratio_v # 中央子图绝对大小（水平）0.2

    edge_h = 0.37 # 竖直边缘间隙
    edge_v = 0.1 # 水平边缘间隙

    mid_h = (1-edge_h*2-absolute_dis_h*4)/3   # 竖直间隙
    mid_v = 0 # 水平间隙……


    # 计算第一层图起点:上层子图  左起点、下起点、宽度比例、高度比例
    ax1 = fig.add_axes([edge_v, 1 - edge_h - absolute_dis_h, absolute_dis_v, absolute_dis_h])
    ax2 = fig.add_axes([edge_v, 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v, absolute_dis_h])
    ax3 = fig.add_axes([edge_v, 1 - edge_h - 3*absolute_dis_h - 2*mid_h, absolute_dis_v, absolute_dis_h])
    ax4 = fig.add_axes([edge_v, 1 - edge_h - 4*absolute_dis_h - 3*mid_h, absolute_dis_v, absolute_dis_h])

    axes = [ax1, ax2, ax3, ax4]

    metric_columns = ['PearsonR', 'Bias', 'RMSE', 'ubRMSE']
    for i, metric in enumerate(metric_columns):
        bottom_labels = True if i in [3] else False
        draw_subplot(axes[i], all_metrics_data[['Method', metric]], metric, bottom_labels=bottom_labels,i=i)

    fig.savefig(r"C:\Users\Administrator\Desktop\Draw\sat_original_statistics.jpeg", dpi=1000, format='jpeg', bbox_inches='tight')

    plt.show()

def draw_subplot(ax, data, metric, bottom_labels,i=None):

    min_val = data[metric].min()
    max_val = data[metric].max()
    range_val = max_val - min_val
    expand_val = range_val * 0.05

    min_val_expanded = min_val - expand_val
    max_val_expanded = max_val + expand_val

    dy = metric
    dx = 'Method'
    ort = "v"

    # Violin Color
    Violin_Color = ['#D1B8DC','#C293D8','#C5CEF5','#7790FB','#6EABAF','#20969E','#81BA9F','#3EAB78']
    all_colors = ['#AA7BC9','#AA7BC9','#4B6CFB','#4B6CFB','#048A93','#048A93','#26A269','#26A269']

    half_violinplot_pal = sns.color_palette(Violin_Color)
    stripplot_pal = sns.color_palette(all_colors)

    ax=pt.half_violinplot(ax=ax, x=dx, y=dy, data=data, palette=half_violinplot_pal, bw=0.3, cut=0., 
                        scale="area", width=0.8, inner=None, orient=ort, linewidth=1, alpha=0.7,order=selected_model_names)
    
    ax = sns.boxplot(ax=ax, x=dx, y=dy, data=data, color="black", width=0.25, zorder=10,
                    showcaps=True, 
                    boxprops={'facecolor':'none', 'edgecolor':'black', 'linewidth':0.8, 'zorder':10}, 
                    medianprops={'color':'black', 'linewidth':1.5}, 
                    whiskerprops={'color':'grey', 'linestyle':'--', 'linewidth':1.2}, 
                    capprops={'color':'black', 'linewidth':1}, 
                    flierprops={'marker':'o', 'markerfacecolor':'none', 'markeredgecolor':'black', 'markersize':3, 'linewidth':0.6},
                    saturation=1, orient=ort,order=selected_model_names)
    
    filtered_data = pd.DataFrame(columns=data.columns)

    groups = data.groupby(dx)
    for name, group in groups:
        Q1 = group[dy].quantile(0.25)
        Q3 = group[dy].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        filtered_group = group[(group[dy] >= lower_bound) & (group[dy] <= upper_bound)]
        filtered_data = pd.concat([filtered_data, filtered_group])

    sns.stripplot(ax=ax, x=dx, y=dy, data=filtered_data, jitter=0.1, alpha=0.5,
                palette=stripplot_pal, edgecolor='none', size=3, orient=ort, zorder=0,order=selected_model_names)
    
    group_means = data.groupby(dx)[dy].mean().reset_index()
    
    for _, (method, mean) in enumerate(group_means.itertuples(index=False)):
        if method in selected_model_names:
            ax.scatter(x=selected_model_names.index(method), y=mean, s=20, color='#E86565', zorder=10)
    
    overall_mean = data[dy].mean()
    ax.axhline(y=overall_mean, color='r', linestyle='--', label=f'Overall Mean: {overall_mean:.2f}')


    # 设置绘制范围
    ylim = [-0.5,1] if metric in ['PearsonR'] else (min_val_expanded, max_val_expanded)
    
    ax.set_ylim(ylim)
    # 设置X轴和Y轴的标签
    ax.spines['bottom'].set_color('black')
    ax.spines['bottom'].set_linewidth(1.5)
    ax.spines['left'].set_color('black')
    ax.spines['left'].set_linewidth(1.5)
    #ax.spines['top'].set_visible(False)
    #ax.spines['right'].set_visible(False)
    ax.spines['top'].set_color('black')
    ax.spines['top'].set_linewidth(1.5)
    ax.spines['right'].set_color('black')
    ax.spines['right'].set_linewidth(1.5)

    # 轴标签可视性
    if not bottom_labels:ax.tick_params(labelbottom=False)
    else: ax.set_xlabel(dx)
    ax.xaxis.label.set_visible(bottom_labels)

    # 轴刻度属性设置
    labelsize_x = 9
    labelsize_y = 10
    pad_x = 4
    pad_y = 4
    ax.tick_params(axis='y', which='both', colors='black', length=3, width=1, labelsize=labelsize_y, direction='in', pad=pad_y, left=True, right=False, labelright=False, labelleft=True)
    ax.tick_params(axis='x', which='both', colors='black', length=3, width=1.5, labelsize=labelsize_x, direction='in', pad=pad_x, top=True, bottom=True, labelbottom=bottom_labels, labeltop=False)
    
    # 轴Label
    if i==0:ax.set_ylabel('Method-R', fontsize=12, fontweight='normal', labelpad=4)
    if i==1:ax.set_ylabel('Method-bias(m³/m³)', fontsize=12, fontweight='normal', labelpad=4)
    if i==2:ax.set_ylabel('Method-RMSE(m³/m³)', fontsize=12, fontweight='normal', labelpad=4)
    if i==3:ax.set_ylabel('Method-ubRMSE(m³/m³)', fontsize=12, fontweight='normal', labelpad=4)

    # x轴刻度标签偏移
    locs = ax.get_xticks()
    new_locs = locs - 0.142  # 向左移动0.2单位
    labels = [item.get_text() for item in ax.get_xticklabels()]
    ax.set_xticks(new_locs)
    ax.set_xticklabels(selected_model_names)

    # x轴标签隐藏
    ax.set_xlabel('')
    ax.grid(False)

    # y轴刻度调整
    ax.yaxis.set_major_formatter(FuncFormatter(custom_formatter))
    i_nbins = [4,4,4,4]
    ax.yaxis.set_major_locator(MaxNLocator(nbins=i_nbins[i]))

create_2d_subplots(all_metrics_data)

### (数据处理) ST/SP产品站点指标统计(4*1 4*8箱线图)
用于STSP模型对比
注：使用的文件是xlsx文件，xlsx文件在最开始的部分生成

In [ ]:
import pandas as pd

# xlxs文件路径
file_paths = {
    'Original': r"D:\Data_Store\Dataset\Reconstruction\37SM\Original_station_metrics.xlsx",
    'PSC(ST)':"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\STPSC-FENet_station_metrics.xlsx",
    'PSC(SP)':"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SPPSC-FENet_station_metrics.xlsx",
    'PSC(ST32)':"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\STPSC-FENet36_station_metrics.xlsx",
    'PSC(SP32)':"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SPPSC-FENet36_station_metrics.xlsx",
    'RF': "D:\Data_Store\Dataset\Reconstruction\RF\RF_station_metrics.xlsx",
    'LSTM': "D:\Data_Store\Dataset\Reconstruction\LSTM\LSTM_station_metrics.xlsx",
    'ResNet-SP': "D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SPRESNET_station_metrics.xlsx",
    'UNet-SP': "D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SPUNET_station_metrics.xlsx",
    'ResNet-ST': "D:\Data_Store\Dataset\Reconstruction\ST\RESALL\STRESNET_station_metrics.xlsx",
    'UNet-ST': "D:\Data_Store\Dataset\Reconstruction\ST\RESALL\STUNET_station_metrics.xlsx",
    'VIT-SP': "D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SPVIT_station_metrics.xlsx",
    'VIT-ST': "D:\Data_Store\Dataset\Reconstruction\ST\RESALL\STVIT_station_metrics.xlsx",
    'Temporal': "D:\Data_Store\Dataset\Reconstruction\ST\RESALL\TEMPORAL_station_metrics.xlsx"
}

selected_model_names = ['PSC(ST)', 'PSC(SP)', 'PSC(ST32)', 'PSC(SP32)', 'ResNet-ST', 'ResNet-SP', 'UNet-ST', 'UNet-SP', 'VIT-ST', 'VIT-SP']

selected_file_paths = {name: paths for name, paths in file_paths.items() if name in selected_model_names}

all_metrics_data = pd.DataFrame()
labels = list(selected_file_paths.keys())
# 加载所有子表，第二行读取
for method, path in selected_file_paths.items():
    xls = pd.ExcelFile(path)
    for sheet_name in xls.sheet_names:
        df = pd.read_excel(path, sheet_name=sheet_name, header=0)
        df['Method'] = method
        all_metrics_data = pd.concat([all_metrics_data, df], ignore_index=True)
all_metrics_data['Method']

### (绘图) V2.0 4*1 4*8 箱线图

In [ ]:
import matplotlib
from matplotlib.font_manager import FontProperties
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.collections as clt
import ptitprince as pt
from matplotlib.ticker import FuncFormatter
from matplotlib.ticker import MaxNLocator

sns.set(style="whitegrid", font='Times New Roman')
matplotlib.rcParams['font.family'] = 'Times New Roman'
legend_font = FontProperties(family='Times New Roman', size=14)
font_prop = FontProperties(family='Times New Roman', size=16)

def custom_formatter(x, pos):
    if x == 0:
        return "0"
    else:
        str_x = "{:.15g}".format(x)
        if 'e' in str_x:
            base, exponent = str_x.split('e')
            base = base.rstrip('0').rstrip('.')
            exponent = exponent.replace('+', '').lstrip('0')
            if exponent:
                return f"{base}e{exponent}"
            else:
                return base
        else:
            return str_x
def clean_axis(ax):
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.tick_params(axis='x', which='both', length=0, labelbottom=False)
    ax.tick_params(axis='y', which='both', length=0, labelleft=False)
    ax.grid(False)

def create_2d_subplots(all_metrics_data, plt_name = None):
    sns.set(style="whitegrid", font='Times New Roman')
    matplotlib.rcParams['font.family'] = 'Times New Roman'
    matplotlib.rcParams['axes.labelweight'] = 'bold'

    # 创建图形对象
    fig = plt.figure(figsize=(18, 24))

    ratio_h = 4 # 竖直比例
    ratio_v = 2.5 # 水平比例

    absolute_dis_h = 0.25/ratio_h # 中央子图绝对大小（竖直）0.15
    absolute_dis_v = 1/ratio_v # 中央子图绝对大小（水平）0.2

    edge_h = 0.37 # 竖直边缘间隙
    edge_v = 0.1 # 水平边缘间隙

    mid_h = (1-edge_h*2-absolute_dis_h*4)/3   # 竖直间隙
    mid_v = 0 # 水平间隙……


    # 计算第一层图起点:上层子图  左起点、下起点、宽度比例、高度比例
    ax1 = fig.add_axes([edge_v, 1 - edge_h - absolute_dis_h, absolute_dis_v, absolute_dis_h])
    ax2 = fig.add_axes([edge_v, 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v, absolute_dis_h])
    ax3 = fig.add_axes([edge_v, 1 - edge_h - 3*absolute_dis_h - 2*mid_h, absolute_dis_v, absolute_dis_h])
    ax4 = fig.add_axes([edge_v, 1 - edge_h - 4*absolute_dis_h - 3*mid_h, absolute_dis_v, absolute_dis_h])

    axes = [ax1, ax2, ax3, ax4]

    metric_columns = ['PearsonR', 'Bias', 'RMSE', 'ubRMSE']
    for i, metric in enumerate(metric_columns):
        bottom_labels = True if i in [3] else False
        draw_subplot(axes[i], all_metrics_data[['Method', metric]], metric, bottom_labels=bottom_labels,i=i)

    fig.savefig(r"C:\Users\Administrator\Desktop\Draw\sat_original_statistics.jpeg", dpi=1000, format='jpeg', bbox_inches='tight')

    plt.show()

def draw_subplot(ax, data, metric, bottom_labels,i=None):

    min_val = data[metric].min()
    max_val = data[metric].max()
    range_val = max_val - min_val
    expand_val = range_val * 0.05

    min_val_expanded = min_val - expand_val
    max_val_expanded = max_val + expand_val

    dy = metric
    dx = 'Method'
    ort = "v"

    # Violin Color
    Violin_Color = ['#D1B8DC','#C293D8','#C5CEF5','#7790FB','#6EABAF','#20969E','#81BA9F','#3EAB78','#77BC00','#558700']
    all_colors = ['#AA7BC9','#AA7BC9','#4B6CFB','#4B6CFB','#048A93','#048A93','#26A269','#26A269','#73B700','#73B700']

    half_violinplot_pal = sns.color_palette(Violin_Color)
    stripplot_pal = sns.color_palette(all_colors)

    ax=pt.half_violinplot(ax=ax, x=dx, y=dy, data=data, palette=half_violinplot_pal, bw=0.3, cut=0., 
                        scale="area", width=1.05, inner=None, orient=ort, linewidth=1, alpha=0.7,order=selected_model_names)
    
    ax = sns.boxplot(ax=ax, x=dx, y=dy, data=data, color="black", width=0.25, zorder=10,
                    showcaps=True, 
                    boxprops={'facecolor':'none', 'edgecolor':'black', 'linewidth':0.8, 'zorder':10}, 
                    medianprops={'color':'black', 'linewidth':1.5}, 
                    whiskerprops={'color':'grey', 'linestyle':'--', 'linewidth':1.2}, 
                    capprops={'color':'black', 'linewidth':1}, 
                    flierprops={'marker':'o', 'markerfacecolor':'none', 'markeredgecolor':'black', 'markersize':3, 'linewidth':0.6},
                    saturation=1, orient=ort,order=selected_model_names)
    
    filtered_data = pd.DataFrame(columns=data.columns)

    groups = data.groupby(dx)
    for name, group in groups:
        Q1 = group[dy].quantile(0.25)
        Q3 = group[dy].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        filtered_group = group[(group[dy] >= lower_bound) & (group[dy] <= upper_bound)]
        filtered_data = pd.concat([filtered_data, filtered_group])

    sns.stripplot(ax=ax, x=dx, y=dy, data=filtered_data, jitter=0.1, alpha=0.5,
                palette=stripplot_pal, edgecolor='none', size=3, orient=ort, zorder=0,order=selected_model_names)
    
    group_means = data.groupby(dx)[dy].mean().reset_index()
    
    for _, (method, mean) in enumerate(group_means.itertuples(index=False)):
        if method in selected_model_names:
            ax.scatter(x=selected_model_names.index(method), y=mean, s=20, color='#E86565', zorder=10)
    
    overall_mean = data[dy].mean()
    ax.axhline(y=overall_mean, color='r', linestyle='--', label=f'Overall Mean: {overall_mean:.2f}')
    print(overall_mean)

    # 设置绘制范围
    ylim = [-0.5,1] if metric in ['PearsonR'] else (min_val_expanded, max_val_expanded)
    
    ax.set_ylim(ylim)
    # 设置X轴和Y轴的标签
    ax.spines['bottom'].set_color('black')
    ax.spines['bottom'].set_linewidth(1.5)
    ax.spines['left'].set_color('black')
    ax.spines['left'].set_linewidth(1.5)
    #ax.spines['top'].set_visible(False)
    #ax.spines['right'].set_visible(False)
    ax.spines['top'].set_color('black')
    ax.spines['top'].set_linewidth(1.5)
    ax.spines['right'].set_color('black')
    ax.spines['right'].set_linewidth(1.5)

    # 轴标签可视性

    if not bottom_labels:ax.tick_params(labelbottom=False)
    else: ax.set_xlabel(dx)
    ax.xaxis.label.set_visible(bottom_labels)

    # 轴刻度属性设置
    labelsize_x = 10
    labelsize_y = 10
    pad_x = 4
    pad_y = 4
    ax.tick_params(axis='y', which='both', colors='black', length=3, width=1, labelsize=labelsize_y, direction='in', pad=pad_y, left=True, right=False, labelright=False, labelleft=True)
    ax.tick_params(axis='x', which='both', colors='black', length=3, width=1.5, labelsize=labelsize_x, direction='in', pad=pad_x, top=True, bottom=True, labelbottom=bottom_labels, labeltop=False)
    '''
    if bottom_labels:
        labels = ax.get_xticklabels()
        for i in range(2):
            labels[i].set_fontsize(7)
    ''' 

    # 轴Label
    if i==0:ax.set_ylabel('R', fontsize=12, fontweight='normal', labelpad=4)
    if i==1:ax.set_ylabel('bias(m³/m³)', fontsize=12, fontweight='normal', labelpad=4)
    if i==2:ax.set_ylabel('RMSE(m³/m³)', fontsize=12, fontweight='normal', labelpad=4)
    if i==3:ax.set_ylabel('ubRMSE(m³/m³)', fontsize=12, fontweight='normal', labelpad=4)

    # x轴刻度标签偏移
    locs = ax.get_xticks()
    new_locs = locs - 0.142  # 向左移动0.2单位
    labels = [item.get_text() for item in ax.get_xticklabels()]
    ax.set_xticks(new_locs)
    ax.set_xticklabels(selected_model_names)

    # x轴标签隐藏
    ax.set_xlabel('')
    ax.grid(False)

    # y轴刻度调整
    ax.yaxis.set_major_formatter(FuncFormatter(custom_formatter))
    i_nbins = [4,4,4,4]
    ax.yaxis.set_major_locator(MaxNLocator(nbins=i_nbins[i]))

create_2d_subplots(all_metrics_data)

### (数据处理) 不同模型、全时刻指标统计(柱状折线图)

In [ ]:
import pandas as pd
time_points = ['0', '3', '6', '9', '12', '15', '18', '21']
network_names = ['Shiquanhe', 'Ali', 'Maqu', 'Naqu', 'CTP']
file_paths = {
    'PSC-FENet-ST': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\STPSCFENet_station_time_metrics.xlsx",
    'PSC-FENet-SP': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SPPSCFENet_station_time_metrics.xlsx",
    'PSC-FENet-ST36': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\STPSCFENet36_station_time_metrics.xlsx",
    'PSC-FENet-SP36': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SPPSCFENet36_station_time_metrics.xlsx",
    'ResNet-ST': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\STRESNET_station_time_metrics.xlsx",
    'ResNet-SP': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SPRESNET_station_time_metrics.xlsx",
    'UNet-ST': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\STUNET_station_time_metrics.xlsx",
    'UNet-SP': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SPUNET_station_time_metrics.xlsx",
    'VIT-ST': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\STVIT_station_time_metrics.xlsx",
    'VIT-SP': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SPVIT_station_time_metrics.xlsx",
    'RF': r"D:\Data_Store\Dataset\Reconstruction\RF\RF_station_time_metrics.xlsx",
    'LSTM': r"D:\Data_Store\Dataset\Reconstruction\LSTM\LSTM_station_time_metrics.xlsx",
    'Temporal': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\TEMPORAL_station_time_metrics.xlsx"
}


def create_combined_data(time_points, network_names, file_paths):
    all_data = {model: {time: {network: pd.DataFrame() for network in network_names} for time in time_points} for model in file_paths.keys()}
    
    for model, file_path in file_paths.items():
        xls = pd.ExcelFile(file_path)
        for time_point in time_points:
            networks_data = {network: pd.DataFrame() for network in network_names}
            for sheet_name in xls.sheet_names:
                if f"_{time_point}h" in sheet_name:
                    df = xls.parse(sheet_name)
                    split_data = df['站点名'].str.rsplit('_', n=2, expand=True)
                    df['point_name'] = split_data[0]
                    df['lon'] = split_data[1].astype(float)
                    df['lat'] = split_data[2].astype(float)
                    df_renamed = df.rename(columns={'PearsonR': 'R'})
                    df_sorted = df_renamed.sort_values(by=['lat', 'lon'], ascending=[False, True])
                    best_records = df_sorted.groupby(['lat', 'lon'], as_index=False).apply(lambda x: x.loc[x['R'].idxmax()]).reset_index(drop=True)
                    best_records = best_records[['point_name', 'lon', 'lat', 'RMSE', 'ubRMSE', 'R', 'Bias']]
                    for network in network_names:
                        if network in sheet_name:
                            networks_data[network] = pd.concat([networks_data[network], best_records], ignore_index=True)
            for network in network_names:
                all_data[model][time_point][network] = pd.concat([all_data[model][time_point][network], networks_data[network]], ignore_index=True)

    return all_data

def calculate_average_metrics(all_data):
    averages_data = {}
    for model in all_data:
        averages_data[model] = {}
        for time_point in all_data[model]:
            combined_df = pd.DataFrame()
            for network in all_data[model][time_point]:
                df = all_data[model][time_point][network]
                if not df.empty:
                    # 平均值
                    df_metrics = df[['RMSE', 'ubRMSE', 'R', 'Bias']]
                    averages = df_metrics.mean().to_frame().T 
                    averages['Network'] = network 
                    combined_df = pd.concat([combined_df, averages], ignore_index=True)
                    combined_df = combined_df[['Network', 'RMSE', 'ubRMSE', 'R', 'Bias']]
            averages_data[model][time_point] = combined_df
    return averages_data

def calculate_all_networks_average_metrics(all_data):  
    all_networks_averages = {}

    for model in all_data:
        all_networks_averages[model] = {}
        combined_df = pd.DataFrame()
        data_for_all_time_points = {network: pd.DataFrame() for network in all_data[model][next(iter(all_data[model]))]}
        for time_point in all_data[model]:
            for network in all_data[model][time_point]:
                df = all_data[model][time_point][network]
                if not df.empty:
                    data_for_all_time_points[network] = pd.concat([data_for_all_time_points[network], df], ignore_index=True)
        for network, df in data_for_all_time_points.items():
            if not df.empty:
                df_metrics = df[['RMSE', 'ubRMSE', 'R', 'Bias']]
                averages = df_metrics.mean().to_frame().T
                averages['Network'] = network
                combined_df = pd.concat([combined_df, averages], ignore_index=True)
        combined_df = combined_df[['Network', 'RMSE', 'ubRMSE', 'R', 'Bias']]
        all_networks_averages[model] = combined_df

    return all_networks_averages


all_data = create_combined_data(time_points, network_names, file_paths)
average_metrics = calculate_average_metrics(all_data)
all_networks_average_metrics = calculate_all_networks_average_metrics(all_data)

'''
df_example = all_data['UNet-SP']['3']['Ali'] 访问模式
model = 'KPNet-ST'
time_point = '18'
print(average_metrics[model][time_point]) 访问模式
model = 'KPNet-ST'
all_networks_average_metrics[model]
'''
# 设置显示浮点数为四位小数
pd.options.display.float_format = '{:,.4f}'.format

# 指定网络的新顺序
new_order = ['Maqu', 'Naqu', 'Ali', 'Shiquanhe', 'CTP']
new_columns_order = ['R', 'Bias', 'RMSE', 'ubRMSE']

# 遍历字典并打印每个调整后的DataFrame
for model, df in all_networks_average_metrics.items():
    # 按新顺序重新索引网络，并选择新的列顺序
    df_ordered = df.set_index('Network').reindex(new_order).reset_index()[['Network'] + new_columns_order]

    print(f"{model} Performance Metrics:")
    print(df_ordered)
    print("\n")

In [ ]:
all_networks_average_metrics


### (数据处理) 选择模型、全时刻指标统计(柱状折线图)

In [ ]:
import pandas as pd

def create_combined_data(time_points, network_names, file_paths):
    all_data = {model: {time: {network: pd.DataFrame() for network in network_names} for time in time_points} for model in file_paths.keys()}
    
    for model, file_path in file_paths.items():
        xls = pd.ExcelFile(file_path)
        for time_point in time_points:
            networks_data = {network: pd.DataFrame() for network in network_names}
            for sheet_name in xls.sheet_names:
                if f"_{time_point}h" in sheet_name:
                    df = xls.parse(sheet_name)
                    split_data = df['站点名'].str.rsplit('_', n=2, expand=True)
                    df['point_name'] = split_data[0]
                    df['lon'] = split_data[1].astype(float)
                    df['lat'] = split_data[2].astype(float)
                    df_renamed = df.rename(columns={'PearsonR': 'R'})
                    df_sorted = df_renamed.sort_values(by=['lat', 'lon'], ascending=[False, True])
                    best_records = df_sorted.groupby(['lat', 'lon'], as_index=False).apply(lambda x: x.loc[x['R'].idxmax()]).reset_index(drop=True)
                    best_records = best_records[['point_name', 'lon', 'lat', 'RMSE', 'ubRMSE', 'R', 'Bias']]
                    for network in network_names:
                        if network in sheet_name:
                            networks_data[network] = pd.concat([networks_data[network], best_records], ignore_index=True)
            for network in network_names:
                all_data[model][time_point][network] = pd.concat([all_data[model][time_point][network], networks_data[network]], ignore_index=True)

    return all_data

def calculate_average_metrics(all_data):
    averages_data = {}
    for model in all_data:
        averages_data[model] = {}
        for time_point in all_data[model]:
            combined_df = pd.DataFrame()
            for network in all_data[model][time_point]:
                df = all_data[model][time_point][network]
                if not df.empty:
                    # 平均值
                    df_metrics = df[['RMSE', 'ubRMSE', 'R', 'Bias']]
                    averages = df_metrics.mean().to_frame().T 
                    averages['Network'] = network 
                    combined_df = pd.concat([combined_df, averages], ignore_index=True)
                    combined_df = combined_df[['Network', 'RMSE', 'ubRMSE', 'R', 'Bias']]
            averages_data[model][time_point] = combined_df
    return averages_data

def calculate_all_networks_average_metrics(all_data):  
    all_networks_averages = {}

    for model in all_data:
        all_networks_averages[model] = {}
        combined_df = pd.DataFrame()
        data_for_all_time_points = {network: pd.DataFrame() for network in all_data[model][next(iter(all_data[model]))]}
        for time_point in all_data[model]:
            for network in all_data[model][time_point]:
                df = all_data[model][time_point][network]
                if not df.empty:
                    data_for_all_time_points[network] = pd.concat([data_for_all_time_points[network], df], ignore_index=True)
        for network, df in data_for_all_time_points.items():
            if not df.empty:
                df_metrics = df[['RMSE', 'ubRMSE', 'R', 'Bias']]
                averages = df_metrics.mean().to_frame().T
                averages['Network'] = network
                combined_df = pd.concat([combined_df, averages], ignore_index=True)
        combined_df = combined_df[['Network', 'RMSE', 'ubRMSE', 'R', 'Bias']]
        all_networks_averages[model] = combined_df

    return all_networks_averages



time_points = ['0', '3', '6', '9', '12', '15', '18', '21']
network_names = ['Shiquanhe', 'Ali', 'Maqu', 'Naqu', 'CTP']
file_paths = {
    'PSC-FENet-ST': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\STPSCFENet_station_time_metrics.xlsx",
    'PSC-FENet-SP': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SPPSCFENet_station_time_metrics.xlsx",
    'ResNet-ST': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\STRESNET_station_time_metrics.xlsx",
    'ResNet-SP': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SPRESNET_station_time_metrics.xlsx",
    'UNet-ST': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\STUNET_station_time_metrics.xlsx",
    'UNet-SP': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SPUNET_station_time_metrics.xlsx",
    'VIT-ST': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\STVIT_station_time_metrics.xlsx",
    'VIT-SP': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SPVIT_station_time_metrics.xlsx",
    'RF': r"D:\Data_Store\Dataset\Reconstruction\RF\RF_station_time_metrics.xlsx",
    'LSTM': r"D:\Data_Store\Dataset\Reconstruction\LSTM\LSTM_station_time_metrics.xlsx",
    'Temporal': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\TEMPORAL_station_time_metrics.xlsx"
}

#selected_model_names = ['PSC-FENet-ST', 'PSC-FENet-SP',  'LSTM', 'RF', 'Temporal']
selected_model_names = ['PSC-FENet-ST', 'PSC-FENet-SP', 'ResNet-ST',  'ResNet-SP', 'UNet-ST', 'UNet-SP', 'VIT-ST', 'VIT-SP', 'LSTM', 'RF', 'Temporal']
selected_reconstruction_paths = {name: paths for name, paths in file_paths.items() if name in selected_model_names}
all_data = create_combined_data(time_points, network_names, selected_reconstruction_paths)
average_metrics = calculate_average_metrics(all_data)
all_networks_average_metrics = calculate_all_networks_average_metrics(all_data)

'''
df_example = all_data['UNet-SP']['3']['Ali'] 访问模式
model = 'KPNet-ST'
time_point = '18'
print(average_metrics[model][time_point]) 访问模式
model = 'KPNet-ST'
all_networks_average_metrics[model]
'''

In [ ]:
import pandas as pd

time_points = ['0', '3', '6', '9', '12', '15', '18', '21']
model_names = ['PSC-FENet-ST', 'PSC-FENet-SP', 'ResNet-ST', 'ResNet-SP', 'UNet-ST', 'UNet-SP', 'VIT-ST', 'VIT-SP', 'LSTM', 'RF', 'Temporal']

# 准备一个字典来存储每个度量的DataFrame
metrics = ['RMSE', 'ubRMSE', 'R', 'Bias']
metric_dfs = {metric: pd.DataFrame(index=time_points, columns=model_names) for metric in metrics}

# 聚合数据并计算平均值
for model in model_names:
    for time in time_points:
        data_frames = [all_data[model][time][network] for network in all_data[model][time] if not all_data[model][time][network].empty]
        if data_frames:
            combined_df = pd.concat(data_frames)
            for metric in metrics:
                metric_dfs[metric].loc[time, model] = combined_df[metric].mean()
# 创建Excel写入器
excel_path = 'C:/Users/Administrator/Desktop/4_metrics.xlsx'  # 修改路径为实际用户名
writer = pd.ExcelWriter(excel_path, engine='xlsxwriter')

# 将每个度量的DataFrame写入不同的工作表
for metric, df in metric_dfs.items():
    df.to_excel(writer, sheet_name=metric)

# 保存并关闭Excel文件
writer.save()


### (绘图) V1.0 柱状折线图 4*2 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import Normalize
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.collections import PolyCollection
import matplotlib.colors as mcolors
from scipy.stats import gaussian_kde
from matplotlib.colors import to_rgba
from matplotlib.font_manager import FontProperties
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import colorsys
def aggregate_data(all_data, time_point=None):
    data_for_plot = {}
    for model in all_data:
        combined_df = pd.DataFrame()  # 聚合network和time
        time_points = [time_point] if time_point else all_data[model].keys()
        for tp in time_points:
            for network in all_data[model][tp]:
                df = all_data[model][tp][network]
                if not df.empty:
                    combined_df = pd.concat([combined_df, df], ignore_index=True)
        if not combined_df.empty:
            data_for_plot[model] = combined_df
    return data_for_plot
def custom_formatter(x, pos):
    if x == 0:
        return "0"
    else:
        str_x = "{:.15g}".format(x)
        if 'e' in str_x:
            base, exponent = str_x.split('e')
            base = base.rstrip('0').rstrip('.')
            exponent = exponent.replace('+', '').lstrip('0')
            if exponent:
                return f"{base}e{exponent}"
            else:
                return base
        else:
            return str_x
def adjust_color(color, brightness_factor):
    rgb = mcolors.hex2color(color)
    hls = colorsys.rgb_to_hls(*rgb)
    new_hls = (hls[0], max(0, min(1, brightness_factor * hls[1])), hls[2])
    # 将调整后的HLS颜色转换回RGB，然后转换为HEX
    new_rgb = colorsys.hls_to_rgb(*new_hls)
    return mcolors.to_hex(new_rgb)
def create_subplots(all_data):
    # 创建图形对象,绘制顺序依赖于fig定义顺序
    legend_font = FontProperties(family='Times New Roman', size=12)
    time_points = ['0', '3', '6', '9', '12', '15', '18', '21']
    colors_third = ['#590d22', '#ff4d6d', '#ffccd5']
    colors_th = ['#590d22', '#c9184a', '#ff8fa3']
    colors_gradient = ['#caf0f8', '#e9f5db']
    fig = plt.figure(figsize=(24, 24))

    ratio_h = 6.7 # 竖直比例 3
    ratio_v = 5.1 # 水平比例

    absolute_dis_h = 0.7/ratio_h # 中央子图绝对大小（竖直）0.7
    absolute_dis_v = 0.7/ratio_v # 中央子图绝对大小（水平）

    edge_h = 0.3803 # 竖直边缘间隙
    edge_v = 0.23 # 水平边缘间隙

    mid_h = (1-edge_h*2-absolute_dis_h*2)/1 # 竖直间隙,可以通过竖直边缘间隙调节
    mid_v = (1-edge_v*2-absolute_dis_v*4)/3 # 水平间隙……

    right_offset_first = 0 # 第一层向右偏移量
    right_offset_four = 0.05 # 第一层向右偏移量

    # 计算第一层图起点:上层子图  左起点、下起点、宽度比例、高度比例 4*2
    ax1 = fig.add_axes([edge_v + right_offset_first, 1 - edge_h - absolute_dis_h, absolute_dis_v, absolute_dis_h])
    ax2 = fig.add_axes([edge_v + right_offset_first + absolute_dis_v + mid_v, 1 - edge_h - absolute_dis_h, absolute_dis_v, absolute_dis_h])
    ax3 = fig.add_axes([edge_v + right_offset_first + 2*(absolute_dis_v + mid_v), 1 - edge_h - absolute_dis_h, absolute_dis_v, absolute_dis_h])
    ax4 = fig.add_axes([edge_v + right_offset_first + 3*(absolute_dis_v + mid_v), 1 - edge_h - absolute_dis_h, absolute_dis_v, absolute_dis_h])

    ax5 = fig.add_axes([edge_v + right_offset_first, 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v, absolute_dis_h])
    ax6 = fig.add_axes([edge_v + right_offset_first + 1*(absolute_dis_v + mid_v), 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v, absolute_dis_h])
    ax7 = fig.add_axes([edge_v + right_offset_first + 2*(absolute_dis_v + mid_v), 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v, absolute_dis_h])
    ax8 = fig.add_axes([edge_v + right_offset_first + 3*(absolute_dis_v + mid_v), 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v, absolute_dis_h])

    draw_subplot(ax1, all_data, time_point=time_points[0],top_spines=False,right_spines=False,bottom_spines=True,left_spines=True,colors_third = colors_third, start_color_rmse = colors_gradient[0], start_color_ubrmse =  colors_gradient[1],colors_th=colors_th)
    draw_subplot(ax2, all_data, time_point=time_points[1],top_spines=False,right_spines=False,bottom_spines=True,left_spines=False,colors_third = colors_third, start_color_rmse = colors_gradient[0], start_color_ubrmse =  colors_gradient[1],colors_th=colors_th)
    draw_subplot(ax3, all_data, time_point=time_points[2],top_spines=False,right_spines=False,bottom_spines=True,left_spines=False,colors_third = colors_third, start_color_rmse = colors_gradient[0], start_color_ubrmse =  colors_gradient[1],colors_th=colors_th)
    draw_subplot(ax4, all_data, time_point=time_points[3],top_spines=False,right_spines=True,bottom_spines=True,left_spines=False,colors_third = colors_third, start_color_rmse = colors_gradient[0], start_color_ubrmse =  colors_gradient[1],colors_th=colors_th)

    draw_subplot(ax5, all_data, time_point=time_points[4],top_spines=True,right_spines=False,bottom_spines=False,left_spines=True,down=True,colors_third = colors_third, start_color_rmse = colors_gradient[0], start_color_ubrmse =  colors_gradient[1],colors_th=colors_th)
    draw_subplot(ax6, all_data, time_point=time_points[5],top_spines=True,right_spines=False,bottom_spines=False,left_spines=False,down=True,colors_third = colors_third, start_color_rmse = colors_gradient[0], start_color_ubrmse =  colors_gradient[1],colors_th=colors_th)
    draw_subplot(ax7, all_data, time_point=time_points[6],top_spines=True,right_spines=False,bottom_spines=False,left_spines=False,down=True,colors_third = colors_third, start_color_rmse = colors_gradient[0], start_color_ubrmse =  colors_gradient[1],colors_th=colors_th)
    draw_subplot(ax8, all_data, time_point=time_points[7],top_spines=True,right_spines=True,bottom_spines=False,left_spines=False,down=True,colors_third = colors_third, start_color_rmse = colors_gradient[0], start_color_ubrmse =  colors_gradient[1],colors_th=colors_th)
    
    colors_rmse = [adjust_color(colors_gradient[0], 1 - 0.02 * i) for i in range(11)]
    colors_ubrmse = [adjust_color(colors_gradient[1], 1 - 0.02 * i) for i in range(11)]
    gradient_patch_rmse = mpatches.Patch(color=colors_rmse[5], label='RMSE')
    gradient_patch_ubrmse = mpatches.Patch(color=colors_ubrmse[5], label='ubRMSE')
    line_r = mlines.Line2D([], [], color='#bc6c25', linestyle='--', marker='X', markeredgecolor='lightgrey', markersize=10, label='R', alpha=0.5)
    line_bias = mlines.Line2D([], [], color='purple', linestyle='--', marker='P', markeredgecolor='grey', markersize=10, label='Bias', alpha=0.5)
    first_legend_handles = [gradient_patch_rmse, gradient_patch_ubrmse, line_r, line_bias]
    plt.figlegend(handles=first_legend_handles, labels=[h.get_label() for h in first_legend_handles], 
                  loc='upper center', ncol=2, bbox_to_anchor=(0.385, edge_h+0.005), fancybox=False, 
                  shadow=False, frameon=False, prop=legend_font,
                  borderpad=1, labelspacing=0.8,handletextpad=0.5, handlelength=1.5)

    colors_top3 = ['#590d22', '#ff4d6d', '#ffccd5']  # 用于条形图
    color_X = ['#8A3C23', '#DD5C48', '#D8B08D']  # 用于'X'标记的散点图
    color_PLUS = ['#750266', '#B82B76', '#BF65AA']  # 用于'+'标记的散点图
    top3_labels = ['1st', '2nd', '3rd']
    top3_handles = []
    for i in range(3):
        bar_handle = mpatches.Patch(color=colors_top3[i], label=' ',alpha =0.8)
        scatter_handle_X = mlines.Line2D([], [], color=color_X[i], marker='X', linestyle='None', markersize=10, markeredgecolor='grey', label=top3_labels[i], linewidth=2)  # 实际标签
        scatter_handle_PLUS = mlines.Line2D([], [], color=color_PLUS[i], marker='P', linestyle='None', markersize=10, markeredgecolor='grey', label=' ', linewidth=2)
        top3_handles.extend([bar_handle, scatter_handle_X, scatter_handle_PLUS])

    plt.figlegend(handles=top3_handles, labels=[h.get_label() for h in top3_handles], 
                  loc='upper center', ncol=3, bbox_to_anchor=(0.29-0.004, edge_h+0.005), 
                  fancybox=False, shadow=False, frameon=False, prop=legend_font,
                  borderpad=1, labelspacing=0,handletextpad=0.5, handlelength=1.5)
    
    fig.text(0.227, 0.493, 'RMSE', ha='left', size=12, fontproperties=legend_font, rotation=90)
    fig.text(0.2168, 0.488, 'ubRMSE', ha='left', size=12, fontproperties=legend_font, rotation=90)
    fig.text(0.7685, 0.488, 'PearsonR', ha='left', size=12, fontproperties=legend_font, rotation=90)
    fig.text(0.7765, 0.496, 'bias', ha='left', size=12, fontproperties=legend_font, rotation=90)

    fig.savefig(r"C:\Users\Administrator\Desktop\Draw\Original_eval.jpeg", dpi=1000, format='jpeg', bbox_inches='tight')
    plt.show()

def draw_subplot(ax, all_data, time_point=None, width=0.41, start_color_rmse = '#caf0f8', start_color_ubrmse = '#e9f5db',Gradation_degree = 0.02,
                 top_spines=True, right_spines=True, bottom_spines=True, left_spines=True,colors_third = None,colors_th=None,
                 down=False):

    sns.set(style="whitegrid", font='Times New Roman')
    matplotlib.rcParams['font.family'] = 'Times New Roman'
    matplotlib.rcParams['axes.labelweight'] = 'bold'
    font_prop = FontProperties(family='Times New Roman', size=10)

    metrics = ['RMSE', 'ubRMSE', 'R', 'Bias']

    data_model = aggregate_data(all_data, time_point)
    ax1 = ax  # RMSE柱状图
    ax2 = ax.twinx()  # ubRMSE柱状图
    ax3 = ax.twinx()  # R散点图
    ax4 = ax.twinx()  # Bias散点图

    x = np.arange(len(data_model.keys()))

    x1 = [i - width/2 for i in x]
    x2 = [i + width/2 for i in x]
    xx = list(data_model.keys())


    # 计算指标
    rmse_values = [data_model[model]['RMSE'].mean() for model in data_model]
    ubrmse_values = [data_model[model]['ubRMSE'].mean() for model in data_model]
    r_values = [data_model[model]['R'].mean() for model in data_model]
    bias_values = [data_model[model]['Bias'].mean() for model in data_model]

    rmse_errors = [(data_model[model]['RMSE'].quantile(0.75) - data_model[model]['RMSE'].quantile(0.25)) for model in data_model]
    ubrmse_errors = [(data_model[model]['ubRMSE'].quantile(0.75) - data_model[model]['ubRMSE'].quantile(0.25)) for model in data_model]

    # 前三索引
    top3_rmse_indices = sorted(range(len(rmse_values)), key=lambda i: rmse_values[i])[:3]
    top3_ubrmse_indices = sorted(range(len(ubrmse_values)), key=lambda i: ubrmse_values[i])[:3]
    top3_r_indices = sorted(range(len(r_values)), key=lambda i: r_values[i], reverse=True)[:3]
    top3_bias_indices = sorted(range(len(bias_values)), key=lambda i: abs(bias_values[i]))[:3]

    # 绘制RMSE柱状图以及误差线
    colors_rmse = [adjust_color(start_color_rmse, 1 - Gradation_degree * i) for i in range(len(data_model.keys()))]
    for i,(value, error_value, pos) in enumerate(zip(rmse_values, rmse_errors, x1)):
        if i not in top3_rmse_indices:
            ax1.bar(pos, value, yerr=error_value, align='center', alpha=0.8, ecolor='black', capsize=2, width=width, color=colors_rmse[i], label='RMSE', linewidth=0, error_kw={'lw': 1},zorder=11)
            ax1.scatter(pos, value, color='black', edgecolors='black', linewidths=0,s=10, alpha=1,zorder=15)

    colors_ubrmse = [adjust_color(start_color_ubrmse, 1 - Gradation_degree * i) for i in range(len(data_model.keys()))]
    for i,(value, error_value, pos) in enumerate(zip(ubrmse_values, ubrmse_errors, x2)):
        if i not in top3_ubrmse_indices:
            ax2.bar(pos, value, yerr=error_value, align='center', alpha=0.8, ecolor='black', capsize=2, width=width, color=colors_ubrmse[i], label='ubRMSE', linewidth=0, error_kw={'lw': 1},zorder=11)
            ax2.scatter(pos, value, color='black', edgecolors='black', linewidths=0,s=10, alpha=1,zorder=15)

    # 绘制R和Bias散点图
    ax3.plot(x1, r_values, label='R', linestyle='--', color='#bc6c25', linewidth=1.5, marker='X', markeredgecolor='lightgrey', markersize=10,alpha=0.5,zorder=10)
    ax4.plot(x2, bias_values, '--', label='Bias',color='purple', linewidth=1.5, marker='P',markeredgecolor='grey', markersize=10,alpha=0.5,zorder=10) 

    # 绘制前三
    for idx in top3_rmse_indices:
        ax1.bar(x1[idx], rmse_values[idx], yerr=rmse_errors[idx], align='center', alpha=0.8, ecolor=colors_th[top3_rmse_indices.index(idx)], capsize=2, width=width, color=colors_third[top3_rmse_indices.index(idx)], label='RMSE' if idx == top3_rmse_indices[0] else "", linewidth=0, error_kw={'lw': 1, 'zorder': 13}, zorder=12)
        ax1.scatter(x1[idx], rmse_values[idx], color='black', edgecolors='black', linewidths=0,s=10, alpha=1,zorder=15)
    for idx in top3_ubrmse_indices:
        ax2.bar(x2[idx], ubrmse_values[idx], yerr=ubrmse_errors[idx], align='center', alpha=0.8, ecolor=colors_th[top3_ubrmse_indices.index(idx)], capsize=2, width=width, color=colors_third[top3_ubrmse_indices.index(idx)], label='ubRMSE' if idx == top3_ubrmse_indices[0] else "", linewidth=0, error_kw={'lw': 1, 'zorder': 13}, zorder=12)
        ax2.scatter(x2[idx], ubrmse_values[idx], color='black', edgecolors='black', linewidths=0,s=10, alpha=1,zorder=15)
    for idx in top3_r_indices:
        ax3.scatter(x1[idx], r_values[idx], linewidths=1.5, marker='X', color=colors_third[top3_r_indices.index(idx)], edgecolor='lightgrey', alpha=1, s=120, zorder=5)
    for idx in top3_bias_indices:
        ax4.scatter(x2[idx], bias_values[idx], linewidths=1.5, marker='P', color=colors_third[top3_bias_indices.index(idx)], edgecolor='grey', alpha=1, s=120, zorder=5)

    grey_len = np.zeros(len(data_model.keys()))+0.3
    ax1.bar(x, grey_len, width=width*2, color='lightgray', alpha=0.2, zorder=0)

    labelsize = 10
    pad_x = 0
    pad_y = 4
    for ax in [ax1, ax2, ax3, ax4]:
        ax.grid(False)
        if left_spines and ax in[ax1, ax2]:
            ax.spines['left'].set_visible(True)
            ax.spines['left'].set_linewidth(1.5)
            ax.spines['left'].set_color('black')
            ax.tick_params(axis='y', which='both', colors='black', length=3, width=1.5, labelsize=labelsize, direction='out', pad=pad_y, left=True, labelleft=True)
        else:
            ax.spines['left'].set_visible(False)
            ax.tick_params(axis='y', which='both', colors='black', length=3, width=1.5, labelsize=labelsize, direction='out', pad=pad_y, left=False, labelleft=False)

        if right_spines and ax in[ax3, ax4]:
            ax.spines['right'].set_visible(True)
            ax.spines['right'].set_linewidth(1.5)
            ax.spines['right'].set_color('black')
            ax.tick_params(axis='y', which='both', colors='black', length=3, width=1.5, labelsize=labelsize, direction='out', pad=pad_y, right=True, labelright=True)
        else:
            ax.spines['right'].set_visible(False)
            ax.tick_params(axis='y', which='both', right=False, labelright=False)

        if bottom_spines:
            ax.spines['bottom'].set_visible(True)
            ax.spines['bottom'].set_linewidth(1.5)
            ax.spines['bottom'].set_color('black')
            ax.tick_params(axis='x', which='both', colors='black', length=3, width=1.5, labelsize=labelsize, direction='out', pad=pad_x, bottom=True, labelbottom=True)
        else:
            ax.spines['bottom'].set_visible(False)
            ax.tick_params(axis='x', which='both', colors='black', length=3, width=1.5, labelsize=labelsize, direction='out', pad=pad_x, bottom=False, labelbottom=False)

        if top_spines:
            ax.spines['top'].set_visible(True)
            ax.spines['top'].set_linewidth(1.5)
            ax.spines['top'].set_color('black')
            ax.tick_params(axis='x', which='both', colors='black', length=3, width=1.5, labelsize=labelsize, direction='out', pad=pad_x, top=True, labeltop=False)
        else:
            ax.spines['top'].set_visible(False)
            ax.tick_params(axis='x', which='both', colors='black', length=3, width=1.5, labelsize=labelsize, direction='out', pad=pad_x, top=False, labeltop=False)
        if ax == ax1:
            ax.spines['left'].set_position(('outward', 0))
        if ax == ax2:
            ax.spines['left'].set_position(('outward', 15))
        if ax == ax3:
            ax.spines['right'].set_position(('axes', 1.0))
        if ax == ax4:
            ax.spines['right'].set_position(('outward', 15))
        # 刻度调节
        plt.setp(ax.get_yticklabels(), rotation=90,ha="center", rotation_mode="anchor")
        # 刻度值格式
        ax.yaxis.set_major_formatter(FuncFormatter(custom_formatter))

    # 设置x轴名
    ax1.set_xticks(x)
    ax1.set_xticklabels(xx, fontproperties=font_prop)
    plt.setp(ax1.get_xticklabels(), rotation=90,ha="center")

    # 设置多轴范围
    rmse_min, rmse_max = float('inf'), 0
    ubrmse_min, ubrmse_max = float('inf'), 0
    r_min, r_max = float('inf'), 0
    bias_min, bias_max = float('inf'), 0
    data_model = aggregate_data(all_data)
    for model, data in data_model.items():
        rmse_min = min(rmse_min, data['RMSE'].min())
        rmse_max = max(rmse_max, data['RMSE'].max())
        ubrmse_min = min(ubrmse_min, data['ubRMSE'].min())
        ubrmse_max = max(ubrmse_max, data['ubRMSE'].max())
        r_min = min(r_min, data['R'].min())
        r_max = max(r_max, data['R'].max())
        bias_min = min(bias_min, data['Bias'].min())
        bias_max = max(bias_max, data['Bias'].max())
    '''
    0.02054428651076537 0.2816327750885776
    0.01710960187069204 0.1287595993351982
    -0.4854811460027492 0.9460179483656507
    -0.2324438218623832 0.2723893704606125
    down_det = 0.1
    ax1.set_ylim(rmse_min - 0.05 * (rmse_max - rmse_min), rmse_max + 0.05 * (rmse_max - rmse_min))
    ax2.set_ylim(ubrmse_min - 0.05 * (ubrmse_max - ubrmse_min), ubrmse_max + 0.05 * (ubrmse_max - ubrmse_min))
    ax3.set_ylim(r_min - 0.05 * (r_max - r_min), r_max + 0.05 * (r_max - r_min))
    ax4.set_ylim(bias_min - 0.05 * (bias_max - bias_min), bias_max + 0.05 * (bias_max - bias_min))
    '''
    # 绘制文本
    hour = int(time_point) % 24
    am_pm = "AM" if hour < 12 else "PM"
    hour_formatted = hour if 0 < hour <= 12 else hour - 12
    if hour == 0:  # 处理午夜是0点的情况
        hour_formatted = 12
    prefix = "(" + chr(ord('a') + time_points.index(time_point)) + ")"
    time_str = f'{prefix} {hour_formatted}:00 {am_pm}'
    

    if down:
        ax1.set_ylim(0.3, 0.01)
        ax2.set_ylim(0.16, 0.01)
        ax3.set_ylim(0.75, 0)
        ax4.set_ylim(0.15, -0.15)
        ax.text(0.03, 0.08, time_str, transform=ax.transAxes, fontsize=14, fontname='Times New Roman', verticalalignment='top')
    else:
        ax1.set_ylim(0.01, 0.3)
        ax2.set_ylim(0.01, 0.16)
        ax3.set_ylim(0, 0.75)
        ax4.set_ylim(-0.15, 0.15)
        ax.text(0.03, 0.999, time_str, transform=ax.transAxes, fontsize=14, fontname='Times New Roman', verticalalignment='top')
    ax1.yaxis.set_major_locator(MaxNLocator(prune='lower', nbins=5))
    ax2.yaxis.set_major_locator(MaxNLocator(nbins=5))
    ax3.yaxis.set_major_locator(MaxNLocator(nbins=5))
    ax4.yaxis.set_major_locator(MaxNLocator(prune='lower', nbins=5))

    # 控制刻度线上下边界缝隙距离
    x_min, x_max = ax1.get_xlim()
    padding = (x_max - x_min) * -0.01
    ax1.set_xlim(x_min - padding, x_max + padding)

    ax1.grid(True, color='lightgray', linestyle='--', alpha=0.5, which='both', axis='x', zorder=0.5)

create_subplots(all_data)

### (绘图) V2.0 柱状折线图 4*2 
这个版本中仅使用KP-ST、RF、LSTM、TP模型

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import FuncFormatter
from matplotlib.colors import Normalize
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.collections import PolyCollection
import matplotlib.colors as mcolors
from scipy.stats import gaussian_kde
from matplotlib.colors import to_rgba
from matplotlib.font_manager import FontProperties
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import colorsys

def aggregate_data(all_data, time_point=None):
    data_for_plot = {}
    for model in all_data:
        combined_df = pd.DataFrame()  # 聚合network和time
        time_points = [time_point] if time_point else all_data[model].keys()
        for tp in time_points:
            for network in all_data[model][tp]:
                df = all_data[model][tp][network]
                if not df.empty:
                    combined_df = pd.concat([combined_df, df], ignore_index=True)
        if not combined_df.empty:
            data_for_plot[model] = combined_df
    return data_for_plot
def custom_formatter(x, pos):
    if x == 0:
        return "0"
    else:
        str_x = "{:.15g}".format(x)
        if 'e' in str_x:
            base, exponent = str_x.split('e')
            base = base.rstrip('0').rstrip('.')
            exponent = exponent.replace('+', '').lstrip('0')
            if exponent:
                return f"{base}e{exponent}"
            else:
                return base
        else:
            return str_x
def adjust_color(color, brightness_factor):
    rgb = mcolors.hex2color(color)
    hls = colorsys.rgb_to_hls(*rgb)
    new_hls = (hls[0], max(0, min(1, brightness_factor * hls[1])), hls[2])
    # 将调整后的HLS颜色转换回RGB，然后转换为HEX
    new_rgb = colorsys.hls_to_rgb(*new_hls)
    return mcolors.to_hex(new_rgb)
def create_subplots(all_data):
    # 创建图形对象,绘制顺序依赖于fig定义顺序
    legend_font = FontProperties(family='Times New Roman', size=12)
    time_points = ['0', '3', '6', '9', '12', '15', '18', '21']
    colors_third = ['#590d22', '#ff4d6d', '#ffccd5']
    colors_th = ['#590d22', '#c9184a', '#ff8fa3']
    colors_gradient = ['#caf0f8', '#e9f5db']
    fig = plt.figure(figsize=(24, 24))

    ratio_h = 6.7 # 竖直比例 3
    ratio_v = 5.1 # 水平比例

    absolute_dis_h = 0.7/ratio_h # 中央子图绝对大小（竖直）0.7
    absolute_dis_v = 0.5/ratio_v # 中央子图绝对大小（水平）

    edge_h = 0.3803 # 竖直边缘间隙
    edge_v = 0.304 # 水平边缘间隙

    mid_h = (1-edge_h*2-absolute_dis_h*2)/1 # 竖直间隙,可以通过竖直边缘间隙调节
    mid_v = (1-edge_v*2-absolute_dis_v*4)/3 # 水平间隙……

    right_offset_first = 0 # 第一层向右偏移量
    right_offset_four = 0.05 # 第一层向右偏移量

    # 计算第一层图起点:上层子图  左起点、下起点、宽度比例、高度比例 4*2
    ax1 = fig.add_axes([edge_v + right_offset_first, 1 - edge_h - absolute_dis_h, absolute_dis_v, absolute_dis_h])
    ax2 = fig.add_axes([edge_v + right_offset_first + absolute_dis_v + mid_v, 1 - edge_h - absolute_dis_h, absolute_dis_v, absolute_dis_h])
    ax3 = fig.add_axes([edge_v + right_offset_first + 2*(absolute_dis_v + mid_v), 1 - edge_h - absolute_dis_h, absolute_dis_v, absolute_dis_h])
    ax4 = fig.add_axes([edge_v + right_offset_first + 3*(absolute_dis_v + mid_v), 1 - edge_h - absolute_dis_h, absolute_dis_v, absolute_dis_h])

    ax5 = fig.add_axes([edge_v + right_offset_first, 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v, absolute_dis_h])
    ax6 = fig.add_axes([edge_v + right_offset_first + 1*(absolute_dis_v + mid_v), 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v, absolute_dis_h])
    ax7 = fig.add_axes([edge_v + right_offset_first + 2*(absolute_dis_v + mid_v), 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v, absolute_dis_h])
    ax8 = fig.add_axes([edge_v + right_offset_first + 3*(absolute_dis_v + mid_v), 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v, absolute_dis_h])

    draw_subplot(ax1, all_data, time_point=time_points[0],top_spines=False,right_spines=False,bottom_spines=True,left_spines=True,colors_third = colors_third, start_color_rmse = colors_gradient[0], start_color_ubrmse =  colors_gradient[1],colors_th=colors_th)
    draw_subplot(ax2, all_data, time_point=time_points[1],top_spines=False,right_spines=False,bottom_spines=True,left_spines=False,colors_third = colors_third, start_color_rmse = colors_gradient[0], start_color_ubrmse =  colors_gradient[1],colors_th=colors_th)
    draw_subplot(ax3, all_data, time_point=time_points[2],top_spines=False,right_spines=False,bottom_spines=True,left_spines=False,colors_third = colors_third, start_color_rmse = colors_gradient[0], start_color_ubrmse =  colors_gradient[1],colors_th=colors_th)
    draw_subplot(ax4, all_data, time_point=time_points[3],top_spines=False,right_spines=True,bottom_spines=True,left_spines=False,colors_third = colors_third, start_color_rmse = colors_gradient[0], start_color_ubrmse =  colors_gradient[1],colors_th=colors_th)

    draw_subplot(ax5, all_data, time_point=time_points[4],top_spines=True,right_spines=False,bottom_spines=False,left_spines=True,down=True,colors_third = colors_third, start_color_rmse = colors_gradient[0], start_color_ubrmse =  colors_gradient[1],colors_th=colors_th)
    draw_subplot(ax6, all_data, time_point=time_points[5],top_spines=True,right_spines=False,bottom_spines=False,left_spines=False,down=True,colors_third = colors_third, start_color_rmse = colors_gradient[0], start_color_ubrmse =  colors_gradient[1],colors_th=colors_th)
    draw_subplot(ax7, all_data, time_point=time_points[6],top_spines=True,right_spines=False,bottom_spines=False,left_spines=False,down=True,colors_third = colors_third, start_color_rmse = colors_gradient[0], start_color_ubrmse =  colors_gradient[1],colors_th=colors_th)
    draw_subplot(ax8, all_data, time_point=time_points[7],top_spines=True,right_spines=True,bottom_spines=False,left_spines=False,down=True,colors_third = colors_third, start_color_rmse = colors_gradient[0], start_color_ubrmse =  colors_gradient[1],colors_th=colors_th)

    colors_rmse = [adjust_color(colors_gradient[0], 1 - 0.02 * i) for i in range(11)]
    colors_ubrmse = [adjust_color(colors_gradient[1], 1 - 0.02 * i) for i in range(11)]
    gradient_patch_rmse = mpatches.Patch(color=colors_rmse[5], label='RMSE')
    gradient_patch_ubrmse = mpatches.Patch(color=colors_ubrmse[5], label='ubRMSE')
    line_r = mlines.Line2D([], [], color='#bc6c25', linestyle='--', marker='X', markeredgecolor='lightgrey', markersize=10, label='R', alpha=0.5)
    line_bias = mlines.Line2D([], [], color='purple', linestyle='--', marker='P', markeredgecolor='grey', markersize=10, label='Bias', alpha=0.5)
    first_legend_handles = [gradient_patch_rmse, gradient_patch_ubrmse, line_r, line_bias]
    plt.figlegend(handles=first_legend_handles, labels=[h.get_label() for h in first_legend_handles], 
                  loc='upper center', ncol=2, bbox_to_anchor=(0.455, edge_h+0.005), fancybox=False, 
                  shadow=False, frameon=False, prop=legend_font,
                  borderpad=1, labelspacing=0.8,handletextpad=0.5, handlelength=1.5)

    colors_top3 = ['#590d22', '#ff4d6d', '#ffccd5']  # 用于条形图
    color_X = ['#8A3C23', '#DD5C48', '#D8B08D']  # 用于'X'标记的散点图
    color_PLUS = ['#750266', '#B82B76', '#BF65AA']  # 用于'+'标记的散点图
    top3_labels = ['1st', '2nd', '3rd']
    top3_handles = []
    for i in range(3):
        bar_handle = mpatches.Patch(color=colors_top3[i], label=' ',alpha =0.8)
        scatter_handle_X = mlines.Line2D([], [], color=color_X[i], marker='X', linestyle='None', markersize=10, markeredgecolor='grey', label=top3_labels[i], linewidth=2)  # 实际标签
        scatter_handle_PLUS = mlines.Line2D([], [], color=color_PLUS[i], marker='P', linestyle='None', markersize=10, markeredgecolor='grey', label=' ', linewidth=2)
        top3_handles.extend([bar_handle, scatter_handle_X, scatter_handle_PLUS])

    plt.figlegend(handles=top3_handles, labels=[h.get_label() for h in top3_handles], 
                  loc='upper center', ncol=3, bbox_to_anchor=(0.36-0.004, edge_h+0.005), 
                  fancybox=False, shadow=False, frameon=False, prop=legend_font,
                  borderpad=1, labelspacing=0,handletextpad=0.5, handlelength=1.5)
    
    colors = ['#bc6c25', 'purple']
    line_styles = [':', '-.']
    labels = ['R Mean', 'Zero Line']
    handles = [
        mlines.Line2D([], [], color=color, linestyle=line_style,
                    linewidth=1.5, label=label)
        for color, line_style, label in zip(colors, line_styles, labels)
    ]
    plt.figlegend(handles=handles, loc='upper center', ncol=1, bbox_to_anchor=(0.525, edge_h+0.005),
                  fancybox=False, 
                  shadow=False, frameon=False, prop=legend_font,
                  borderpad=1, labelspacing=0.8,handletextpad=0.5, handlelength=1.5)
    
    offset = 0.006
    fig.text(0.297+offset, 0.493, 'RMSE', ha='left', size=12, fontproperties=legend_font, rotation=90)
    fig.text(0.2868+offset, 0.488, 'ubRMSE', ha='left', size=12, fontproperties=legend_font, rotation=90)
    fig.text(0.6985-offset, 0.488, 'PearsonR', ha='left', size=12, fontproperties=legend_font, rotation=90)
    fig.text(0.7065-offset, 0.496, 'bias', ha='left', size=12, fontproperties=legend_font, rotation=90)

    fig.savefig(r"C:\Users\Administrator\Desktop\Draw\Original_eval.jpeg", dpi=1000, format='jpeg', bbox_inches='tight')

    plt.show()

def draw_subplot(ax, all_data, time_point=None, width=0.41, start_color_rmse = '#caf0f8', start_color_ubrmse = '#e9f5db',Gradation_degree = 0.02,
                 top_spines=True, right_spines=True, bottom_spines=True, left_spines=True,colors_third = None,colors_th=None,
                 down=False):

    sns.set(style="whitegrid", font='Times New Roman')
    matplotlib.rcParams['font.family'] = 'Times New Roman'
    matplotlib.rcParams['axes.labelweight'] = 'bold'
    font_prop = FontProperties(family='Times New Roman', size=10)

    metrics = ['RMSE', 'ubRMSE', 'R', 'Bias']

    data_model = aggregate_data(all_data, time_point)
    ax1 = ax  # RMSE柱状图
    ax3 = ax.twinx()  # R散点图
    ax4 = ax.twinx()  # Bias散点图
    ax2 = ax.twinx()  # ubRMSE柱状图


    x = np.arange(len(data_model.keys()))

    x1 = [i - width/2 for i in x]
    x2 = [i + width/2 for i in x]
    xx = list(data_model.keys())


    # 计算指标
    rmse_values = [data_model[model]['RMSE'].mean() for model in data_model]
    ubrmse_values = [data_model[model]['ubRMSE'].mean() for model in data_model]
    r_values = [data_model[model]['R'].mean() for model in data_model]
    bias_values = [data_model[model]['Bias'].mean() for model in data_model]

    rmse_errors = [(data_model[model]['RMSE'].quantile(0.75) - data_model[model]['RMSE'].quantile(0.25)) for model in data_model]
    ubrmse_errors = [(data_model[model]['ubRMSE'].quantile(0.75) - data_model[model]['ubRMSE'].quantile(0.25)) for model in data_model]

    # 前三索引
    top3_rmse_indices = sorted(range(len(rmse_values)), key=lambda i: rmse_values[i])[:3]
    top3_ubrmse_indices = sorted(range(len(ubrmse_values)), key=lambda i: ubrmse_values[i])[:3]
    top3_r_indices = sorted(range(len(r_values)), key=lambda i: r_values[i], reverse=True)[:3]
    top3_bias_indices = sorted(range(len(bias_values)), key=lambda i: abs(bias_values[i]))[:3]

    # 绘制RMSE柱状图以及误差线
    colors_rmse = [adjust_color(start_color_rmse, 1 - Gradation_degree * i) for i in range(len(data_model.keys()))]
    for i,(value, error_value, pos) in enumerate(zip(rmse_values, rmse_errors, x1)):
        if i not in top3_rmse_indices:
            ax1.bar(pos, value, yerr=error_value, align='center', alpha=0.8, ecolor='black', capsize=2, width=width, color=colors_rmse[i], label='RMSE', linewidth=0, error_kw={'lw': 1,'zorder':16},zorder=16)
            ax1.scatter(pos, value, color='black', edgecolors='black', linewidths=0,s=10, alpha=1,zorder=16)

    colors_ubrmse = [adjust_color(start_color_ubrmse, 1 - Gradation_degree * i) for i in range(len(data_model.keys()))]
    for i,(value, error_value, pos) in enumerate(zip(ubrmse_values, ubrmse_errors, x2)):
        if i not in top3_ubrmse_indices:
            ax2.bar(pos, value, yerr=error_value, align='center', alpha=0.8, ecolor='black', capsize=2, width=width, color=colors_ubrmse[i], label='ubRMSE', linewidth=0, error_kw={'lw': 1,'zorder':16},zorder=16)
            ax2.scatter(pos, value, color='black', edgecolors='black', linewidths=0,s=10, alpha=1,zorder=16)

    # 绘制R和Bias散点图
    ax3.plot(x1, r_values, label='R', linestyle='--', color='#bc6c25', linewidth=1.5, marker='X', markeredgecolor='lightgrey', markersize=10,alpha=0.5,zorder=10)
    ax4.plot(x2, bias_values, '--', label='Bias',color='purple', linewidth=1.5, marker='P',markeredgecolor='grey', markersize=10,alpha=0.5,zorder=10) 
    
    # 绘制均值线和0线
    r_mean = np.mean(r_values)
    ax3.axhline(y=r_mean, label='R mean', linestyle=':', color='#bc6c25', linewidth=1.5, alpha=0.6, zorder=10)
    ax4.axhline(y=0, label='Bias mean', linestyle='-.', color='purple', linewidth=1, alpha=0.5, zorder=10)

    # 绘制前三
    for idx in top3_r_indices:
        ax3.scatter(x1[idx], r_values[idx], linewidths=1.5, marker='X', color=colors_third[top3_r_indices.index(idx)], edgecolor='lightgrey', alpha=1, s=120, zorder=5)
    for idx in top3_bias_indices:
        ax4.scatter(x2[idx], bias_values[idx], linewidths=1.5, marker='P', color=colors_third[top3_bias_indices.index(idx)], edgecolor='grey', alpha=1, s=120, zorder=5)

    for idx in top3_rmse_indices:
        ax1.bar(x1[idx], rmse_values[idx], yerr=rmse_errors[idx], align='center', alpha=0.8, ecolor=colors_th[top3_rmse_indices.index(idx)], capsize=2, width=width, color=colors_third[top3_rmse_indices.index(idx)], label='RMSE' if idx == top3_rmse_indices[0] else "", linewidth=0, error_kw={'lw': 1, 'zorder': 16}, zorder=12)
        ax1.scatter(x1[idx], rmse_values[idx], color='black', edgecolors='black', linewidths=0,s=10, alpha=1,zorder=15)
    for idx in top3_ubrmse_indices:
        ax2.bar(x2[idx], ubrmse_values[idx], yerr=ubrmse_errors[idx], align='center', alpha=0.8, ecolor=colors_th[top3_ubrmse_indices.index(idx)], capsize=2, width=width, color=colors_third[top3_ubrmse_indices.index(idx)], label='ubRMSE' if idx == top3_ubrmse_indices[0] else "", linewidth=0, error_kw={'lw': 1, 'zorder': 16}, zorder=12)
        ax2.scatter(x2[idx], ubrmse_values[idx], color='black', edgecolors='black', linewidths=0,s=10, alpha=1,zorder=15)

    grey_len = np.zeros(len(data_model.keys()))+0.3
    ax1.bar(x, grey_len, width=width*2, color='lightgray', alpha=0.2, zorder=0)

    labelsize = 10
    pad_x = 0
    pad_y = 4
    for ax in [ax1, ax2, ax3, ax4]:
        ax.grid(False)
        if left_spines and ax in[ax1, ax2]:
            ax.spines['left'].set_visible(True)
            ax.spines['left'].set_linewidth(1.5)
            ax.spines['left'].set_color('black')
            color = '#49E249' if ax in [ax1] else '#48A0CB'
            ax.tick_params(axis='y', which='both', colors=color, length=3, width=1.5, labelsize=labelsize, direction='out', pad=pad_y, left=True, labelleft=True)
        else:
            ax.spines['left'].set_visible(False)
            ax.tick_params(axis='y', which='both', colors='black', length=3, width=1.5, labelsize=labelsize, direction='out', pad=pad_y, left=False, labelleft=False)

        if right_spines and ax in[ax3, ax4]:
            ax.spines['right'].set_visible(True)
            ax.spines['right'].set_linewidth(1.5)
            ax.spines['right'].set_color('black')
            color = '#bc6c25' if ax in [ax3] else 'purple'
            ax.tick_params(axis='y', which='both', colors=color, length=3, width=1.5, labelsize=labelsize, direction='out', pad=pad_y, right=True, labelright=True)
        else:
            ax.spines['right'].set_visible(False)
            ax.tick_params(axis='y', which='both', right=False, labelright=False)

        if bottom_spines:
            ax.spines['bottom'].set_visible(True)
            ax.spines['bottom'].set_linewidth(1.5)
            ax.spines['bottom'].set_color('black')
            ax.tick_params(axis='x', which='both', colors='black', length=3, width=1.5, labelsize=labelsize, direction='out', pad=pad_x, bottom=True, labelbottom=True)
        else:
            ax.spines['bottom'].set_visible(False)
            ax.tick_params(axis='x', which='both', colors='black', length=3, width=1.5, labelsize=labelsize, direction='out', pad=pad_x, bottom=False, labelbottom=False)

        if top_spines:
            ax.spines['top'].set_visible(True)
            ax.spines['top'].set_linewidth(1.5)
            ax.spines['top'].set_color('black')
            ax.tick_params(axis='x', which='both', colors='black', length=3, width=1.5, labelsize=labelsize, direction='out', pad=pad_x, top=True, labeltop=False)
        else:
            ax.spines['top'].set_visible(False)
            ax.tick_params(axis='x', which='both', colors='black', length=3, width=1.5, labelsize=labelsize, direction='out', pad=pad_x, top=False, labeltop=False)
        if ax == ax1:
            ax.spines['left'].set_position(('outward', 0))
        if ax == ax2:
            ax.spines['left'].set_position(('outward', 15))
        if ax == ax3:
            ax.spines['right'].set_position(('axes', 1.0))
        if ax == ax4:
            ax.spines['right'].set_position(('outward', 15))
        # 刻度调节
        plt.setp(ax.get_yticklabels(), rotation=90,ha="center", rotation_mode="anchor")
        # 刻度值格式
        ax.yaxis.set_major_formatter(FuncFormatter(custom_formatter))

    # 设置x轴名
    ax1.set_xticks(x)
    ax1.set_xticklabels(xx, fontproperties=font_prop)
    if bottom_spines:
        labels = ax1.get_xticklabels()
        labels[0].set_fontsize(7)
    plt.setp(ax1.get_xticklabels(), rotation=90,ha="center")

    # 设置多轴范围
    rmse_min, rmse_max = float('inf'), 0
    ubrmse_min, ubrmse_max = float('inf'), 0
    r_min, r_max = float('inf'), 0
    bias_min, bias_max = float('inf'), 0
    data_model = aggregate_data(all_data)
    for model, data in data_model.items():
        rmse_min = min(rmse_min, data['RMSE'].min())
        rmse_max = max(rmse_max, data['RMSE'].max())
        ubrmse_min = min(ubrmse_min, data['ubRMSE'].min())
        ubrmse_max = max(ubrmse_max, data['ubRMSE'].max())
        r_min = min(r_min, data['R'].min())
        r_max = max(r_max, data['R'].max())
        bias_min = min(bias_min, data['Bias'].min())
        bias_max = max(bias_max, data['Bias'].max())

    '''
    0.02054428651076537 0.2816327750885776
    0.01710960187069204 0.1287595993351982
    -0.4854811460027492 0.9460179483656507
    -0.2324438218623832 0.2723893704606125
    down_det = 0.1
    ax1.set_ylim(rmse_min - 0.05 * (rmse_max - rmse_min), rmse_max + 0.05 * (rmse_max - rmse_min))
    ax2.set_ylim(ubrmse_min - 0.05 * (ubrmse_max - ubrmse_min), ubrmse_max + 0.05 * (ubrmse_max - ubrmse_min))
    ax3.set_ylim(r_min - 0.05 * (r_max - r_min), r_max + 0.05 * (r_max - r_min))
    ax4.set_ylim(bias_min - 0.05 * (bias_max - bias_min), bias_max + 0.05 * (bias_max - bias_min))
    '''

    # 绘制文本
    hour = int(time_point) % 24
    am_pm = "AM" if hour < 12 else "PM"
    hour_formatted = hour if 0 < hour <= 12 else hour - 12
    if hour == 0:  # 处理午夜是0点的情况
        hour_formatted = 12
    prefix = "(" + chr(ord('a') + time_points.index(time_point)) + ")"
    time_str = f'{prefix} {hour_formatted}:00 {am_pm}'
    
    if down:
        ax1.set_ylim(0.3, 0.01)
        ax2.set_ylim(0.16, 0.01)
        ax3.set_ylim(0.75, 0)
        ax4.set_ylim(0.15, -0.30)
        ax.text(0.03, 0.08, time_str, transform=ax.transAxes, fontsize=14, fontname='Times New Roman', verticalalignment='top')
    else:
        ax1.set_ylim(0.01, 0.3)
        ax2.set_ylim(0.01, 0.16)
        ax3.set_ylim(0, 0.75)
        ax4.set_ylim(-0.30, 0.15)
        ax.text(0.03, 0.999, time_str, transform=ax.transAxes, fontsize=14, fontname='Times New Roman', verticalalignment='top')
    ax1.yaxis.set_major_locator(MaxNLocator(prune='lower', nbins=5))
    ax2.yaxis.set_major_locator(MaxNLocator(nbins=5))
    ax3.yaxis.set_major_locator(MaxNLocator(nbins=5))
    ax4.yaxis.set_major_locator(MaxNLocator(prune='lower', nbins=5))

    # 控制刻度线上下边界缝隙距离
    x_min, x_max = ax1.get_xlim()
    padding = (x_max - x_min) * -0.01
    ax1.set_xlim(x_min - padding, x_max + padding)

    ax1.grid(True, color='lightgray', linestyle='--', alpha=0.5, which='both', axis='x', zorder=0.5)

create_subplots(all_data)

### (绘图) V2.0 柱状折线图 4*2 
这个版本中仅使用KP-ST、RF、LSTM、TP模型

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import FuncFormatter
from matplotlib.colors import Normalize
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.collections import PolyCollection
import matplotlib.colors as mcolors
from scipy.stats import gaussian_kde
from matplotlib.colors import to_rgba
from matplotlib.font_manager import FontProperties
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import colorsys

def aggregate_data(all_data, time_point=None):
    data_for_plot = {}
    for model in all_data:
        combined_df = pd.DataFrame()  # 聚合network和time
        time_points = [time_point] if time_point else all_data[model].keys()
        for tp in time_points:
            for network in all_data[model][tp]:
                df = all_data[model][tp][network]
                if not df.empty:
                    combined_df = pd.concat([combined_df, df], ignore_index=True)
        if not combined_df.empty:
            data_for_plot[model] = combined_df
    return data_for_plot
def custom_formatter(x, pos):
    if x == 0:
        return "0"
    else:
        str_x = "{:.15g}".format(x)
        if 'e' in str_x:
            base, exponent = str_x.split('e')
            base = base.rstrip('0').rstrip('.')
            exponent = exponent.replace('+', '').lstrip('0')
            if exponent:
                return f"{base}e{exponent}"
            else:
                return base
        else:
            return str_x
def adjust_color(color, brightness_factor):
    rgb = mcolors.hex2color(color)
    hls = colorsys.rgb_to_hls(*rgb)
    new_hls = (hls[0], max(0, min(1, brightness_factor * hls[1])), hls[2])
    # 将调整后的HLS颜色转换回RGB，然后转换为HEX
    new_rgb = colorsys.hls_to_rgb(*new_hls)
    return mcolors.to_hex(new_rgb)
def create_subplots(all_data):
    # 创建图形对象,绘制顺序依赖于fig定义顺序
    legend_font = FontProperties(family='Times New Roman', size=12)
    time_points = ['0', '3', '6', '9', '12', '15', '18', '21']
    colors_third = ['#590d22', '#ff4d6d', '#ffccd5']
    colors_th = ['#590d22', '#c9184a', '#ff8fa3']
    colors_gradient = ['#caf0f8', '#e9f5db']
    fig = plt.figure(figsize=(24, 24))

    ratio_h = 6.7 # 竖直比例 3
    ratio_v = 5.1 # 水平比例

    absolute_dis_h = 0.7/ratio_h # 中央子图绝对大小（竖直）0.7
    absolute_dis_v = 0.57/ratio_v # 中央子图绝对大小（水平）

    edge_h = 0.3803 # 竖直边缘间隙
    edge_v = 0.28 # 水平边缘间隙

    mid_h = (1-edge_h*2-absolute_dis_h*2)/1 # 竖直间隙,可以通过竖直边缘间隙调节
    mid_v = (1-edge_v*2-absolute_dis_v*4)/3 # 水平间隙……

    right_offset_first = 0 # 第一层向右偏移量
    right_offset_four = 0.05 # 第一层向右偏移量

    # 计算第一层图起点:上层子图  左起点、下起点、宽度比例、高度比例 4*2
    ax1 = fig.add_axes([edge_v + right_offset_first, 1 - edge_h - absolute_dis_h, absolute_dis_v, absolute_dis_h])
    ax2 = fig.add_axes([edge_v + right_offset_first + absolute_dis_v + mid_v, 1 - edge_h - absolute_dis_h, absolute_dis_v, absolute_dis_h])
    ax3 = fig.add_axes([edge_v + right_offset_first + 2*(absolute_dis_v + mid_v), 1 - edge_h - absolute_dis_h, absolute_dis_v, absolute_dis_h])
    ax4 = fig.add_axes([edge_v + right_offset_first + 3*(absolute_dis_v + mid_v), 1 - edge_h - absolute_dis_h, absolute_dis_v, absolute_dis_h])

    ax5 = fig.add_axes([edge_v + right_offset_first, 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v, absolute_dis_h])
    ax6 = fig.add_axes([edge_v + right_offset_first + 1*(absolute_dis_v + mid_v), 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v, absolute_dis_h])
    ax7 = fig.add_axes([edge_v + right_offset_first + 2*(absolute_dis_v + mid_v), 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v, absolute_dis_h])
    ax8 = fig.add_axes([edge_v + right_offset_first + 3*(absolute_dis_v + mid_v), 1 - edge_h - 2*absolute_dis_h - mid_h, absolute_dis_v, absolute_dis_h])

    draw_subplot(ax1, all_data, time_point=time_points[0],top_spines=False,right_spines=False,bottom_spines=True,left_spines=True,colors_third = colors_third, start_color_rmse = colors_gradient[0], start_color_ubrmse =  colors_gradient[1],colors_th=colors_th)
    draw_subplot(ax2, all_data, time_point=time_points[1],top_spines=False,right_spines=False,bottom_spines=True,left_spines=False,colors_third = colors_third, start_color_rmse = colors_gradient[0], start_color_ubrmse =  colors_gradient[1],colors_th=colors_th)
    draw_subplot(ax3, all_data, time_point=time_points[2],top_spines=False,right_spines=False,bottom_spines=True,left_spines=False,colors_third = colors_third, start_color_rmse = colors_gradient[0], start_color_ubrmse =  colors_gradient[1],colors_th=colors_th)
    draw_subplot(ax4, all_data, time_point=time_points[3],top_spines=False,right_spines=True,bottom_spines=True,left_spines=False,colors_third = colors_third, start_color_rmse = colors_gradient[0], start_color_ubrmse =  colors_gradient[1],colors_th=colors_th)

    draw_subplot(ax5, all_data, time_point=time_points[4],top_spines=True,right_spines=False,bottom_spines=False,left_spines=True,down=True,colors_third = colors_third, start_color_rmse = colors_gradient[0], start_color_ubrmse =  colors_gradient[1],colors_th=colors_th)
    draw_subplot(ax6, all_data, time_point=time_points[5],top_spines=True,right_spines=False,bottom_spines=False,left_spines=False,down=True,colors_third = colors_third, start_color_rmse = colors_gradient[0], start_color_ubrmse =  colors_gradient[1],colors_th=colors_th)
    draw_subplot(ax7, all_data, time_point=time_points[6],top_spines=True,right_spines=False,bottom_spines=False,left_spines=False,down=True,colors_third = colors_third, start_color_rmse = colors_gradient[0], start_color_ubrmse =  colors_gradient[1],colors_th=colors_th)
    draw_subplot(ax8, all_data, time_point=time_points[7],top_spines=True,right_spines=True,bottom_spines=False,left_spines=False,down=True,colors_third = colors_third, start_color_rmse = colors_gradient[0], start_color_ubrmse =  colors_gradient[1],colors_th=colors_th)

    colors_rmse = [adjust_color(colors_gradient[0], 1 - 0.02 * i) for i in range(11)]
    colors_ubrmse = [adjust_color(colors_gradient[1], 1 - 0.02 * i) for i in range(11)]
    gradient_patch_rmse = mpatches.Patch(color=colors_rmse[5], label='RMSE')
    gradient_patch_ubrmse = mpatches.Patch(color=colors_ubrmse[5], label='ubRMSE')
    line_r = mlines.Line2D([], [], color='#bc6c25', linestyle='--', marker='X', markeredgecolor='lightgrey', markersize=10, label='R', alpha=0.5)
    line_bias = mlines.Line2D([], [], color='purple', linestyle='--', marker='P', markeredgecolor='grey', markersize=10, label='Bias', alpha=0.5)
    first_legend_handles = [gradient_patch_rmse, gradient_patch_ubrmse, line_r, line_bias]
    plt.figlegend(handles=first_legend_handles, labels=[h.get_label() for h in first_legend_handles], 
                  loc='upper center', ncol=2, bbox_to_anchor=(0.455, edge_h+0.005), fancybox=False, 
                  shadow=False, frameon=False, prop=legend_font,
                  borderpad=1, labelspacing=0.8,handletextpad=0.5, handlelength=1.5)

    colors_top3 = ['#590d22', '#ff4d6d', '#ffccd5']  # 用于条形图
    color_X = ['#8A3C23', '#DD5C48', '#D8B08D']  # 用于'X'标记的散点图
    color_PLUS = ['#750266', '#B82B76', '#BF65AA']  # 用于'+'标记的散点图
    top3_labels = ['1st', '2nd', '3rd']
    top3_handles = []
    for i in range(3):
        bar_handle = mpatches.Patch(color=colors_top3[i], label=' ',alpha =0.8)
        scatter_handle_X = mlines.Line2D([], [], color=color_X[i], marker='X', linestyle='None', markersize=10, markeredgecolor='grey', label=top3_labels[i], linewidth=2)  # 实际标签
        scatter_handle_PLUS = mlines.Line2D([], [], color=color_PLUS[i], marker='P', linestyle='None', markersize=10, markeredgecolor='grey', label=' ', linewidth=2)
        top3_handles.extend([bar_handle, scatter_handle_X, scatter_handle_PLUS])

    plt.figlegend(handles=top3_handles, labels=[h.get_label() for h in top3_handles], 
                  loc='upper center', ncol=3, bbox_to_anchor=(0.36-0.004, edge_h+0.005), 
                  fancybox=False, shadow=False, frameon=False, prop=legend_font,
                  borderpad=1, labelspacing=0,handletextpad=0.5, handlelength=1.5)
    
    colors = ['#bc6c25', 'purple']
    line_styles = [':', '-.']
    labels = ['R Mean', 'Zero Line']
    handles = [
        mlines.Line2D([], [], color=color, linestyle=line_style,
                    linewidth=1.5, label=label)
        for color, line_style, label in zip(colors, line_styles, labels)
    ]
    plt.figlegend(handles=handles, loc='upper center', ncol=1, bbox_to_anchor=(0.525, edge_h+0.005),
                  fancybox=False, 
                  shadow=False, frameon=False, prop=legend_font,
                  borderpad=1, labelspacing=0.8,handletextpad=0.5, handlelength=1.5)
    
    offset = 0.006
    fig.text(0.271+offset, 0.493, 'RMSE', ha='left', size=12, fontproperties=legend_font, rotation=90)
    fig.text(0.2608+offset, 0.488, 'ubRMSE', ha='left', size=12, fontproperties=legend_font, rotation=90)
    fig.text(0.7245-offset, 0.488, 'PearsonR', ha='left', size=12, fontproperties=legend_font, rotation=90)
    fig.text(0.7325-offset, 0.496, 'bias', ha='left', size=12, fontproperties=legend_font, rotation=90)

    fig.savefig(r"C:\Users\Administrator\Desktop\Draw\Original_eval.jpeg", dpi=1000, format='jpeg', bbox_inches='tight')

    plt.show()

def draw_subplot(ax, all_data, time_point=None, width=0.41, start_color_rmse = '#caf0f8', start_color_ubrmse = '#e9f5db',Gradation_degree = 0.02,
                 top_spines=True, right_spines=True, bottom_spines=True, left_spines=True,colors_third = None,colors_th=None,
                 down=False):

    sns.set(style="whitegrid", font='Times New Roman')
    matplotlib.rcParams['font.family'] = 'Times New Roman'
    matplotlib.rcParams['axes.labelweight'] = 'bold'
    font_prop = FontProperties(family='Times New Roman', size=10)

    metrics = ['RMSE', 'ubRMSE', 'R', 'Bias']

    data_model = aggregate_data(all_data, time_point)
    ax1 = ax  # RMSE柱状图
    ax3 = ax.twinx()  # R散点图
    ax4 = ax.twinx()  # Bias散点图
    ax2 = ax.twinx()  # ubRMSE柱状图


    x = np.arange(len(data_model.keys()))

    x1 = [i - width/2 for i in x]
    x2 = [i + width/2 for i in x]
    xx = list(data_model.keys())


    # 计算指标
    rmse_values = [data_model[model]['RMSE'].mean() for model in data_model]
    ubrmse_values = [data_model[model]['ubRMSE'].mean() for model in data_model]
    r_values = [data_model[model]['R'].mean() for model in data_model]
    bias_values = [data_model[model]['Bias'].mean() for model in data_model]

    rmse_errors = [(data_model[model]['RMSE'].quantile(0.75) - data_model[model]['RMSE'].quantile(0.25)) for model in data_model]
    ubrmse_errors = [(data_model[model]['ubRMSE'].quantile(0.75) - data_model[model]['ubRMSE'].quantile(0.25)) for model in data_model]

    # 前三索引
    top3_rmse_indices = sorted(range(len(rmse_values)), key=lambda i: rmse_values[i])[:3]
    top3_ubrmse_indices = sorted(range(len(ubrmse_values)), key=lambda i: ubrmse_values[i])[:3]
    top3_r_indices = sorted(range(len(r_values)), key=lambda i: r_values[i], reverse=True)[:3]
    top3_bias_indices = sorted(range(len(bias_values)), key=lambda i: abs(bias_values[i]))[:3]

    # 绘制RMSE柱状图以及误差线
    colors_rmse = [adjust_color(start_color_rmse, 1 - Gradation_degree * i) for i in range(len(data_model.keys()))]
    for i,(value, error_value, pos) in enumerate(zip(rmse_values, rmse_errors, x1)):
        if i not in top3_rmse_indices:
            ax1.bar(pos, value, yerr=error_value, align='center', alpha=0.8, ecolor='black', capsize=2, width=width, color=colors_rmse[i], label='RMSE', linewidth=0, error_kw={'lw': 1,'zorder':16},zorder=16)
            ax1.scatter(pos, value, color='black', edgecolors='black', linewidths=0,s=10, alpha=1,zorder=16)
            
    colors_ubrmse = [adjust_color(start_color_ubrmse, 1 - Gradation_degree * i) for i in range(len(data_model.keys()))]
    for i,(value, error_value, pos) in enumerate(zip(ubrmse_values, ubrmse_errors, x2)):
        if i not in top3_ubrmse_indices:
            ax2.bar(pos, value, yerr=error_value, align='center', alpha=0.8, ecolor='black', capsize=2, width=width, color=colors_ubrmse[i], label='ubRMSE', linewidth=0, error_kw={'lw': 1,'zorder':16},zorder=16)
            ax2.scatter(pos, value, color='black', edgecolors='black', linewidths=0,s=10, alpha=1,zorder=16)

    # 绘制R和Bias散点图
    ax3.plot(x1, r_values, label='R', linestyle='--', color='#bc6c25', linewidth=1.5, marker='X', markeredgecolor='lightgrey', markersize=10,alpha=0.5,zorder=10)
    ax4.plot(x2, bias_values, '--', label='Bias',color='purple', linewidth=1.5, marker='P',markeredgecolor='grey', markersize=10,alpha=0.5,zorder=10) 
    
    # 绘制均值线和0线
    r_mean = np.mean(r_values)
    ax3.axhline(y=r_mean, label='R mean', linestyle=':', color='#bc6c25', linewidth=1.5, alpha=0.6, zorder=10)
    ax4.axhline(y=0, label='Bias mean', linestyle='-.', color='purple', linewidth=1, alpha=0.5, zorder=10)

    # 绘制前三
    for idx in top3_r_indices:
        ax3.scatter(x1[idx], r_values[idx], linewidths=1.5, marker='X', color=colors_third[top3_r_indices.index(idx)], edgecolor='lightgrey', alpha=1, s=120, zorder=5)
    for idx in top3_bias_indices:
        ax4.scatter(x2[idx], bias_values[idx], linewidths=1.5, marker='P', color=colors_third[top3_bias_indices.index(idx)], edgecolor='grey', alpha=1, s=120, zorder=5)

    for idx in top3_rmse_indices:
        ax1.bar(x1[idx], rmse_values[idx], yerr=rmse_errors[idx], align='center', alpha=0.8, ecolor=colors_th[top3_rmse_indices.index(idx)], capsize=2, width=width, color=colors_third[top3_rmse_indices.index(idx)], label='RMSE' if idx == top3_rmse_indices[0] else "", linewidth=0, error_kw={'lw': 1, 'zorder': 16}, zorder=12)
        ax1.scatter(x1[idx], rmse_values[idx], color='black', edgecolors='black', linewidths=0,s=10, alpha=1,zorder=15)
    for idx in top3_ubrmse_indices:
        ax2.bar(x2[idx], ubrmse_values[idx], yerr=ubrmse_errors[idx], align='center', alpha=0.8, ecolor=colors_th[top3_ubrmse_indices.index(idx)], capsize=2, width=width, color=colors_third[top3_ubrmse_indices.index(idx)], label='ubRMSE' if idx == top3_ubrmse_indices[0] else "", linewidth=0, error_kw={'lw': 1, 'zorder': 16}, zorder=12)
        ax2.scatter(x2[idx], ubrmse_values[idx], color='black', edgecolors='black', linewidths=0,s=10, alpha=1,zorder=15)

    grey_len = np.zeros(len(data_model.keys()))+0.3
    ax1.bar(x, grey_len, width=width*2, color='lightgray', alpha=0.2, zorder=0)

    labelsize = 10
    pad_x = 0
    pad_y = 4
    for ax in [ax1, ax2, ax3, ax4]:
        ax.grid(False)
        if left_spines and ax in[ax1, ax2]:
            ax.spines['left'].set_visible(True)
            ax.spines['left'].set_linewidth(1.5)
            ax.spines['left'].set_color('black')
            color = '#49E249' if ax in [ax1] else '#48A0CB'
            ax.tick_params(axis='y', which='both', colors=color, length=3, width=1.5, labelsize=labelsize, direction='out', pad=pad_y, left=True, labelleft=True)
        else:
            ax.spines['left'].set_visible(False)
            ax.tick_params(axis='y', which='both', colors='black', length=3, width=1.5, labelsize=labelsize, direction='out', pad=pad_y, left=False, labelleft=False)

        if right_spines and ax in[ax3, ax4]:
            ax.spines['right'].set_visible(True)
            ax.spines['right'].set_linewidth(1.5)
            ax.spines['right'].set_color('black')
            color = '#bc6c25' if ax in [ax3] else 'purple'
            ax.tick_params(axis='y', which='both', colors=color, length=3, width=1.5, labelsize=labelsize, direction='out', pad=pad_y, right=True, labelright=True)
        else:
            ax.spines['right'].set_visible(False)
            ax.tick_params(axis='y', which='both', right=False, labelright=False)

        if bottom_spines:
            ax.spines['bottom'].set_visible(True)
            ax.spines['bottom'].set_linewidth(1.5)
            ax.spines['bottom'].set_color('black')
            ax.tick_params(axis='x', which='both', colors='black', length=3, width=1.5, labelsize=labelsize, direction='out', pad=pad_x, bottom=True, labelbottom=True)
        else:
            ax.spines['bottom'].set_visible(False)
            ax.tick_params(axis='x', which='both', colors='black', length=3, width=1.5, labelsize=labelsize, direction='out', pad=pad_x, bottom=False, labelbottom=False)

        if top_spines:
            ax.spines['top'].set_visible(True)
            ax.spines['top'].set_linewidth(1.5)
            ax.spines['top'].set_color('black')
            ax.tick_params(axis='x', which='both', colors='black', length=3, width=1.5, labelsize=labelsize, direction='out', pad=pad_x, top=True, labeltop=False)
        else:
            ax.spines['top'].set_visible(False)
            ax.tick_params(axis='x', which='both', colors='black', length=3, width=1.5, labelsize=labelsize, direction='out', pad=pad_x, top=False, labeltop=False)
        if ax == ax1:
            ax.spines['left'].set_position(('outward', 0))
        if ax == ax2:
            ax.spines['left'].set_position(('outward', 15))
        if ax == ax3:
            ax.spines['right'].set_position(('axes', 1.0))
        if ax == ax4:
            ax.spines['right'].set_position(('outward', 15))
        # 刻度调节
        plt.setp(ax.get_yticklabels(), rotation=90,ha="center", rotation_mode="anchor")
        # 刻度值格式
        ax.yaxis.set_major_formatter(FuncFormatter(custom_formatter))

    # 设置x轴名
    ax1.set_xticks(x)
    ax1.set_xticklabels(xx, fontproperties=font_prop)
    if bottom_spines:
        labels = ax1.get_xticklabels()
        labels[0].set_fontsize(7)
        labels[1].set_fontsize(7)
    plt.setp(ax1.get_xticklabels(), rotation=90,ha="center")

    # 设置多轴范围
    rmse_min, rmse_max = float('inf'), 0
    ubrmse_min, ubrmse_max = float('inf'), 0
    r_min, r_max = float('inf'), 0
    bias_min, bias_max = float('inf'), 0
    data_model = aggregate_data(all_data)
    for model, data in data_model.items():
        rmse_min = min(rmse_min, data['RMSE'].min())
        rmse_max = max(rmse_max, data['RMSE'].max())
        ubrmse_min = min(ubrmse_min, data['ubRMSE'].min())
        ubrmse_max = max(ubrmse_max, data['ubRMSE'].max())
        r_min = min(r_min, data['R'].min())
        r_max = max(r_max, data['R'].max())
        bias_min = min(bias_min, data['Bias'].min())
        bias_max = max(bias_max, data['Bias'].max())

    '''
    0.02054428651076537 0.2816327750885776
    0.01710960187069204 0.1287595993351982
    -0.4854811460027492 0.9460179483656507
    -0.2324438218623832 0.2723893704606125
    down_det = 0.1
    ax1.set_ylim(rmse_min - 0.05 * (rmse_max - rmse_min), rmse_max + 0.05 * (rmse_max - rmse_min))
    ax2.set_ylim(ubrmse_min - 0.05 * (ubrmse_max - ubrmse_min), ubrmse_max + 0.05 * (ubrmse_max - ubrmse_min))
    ax3.set_ylim(r_min - 0.05 * (r_max - r_min), r_max + 0.05 * (r_max - r_min))
    ax4.set_ylim(bias_min - 0.05 * (bias_max - bias_min), bias_max + 0.05 * (bias_max - bias_min))
    '''

    # 绘制文本
    hour = int(time_point) % 24
    am_pm = "AM" if hour < 12 else "PM"
    hour_formatted = hour if 0 < hour <= 12 else hour - 12
    if hour == 0:  # 处理午夜是0点的情况
        hour_formatted = 12
    prefix = "(" + chr(ord('a') + time_points.index(time_point)) + ")"
    time_str = f'{prefix} {hour_formatted}:00 {am_pm}'
    # 绘制RMSE柱状图以及误差线
    offset_y1 = 0.03 if bottom_spines is True else 0.025
    offset_y2 = 0.0202 if bottom_spines is True else 0.0176
    for i,(value, error_value, pos) in enumerate(zip(rmse_values, rmse_errors, x1)):
        ax1.text(pos, offset_y1, f'{value:.3f}', ha='center', va='center', fontproperties=font_prop, size=8, color='black', zorder=17, rotation=90, rotation_mode='anchor')
    for i,(value, error_value, pos) in enumerate(zip(ubrmse_values, ubrmse_errors, x2)):
        ax2.text(pos, offset_y2, f'{value:.3f}', ha='center', va='center', fontproperties=font_prop, size=8, color='black', zorder=17, rotation=90, rotation_mode='anchor')
    
    offset_y3 = 0.065 if bottom_spines is True else 0.06
    offset_y4 = 0.035 if bottom_spines is True else 0.032
    for i, (value, pos) in enumerate(zip(r_values, x1)):
        ax3.text(pos, value + offset_y3, f'{value:.3f}', ha='center', va='center', fontproperties=font_prop, size=8, color='#bc6c25', zorder=17, rotation=90, rotation_mode='anchor')
    for i, (value, pos) in enumerate(zip(bias_values, x2)):
        ax4.text(pos, value + offset_y4, f'{value:.3f}', ha='center', va='center', fontproperties=font_prop, size=8, color='purple', zorder=17, rotation=90, rotation_mode='anchor')

    if down:
        ax1.set_ylim(0.3, 0.01)
        ax2.set_ylim(0.16, 0.01)
        ax3.set_ylim(0.8, 0)
        ax4.set_ylim(0.15, -0.25)
        ax.text(0.03, 0.08, time_str, transform=ax.transAxes, fontsize=14, fontname='Times New Roman', verticalalignment='top')
    else:
        ax1.set_ylim(0.01, 0.3)
        ax2.set_ylim(0.01, 0.16)
        ax3.set_ylim(0, 0.8)
        ax4.set_ylim(-0.25, 0.15)
        ax.text(0.03, 0.999, time_str, transform=ax.transAxes, fontsize=14, fontname='Times New Roman', verticalalignment='top')
    ax1.yaxis.set_major_locator(MaxNLocator(prune='lower', nbins=5))
    ax2.yaxis.set_major_locator(MaxNLocator(nbins=5))
    ax3.yaxis.set_major_locator(MaxNLocator(nbins=5))
    ax4.yaxis.set_major_locator(MaxNLocator(prune='lower', nbins=5))

    # 控制刻度线上下边界缝隙距离
    x_min, x_max = ax1.get_xlim()
    padding = (x_max - x_min) * -0.01
    ax1.set_xlim(x_min - padding, x_max + padding)

    ax1.grid(True, color='lightgray', linestyle='--', alpha=0.5, which='both', axis='x', zorder=0.5)

create_subplots(all_data)

### (数据处理) 原点网络均值计算(时间动态图)

In [ ]:
import sys
sys.path.append(r"C:\Users\Administrator\Desktop\code\ST-Conv")
import argparse
import pandas as pd
import numpy as np
from Original_Point_Data.tool.origin_utils import OriginUtils
from tool.utils import Util

def calculate_network_avg(modes, info_path, find_mode='0.1'):
    # 初始化存储最终均值数据的DataFrame
    avg_data_df = pd.DataFrame()

    for mode in modes:
        ali_list = OriginUtils.find_grid_points(mode, info_path, find_mode=find_mode)
        point_list = [[item[2], item[1], item[0], item[3]] for item in ali_list]  # 保证顺序：纬度，经度，站点名，文件路径
        point_list.sort(key=lambda x: (-x[0], x[1]))  # 根据纬度降序，经度升序排序

        point_data_frames = []

        for lat, lon, point_name, file_path in point_list:
            # 读取并处理每个站点的数据
            point_data = pd.read_csv(file_path, header=0)  # 读取CSV文件
            point_data['Time'] = pd.to_datetime(point_data.iloc[:, 0])  # 转换时间列为DateTime对象
            point_data.set_index('Time', inplace=True)  # 设置时间列为索引
            point_data_frames.append(point_data['SM'])  # 假设感兴趣的数据在'SM'列

        # 时间对齐并计算均值
        if point_data_frames:
            aligned_data = pd.concat(point_data_frames, axis=1, join='outer').mean(axis=1)
            avg_data_df[mode] = aligned_data

    return avg_data_df

def main(args):
    Original_Point_path_argu = Util.load_config(args.Original_Point_path_config)
    path_argu = Util.load_config(args.path_config_path)
    return Original_Point_path_argu, path_argu

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--Original_Point_path_config', type=str, default='../Original_Point_Data/config/path_config.yaml')
    parser.add_argument('--path_config_path', type=str, default='../config/path_config.yaml')
    args = parser.parse_known_args()[0]
    Original_Point_path_argu, path_argu = main(args)
    modes = ['Shiquanhe', 'Ali', 'Maqu', 'Naqu', 'CTP']  # 所有模式
    cal_time = '3'
    csv_path = Original_Point_path_argu['info']
    avg_data_df = calculate_network_avg(modes, csv_path, find_mode='0.1')

### (数据处理) 重建数据均值计算(时间动态图)

In [ ]:
import sys
sys.path.append(r"C:\Users\Administrator\Desktop\code\ST-Conv")
import argparse
import pandas as pd
import numpy as np
from Original_Point_Data.tool.origin_utils import OriginUtils
from tool.utils import Util

def calculate_reconstruction_avg(modes, info_path, reconstruction_paths, find_mode='0.1'):
    all_networks_recon_avg = {}  # 所有网络和所有重建方法的结果

    for mode in modes:
        print(f"Processing network: {mode}")
        network_avg_data = {}  # 当前网络的重建数据均值

        for method, recon_path in reconstruction_paths.items():
            print(f"Processing reconstruction method: {method}")
            ali_list = OriginUtils.find_grid_points(mode, info_path, find_mode=find_mode)
            point_list = [[item[2], item[1], item[0]] for item in ali_list]
            point_list.sort(key=lambda x: (-x[0], x[1]))

            recon_data_frames = []
            recon_data = np.load(recon_path)

            for lat, lon, point_name in point_list:
                _, (lat_ind, lon_ind) = OriginUtils.find_grid_coordinates(lon, lat, find_mode=find_mode)
                point_recon_data = recon_data[:, lat_ind, lon_ind]
                time_index = pd.date_range(start='2016-01-01 03:00:00', periods=len(point_recon_data), freq='3H')
                point_recon_df = pd.DataFrame(point_recon_data, index=time_index, columns=[point_name])
                recon_data_frames.append(point_recon_df)

            # 时间对齐并计算均值
            if recon_data_frames:
                aligned_recon_data = pd.concat(recon_data_frames, axis=1, join='outer').mean(axis=1)
                network_avg_data[method] = aligned_recon_data

        # 将当前网络的结果添加到所有网络的结果中
        all_networks_recon_avg[mode] = network_avg_data

    return all_networks_recon_avg


def main(args):
    Original_Point_path_argu = Util.load_config(args.Original_Point_path_config)
    path_argu = Util.load_config(args.path_config_path)
    return Original_Point_path_argu, path_argu

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--Original_Point_path_config', type=str, default='../Original_Point_Data/config/path_config.yaml')
    parser.add_argument('--path_config_path', type=str, default='../config/path_config.yaml')
    args = parser.parse_known_args()[0]
    Original_Point_path_argu, path_argu = main(args)
    
    start_time = '2016-01-01 03:00:00'
    modes = ['Shiquanhe', 'Ali', 'Maqu', 'Naqu', 'CTP']
    cal_time = '3'
    csv_path = Original_Point_path_argu['info']
    reconstruction_paths = {
        'PSC-FENet-SP': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SP_PSC-FENet.npy",
        'PSC-FENet-ST': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\ST_PSC-FENet.npy",
        'LSTM': r"D:\Data_Store\Dataset\Reconstruction\LSTM\LSTM1.npy",
        'RF': r"D:\Data_Store\Dataset\Reconstruction\RF\RF_reconstructed_data.npy",
        'ResNet-SP': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SP_ResNet.npy",
        'ResNet-ST': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\ST_ResNet.npy",
        'UNet-SP': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SP_UNet.npy",
        'UNet-ST': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\ST_UNet.npy",
        'VIT-SP': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SP_ViT.npy",
        'VIT-ST': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\ST_ViT.npy",
        'Temporal': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\Temporal.npy"
    }

    #selected_model_names = ['PSC-FENet-ST','PSC-FENet-SP','ResNet-ST', 'UNet-ST', 'VIT-ST',  'LSTM', 'RF', 'Temporal']
    selected_model_names = ['PSC-FENet-ST','PSC-FENet-SP', 'LSTM', 'RF', 'Temporal']
    selected_reconstruction_paths = {name: paths for name, paths in reconstruction_paths.items() if name in selected_model_names}
    all_networks_recon_avg = calculate_reconstruction_avg(modes, csv_path , selected_reconstruction_paths, find_mode='0.1')
    np.save(r"D:\Data_Store\Dataset\Results\all_networks_recon_avg.npy",all_networks_recon_avg)
    
    Original_paths = {'Original': r"D:\Data_Store\Dataset\Reconstruction\37SM\soil_moisture.npy"}
    all_networks_origin_avg = calculate_reconstruction_avg(modes, csv_path , Original_paths, find_mode='0.37')
    np.save(r"D:\Data_Store\Dataset\Results\all_networks_origin_avg.npy",all_networks_origin_avg)

    Precipitation_paths = {'Precipitation': r"D:\\Data_Store\\Dataset\\Reconstruction\\37SM\\hourlyPrecipRateGC.npy"}
    all_networks_Precipitation_avg = calculate_reconstruction_avg(modes, csv_path , Precipitation_paths, find_mode='0.1')
    np.save(r"D:\Data_Store\Dataset\Results\all_networks_Precipitation_avg.npy",all_networks_Precipitation_avg)
    for network, methods_data in all_networks_Precipitation_avg.items():
        print(f"Network: {network}")
        for method, data in methods_data.items():
            print(f"Method: {method}")
            print(data.head())
            print("\n")

### (数据处理) 拼接原位、重建和卫星均值数据(时间动态图)

In [ ]:
def calculate_all_networks_data(modes, avg_data_df, all_networks_recon_avg, all_networks_origin_avg, all_networks_Precipitation_avg):
    all_networks_data = {}
    end_time = pd.Timestamp('2018-12-31 21:00:00')

    for network in modes:
        print(f"Processing data for network: {network}")
        network_data_frames = []

        # 添加原始站点数据
        if network in avg_data_df:
            filtered_avg_data = avg_data_df[network][avg_data_df[network].index <= end_time]
            network_data_frames.append(filtered_avg_data.rename('Original_Data'))

        # 添加原始卫星数据
        if network in all_networks_origin_avg and 'Original' in all_networks_origin_avg[network]:
            filtered_origin_data = all_networks_origin_avg[network]['Original'][all_networks_origin_avg[network]['Original'].index <= end_time]
            network_data_frames.append(filtered_origin_data.rename('Satellite_Data'))

        # 添加各种重建方法的数据
        for method, recon_data in all_networks_recon_avg[network].items():
            filtered_recon_data = recon_data[recon_data.index <= end_time]
            network_data_frames.append(filtered_recon_data.rename(method))

        # 添加降水量数据
        if network in all_networks_Precipitation_avg:
            filtered_precip_data = all_networks_Precipitation_avg[network]['Precipitation'][all_networks_Precipitation_avg[network]['Precipitation'].index <= end_time]
            network_data_frames.append(filtered_precip_data.rename('Precipitation'))

        # 合并所有数据帧，确保使用外连接以保留所有时间点
        if network_data_frames:
            network_combined_data = pd.concat(network_data_frames, axis=1, join='outer')
            network_combined_data = network_combined_data[network_combined_data.index <= end_time]
            all_networks_data[network] = network_combined_data

    return all_networks_data

def filter_data_for_spring_summer(all_networks_data):
    filtered_networks_data = {}

    for network, data in all_networks_data.items():
        filtered_data = data[(data.index.month >= 4) & (data.index.month <= 10)]
        filtered_networks_data[network] = filtered_data

    return filtered_networks_data

all_networks_data = calculate_all_networks_data(modes, avg_data_df, all_networks_recon_avg, all_networks_origin_avg, all_networks_Precipitation_avg)
filtered_all_networks_data = filter_data_for_spring_summer(all_networks_data)

for network, data in filtered_all_networks_data.items():
    print(f"Data for network {network}:")
    print(data.head())
    print("\n")

### (绘图) V1.0 时间动态图

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.font_manager import FontProperties
from matplotlib.collections import PathCollection

def create_and_draw_subplots(filtered_all_networks_data):
    # 创建图形对象,绘制顺序依赖于fig定义顺序
    fig = plt.figure(figsize=(36, 30))

    total_height = 0.8
    single_height = total_height / 5
    gap = 0.018

    ax1 = fig.add_axes([0.05, 0.05 + 4 * (single_height + gap), 0.9, single_height]) # 上1
    ax2 = fig.add_axes([0.05, 0.05 + 3 * (single_height + gap), 0.9, single_height]) # 上2
    ax3 = fig.add_axes([0.05, 0.05 + 2 * (single_height + gap), 0.9, single_height]) # 上3
    ax4 = fig.add_axes([0.05, 0.05 + 1 * (single_height + gap), 0.9, single_height]) # 上4
    ax5 = fig.add_axes([0.05, 0.05, 0.9, single_height]) # 上5

    # 绘制网络数据
    plot_network_data(ax1, filtered_all_networks_data['Shiquanhe'], '(a)Shiquanhe', axhline_low=0.15, axhline_up=0.3, use_legend=True, use_xlabel=False)
    plot_network_data(ax2, filtered_all_networks_data['Ali'], '(b)Ali', axhline_low=0.15, axhline_up=0.3, use_legend=False, use_xlabel=False)
    plot_network_data(ax3, filtered_all_networks_data['Maqu'], '(c)Maqu', axhline_low=0.2, axhline_up=0.4, use_legend=False, use_xlabel=False)
    plot_network_data(ax4, filtered_all_networks_data['Naqu'], '(d)Naqu', axhline_low=0.2, axhline_up=0.4, use_legend=False, use_xlabel=False)
    plot_network_data(ax5, filtered_all_networks_data['CTP'], '(e)CTP', axhline_low=0.2, axhline_up=0.4, use_legend=False, use_xlabel=True)

    plt.show()
def plot_network_data(ax, network_data, network_name,axhline_low=0.2,axhline_up=0.4, use_legend=True, use_xlabel=True):
    font = FontProperties()
    font.set_family('Times New Roman')
    font.set_size(24)
    plt.rcParams['font.family'] = 'Times New Roman'
    plt.rcParams['font.size'] = 24

    dates = network_data.index
    dates_shift = np.arange(len(dates))
    plot_settings = {
        'Original_Data': {'color': 'red', 'label': 'Original_Data', 'linewidth': 2, 'linestyle': '-', 'zorder': 10, 'alpha': 1.0},
        'Satellite_Data': {'color': 'black', 'label': 'Satellite_Data', 'linewidth': 2, 'linestyle': '-', 'zorder': 9, 'alpha': 1.0},
        'PSC-FENet-ST': {'color': '#a40062', 'label': 'PSC-FENet-ST', 's': 20, 'marker': 'D', 'edgecolors': '#a40062', 'facecolors': 'none', 'zorder': 8, 'alpha': 0.8, 'linewidths': 1},
        'RF': {'color': '#0b6909', 'label': 'Random Forest', 's': 20, 'marker': '+', 'edgecolors': '#0b6909', 'facecolors': '#0b6909', 'zorder': 7, 'alpha': 0.85, 'linewidths': 1.5},
        'ResNet-ST': {'color': '#005fb8', 'label': 'ResNet-ST', 's': 20, 'marker': '^', 'edgecolors': '#005fb8', 'facecolors': 'none', 'zorder': 6, 'alpha': 0.75, 'linewidths': 1},
        'UNet-ST': {'color': '#70e000', 'label': 'UNet-ST', 's': 20, 'marker': 'v', 'edgecolors': '#70e000', 'facecolors': 'none', 'zorder': 5, 'alpha': 0.70, 'linewidths': 1},
        'LSTM': {'color': '#694e41', 'label': 'LSTM', 's': 20, 'marker': 'X', 'edgecolors': '#694e41', 'facecolors': 'none', 'zorder': 4, 'alpha': 0.65, 'linewidths': 1},
        'VIT-ST': {'color': '#faa307', 'label': 'VIT-ST', 's': 20, 'marker': '*', 'edgecolors': '#faa307', 'facecolors': 'none', 'zorder': 3, 'alpha': 0.60, 'linewidths': 1},
        'Temporal': {'color': '#ff8fa3', 'label': 'Temporal', 's': 20, 'marker': 'h', 'edgecolors': '#ff8fa3', 'facecolors': 'none', 'zorder': 2, 'alpha': 0.55, 'linewidths': 1},
    }

    ax2 = ax.twinx()
    for year in range(dates[0].year, dates[-1].year + 1):
        start_date = pd.Timestamp(year=year, month=4, day=1)
        start_idx = np.where(dates == start_date)[0][0]
        # 绘制虚线
        ax.axvline(x=dates_shift[start_idx], color='grey', linestyle='--', linewidth=1)

    if 'Original_Data' in network_data.columns:
        ax.plot(dates_shift, network_data['Original_Data'], **plot_settings['Original_Data'])
    
    if 'Satellite_Data' in network_data.columns:
        os_data = network_data['Satellite_Data'].dropna()
        os_dates_shift = dates_shift[network_data.index.isin(os_data.index)]
        ax.plot(os_dates_shift, os_data, **plot_settings['Satellite_Data'])

    for column in network_data.columns:
        if column not in ['Original_Data', 'Satellite_Data', 'Precipitation'] and column in plot_settings:
            ax.scatter(dates_shift, network_data[column], **plot_settings[column])

    if 'Precipitation' in network_data.columns:
        # 绘制柱状图
        ax2.patch.set_alpha(0.0)
        ax2.bar(dates_shift, network_data['Precipitation'], width=0.02, color='#47a0ff', edgecolor='#47a0ff', label='Precipitation', alpha=0.8, zorder=10)
        ax2.set_ylabel('Precipitation (mm)', fontproperties=font, color='#47a0ff')

    # 设置时间刻度
    ax.set_xticks(dates_shift[::480])
    ax.set_xticklabels([date.strftime('%Y-%m') for date in dates[::480]], rotation=0, fontproperties=font)
    # 设置刻度
    ax.tick_params(axis='x', which='both', colors='black', length=4, width=2, labelsize=24, direction='out', pad=6, bottom=True, top=False, labelbottom=True,labeltop=False)
    ax.tick_params(axis='y', which='both', colors='black', length=4, width=2, labelsize=24, direction='out', pad=6, left=True, right=False, labelleft=True,labelright=False)
    ax2.tick_params(axis='y', which='both', colors='#47a0ff', length=4, width=2, labelsize=24, direction='out', pad=6, left=False, right=True, labelleft=False,labelright=True)
    # 设置图例
    lines, labels = ax.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax.legend(lines + lines2, labels + labels2, loc='upper right', fancybox=True, shadow=True, ncol=5, prop=font)

    plt.xlim([dates_shift.min(), dates_shift.max()])
    plt.ylim([0, network_data.max().max() + 0.1])

    ax.axhline(axhline_low, color='black', linestyle='--', linewidth=1.5)
    ax.axhline(axhline_up, color='black', linestyle='--', linewidth=1.5)
    # text绘制图名
    ax.text(0.001, 0.945, f'{network_name}', transform=ax.transAxes, fontsize=30, fontname='Times New Roman', verticalalignment='top')

    if use_legend:
        lines, labels = ax.get_legend_handles_labels()
        lines2, labels2 = ax2.get_legend_handles_labels()
        legend = ax.legend(lines + lines2, labels + labels2, loc='upper left', fancybox=False, shadow=False, ncol=5, frameon=False, prop=font, bbox_to_anchor=(0.065, 1.035))
        for legend_handle in legend.legendHandles:
            if isinstance(legend_handle, PathCollection):
                legend_handle.set_sizes([80])
    else:
        ax.legend().set_visible(False)
        ax2.legend().set_visible(False)
    if use_xlabel==True:
        ax.set_xlabel('Date', fontsize=26, fontproperties=font)
    ax.set_ylabel(r'Soil Moisture ($m^3/m^{-3}$)', fontsize=26, fontproperties=font)
    ax2.set_ylabel(r'Precipitation ($mm /3\,hours$)', fontsize=26, fontproperties=font)
    for label in ax.get_xticklabels() + ax.get_yticklabels() + ax2.get_yticklabels():
        label.set_fontsize(24)
    ax2.grid(False)
    ax.grid(False)
    plt.savefig(r"C:\Users\Administrator\Desktop\Draw\zhexiantu.jpeg", dpi=500, format='jpeg', bbox_inches='tight')

create_and_draw_subplots(filtered_all_networks_data)

### (绘图) V2.0 时间动态图

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.font_manager import FontProperties
from matplotlib.collections import PathCollection

def create_and_draw_subplots(filtered_all_networks_data):
    # 创建图形对象,绘制顺序依赖于fig定义顺序
    fig = plt.figure(figsize=(36, 30))

    total_height = 0.8
    single_height = total_height / 5
    gap = 0.018

    ax1 = fig.add_axes([0.05, 0.05 + 4 * (single_height + gap), 0.9, single_height]) # 上1
    ax2 = fig.add_axes([0.05, 0.05 + 3 * (single_height + gap), 0.9, single_height]) # 上2
    ax3 = fig.add_axes([0.05, 0.05 + 2 * (single_height + gap), 0.9, single_height]) # 上3
    ax4 = fig.add_axes([0.05, 0.05 + 1 * (single_height + gap), 0.9, single_height]) # 上4
    ax5 = fig.add_axes([0.05, 0.05, 0.9, single_height]) # 上5

    # 绘制网络数据
    plot_network_data(ax1, filtered_all_networks_data['Shiquanhe'], '(a)Shiquanhe', axhline_low=0.15, axhline_up=0.3, use_legend=True, use_xlabel=False)
    plot_network_data(ax2, filtered_all_networks_data['Ali'], '(b)Ali', axhline_low=0.15, axhline_up=0.3, use_legend=False, use_xlabel=False)
    plot_network_data(ax3, filtered_all_networks_data['Maqu'], '(c)Maqu', axhline_low=0.2, axhline_up=0.4, use_legend=False, use_xlabel=False)
    plot_network_data(ax4, filtered_all_networks_data['Naqu'], '(d)Naqu', axhline_low=0.2, axhline_up=0.4, use_legend=False, use_xlabel=False)
    plot_network_data(ax5, filtered_all_networks_data['CTP'], '(e)CTP', axhline_low=0.2, axhline_up=0.4, use_legend=False, use_xlabel=True)

    plt.show()
def plot_network_data(ax, network_data, network_name,axhline_low=0.2,axhline_up=0.4, use_legend=True, use_xlabel=True):
    font = FontProperties()
    font.set_family('Times New Roman')
    font.set_size(24)
    plt.rcParams['font.family'] = 'Times New Roman'
    plt.rcParams['font.size'] = 24

    dates = network_data.index
    dates_shift = np.arange(len(dates))
    plot_settings = {
        'Original_Data': {'color': 'red', 'label': 'Original_Data', 'linewidth': 2, 'linestyle': '-', 'zorder': 10, 'alpha': 1.0},
        'Satellite_Data': {'color': 'black', 'label': 'Satellite_Data', 'linewidth': 2, 'linestyle': '-', 'zorder': 9, 'alpha': 1.0},
        'PSC-FENet-ST': {'color': '#a40062', 'label': 'PSC-FENet-ST', 's': 20, 'marker': '^', 'edgecolors': '#a40062', 'facecolors': 'none', 'zorder': 6, 'alpha': 0.9, 'linewidths': 1},
        'PSC-FENet-SP': {'color': '#694e41', 'label': 'PSC-FENet-SP', 's': 20, 'marker': 'v', 'edgecolors': '#694e41', 'facecolors': 'none', 'zorder': 6, 'alpha': 0.9, 'linewidths': 1},
        'RF': {'color': '#70e000', 'label': 'Random Forest', 's': 20, 'marker': '*', 'edgecolors': '#70e000', 'facecolors': '#0b6909', 'zorder': 8, 'alpha': 0.85, 'linewidths': 1},
        'LSTM': {'color': '#005fb8', 'label': 'LSTM', 's': 20, 'marker': 'X', 'edgecolors': '#005fb8', 'facecolors': 'none', 'zorder': 7, 'alpha': 0.80, 'linewidths': 1},
        'Temporal': {'color': '#faa307', 'label': 'Temporal', 's': 20, 'marker': 'D', 'edgecolors': '#faa307', 'facecolors': 'none', 'zorder': 4, 'alpha': 0.70, 'linewidths': 1},
    }

    ax2 = ax.twinx()
    for year in range(dates[0].year, dates[-1].year + 1):
        start_date = pd.Timestamp(year=year, month=4, day=1)
        start_idx = np.where(dates == start_date)[0][0]
        # 绘制虚线
        ax.axvline(x=dates_shift[start_idx], color='grey', linestyle='--', linewidth=1)

    if 'Original_Data' in network_data.columns:
        ax.plot(dates_shift, network_data['Original_Data'], **plot_settings['Original_Data'])
    
    if 'Satellite_Data' in network_data.columns:
        os_data = network_data['Satellite_Data'].dropna()
        os_dates_shift = dates_shift[network_data.index.isin(os_data.index)]
        ax.plot(os_dates_shift, os_data, **plot_settings['Satellite_Data'])

    for column in network_data.columns:
        if column not in ['Original_Data', 'Satellite_Data', 'Precipitation'] and column in plot_settings:
            ax.scatter(dates_shift, network_data[column], **plot_settings[column])

    if 'Precipitation' in network_data.columns:
        # 绘制柱状图
        ax2.patch.set_alpha(0.0)
        ax2.bar(dates_shift, network_data['Precipitation'], width=0.02, color='#47a0ff', edgecolor='#47a0ff', label='Precipitation', alpha=0.8, zorder=10)
        ax2.set_ylabel('Precipitation (mm)', fontproperties=font, color='#47a0ff')

    # 设置时间刻度
    ax.set_xticks(dates_shift[::480])
    ax.set_xticklabels([date.strftime('%Y-%m') for date in dates[::480]], rotation=0, fontproperties=font)
    # 设置刻度
    ax.tick_params(axis='x', which='both', colors='black', length=4, width=2, labelsize=24, direction='out', pad=6, bottom=True, top=False, labelbottom=True,labeltop=False)
    ax.tick_params(axis='y', which='both', colors='black', length=4, width=2, labelsize=24, direction='out', pad=6, left=True, right=False, labelleft=True,labelright=False)
    ax2.tick_params(axis='y', which='both', colors='#47a0ff', length=4, width=2, labelsize=24, direction='out', pad=6, left=False, right=True, labelleft=False,labelright=True)


    plt.xlim([dates_shift.min(), dates_shift.max()])
    plt.ylim([0, network_data.max().max() + 0.1])

    ax.axhline(axhline_low, color='black', linestyle='--', linewidth=1.5)
    ax.axhline(axhline_up, color='black', linestyle='--', linewidth=1.5)
    # text绘制图名
    ax.text(0.001, 0.945, f'{network_name}', transform=ax.transAxes, fontsize=30, fontname='Times New Roman', verticalalignment='top')

    # 设置图例
    if use_legend:
        lines, labels = ax.get_legend_handles_labels()
        lines2, labels2 = ax2.get_legend_handles_labels()
        legend = ax.legend(lines + lines2, labels + labels2, loc='upper left', fancybox=False, shadow=False, ncol=4, frameon=False, prop=font, bbox_to_anchor=(0.065, 1.035))
        for legend_handle in legend.legendHandles:
            if isinstance(legend_handle, PathCollection):
                legend_handle.set_sizes([80])
    else:
        ax.legend().set_visible(False)
        ax2.legend().set_visible(False)
    if use_xlabel==True:
        ax.set_xlabel('Date', fontsize=26, fontproperties=font)
    ax.set_ylabel(r'Soil Moisture ($m^3/m^{-3}$)', fontsize=26, fontproperties=font)
    ax2.set_ylabel(r'Precipitation ($mm /3\,hours$)', fontsize=26, fontproperties=font)
    for label in ax.get_xticklabels() + ax.get_yticklabels() + ax2.get_yticklabels():
        label.set_fontsize(24)
    ax2.grid(False)
    ax.grid(False)
    plt.savefig(r"C:\Users\Administrator\Desktop\Draw\zhexiantu.jpeg", dpi=500, format='jpeg', bbox_inches='tight')

create_and_draw_subplots(filtered_all_networks_data)

### (数据处理) 逐站点指标计算

In [ ]:
import pandas as pd

def process_station_data(file_path):
    networks_data = {
        'Other Networks': pd.DataFrame(),
        'CTP': pd.DataFrame(),
    }
    xls = pd.ExcelFile(file_path)
    for sheet_name in xls.sheet_names:
        df = xls.parse(sheet_name)
        split_data = df['站点名'].str.rsplit('_', n=2, expand=True)
        df['point_name'] = split_data[0]
        df['lon'] = split_data[1].astype(float)
        df['lat'] = split_data[2].astype(float)
        df_renamed = df.rename(columns={'PearsonR': 'R'})
        df_sorted = df_renamed.sort_values(by=['lat', 'lon'], ascending=[True, False])
        best_records = df_sorted.groupby(['lat','lon'], as_index=False).apply(lambda x: x.loc[x['R'].idxmax()]).reset_index(drop=True)
        best_records = best_records[['point_name', 'lon', 'lat', 'RMSE', 'ubRMSE', 'R', 'Bias']]
        if sheet_name in ['Shiquanhe', 'Ali','Maqu', 'Naqu']:
            networks_data['Other Networks'] = pd.concat([networks_data['Other Networks'], best_records])
        elif sheet_name in ['CTP']:
            networks_data[sheet_name] = pd.concat([networks_data[sheet_name], best_records])
        if not networks_data['Other Networks'].empty:
            networks_data['Other Networks'] = networks_data['Other Networks'].sort_values(by=['lat', 'lon'], ascending=[True, False])
        if not networks_data['CTP'].empty:
            networks_data['CTP'] = networks_data['CTP'].sort_values(by=['lat', 'lon'], ascending=[True, False])

    selected_points = set()
    for key in networks_data:
        if not networks_data[key].empty:
            selected_points.update(networks_data[key]['point_name'].unique())

    return networks_data['Other Networks'], networks_data['CTP'], selected_points


def process_station_file_paths(file_paths, selected_points):
    networks_data_all_files = {
        'Other Networks': [],
        'CTP': [],
    }

    for network_name, file_path in file_paths.items():
        xls = pd.ExcelFile(file_path)
        networks_data_single_file = {
            'Other Networks': pd.DataFrame(),
            'CTP': pd.DataFrame(),
        }
        for sheet_name in xls.sheet_names:
            df = xls.parse(sheet_name)
            split_data = df['站点名'].str.rsplit('_', n=2, expand=True)
            df['point_name'] = split_data[0]
            df['lon'] = split_data[1].astype(float)
            df['lat'] = split_data[2].astype(float)
            df_renamed = df.rename(columns={'PearsonR': 'R'})
            df_sorted = df_renamed.sort_values(by=['lat', 'lon'], ascending=[True, False])

            best_records = df_sorted[df_sorted['point_name'].isin(selected_points)]
            best_records = best_records[['point_name', 'lon', 'lat', 'RMSE', 'ubRMSE', 'R', 'Bias']]

            if sheet_name in ['Shiquanhe', 'Ali', 'Maqu', 'Naqu']:
                networks_data_single_file['Other Networks'] = pd.concat([networks_data_single_file['Other Networks'], best_records])
            elif sheet_name in ['CTP']:
                networks_data_single_file['CTP'] = pd.concat([networks_data_single_file['CTP'], best_records])
                
        for network_type in networks_data_single_file:
            if not networks_data_single_file[network_type].empty:
                networks_data_all_files[network_type].append(networks_data_single_file[network_type])

    for network_type in networks_data_all_files:
        if networks_data_all_files[network_type]:
            combined_data = pd.concat(networks_data_all_files[network_type])
            mean_data = combined_data.groupby(['point_name', 'lon', 'lat'], as_index=False).mean()
            networks_data_all_files[network_type] = mean_data.sort_values(by=['lat', 'lon'], ascending=[True, False])
        else:
            networks_data_all_files[network_type] = pd.DataFrame()

    return networks_data_all_files['Other Networks'], networks_data_all_files['CTP']
'''
file_paths = {
    'PSC-FENet-ST': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\STPSC-FENet_station_metrics.xlsx",
    'PSC-FENet-SP': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SPPSC-FENet_station_metrics.xlsx",
    'PSC-FENet-ST32': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\STPSC-FENet36_station_metrics.xlsx",
    'PSC-FENet-SP32': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SPPSC-FENet36_station_metrics.xlsx",
    'ResNet-ST': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\STRESNET_station_metrics.xlsx",
    'ResNet-SP': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SPRESNET_station_metrics.xlsx",
    'UNet-ST': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\STUNET_station_metrics.xlsx",
    'UNet-SP': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SPUNET_station_metrics.xlsx",
    'VIT-ST': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\STVIT_station_metrics.xlsx",
    'VIT-SP': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SPVIT_station_metrics.xlsx",
    'RF': r"D:\Data_Store\Dataset\Reconstruction\RF\RF_station_metrics.xlsx",
    'LSTM': r"D:\Data_Store\Dataset\Reconstruction\LSTM\LSTM_station_metrics.xlsx",
    'Temporal': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\TEMPORAL_station_metrics.xlsx"
}
'''

file_paths = {
    'PSC-FENet-ST': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\STPSC-FENet_station_metrics.xlsx",
    'RF': r"D:\Data_Store\Dataset\Reconstruction\RF\RF_station_metrics.xlsx",
    'LSTM': r"D:\Data_Store\Dataset\Reconstruction\LSTM\LSTM_station_metrics.xlsx",
    'Temporal': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\TEMPORAL_station_metrics.xlsx"
}

file_path = r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\STPSC-FENet_station_metrics.xlsx"

df_other_networks, df_ctp, selected_points = process_station_data(file_path)
all_methods_other_networks,all_methods_ctp = process_station_file_paths(file_paths, selected_points)

print(all_methods_other_networks.head(5))
print(all_methods_ctp.head(5))

print(df_other_networks.head(5))
print(df_ctp.head(5))

### (数据处理) 逐站点全时刻指标均值

In [ ]:
import pandas as pd

def create_combined_data(time_points, network_names, file_paths):
    all_data = {time: {network: pd.DataFrame() for network in network_names} for time in time_points}
    for model, file_path in file_paths.items():
        xls = pd.ExcelFile(file_path)
        for time_point in time_points:
            networks_data = {network: pd.DataFrame() for network in network_names}
            for sheet_name in xls.sheet_names:
                if f"_{time_point}h" in sheet_name:
                    df = xls.parse(sheet_name)
                    split_data = df['站点名'].str.rsplit('_', n=2, expand=True)
                    df['point_name'] = split_data[0]
                    df['lon'] = split_data[1].astype(float)
                    df['lat'] = split_data[2].astype(float)
                    df_renamed = df.rename(columns={'PearsonR': 'R'})
                    df_sorted = df_renamed.sort_values(by=['lat', 'lon'], ascending=[False, True])
                    best_records = df_sorted.groupby(['lat', 'lon'], as_index=False).apply(lambda x: x.loc[x['R'].idxmax()]).reset_index(drop=True)
                    best_records = best_records[['point_name', 'lon', 'lat', 'RMSE', 'ubRMSE', 'R', 'Bias']]
                    for network in network_names:
                        if network in sheet_name:
                            networks_data[network] = pd.concat([networks_data[network], best_records], ignore_index=True)
            for network in network_names:
                all_data[time_point][network] = pd.concat([all_data[time_point][network], networks_data[network]], ignore_index=True)

    return all_data

time_points = ['0', '3', '6', '9', '12', '15', '18', '21']
network_names = ['Shiquanhe', 'Ali', 'Maqu', 'Naqu', 'CTP']
'''
file_paths = {
    'PSC-FENet-ST': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\STPSCFENet_station_time_metrics.xlsx",
    'PSC-FENet-SP': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SPPSCFENet_station_time_metrics.xlsx",
    'PSC-FENet-ST32': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\STPSC-FENet36_station_metrics.xlsx",
    'PSC-FENet-SP32': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SPPSC-FENet36_station_metrics.xlsx",
    'ResNet-ST': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\STRESNET_station_time_metrics.xlsx",
    'ResNet-SP': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SPRESNET_station_time_metrics.xlsx",
    'UNet-ST': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\STUNET_station_time_metrics.xlsx",
    'UNet-SP': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SPUNET_station_time_metrics.xlsx",
    'VIT-ST': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\STVIT_station_time_metrics.xlsx",
    'VIT-SP': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SPVIT_station_time_metrics.xlsx",
    'RF': r"D:\Data_Store\Dataset\Reconstruction\RF\RF_station_time_metrics.xlsx",
    'LSTM': r"D:\Data_Store\Dataset\Reconstruction\LSTM\LSTM_station_time_metrics.xlsx",
    'Temporal': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\TEMPORAL_station_time_metrics.xlsx"
}
'''
file_paths = {
    'PSC-FENet-ST': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\STPSCFENet_station_time_metrics.xlsx",
    'LSTM': r"D:\Data_Store\Dataset\Reconstruction\LSTM\LSTM_station_time_metrics.xlsx",
    'Temporal': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\TEMPORAL_station_time_metrics.xlsx"
}
#selected_models = ['PSC-FENet-ST', 'PSC-FENet-SP','PSC-FENet-ST32', 'PSC-FENet-SP32','RF',  'LSTM', 'ResNet-SP', 'UNet-SP', 'ResNet-ST', 'UNet-ST', 'VIT-SP','Temporal']
selected_models = ['PSC-FENet-ST','RF',  'LSTM','Temporal']
selected_reconstruction_paths = {name: paths for name, paths in file_paths.items() if name in selected_models}

all_data = create_combined_data(time_points, network_names, selected_reconstruction_paths)

df_example = all_data['3']['Ali']  # 获取3:00的CTP网络的DataFrame
print(df_example.head(50))


### (绘图) V2.0 原位站点误差 柱状-折线图 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.font_manager import FontProperties
from matplotlib.ticker import MultipleLocator
from matplotlib.ticker import MaxNLocator
import colorsys
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
from matplotlib.patches import FancyBboxPatch
import matplotlib.colors as mcolors
import seaborn as sns

np.random.seed(0)  # For reproducibility
data3 = pd.DataFrame({
    'point_name': [f'SKU{i}' for i in range(1, 11)],
    'RMSE': np.random.rand(10) * 10 + 50,
    'ubRMSE': np.random.rand(10) * 5 + 25,
    'R': np.random.rand(10) * 1,
    'Bias': np.random.rand(10) * 0.05
})

def plot_design_changes_extended(data, all_method_data, height = 0.4, Gradation_degree = 0.014, Gradation_degree_all_method = 0.024, figsize = (6,12), start_colors_rmse = ['#caf0f8','#DFD8FD'], start_colors_ubrmse = ['#e9f5db','#6BF7D4'],name = None, method=None):

    def adjust_color(color, brightness_factor):
        rgb = mcolors.hex2color(color)
        hls = colorsys.rgb_to_hls(*rgb)
        new_hls = (hls[0], max(0, min(1, brightness_factor * hls[1])), hls[2])
        # 将调整后的HLS颜色转换回RGB，然后转换为HEX
        new_rgb = colorsys.hls_to_rgb(*new_hls)
        return mcolors.to_hex(new_rgb)
    
    sns.set(style="whitegrid", font='Times New Roman')
    matplotlib.rcParams['font.family'] = 'Times New Roman'
    matplotlib.rcParams['axes.labelweight'] = 'bold'
    legend_font = FontProperties(family='Times New Roman', size=12)
    font_prop = FontProperties(family='Times New Roman', size=12)
    fig, ax1 = plt.subplots(figsize=figsize)
    for item in ([ax1.xaxis.label, ax1.yaxis.label] + ax1.get_xticklabels() + ax1.get_yticklabels()):
        item.set_fontproperties(font_prop)

    # 避免重叠
    y = np.arange(len(data))
    y1 = [i - height/1.8 for i in y]
    y2 = [i + height/1.8 for i in y]
    yy = data['point_name'].to_list()

    # ax1 RMSE
    ha= 'left' if method == 'left' else 'right'
    colors_rmse = [adjust_color(start_colors_rmse[0], 1 - Gradation_degree * i) for i in range(len(data['RMSE']))]
    colors_rmse_all_method = [adjust_color(start_colors_rmse[1], 1 - Gradation_degree_all_method * i) for i in range(len(data['RMSE']))]
    for i, (value,all_method_value, pos) in enumerate(zip(data['RMSE'], all_method_data['RMSE'], y1)):
        ax1.barh(pos, all_method_value, height=height, color=colors_rmse_all_method[-i-1], alpha=0.6, zorder=5)

        ax1.barh(pos, value, height=height, color=colors_rmse[-i-1], alpha=0.6,zorder=5)
        ax1.text(0, pos, f'{value:.3f}', size=10, ha=ha, va='center', fontproperties=font_prop, zorder=5)

    ax1.set_yticks(y)
    ax1.set_yticklabels(yy)

    # ax2 ubRMSE
    ax2 = ax1.twiny()
    colors_ubrmse = [adjust_color(start_colors_ubrmse[0], 1 - Gradation_degree * i) for i in range(len(data['ubRMSE']))]
    colors_ubrmse_all_method = [adjust_color(start_colors_ubrmse[1], 1 - Gradation_degree_all_method * i) for i in range(len(data['ubRMSE']))]
    for i, (value,all_method_value, pos) in enumerate(zip(data['ubRMSE'], all_method_data['ubRMSE'], y2)):
        ax2.barh(pos, all_method_value, height=height, color=colors_ubrmse_all_method[-i-1], alpha=0.6, zorder=5)

        ax2.barh(pos, value, height=height, color=colors_ubrmse[-i-1], alpha=0.6, zorder=5)
        ax2.text(0, pos, f'{value:.3f}', size=10, ha=ha, va='center', fontproperties=font_prop, zorder=5)

    # ax3 R
    color_ax3 = ['#FFB110', '#FFF571']
    ax3 = ax1.twiny()
    ax3.plot(data['R'], y2, label='R',linestyle='--', color=color_ax3[0], linewidth=1.5, marker='X', markeredgecolor='lightgrey', markersize=10,alpha=0.5,zorder=1)
    ax3.scatter(all_method_data['R'], y2, marker='X', edgecolors='black', color=color_ax3[1], alpha=0.6, zorder=2, s=40) 
    for (r_val, y_val), r_all_method in zip(zip(data['R'], y2), all_method_data['R']):
        ax3.plot([r_val, r_all_method], [y_val, y_val], color=color_ax3[1], linewidth=1.5, zorder=1.5)

    # ax4 Bias
    color_ax4 = ['#9602C0', '#C992EC']
    ax4 = ax1.twiny()
    ax4.plot(data['Bias'], y1, '--', label='Bias',color=color_ax4[0], linewidth=1.5, marker='P',markeredgecolor='grey', markersize=10,alpha=0.5,zorder=1) 
    ax4.scatter(all_method_data['Bias'], y1, marker='P', edgecolors='black', color=color_ax4[1], alpha=0.6, zorder=2, s=50)
    for (bias_val, y_val), bias_all_method in zip(zip(data['Bias'], y1), all_method_data['Bias']):
        ax4.plot([bias_val, bias_all_method], [y_val, y_val], color=color_ax4[1], linewidth=1.5, zorder=1.5)

    if method == 'left':
        split_points = ['SQ14', 'ALI01', 'NST_05']
        current_order_index = {name: i for i, name in enumerate(data['point_name'])}
        split_indices = [current_order_index[point] for point in split_points if point in current_order_index]
        split_indices.append(len(data))

        for i in range(len(split_indices)):
            # 计算开始和结束位置
            start = 0 if i == 0 else split_indices[i - 1]
            end = len(data['point_name']) if i == len(split_indices)-1 else split_indices[i]
            # 提取对应段的数据
            segment = data.iloc[start:end]
            mean_R = np.mean(segment['R'])
            # 计算y轴的范围
            y_start = y1[start] - height / 1.8 if start < len(y1) else y1[-1] - height / 1.8
            y_end = y2[end - 1] + height / 1.8 if end - 1 < len(y2) else y2[-1] + height / 1.8
            # 绘制线段
            ax3.plot([mean_R, mean_R], [y_start, y_end], linestyle=':', color=color_ax3[0], linewidth=1.3, alpha=0.8, zorder=1)
            
    else:
        mean_R = np.mean(data['R'])
        ax3.axvline(x=mean_R, linestyle=':', color=color_ax3[0], linewidth=1.3, alpha=0.8, zorder=1)

    ax4.axvline(x=0, linestyle=':', color=color_ax4[0], linewidth=1, alpha=0.5, zorder=1)

    # 添加文本
    ha= 'right' if method == 'left' else 'left'
    r_range = data['R'].max() - data['R'].min()
    bias_range = data['Bias'].max() - data['Bias'].min()
    r_offset = r_range * 0.15 if method =='left' else r_range * 0.065
    bias_offset = bias_range * 0.05
    for x, y_val in zip(data['R'], y2):
        ax3.text(x - r_offset, y_val, f' {x:.3f}', color=color_ax3[0], va='center', ha=ha, fontproperties=font_prop,zorder=2, size=10)
    for x, y_val in zip(data['Bias'], y1):
        ax4.text(x - bias_offset, y_val, f' {x:.3f}', color='purple', va='center', ha=ha, fontproperties=font_prop,zorder=2, size=10)

    # 轴位置控制
    ax1.spines['bottom'].set_position(('outward', 0))
    ax1.xaxis.set_ticks_position('bottom')
    ax1.xaxis.set_label_position('bottom')
    ax2.spines['bottom'].set_position(('outward', 12))
    ax2.xaxis.set_ticks_position('bottom')
    ax2.xaxis.set_label_position('bottom')
    ax3.spines['top'].set_position(('axes', 1.0))
    ax3.xaxis.set_ticks_position('top')
    ax3.xaxis.set_label_position('top')
    ax4.spines['top'].set_position(('outward', 12))
    ax4.xaxis.set_ticks_position('top')
    ax4.xaxis.set_label_position('top')

    # 轴范围控制(先计算拓展范围)
    expand_ratio = 0.8
    data_RMSE_max = data['RMSE'].max()
    data_ubRMSE_max = data['ubRMSE'].max()
    RMSE_new_max = data_RMSE_max + (data_RMSE_max) * expand_ratio
    ubRMSE_new_max = data_ubRMSE_max + (data_ubRMSE_max) * expand_ratio

    expand_ratio = 0.5
    shift_ratio = 0.1
    data_R_min, data_R_max = data['R'].min(), data['R'].max()
    R_range = data_R_max - data_R_min
    R_new_min = data_R_min - R_range * expand_ratio * (1 - shift_ratio)
    R_new_max = data_R_max + R_range * expand_ratio * shift_ratio
    data_Bias_min, data_Bias_max = data['Bias'].min(), data['Bias'].max()
    Bias_range = data_Bias_max - data_Bias_min
    Bias_new_min = data_Bias_min - Bias_range * expand_ratio * (1 - shift_ratio)
    Bias_new_max = data_Bias_max + Bias_range * expand_ratio * shift_ratio

    ax1.set_xlim([0.0, 0.28])
    ax2.set_xlim([0, 0.26])
    ax3.set_xlim([-2, 1])
    ax4.set_xlim([-0.4, 0.3])
    ax1.barh(y, RMSE_new_max, height=height*2, color='lightgray', alpha=0.4, zorder=0)
    if method =='left':ax1.yaxis.tick_left()
    ax1.tick_params('y', colors='black', length=3, width=1.5, labelsize=12, direction='out',pad=4)
    ax1.tick_params('x', colors=colors_rmse[-1], length=3, width=1, labelsize=10,direction='in', pad=1)
    ax2.tick_params('x', colors=colors_ubrmse[-1], length=3, width=1, labelsize=10,direction='in', pad=1)
    ax3.tick_params(axis='x', colors=color_ax3[0], length=3, width=1, labelsize=10,direction='in', pad=0)
    ax4.tick_params('x', colors='purple', length=3, width=1, labelsize=10,direction='in', pad=0)
    ax1.xaxis.set_major_locator(MaxNLocator(prune='lower', nbins=5))
    ax2.xaxis.set_major_locator(MaxNLocator(prune='lower', nbins=4))
    ax3.xaxis.set_major_locator(MaxNLocator(prune='lower', nbins=4))
    ax4.xaxis.set_major_locator(MaxNLocator(prune='lower', nbins=3))

    # x轴绘制
    ax1.set_xlabel('RMSE', fontproperties=font_prop)
    ax2.set_xlabel('ubRMSE', fontproperties=font_prop)
    ax3.set_xlabel('Bias', fontproperties=font_prop)
    ax4.set_xlabel('R', fontproperties=font_prop)

    # 刻度线控制
    ax2.yaxis.set_ticks_position('none')
    ax3.yaxis.set_ticks_position('none')
    ax4.yaxis.set_ticks_position('none')
    if method == 'right':
        for ax in [ax1, ax2, ax3, ax4]:
            ax.yaxis.tick_right()
            ax.yaxis.set_label_position("right")
            ax.invert_xaxis()
    # 控制刻度线上下边界缝隙距离
    y_min, y_max = ax1.get_ylim()
    padding = (y_max - y_min) * -0.04
    ax1.set_ylim(y_min - padding, y_max + padding)
    if method == 'left':
        # 绘制图例
        original_handles = []
        original_labels = []
        for ax in [ax1,ax2, ax3, ax4]:
            handles, labels = ax.get_legend_handles_labels()
            original_handles.extend(handles)
            original_labels.extend(labels)
        gradient_patch_rmse = [mpatches.Patch(color=colors_rmse[5], label='PSC-FENet RMSE')]
        gradient_patch_ubrmse = [mpatches.Patch(color=colors_ubrmse[5], label='PSC-FENet ubRMSE')]
        all_method_gradient_patch_rmse = [mpatches.Patch(color=colors_rmse_all_method[5], label='Mean RMSE')]
        all_method_gradient_patch_ubrmse = [mpatches.Patch(color=colors_ubrmse_all_method[5], label='Mean ubRMSE')]

        mean_r_scatter = mlines.Line2D([], [], color=color_ax3[1], marker='X', linestyle='None', markersize=10, label='Mean R', alpha=0.6, markeredgewidth=1, markeredgecolor='black')
        mean_bias_scatter = mlines.Line2D([], [], color=color_ax4[1], marker='P', linestyle='None', markersize=10, label='Mean Bias', alpha=0.6, markeredgewidth=1, markeredgecolor='black')
        zero_line = mlines.Line2D([], [], color=color_ax4[0], linestyle=':', linewidth=1, label='Zero Line', alpha=0.5)
        kpnet_mean_r_line = mlines.Line2D([], [], color=color_ax3[0], linestyle=':', linewidth=1, label='PSC-FENet Mean R', alpha=0.5)

        final_handles = original_handles + gradient_patch_rmse + gradient_patch_ubrmse + all_method_gradient_patch_rmse + all_method_gradient_patch_ubrmse+[mean_r_scatter, mean_bias_scatter, zero_line, kpnet_mean_r_line]
        final_labels = original_labels + ['PSC-FENet RMSE', 'PSC-FENet ubRMSE', 'Mean RMSE', 'Mean ubRMSE', 'Mean R', 'Mean Bias', 'Zero Line', 'PSC-FENet Mean R']
        legend = plt.legend(final_handles, final_labels, 
           loc='upper left', 
           bbox_to_anchor=(-0.52, 0.995), 
           fancybox=False, 
           shadow=False, 
           ncol=1, 
           frameon=False, 
           prop=legend_font, 
           fontsize=10, 
           borderpad=0,
           labelspacing=1,
           handletextpad=.2)
        for text, label in zip(legend.get_texts(), final_labels):
            if label in ['PSC-FENet Mean R', 'PSC-FENet ubRMSE', 'PSC-FENet RMSE']:
                text.set_fontsize(10)
    # 文本标签
    y_min, y_max = ax1.get_ylim()
    bias_label_y = y_max + (y_max - y_min) * 0.02
    r_label_y = bias_label_y - (y_max - y_min) * 0.025
    rmse_label_y = y_min - (y_max - y_min) * 0.005
    ubrmse_label_y = rmse_label_y - (y_max - y_min) * 0.02
    x = -0.005

    ax1.text(x, bias_label_y, 'Bias', size=12, fontproperties=font_prop, zorder=5, ha=ha)
    ax1.text(x, r_label_y, 'PearsonR', size=12, fontproperties=font_prop, zorder=5, ha=ha)
    ax1.text(x, rmse_label_y, 'RMSE', size=12, fontproperties=font_prop, zorder=5, ha=ha)
    ax1.text(x, ubrmse_label_y, 'ubRMSE', size=12, fontproperties=font_prop, zorder=5, ha=ha)

    # 控制y轴间隙
    current_num_ticks = len(ax1.get_yticks())
    new_num_ticks = int(current_num_ticks * 1.1)
    ax1.yaxis.set_major_locator(MaxNLocator(new_num_ticks))

    # 共同控制：隐藏右侧指标轴、x轴label
    for ax in [ax1, ax2, ax3, ax4]:
        ax.spines[ha].set_visible(False)
        ax.xaxis.label.set_visible(False)
    for ax in [ax1, ax2, ax3, ax4]:ax.grid(False)
    ax3.grid(True, color=color_ax3[0], linestyle='--',alpha=0.5,zorder=0.1)
    ax4.grid(True, color='purple', linestyle='--',alpha=0.5,zorder=0.1)
    for ax in [ax1, ax2, ax3, ax4]:ax.grid(False)
    ax1.grid(True, color='lightgray', linestyle='--', alpha=0.5, which='both', axis='y', zorder=0.5)
    file_path = r'C:\Users\Administrator\Desktop\\' + name + '.jpeg'
    fig.savefig(file_path, dpi=1000, format='jpeg', bbox_inches='tight')
    plt.show()

#plot_design_changes_extended(df_other_networks)
plot_design_changes_extended(df_ctp, all_methods_ctp, name = 'b',method='right')
plot_design_changes_extended(df_other_networks, all_methods_other_networks,name = "a",method='left')

### (数据处理) 原位误差平面图

In [ ]:
import numpy as np
import pandas as pd
data_path = r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\ST_PSC-FENet.npy"
data = np.load(data_path)
overall_mean = np.nanmean(data, axis=0)
# 时间起点
start_time = pd.Timestamp('2016-01-01 03:00')

# 生成完整的时间序列
times = pd.date_range(start=start_time, periods=data.shape[2], freq='3H')
save_path = r"D:\Data_Store\Dataset\Reconstruction\RF\RF_reconstructed_data_cor.npy"
np.save(save_path, overall_mean)
print(save_path)

In [ ]:
import numpy as np
import rasterio
from rasterio.transform import from_origin
lat_min = 25.550117
lat_max = 40.450184
lon_min = 66.95031
lon_max = 104.95048
lat_points = 150
lon_points = 381

new_lat_max = lat_max + ((lat_max - lat_min) / (lat_points - 1) / 2)
new_lon_min = lon_min - ((lon_max - lon_min) / (lon_points - 1) / 2)

transform = from_origin(new_lon_min, new_lat_max, (lon_max - lon_min) / (lon_points - 1), (lat_max - lat_min) / (lat_points - 1))

metadata = {
    'driver': 'GTiff',
    'height': lat_points,
    'width': lon_points,
    'count': 1,
    'dtype': 'float32',
    'crs': '+proj=latlong',
    'transform': transform
}
data = np.load(r"D:\Data_Store\Dataset\Reconstruction\RF\RF_reconstructed_data_cor.npy")
data_flipped = np.flipud(data)
with rasterio.open(r'D:\Data_Store\Dataset\Reconstruction\RF\RF_reconstructed_data_geotiff.tif', 'w', **metadata) as dst:
    dst.write(data_flipped, 1)

## 模型间不确定度  3H/12H不确定度图、全域最小不确定性图

### (数据处理) 3H 全年不确定性计算

In [ ]:
import numpy as np
import scipy.optimize as opt
from numpy.linalg import inv, det
import matplotlib.pyplot as plt

def calculate_uncertainty(models, reference, model_names):
    combined_data = np.array(models + [reference])
    valid_indices = ~np.isnan(combined_data).any(axis=0)
    filtered_models = [model[valid_indices] for model in models]
    filtered_reference = reference[valid_indices]
    deviations = [model - filtered_reference for model in filtered_models]
    y = np.array(deviations)
    S = np.cov(y)
    n = S.shape[0] + 1
    R = np.zeros((n, n))
    R[n-1, n-1] = 1 / (2 * np.sum(np.sum(inv(S))))
    for i in range(n-1):
        for j in range(n-1):
            R[i, j] = S[i, j] - R[n-1, n-1] + R[i, n-1] + R[j, n-1]
    def fun(x):
        return np.sum((S - x[-1] + x[:n-1] + x[:n-1, None])**2) + np.sum(x[:n-1]**2)
    def mycon(x):
        b = x[-1] - x[:n-1]
        return np.dot(b, np.dot(inv(S), b)) - x[-1],
    x0 = np.zeros(n)
    bnds = [(0, None) for _ in range(n)]
    cons = {'type': 'eq', 'fun': mycon}
    res = opt.minimize(fun, x0, method='SLSQP', bounds=bnds, constraints=cons)
    x = res.x
    R[:n-1, n-1] = x[:n-1]
    R[n-1, :n-1] = x[:n-1]
    for i in range(n-1):
        for j in range(n-1):
            R[i, j] = S[i, j] - R[n-1, n-1] + R[i, n-1] + R[j, n-1]
    uncertainties = R.diagonal()[:-1]
    sorted_indices = np.argsort(uncertainties)
    sorted_models = {model_names[sorted_indices[i]]: i + 1 for i in range(len(uncertainties))}
    model_uncertainties = {model_names[i]: uncertainties[i] for i in range(len(uncertainties))}
    return sorted_models, model_uncertainties

reconstruction_paths = {
    'PSC-FENet-SP': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SP_PSC-FENet.npy",
    'PSC-FENet-ST': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\ST_PSC-FENet.npy",
    'LSTM': r"D:\Data_Store\Dataset\Reconstruction\LSTM\LSTM1.npy",
    'RF': r"D:\Data_Store\Dataset\Reconstruction\RF\RF_reconstructed_data.npy",
    'ResNet-SP': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SP_ResNet.npy",
    'ResNet-ST': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\ST_ResNet.npy",
    'UNet-SP': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SP_UNet.npy",
    'UNet-ST': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\ST_UNet.npy",
    'VIT-SP': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SP_ViT.npy",
    'VIT-ST': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\ST_ViT.npy",
    'Temporal': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\Temporal.npy"
}

selected_model_names = ['PSC-FENet-ST', 'PSC-FENet-SP', 'LSTM', 'RF', 'Temporal']
selected_paths = {name: reconstruction_paths[name] for name in selected_model_names if name in reconstruction_paths}
model_data = [np.load(path)[:8767, :, :] for path in selected_paths.values()]

model_names = list(selected_paths.keys())
reference_data = np.zeros(model_data[0].shape)
time_steps, lons, lats = model_data[0].shape

model_ranks = np.zeros((lons, lats, len(model_names)), dtype=int)
model_uncertainty_values = np.zeros((lons, lats, len(model_names)))
for lon in range(lons):
    for lat in range(lats):
        current_data = [model[:, lon, lat] for model in model_data]
        sorted_models, model_uncertainties = calculate_uncertainty(current_data, reference_data[:, lon, lat], model_names)
        # 排名
        for model_name, rank in sorted_models.items():
            model_index = model_names.index(model_name)
            model_ranks[lon, lat, model_index] = rank
        # 不确定性
        for model_name, uncertainty in model_uncertainties.items():
            model_index = model_names.index(model_name)
            model_uncertainty_values[lon, lat, model_index] = uncertainty
del model_data

### (数据处理) 12H 全年不确定性计算

In [ ]:
import numpy as np
import scipy.optimize as opt
from numpy.linalg import inv, det
import matplotlib.pyplot as plt
import pandas as pd

def calculate_uncertainty(models, reference, model_names):
    combined_data = np.array(models + [reference])
    valid_indices = ~np.isnan(combined_data).any(axis=0)
    filtered_models = [model[valid_indices] for model in models]
    filtered_reference = reference[valid_indices]
    deviations = [model - filtered_reference for model in filtered_models]
    y = np.array(deviations)
    S = np.cov(y)
    n = S.shape[0] + 1
    R = np.zeros((n, n))
    R[n-1, n-1] = 1 / (2 * np.sum(np.sum(inv(S))))
    for i in range(n-1):
        for j in range(n-1):
            R[i, j] = S[i, j] - R[n-1, n-1] + R[i, n-1] + R[j, n-1]
    def fun(x):
        return np.sum((S - x[-1] + x[:n-1] + x[:n-1, None])**2) + np.sum(x[:n-1]**2)
    def mycon(x):
        b = x[-1] - x[:n-1]
        return np.dot(b, np.dot(inv(S), b)) - x[-1],
    x0 = np.zeros(n)
    bnds = [(0, None) for _ in range(n)]
    cons = {'type': 'eq', 'fun': mycon}
    res = opt.minimize(fun, x0, method='SLSQP', bounds=bnds, constraints=cons)
    x = res.x
    R[:n-1, n-1] = x[:n-1]
    R[n-1, :n-1] = x[:n-1]
    for i in range(n-1):
        for j in range(n-1):
            R[i, j] = S[i, j] - R[n-1, n-1] + R[i, n-1] + R[j, n-1]
    uncertainties = R.diagonal()[:-1]
    sorted_indices = np.argsort(uncertainties)
    sorted_models = {model_names[sorted_indices[i]]: i + 1 for i in range(len(uncertainties))}
    model_uncertainties = {model_names[i]: uncertainties[i] for i in range(len(uncertainties))}
    return sorted_models, model_uncertainties

reconstruction_paths = {
    'PSC-FENet-SP': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SP_PSC-FENet.npy",
    'PSC-FENet-ST': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\ST_PSC-FENet.npy",
    'LSTM': r"D:\Data_Store\Dataset\Reconstruction\LSTM\LSTM1.npy",
    'RF': r"D:\Data_Store\Dataset\Reconstruction\RF\RF_reconstructed_data.npy",
    'ResNet-SP': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SP_ResNet.npy",
    'ResNet-ST': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\ST_ResNet.npy",
    'UNet-SP': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SP_UNet.npy",
    'UNet-ST': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\ST_UNet.npy",
    'VIT-SP': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SP_ViT.npy",
    'VIT-ST': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\ST_ViT.npy",
    'Temporal': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\Temporal.npy"
}

dates = pd.date_range(start='2016-01-01 03:00', periods=8768, freq='3H')
filter_mask = ((dates.hour == 6) | (dates.hour == 18))
filtered_indices = np.where(filter_mask)[0]

selected_model_names = ['PSC-FENet-ST', 'PSC-FENet-SP', 'LSTM', 'RF', 'Temporal']

#selected_model_names = ['PSC-FENet-ST','PSC-FENet-SP',  'ResNet-ST', 'ResNet-SP', 'UNet-ST', 'UNet-SP', 'VIT-ST', 'VIT-SP',  'LSTM', 'RF', 'Temporal']
selected_paths = {name: reconstruction_paths[name] for name in selected_model_names if name in reconstruction_paths}
model_data = [np.load(path)[filtered_indices, :, :] for path in selected_paths.values()]

model_names = list(selected_paths.keys())
reference_data = np.zeros(model_data[0].shape)
time_steps, lons, lats = model_data[0].shape

model_ranks_12 = np.zeros((lons, lats, len(model_names)), dtype=int)
model_uncertainty_values_12 = np.zeros((lons, lats, len(model_names)))

for lon in range(lons):
    for lat in range(lats):
        current_data = [model[:, lon, lat] for model in model_data]
        sorted_models, model_uncertainties = calculate_uncertainty(current_data, reference_data[:, lon, lat], model_names)
        # 排名
        for model_name, rank in sorted_models.items():
            model_index = model_names.index(model_name)                                                                                                                                                                                                                                                                                                                                 
            model_ranks_12[lon, lat, model_index] = rank
        # 不确定性
        for model_name, uncertainty in model_uncertainties.items():
            model_index = model_names.index(model_name)
            model_uncertainty_values_12[lon, lat, model_index] = uncertainty
del model_data

### (数据处理) 每日均值全年不确定性计算

In [ ]:
import numpy as np
import scipy.optimize as opt
from numpy.linalg import inv, det
import matplotlib.pyplot as plt
import pandas as pd

def calculate_uncertainty(models, reference, model_names):
    combined_data = np.array(models + [reference])
    valid_indices = ~np.isnan(combined_data).any(axis=0)
    filtered_models = [model[valid_indices] for model in models]
    filtered_reference = reference[valid_indices]
    deviations = [model - filtered_reference for model in filtered_models]
    y = np.array(deviations)
    S = np.cov(y)
    n = S.shape[0] + 1
    R = np.zeros((n, n))
    R[n-1, n-1] = 1 / (2 * np.sum(np.sum(inv(S))))
    for i in range(n-1):
        for j in range(n-1):
            R[i, j] = S[i, j] - R[n-1, n-1] + R[i, n-1] + R[j, n-1]
    def fun(x):
        return np.sum((S - x[-1] + x[:n-1] + x[:n-1, None])**2) + np.sum(x[:n-1]**2)
    def mycon(x):
        b = x[-1] - x[:n-1]
        return np.dot(b, np.dot(inv(S), b)) - x[-1],
    x0 = np.zeros(n)
    bnds = [(0, None) for _ in range(n)]
    cons = {'type': 'eq', 'fun': mycon}
    res = opt.minimize(fun, x0, method='SLSQP', bounds=bnds, constraints=cons)
    x = res.x
    R[:n-1, n-1] = x[:n-1]
    R[n-1, :n-1] = x[:n-1]
    for i in range(n-1):
        for j in range(n-1):
            R[i, j] = S[i, j] - R[n-1, n-1] + R[i, n-1] + R[j, n-1]
    uncertainties = R.diagonal()[:-1]
    sorted_indices = np.argsort(uncertainties)
    sorted_models = {model_names[sorted_indices[i]]: i + 1 for i in range(len(uncertainties))}
    model_uncertainties = {model_names[i]: uncertainties[i] for i in range(len(uncertainties))}
    return sorted_models, model_uncertainties

reconstruction_paths = {
    'PSC-FENet-SP': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SP_PSC-FENet.npy",
    'PSC-FENet-ST': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\ST_PSC-FENet.npy",
    'LSTM': r"D:\Data_Store\Dataset\Reconstruction\LSTM\LSTM1.npy",
    'RF': r"D:\Data_Store\Dataset\Reconstruction\RF\RF_reconstructed_data.npy",
    'ResNet-SP': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SP_ResNet.npy",
    'ResNet-ST': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\ST_ResNet.npy",
    'UNet-SP': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SP_UNet.npy",
    'UNet-ST': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\ST_UNet.npy",
    'VIT-SP': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\SP_ViT.npy",
    'VIT-ST': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\ST_ViT.npy",
    'Temporal': r"D:\Data_Store\Dataset\Reconstruction\ST\RESALL\Temporal.npy"
}


dates = pd.date_range(start='2016-01-01 03:00', periods=8767, freq='3H')
n_per_day = 8

full_groups = len(dates) // n_per_day * n_per_day
daily_means_indices = [np.arange(i, i + n_per_day) for i in range(0, full_groups, n_per_day)]

if len(dates) % n_per_day != 0:
    daily_means_indices.append(np.arange(full_groups, len(dates)))

daily_means_indices = np.array(daily_means_indices)
selected_model_names = ['PSC-FENet-ST', 'PSC-FENet-SP', 'LSTM', 'RF', 'Temporal']
selected_paths = {name: reconstruction_paths[name] for name in selected_model_names if name in reconstruction_paths}

model_data = [np.load(path)[:8767] for path in selected_paths.values()]

daily_means_data = []
for data in model_data:
    daily_means = np.array([np.mean(data[indices, :, :], axis=0) for indices in daily_means_indices])
    daily_means_data.append(daily_means)

model_names = list(selected_paths.keys())
reference_data = np.zeros(daily_means_data[0].shape)
time_steps, lons, lats = daily_means_data[0].shape

model_ranks_daily = np.zeros((lons, lats, len(model_names)), dtype=int)
model_uncertainty_values_daily = np.zeros((lons, lats, len(model_names)))

for lon in range(lons):
    for lat in range(lats):
        current_data = [model[:, lon, lat] for model in daily_means_data]
        sorted_models, model_uncertainties = calculate_uncertainty(current_data, reference_data[:, lon, lat], model_names)
        # 排名
        for model_name, rank in sorted_models.items():
            model_index = model_names.index(model_name)
            model_ranks_daily[lon, lat, model_index] = rank
        # 不确定性
        for model_name, uncertainty in model_uncertainties.items():
            model_index = model_names.index(model_name)
            model_uncertainty_values_daily[lon, lat, model_index] = uncertainty

del model_data, daily_means_data  # 清理内存

### (数据处理) 最优模型计算
运行之前先运行前面两步
selected_models 已经在前面定义

In [ ]:
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from shapely.geometry import Point

def extract_best_model_for_each_point(model_ranks, model_names, selected_models):

    model_name_to_original_index = {name: i for i, name in enumerate(model_names)}
    model_name_to_selected_index = {name: i for i, name in enumerate(selected_models)}
    best_models = np.zeros((model_ranks.shape[0], model_ranks.shape[1]), dtype=int)
    for lat in range(model_ranks.shape[0]):
        for lon in range(model_ranks.shape[1]):
            ranks = model_ranks[lat, lon, :]
            best_rank = float('inf')
            best_model_index = -1
            for model in selected_models:
                original_index = model_name_to_original_index[model]
                if ranks[original_index] < best_rank:
                    best_rank = ranks[original_index]
                    best_model_index = model_name_to_selected_index[model]
            if best_model_index >= 0:
                best_models[lat, lon] = best_model_index + 1

    return best_models

def extract_best_value_for_each_point(model_uncertainty_values, model_names, selected_models):
    model_name_to_original_index = {name: i for i, name in enumerate(model_names)}
    best_models_values = np.zeros((model_uncertainty_values.shape[0], model_uncertainty_values.shape[1]))

    for lat in range(model_uncertainty_values.shape[0]):
        for lon in range(model_uncertainty_values.shape[1]):
            best_value = float('inf')
            for model in selected_models:
                original_index = model_name_to_original_index[model]
                value = model_uncertainty_values[lat, lon, original_index]
                if value < best_value:
                    best_value = value
            best_models_values[lat, lon] = best_value

    return best_models_values

def apply_clipping(data, lon_grid, lat_grid, shp_path):
    """
    使用.shp文件裁剪数据：在边界外的数据设置为NaN。
    """
    gdf = gpd.read_file(shp_path)
    # 获取.shp文件的几何形状
    poly = gdf.unary_union

    clipped_data = np.full(data.shape, np.nan)

    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            point = Point(lon_grid[i, j], lat_grid[i, j])
            if point.within(poly):
                clipped_data[i, j] = data[i, j]

    return clipped_data

lats, lons, _ = model_ranks.shape

# 可使用model_names, selected_models匹配model_names中selected_models正确排名，这里使用相同列表
best_models = extract_best_model_for_each_point(model_ranks, selected_model_names, selected_model_names)
best_models_values = extract_best_value_for_each_point(model_uncertainty_values, selected_model_names, selected_model_names)

best_models_12 = extract_best_model_for_each_point(model_ranks_12, selected_model_names, selected_model_names)
best_models_values_12 = extract_best_value_for_each_point(model_uncertainty_values_12, selected_model_names, selected_model_names)

best_models_daily = extract_best_model_for_each_point(model_ranks_daily, selected_model_names, selected_model_names)
best_models_values_daily = extract_best_value_for_each_point(model_uncertainty_values_daily, selected_model_names, selected_model_names)

# 数据裁剪
lat_min = 25.550117
lat_max = 40.450184
lon_min = 66.95031
lon_max = 104.95048
lat_points = 150
lon_points = 381
lat_span = (lat_max - lat_min) / (lat_points - 1)
lon_span = (lon_max - lon_min) / (lon_points - 1)
new_lat_min = lat_min - lat_span / 2
new_lat_max = lat_max + lat_span / 2
new_lon_min = lon_min - lon_span / 2
new_lon_max = lon_max + lon_span / 2

lon_range = np.linspace(lon_min, lon_max, lon_points)
lat_range = np.linspace(lat_min, lat_max, lat_points)
lon_grid, lat_grid = np.meshgrid(lon_range, lat_range)

shp_path = "D:\\Data_Store\\TPBoundary_new(2021)\\TPBoundary_new(2021).shp"
clipped_data = apply_clipping(best_models, lon_grid, lat_grid, shp_path)
clipped_data_12 = apply_clipping(best_models_12, lon_grid, lat_grid, shp_path)

### (数据处理) 数据裁剪

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from shapely.geometry import Point
from matplotlib import rcParams
import geopandas as gpd
import matplotlib.colors as mcolors
from matplotlib.ticker import MaxNLocator

def apply_clipping(data, lon_grid, lat_grid, shp_path):
    """
    使用.shp文件裁剪数据：在边界外的数据设置为NaN。
    """
    gdf = gpd.read_file(shp_path)
    # 获取.shp文件的几何形状
    poly = gdf.unary_union

    clipped_data = np.full(data.shape, np.nan)

    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            point = Point(lon_grid[i, j], lat_grid[i, j])
            if point.within(poly):
                clipped_data[i, j] = data[i, j]

    return clipped_data

lat_min = 25.550117
lat_max = 40.450184
lon_min = 66.95031
lon_max = 104.95048
lat_points = 150
lon_points = 381

lon_range = np.linspace(lon_min, lon_max, lon_points)
lat_range = np.linspace(lat_min, lat_max, lat_points)
lon_grid, lat_grid = np.meshgrid(lon_range, lat_range)

#model_names = ['PSC-FENet-ST', 'ResNet-ST', 'UNet-ST', 'VIT-ST',  'LSTM', 'RF', 'Temporal']
model_names = ['PSC-FENet-ST','PSC-FENet-SP',  'ResNet-ST', 'ResNet-SP', 'UNet-ST', 'UNet-SP', 'VIT-ST', 'VIT-SP',  'LSTM', 'RF', 'Temporal']

data_pieces,data_pieces_12 = {},{}
for i, model_name in enumerate(model_names):
    model_uncertainty = model_uncertainty_values[:, :, i]
    model_uncertainty_12 = model_uncertainty_values_12[:, :, i]
    data_pieces[model_name] = model_uncertainty 
    data_pieces_12[model_name] = model_uncertainty_12

clipped_data_pieces,clipped_data_pieces_12 = {},{}
shp_path = "D:\\Data_Store\\TPBoundary_new(2021)\\TPBoundary_new(2021).shp"
for name, data in data_pieces.items():
    if name in ['PSC-FENet-ST', 'ResNet-ST', 'UNet-ST', 'VIT-ST',  'LSTM', 'RF', 'Temporal']:
        clipped_data = apply_clipping(data, lon_grid, lat_grid, shp_path)
        clipped_data_pieces[name] = clipped_data

for name_12, data_12 in data_pieces_12.items():
    if name in ['PSC-FENet-ST', 'ResNet-ST', 'UNet-ST', 'VIT-ST',  'LSTM', 'RF', 'Temporal']:
        clipped_data_12 = apply_clipping(data_12, lon_grid, lat_grid, shp_path)
        clipped_data_pieces_12[name_12] = clipped_data_12

### (数据处理)优化后的数据裁剪

In [ ]:
def generate_clipping_mask(lon_grid, lat_grid, shp_path):
    """
    使用.shp文件生成数据掩码：在边界外的数据掩码为0，在边界内的数据掩码为1。
    """
    import geopandas as gpd
    from shapely.geometry import Point
    import numpy as np

    gdf = gpd.read_file(shp_path)
    poly = gdf.unary_union
    mask = np.zeros_like(lon_grid, dtype=bool)

    for i in range(lon_grid.shape[0]):
        for j in range(lon_grid.shape[1]):
            point = Point(lon_grid[i, j], lat_grid[i, j])
            if point.within(poly):
                mask[i, j] = True

    return mask

shp_path = r"D:\\Data_Store\\TPBoundary_new(2021)\\TPBoundary_new(2021).shp"
model_names = ['PSC-FENet-ST', 'PSC-FENet-SP', 'LSTM', 'RF', 'Temporal']
#model_names = ['PSC-FENet-ST','PSC-FENet-SP',  'ResNet-ST', 'ResNet-SP', 'UNet-ST', 'UNet-SP', 'VIT-ST', 'VIT-SP',  'LSTM', 'RF', 'Temporal']

settings = {0.1: (25.550117, 40.450184, 66.95031, 104.95048, 150, 381)}


data_pieces,data_pieces_12,data_pieces_daily = {},{},{}
for i, model_name in enumerate(model_names):
    model_uncertainty = model_uncertainty_values[:, :, i]
    model_uncertainty_12 = model_uncertainty_values_12[:, :, i]
    model_uncertainty_daily = model_uncertainty_values_daily[:, :, i]
    data_pieces[model_name] = model_uncertainty 
    data_pieces_12[model_name] = model_uncertainty_12
    data_pieces_daily[model_name] = model_uncertainty_daily

masks = {}
for mode, (lat_min, lat_max, lon_min, lon_max, lat_points, lon_points) in settings.items():
    lon_step = (lon_max - lon_min) / (lon_points - 1)
    lat_step = (lat_max - lat_min) / (lat_points - 1)
    new_lon_min = lon_min - lon_step / 2
    new_lon_max = lon_max + lon_step / 2
    new_lat_min = lat_min - lat_step / 2
    new_lat_max = lat_max + lat_step / 2
    lon_range, lat_range = np.linspace(new_lon_min, new_lon_max, lon_points), np.linspace(new_lat_min, new_lat_max, lat_points)
    lon_grid, lat_grid = np.meshgrid(lon_range, lat_range)
    masks[mode] = generate_clipping_mask(lon_grid, lat_grid, shp_path)

mode = 0.1
mask = masks[mode]
clipped_data_pieces,clipped_data_pieces_12,clipped_data_pieces_daily = {},{},{}

for name, data in data_pieces.items():
        clipped_data_pieces[name] = np.where(mask, data, np.nan)
for name_12, data_12 in data_pieces_12.items():
        clipped_data_pieces_12[name_12] = np.where(mask, data_12, np.nan)
for name_daily, data_daily in data_pieces_daily.items():
        clipped_data_pieces_daily[name_daily] = np.where(mask, data_daily, np.nan)

### (绘图) 多模型不确定度图

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.ticker import MaxNLocator
from matplotlib import rcParams

def add_degree_E(x, pos):
    return f"{int(x)}°E"

def add_degree_N(y, pos):
    return f"{int(y)}°N"
def plot_data_slices(data_pieces, lon_range, lat_range, name):
    fig = plt.figure(figsize=(36, 24))
    ax = fig.add_subplot(111, projection='3d')
    lon_grid, lat_grid = np.meshgrid(lon_range, lat_range)
    y_positions = np.arange(len(data_pieces))

    # 收集所有数据用于计算颜色范围
    #all_data = np.concatenate([data.flatten() for _, data in data_pieces.items()])
    vmin = 0
    vmax = 0.025
    '''
    colors = ["#ffc43d", "#60d394"]
    cmap_name = "custom_orange_green"
    cm = LinearSegmentedColormap.from_list(cmap_name, colors, N=100)

    # Create a gradient image
    gradient = np.linspace(0, 1, 256)
    gradient = np.vstack((gradient, gradient))
    '''
    # 创建归一化和映射对象
    norm = Normalize(vmin=vmin, vmax=vmax)
    mappable = ScalarMappable(norm=norm, cmap='RdYlGn_r')

    # 绘制每个数据片
    for i, (indicator, data) in enumerate(data_pieces.items()):
        colors = mappable.to_rgba(data)
        colors[np.isnan(data), -1] = 0
        ax.plot_surface(lon_grid, np.full(lon_grid.shape, y_positions[i]), lat_grid,
                        facecolors=colors, rstride=1, cstride=1, linewidth=0, antialiased=False)

    # 设置colorbar
    cbar = plt.colorbar(mappable, ax=ax, aspect=20, shrink=0.2, pad=0.02, orientation='vertical', extend='both')
    cbar.set_label('Uncertainty ($m^3/m^{-3}$)', rotation=90, labelpad=10, size=16)
    cbar.ax.tick_params(labelsize=14)

    # 设置刻度和标签
    #ax.set_xlabel('Longitude(°)', labelpad=30,size=18, rotation=90)
    #ax.set_zlabel('Latitude(°)', labelpad=5,size=18)
    ax.tick_params(axis='y', colors='black', length=8, width=1.5, labelsize=16, direction='out', pad=10)
    ax.tick_params(axis='x', colors='black', length=8, width=1.5, labelsize=16, direction='out', pad=0)
    ax.tick_params(axis='z', colors='black', length=5, width=1.5, labelsize=16, direction='out', pad=5)
    ax.set_yticks(y_positions)
    ax.set_yticklabels(list(data_pieces.keys()),size=16)

    # 设置绘图范围和刻度数目
    ax.set_xlim(max(lon_range)+0.01, min(lon_range)-0.01)
    ax.set_ylim(min(y_positions)+0.1, max(y_positions)-0.01)
    ax.set_zlim(min(lat_range)-0.01, max(lat_range)+0.01)
    ax.zaxis.set_major_locator(MaxNLocator(prune='lower', nbins=5))
    ax.xaxis.set_major_locator(MaxNLocator(prune='lower', nbins=5))
    ax.xaxis.set_major_formatter(FuncFormatter(add_degree_E))
    ax.zaxis.set_major_formatter(FuncFormatter(add_degree_N))

    # 去除背景色并调整视角
    ax.xaxis.pane.fill = False
    ax.yaxis.pane.fill = False
    ax.zaxis.pane.fill = False
    ax.set_proj_type('ortho')
    ax.set_box_aspect([3, 6, 1])
    ax.view_init(elev=3, azim=28)
    plt.savefig(r"C:\Users\Administrator\Desktop\Draw\uncertain"+name+".jpeg", dpi=1000, format='jpeg')
    plt.show()

rcParams['font.family'] = 'serif'
rcParams['font.serif'] = ['Times New Roman']

plot_data_slices(clipped_data_pieces, lon_range, lat_range, name='a')
#plot_data_slices(clipped_data_pieces_12, lon_range, lat_range, name='b')
plot_data_slices(clipped_data_pieces_daily, lon_range, lat_range, name='b')

### (图像裁剪)  手动裁剪
bbox_inches='tight'不起作用

In [ ]:
from PIL import Image, ImageFile

# 修改Pillow的像素限制（默认是 89478485 pixels，可以根据需要调整这个值）
Image.MAX_IMAGE_PIXELS = None  # 移除限制，慎用！
ImageFile.LOAD_TRUNCATED_IMAGES = True  # 允许加载截断的图像

def crop_image(input_path, output_path):
    img = Image.open(input_path)
    width, height = img.size
    
    # 裁剪参数调整
    top = int(height / 2.6)
    bottom = height - top
    # 左右裁剪参数调整，剪切20%
    left = int(width * 0.23)
    right = width - int(width * 0.18) 
    
    # 应用裁剪
    img_cropped = img.crop((left, top, right, bottom))
    img_cropped.save(output_path)

input_paths = ["C:/Users/Administrator/Desktop/Draw/uncertainb.jpeg", "C:/Users/Administrator/Desktop/Draw/uncertaina.jpeg"]
output_paths = ["C:/Users/Administrator/Desktop/Draw/uncertainb_cropped.jpeg", "C:/Users/Administrator/Desktop/Draw/uncertaina_cropped.jpeg"]

for input_path, output_path in zip(input_paths, output_paths):
    crop_image(input_path, output_path)

### (绘图) 全域最小不确定度图

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.gridspec as gridspec
from shapely.geometry import Point
from matplotlib import rcParams
import geopandas as gpd
import seaborn as sns
from matplotlib.font_manager import FontProperties
import matplotlib
from matplotlib.ticker import MaxNLocator
from collections import Counter
from matplotlib.legend_handler import HandlerTuple
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
from matplotlib.ticker import FuncFormatter

sns.set(style="whitegrid", font='Times New Roman')
matplotlib.rcParams['font.family'] = 'Times New Roman'
#matplotlib.rcParams['axes.labelweight'] = 'bold'
legend_font = FontProperties(family='Times New Roman', size=14)
font_prop = FontProperties(family='Times New Roman', size=16)
def add_degree_E(x, pos):
    return f"{int(x)}°E"

def add_degree_N(y, pos):
    return f"{int(y)}°N"

def create_and_draw_subplots(model_ranks, lon_range, lat_range, selected_models,name):
    # 创建图形对象,绘制顺序依赖于fig定义顺序
    colors = ['#afd189', '#ffd60a', '#62e3dc', '#2a6858', '#1f7ea1', '#2b0000', '#E0E0E0', '#62e3dc', '#2a6858', '#1f7ea1', '#2b0000', '#E0E0E0']
    fig = plt.figure(figsize=(14, 8))
    ax1 = fig.add_axes([0.146, 0.4012, 0.510, 0.14]) # 左上
    ax2 = fig.add_axes([0.715, 0.400, 0.060, 0.15]) # 右上
    ax4 = fig.add_axes([0.655, 0.05, 0.12, 0.35]) # 右下
    ax3 = fig.add_axes([0.05, 0.05, 0.7, 0.35]) # 左下

    draw_subplot1(ax1, colors, model_ranks, selected_models)
    draw_subplot2(ax2, model_ranks, selected_models, colors)
    draw_subplot4(ax4, colors, model_ranks, selected_models)
    draw_subplot3(ax3, colors, model_ranks, lon_range, lat_range, selected_models,name)

    plt.tight_layout()

    plt.show()

# 折线图（上）
def draw_subplot1(ax, colors, model_ranks, selected_models):
    _, lons = best_models.shape
    x = np.arange(lons)
    row_list = np.zeros((lons, len(selected_models)), dtype=int)
    for i in range(lons):
        # 排除NaN值
        valid_indices = model_ranks[:, i][~np.isnan(model_ranks[:, i])]
        model_counts = Counter(valid_indices.astype(int))
        for model_idx, count in model_counts.items():
            # 确保model_idx在selected_models的索引范围内
            if 0 < model_idx <= len(selected_models):
                row_list[i, model_idx - 1] = count
    
    for i, model in enumerate(selected_models):
        if i < len(colors):
            ax.plot(x, row_list[:, i], linestyle='-', color=colors[i], linewidth=1, zorder=1, label=model)
    
    ax.set_xlim(0, lons)
    ax.tick_params(axis='x', which='both', colors='black', length=3, width=1.5, labelsize=14, bottom=False, top=False, labelbottom=False, labeltop=False)
    ax.tick_params(axis='y', which='both', colors='black', length=3, width=1.5, labelsize=14, direction='out', pad=6, left=True, right=False, labelright=False)
    plt.setp(ax.get_yticklabels(), rotation=90,ha="center", rotation_mode="anchor")
    ax.xaxis.set_major_locator(MaxNLocator(prune='both', nbins=4))
    ax.set_ylabel('Count',size=18)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_color('black')
    ax.spines['bottom'].set_linewidth(1.5)
    ax.spines['left'].set_color('black')
    ax.spines['left'].set_linewidth(1.5)
    ax.grid(False)

# 折线图（右）
def draw_subplot4(ax, colors, model_ranks, selected_models):
    lats, _ = model_ranks.shape
    x = np.arange(lats)
    row_list = np.zeros((lats, len(selected_models)), dtype=int)

    for i in range(lats):
        valid_indices = model_ranks[i, :][~np.isnan(model_ranks[i, :])]
        model_counts = Counter(valid_indices.astype(int))
        for model_idx, count in model_counts.items():
            if 0 < model_idx <= len(selected_models):
                row_list[i, model_idx - 1] = count

    for i, model in enumerate(selected_models):
        if i < len(colors):
            ax.plot(row_list[:, i], x, linestyle='-', color=colors[i], linewidth=1, zorder=1, label=model)
    ax.invert_yaxis()
    ax.set_ylim(0, lats)
    ax.tick_params(axis='x', which='both', colors='black', length=3, width=1.5, labelsize=14, direction='out', pad=3, bottom=True, top=False, labelbottom=True)
    ax.tick_params(axis='y', which='both', colors='black', length=3, width=1.5, labelsize=14, direction='in', left=False, right=False, labelright=False,labelleft=False)
    ax.xaxis.set_major_locator(MaxNLocator(nbins=4))
    ax.yaxis.set_major_locator(MaxNLocator(nbins=5))
    ax.set_xlabel('Count',size=18)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_color('black')
    ax.spines['bottom'].set_linewidth(1.5)
    ax.spines['left'].set_color('black')
    ax.spines['left'].set_linewidth(1.5)
    ax.grid(False)

# 图例
'''
def draw_subplot2(ax, model_ranks, selected_models, colors):
    line_handles = [Line2D([0], [0], color=color, linewidth=3) for color in colors]
    patch_handles = [Patch(color=color) for color in colors]
    legend_elements = [(line, patch) for line, patch in zip(line_handles, patch_handles)]
    ax.legend(legend_elements, selected_models, handler_map={tuple: HandlerTuple(ndivide=None)}, handlelength=1.5, loc='center',fancybox=False, shadow=False, ncol=2, frameon=False, prop=legend_font, columnspacing=-0.3, handletextpad=0.5, labelspacing=0.7)
    ax.axis('off')
'''
def draw_subplot2(ax, model_ranks, selected_models, colors):
    total_points = (~np.isnan(model_ranks)).sum()
    model_counts = Counter(model_ranks.flatten()) 
    percentages = [model_counts.get(i, 0) / total_points for i in range(1, len(selected_models) + 1)]
    bar_width = 0.6
    for i, model in enumerate(selected_models):
        if i < len(colors):
            ax.barh(i+bar_width/2, percentages[i], color=colors[i], height=bar_width)
            pct_text = "{:.1%}".format(percentages[i])
            ax.text(percentages[i] + 0.01, i + bar_width / 2, pct_text, va='center',fontproperties=font_prop, size=14)

    ax.set_yticks(np.arange(len(selected_models)) + bar_width / 2)
    yticklabels = ax.set_yticklabels(selected_models)
    
    # Set properties individually
    for label, model in zip(yticklabels, selected_models):
        if model in ['PSC-FENet-ST', 'PSC-FENet-SP']:
            label.set_fontproperties(FontProperties(family='Times New Roman', size=10))
        else:
            label.set_fontproperties(FontProperties(family='Times New Roman', size=16))

    ax.set_xlabel('Percentage')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(True)
    ax.spines['bottom'].set_visible(False)
    ax.tick_params('y', colors='black', length=3, width=1.5, direction='out',pad=2, which='both', left=True, right=False)
    ax.tick_params(axis='x', which='both', bottom=False, top=False, labeltop=False)
    ax.set_facecolor('none')
    ax.axis('on')
    ax.grid(False)

def draw_subplot3(ax, colors, model_ranks, lon_range, lat_range, selected_models,name):
    color_list = [colors[i] for i in range(len(selected_models))]
    # 颜色映射
    cmap = mcolors.ListedColormap(color_list)
    gdf = gpd.read_file(shp_path)
    gdf.plot(ax=ax, edgecolor='black', facecolor='none', linewidth=2)
    image = ax.imshow(model_ranks, cmap=cmap, origin='lower', extent=[new_lon_min, new_lon_max, new_lat_min, new_lat_max])
    ax.set_xlabel('Longitude(°)',size=18)
    ax.set_ylabel('Latitude(°)',size=18)
    ax.xaxis.set_major_locator(MaxNLocator(nbins=10))
    ax.yaxis.set_major_locator(MaxNLocator(nbins=6))
    ax.xaxis.set_major_formatter(FuncFormatter(add_degree_E))
    ax.yaxis.set_major_formatter(FuncFormatter(add_degree_N))
    ax.tick_params(axis='x', which='both', colors='black', length=3, width=1.5, labelsize=14, direction='out', pad=2, bottom=True, top=True, labelbottom=True,labeltop=False)
    ax.tick_params(axis='y', which='both', colors='black', length=3, width=1.5, labelsize=14, direction='out', pad=6, left=True, right=True, labelleft=True,labelright=False)
    plt.setp(ax.get_xticklabels(), rotation=0,rotation_mode="anchor")
    plt.setp(ax.get_yticklabels(), rotation=90,ha="center", rotation_mode="anchor")
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_color('black')
    ax.spines['bottom'].set_linewidth(1.5)
    ax.spines['left'].set_color('black')
    ax.spines['left'].set_linewidth(1.5)
    ax.grid(False)
    #ax.grid(True, color='black', linestyle='--',alpha=0.5,zorder=0.1)
    plt.savefig(r"C:\Users\Administrator\Desktop\Draw\min_uncertain"+name+".jpeg", dpi=1000, format='jpeg', bbox_inches='tight')

    plt.show()


colors = ['#afd189', '#88352b', '#2a6858', '#62e3dc', '#1f7ea1', '#2b0000', '#F0F0F0', '#62e3dc', '#1f7ea1', '#2b0000', '#F0F0F0', '#2b0000', '#F0F0F0']
create_and_draw_subplots(clipped_data, lon_range, lat_range, selected_model_names, name='a')
create_and_draw_subplots(clipped_data_12, lon_range, lat_range, selected_model_names, name='b')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.gridspec as gridspec
from shapely.geometry import Point
from matplotlib import rcParams
import geopandas as gpd
import seaborn as sns
from matplotlib.font_manager import FontProperties
import matplotlib
from matplotlib.ticker import MaxNLocator
from collections import Counter
from matplotlib.legend_handler import HandlerTuple
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
from matplotlib.ticker import FixedLocator, FixedFormatter

sns.set(style="whitegrid", font='Times New Roman')
matplotlib.rcParams['font.family'] = 'Times New Roman'
#matplotlib.rcParams['axes.labelweight'] = 'bold'
legend_font = FontProperties(family='Times New Roman', size=14)
font_prop = FontProperties(family='Times New Roman', size=16)
def add_degree_E(x, pos):
    return f"{int(x)}°E"

def add_degree_N(y, pos):
    return f"{int(y)}°N"

def create_and_draw_subplots(model_ranks, lon_range, lat_range, selected_models,name):
    # 创建图形对象,绘制顺序依赖于fig定义顺序
    colors = ['#afd189', '#ffd60a', '#62e3dc', '#2a6858', '#1f7ea1', '#2b0000', '#E0E0E0', '#62e3dc', '#2a6858', '#1f7ea1', '#2b0000', '#E0E0E0']
    fig = plt.figure(figsize=(14, 8))
    ax1 = fig.add_axes([0.146, 0.4012, 0.510, 0.14]) # 左上
    ax2 = fig.add_axes([0.715, 0.400, 0.060, 0.15]) # 右上
    ax4 = fig.add_axes([0.655, 0.05, 0.12, 0.35]) # 右下
    ax3 = fig.add_axes([0.05, 0.05, 0.7, 0.35]) # 左下

    draw_subplot1(ax1, colors, model_ranks, selected_models)
    draw_subplot2(ax2, model_ranks, selected_models, colors)
    draw_subplot4(ax4, colors, model_ranks, selected_models)
    draw_subplot3(ax3, colors, model_ranks, lon_range, lat_range, selected_models,name)

    plt.tight_layout()

    plt.show()

# 折线图（上）
def draw_subplot1(ax, colors, model_ranks, selected_models):
    _, lons = best_models.shape
    x = np.arange(lons)
    row_list = np.zeros((lons, len(selected_models)), dtype=int)
    for i in range(lons):
        valid_indices = model_ranks[:, i][~np.isnan(model_ranks[:, i])]
        total_valid = len(valid_indices)
        if total_valid > 0:
            model_counts = Counter(valid_indices.astype(int))
        
            for model_idx, count in model_counts.items():
                if 0 < model_idx <= len(selected_models):
                    row_list[i, model_idx - 1] = (count / total_valid) * 100
        else:
            row_list[i, :] = 0
    for i, model in enumerate(selected_models):
        if i < len(colors):
            ax.plot(x, row_list[:, i], linestyle='-', color=colors[i], linewidth=1, zorder=1, label=model)
    
    ax.set_xlim(0, lons)
    ax.tick_params(axis='x', which='both', colors='black', length=3, width=1.5, labelsize=14, bottom=False, top=False, labelbottom=False, labeltop=False)
    ax.tick_params(axis='y', which='both', colors='black', length=3, width=1.5, labelsize=14, direction='out', pad=6, left=True, right=False, labelright=False)
    plt.setp(ax.get_yticklabels(), rotation=90,ha="center", rotation_mode="anchor")
    ax.yaxis.set_major_locator(FixedLocator([lats * 0.25, lats * 0.5, lats * 0.75, lats - 1]))
    ax.yaxis.set_major_formatter(FixedFormatter(['25%', '50%', '75%', '100%']))
    ax.xaxis.set_major_locator(MaxNLocator(prune='both', nbins=4))
    ax.set_ylabel('Count',size=18)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_color('black')
    ax.spines['bottom'].set_linewidth(1.5)
    ax.spines['left'].set_color('black')
    ax.spines['left'].set_linewidth(1.5)
    ax.grid(False)

# 折线图（右）
def draw_subplot4(ax, colors, model_ranks, selected_models):
    lats, _ = model_ranks.shape
    x = np.arange(lats)
    row_list = np.zeros((lats, len(selected_models)), dtype=float)  # 存储百分比，使用float类型

    for i in range(lats):
        valid_indices = model_ranks[i, :][~np.isnan(model_ranks[i, :])]
        total_valid = len(valid_indices)
        model_counts = Counter(valid_indices.astype(int)) if total_valid > 0 else {}
        
        for model_idx, count in model_counts.items():
            if 0 < model_idx <= len(selected_models):
                # 转换为百分比
                row_list[i, model_idx - 1] = (count / total_valid * 100) if total_valid > 0 else 0

    for i, model in enumerate(selected_models):
        if i < len(colors):
            ax.plot(row_list[:, i],x, linestyle='-', color=colors[i], linewidth=1, zorder=1, label=model)

    ax.invert_yaxis()
    ax.set_ylim(0, lats)

    ax.tick_params(axis='x', which='both', colors='black', length=3, width=1.5, labelsize=14, direction='out', pad=3, bottom=True, top=False, labelbottom=True)
    ax.tick_params(axis='y', which='both', colors='black', length=3, width=1.5, labelsize=14, direction='in', left=False, right=False, labelright=False,labelleft=False)
    ax.xaxis.set_major_locator(FixedLocator([lats * 0.25, lats * 0.5, lats * 0.75, lats - 1]))
    ax.xaxis.set_major_formatter(FixedFormatter(['25%', '50%', '75%', '100%']))
    ax.yaxis.set_major_locator(MaxNLocator(nbins=5))
    ax.set_xlabel('Count',size=18)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_color('black')
    ax.spines['bottom'].set_linewidth(1.5)
    ax.spines['left'].set_color('black')
    ax.spines['left'].set_linewidth(1.5)
    ax.grid(False)

# 图例
'''
def draw_subplot2(ax, model_ranks, selected_models, colors):
    line_handles = [Line2D([0], [0], color=color, linewidth=3) for color in colors]
    patch_handles = [Patch(color=color) for color in colors]
    legend_elements = [(line, patch) for line, patch in zip(line_handles, patch_handles)]
    ax.legend(legend_elements, selected_models, handler_map={tuple: HandlerTuple(ndivide=None)}, handlelength=1.5, loc='center',fancybox=False, shadow=False, ncol=2, frameon=False, prop=legend_font, columnspacing=-0.3, handletextpad=0.5, labelspacing=0.7)
    ax.axis('off')
'''
def draw_subplot2(ax, model_ranks, selected_models, colors):
    total_points = (~np.isnan(model_ranks)).sum()
    model_counts = Counter(model_ranks.flatten()) 
    percentages = [model_counts.get(i, 0) / total_points for i in range(1, len(selected_models) + 1)]
    bar_width = 0.6
    for i, model in enumerate(selected_models):
        if i < len(colors):
            ax.barh(i+bar_width/2, percentages[i], color=colors[i], height=bar_width)
            pct_text = "{:.1%}".format(percentages[i])
            ax.text(percentages[i] + 0.01, i + bar_width / 2, pct_text, va='center',fontproperties=font_prop, size=14)

    ax.set_yticks(np.arange(len(selected_models)) + bar_width / 2)
    yticklabels = ax.set_yticklabels(selected_models)
    
    # Set properties individually
    for label, model in zip(yticklabels, selected_models):
        if model in ['PSC-FENet-ST', 'PSC-FENet-SP']:
            label.set_fontproperties(FontProperties(family='Times New Roman', size=10))
        else:
            label.set_fontproperties(FontProperties(family='Times New Roman', size=16))

    ax.set_xlabel('Percentage')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(True)
    ax.spines['bottom'].set_visible(False)
    ax.tick_params('y', colors='black', length=3, width=1.5, direction='out',pad=2, which='both', left=True, right=False)
    ax.tick_params(axis='x', which='both', bottom=False, top=False, labeltop=False)
    ax.set_facecolor('none')
    ax.axis('on')
    ax.grid(False)

def draw_subplot3(ax, colors, model_ranks, lon_range, lat_range, selected_models,name):
    color_list = [colors[i] for i in range(len(selected_models))]
    # 颜色映射
    cmap = mcolors.ListedColormap(color_list)
    gdf = gpd.read_file(shp_path)
    gdf.plot(ax=ax, edgecolor='black', facecolor='none', linewidth=2)
    image = ax.imshow(model_ranks, cmap=cmap, origin='lower', extent=[new_lon_min, new_lon_max, new_lat_min, new_lat_max])
    ax.set_xlabel('Longitude(°)',size=18)
    ax.set_ylabel('Latitude(°)',size=18)
    ax.xaxis.set_major_locator(MaxNLocator(nbins=10))
    ax.yaxis.set_major_locator(MaxNLocator(nbins=6))
    ax.xaxis.set_major_formatter(FuncFormatter(add_degree_E))
    ax.yaxis.set_major_formatter(FuncFormatter(add_degree_N))
    ax.tick_params(axis='x', which='both', colors='black', length=3, width=1.5, labelsize=14, direction='out', pad=2, bottom=True, top=True, labelbottom=True,labeltop=False)
    ax.tick_params(axis='y', which='both', colors='black', length=3, width=1.5, labelsize=14, direction='out', pad=6, left=True, right=True, labelleft=True,labelright=False)
    plt.setp(ax.get_xticklabels(), rotation=0,rotation_mode="anchor")
    plt.setp(ax.get_yticklabels(), rotation=90,ha="center", rotation_mode="anchor")
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_color('black')
    ax.spines['bottom'].set_linewidth(1.5)
    ax.spines['left'].set_color('black')
    ax.spines['left'].set_linewidth(1.5)
    ax.grid(False)
    #ax.grid(True, color='black', linestyle='--',alpha=0.5,zorder=0.1)
    plt.savefig(r"C:\Users\Administrator\Desktop\Draw\min_uncertain2"+name+".jpeg", dpi=1000, format='jpeg', bbox_inches='tight')

    plt.show()


colors = ['#afd189', '#88352b', '#2a6858', '#62e3dc', '#1f7ea1', '#2b0000', '#F0F0F0', '#62e3dc', '#1f7ea1', '#2b0000', '#F0F0F0', '#2b0000', '#F0F0F0']
create_and_draw_subplots(clipped_data, lon_range, lat_range, selected_model_names, name='a')
create_and_draw_subplots(clipped_data_12, lon_range, lat_range, selected_model_names, name='b')

## colorbar测试

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

colors = ["#ffc43d", "#60d394"] 
cmap_name = "custom_orange_green"
cm = LinearSegmentedColormap.from_list(cmap_name, colors, N=100)

gradient = np.linspace(0, 1, 256)
gradient = np.vstack((gradient, gradient))

fig, ax = plt.subplots(figsize=(6, 2))
ax.imshow(gradient, aspect='auto', cmap=cm)
ax.set_axis_off()

plt.show()